## Get 2 incidents

In [2]:
import sys, os

sys.path.append(os.environ["BUILD_WORKSPACE_DIRECTORY"])
sys.path.append(
    os.path.join(os.environ["BUILD_WORKSPACE_DIRECTORY"], "bazel-bin")
)

In [ ]:
CAMERAS = "configs/cameras/cameras"
from core.utils.yaml_jinja import load_yaml_with_jinja
from datetime import datetime, timedelta
from loguru import logger
from core.utils.perception_portal_graphql_session import (
    PerceptionPortalSession,
)
from core.common.queries import FILTERED_RAW_INCIDENTS_QUERY
import json

from tqdm import tqdm
import tempfile
from core.utils.aws_utils import (
    download_to_file,
    glob_from_bucket,
    upload_file,
)

In [ ]:
from dataclasses import dataclass
from typing import Optional


@dataclass
class Incident:
    camera_uuid: str
    video_s3_path: str
    incident_type_id: str
    incident_uuid: str
    organization: str
    zone: str
    experimental: bool
    created_at: str
    feedback_type: str
    scenario_type: str
    copied_s3_path: Optional[str] = None


feedback_scenario_map = {
    "valid": "POSITIVE",
    "invalid": "NEGATIVE",
    "all": "UNKNOWN",
}

In [ ]:
def _get_camera_uuid_version(cameras):
    camera_uuids = {}
    for config_path in cameras:
        config = load_yaml_with_jinja(config_path)
        camera_uuids[config["camera_uuid"]] = config["incident"][
            "state_machine_monitors_requested"
        ]
    return camera_uuids


with open(CAMERAS, encoding="utf-8") as f:
    cameras = []
    for line in f:
        cameras.append(line.rstrip("\n"))
    camera_uuids = _get_camera_uuid_version(cameras)
    print(camera_uuids)

In [ ]:
start_date = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")
end_date = (datetime.now()).strftime("%Y-%m-%d")

if start_date:
    start_date = datetime.strptime(start_date, "%Y-%m-%d").isoformat()
if end_date:
    end_date = datetime.strptime(end_date, "%Y-%m-%d").isoformat()

In [ ]:
FILTERED_RAW_INCIDENTS_QUERY = """
query GetFilteredIncidents($fromUtc: DateTime, $toUtc: DateTime, $incidentTypeFilter: String, $organizationKey: [String], $zoneKey: [String], $cameraUuid: [String], $feedbackType: String, $first: Int, $after: String) {
    integrations {
        filteredRawIncidents(
            fromUtc: $fromUtc
            toUtc: $toUtc
            incidentTypeFilter: $incidentTypeFilter
            organizationKey: $organizationKey
            zoneKey: $zoneKey
            cameraUuid: $cameraUuid
            feedbackType: $feedbackType
            first: $first
            after: $after
        ) {
            pageInfo {
                hasNextPage
                endCursor
            }
            edges {
                cursor
                node {
                    uuid
                    data
                    createdAt
                    cameraUuid
                    experimental
                    organization {
                        key
                    }
                    zone {
                        key
                    }
                    incidentType {
                        key
                    }
                }
            }
        }
    }
}
"""

In [ ]:
incidents = []
use_experimental_incidents = False
with PerceptionPortalSession("PROD") as perception_portal_session:
    for camera_uuid, machines in camera_uuids.items():
        for machine in machines:
            print(f"{camera} , {machine}")

            variables = {
                "fromUtc": start_date,
                "toUtc": end_date,
                "incidentTypeFilter": machine,
                "feedbackType": "valid",
                "first": 2,
                "cameraUuid": camera_uuid,
                "after": None,
                "organizationKey": None,
                "zoneKey": None,
            }
            # Execute graphql query
            response = perception_portal_session.session.post(
                f"{perception_portal_session.host}/graphql/",
                json={
                    "query": FILTERED_RAW_INCIDENTS_QUERY,
                    "variables": variables,
                },
                headers=perception_portal_session.headers,
            )

            if response.status_code != 200:
                raise RuntimeError(f"Query failed: {response.reason}")

            result_json = json.loads(response.text)["data"]["integrations"][
                "filteredRawIncidents"
            ]
            for node in result_json["edges"]:
                # skip video if it is already in metaverse
                # skip video if experimental flag does not match
                if use_experimental_incidents != node["node"]["experimental"]:
                    remaining_incidents_to_load += 1
                    continue
                print(node["node"])
                data = json.loads(node["node"]["data"])
                incident = Incident(
                    camera_uuid=data["camera_uuid"],
                    video_s3_path=data["video_s3_path"],
                    created_at=node["node"]["createdAt"],
                    incident_type_id=node["node"]["incidentType"]["key"],
                    incident_uuid=node["node"]["uuid"],
                    organization=node["node"]["organization"]["key"],
                    zone=node["node"]["zone"]["key"],
                    experimental=node["node"]["experimental"],
                    feedback_type="valid",
                    scenario_type=feedback_scenario_map.get(
                        "valid", "UNKNOWN"
                    ),
                )
                print(incident)
                incidents.append(incident)
print(incidents)

In [ ]:
copy_incidents_to_bucket(incidents)

In [ ]:
from typing import List


def copy_incidents_to_bucket(
    incidents: List[Incident],
    output_bucket: str = "voxel-raw-logs",
    output_path: str = "sandbox",
    environment: str = "production",
):
    """Moves incident videos to a S3 bucket

    Args:
        incidents (list): list of incidents
        output_bucket (str, optional): Bucket to copy to. Defaults to "voxel_lightly_input".
        output_path (str, optional): Path in S3 bucket to copy videos to. Defaults to "sandbox".
        environment (str): Environment operation is being run in. Defaults to "production".

    """
    incidents_s3_paths = []
    logger.info(f"Copying over {len(incidents)} incidents to AWS bucket..")
    for incident in tqdm(incidents):
        with tempfile.NamedTemporaryFile() as temporary_file:

            # Download incidents from aws
            incident_files_s3 = glob_from_bucket(
                bucket=f"voxel-portal-{environment}",
                prefix=f"{incident.organization.lower()}/{incident.zone.lower()}/incidents/{incident.incident_uuid}",
                extensions=("mp4"),
            )

            def get_incident_videos(files: List[str]) -> List[str]:
                """Helper method to get incident video

                Args:
                    files (List[str]): list of files

                Returns:
                    List[str]: list of files filtered
                """

                def filter_incident(file_type: str) -> List[str]:
                    """Helper method to filter incident files

                    Args:
                        file_type (str): file type to search for

                    Returns:
                        List[str]: filtered list of incidents
                    """
                    return [
                        incident_video_file
                        for incident_video_file in files
                        if file_type in incident_video_file
                    ]

                incident_videos = filter_incident("original")  # h265 video
                if not incident_videos:
                    incident_videos = filter_incident("mp4")  # h264 video

                return incident_videos

            if len(incident_files_s3) > 0:

                # download
                incident_video = get_incident_videos(incident_files_s3)

                download_to_file(
                    f"voxel-portal-{environment}",
                    incident_video[0],
                    temporary_file.name,
                )

                s3_path = upload_file(
                    bucket=output_bucket,
                    local_path=temporary_file.name,
                    s3_path=f"{incident.camera_uuid}/scenarios/{incident.incident_type_id}/POSITIVE/{incident.incident_uuid}.mp4",
                )
                incident.copied_s3_path = f"s3://{output_bucket}/{s3_path}"
                incidents_s3_paths.append(
                    f"{incident.camera_uuid}/{incident.incident_uuid}"
                )
                print(f"{incident.camera_uuid}/{incident.incident_uuid}")
    logger.info("Done")

In [ ]:
import boto3

from core.utils.aws_utils import upload_file
from urllib.parse import urlparse, urlunparse
from typing import Tuple


def get_bucket_path_from_s3_uri(s3_uri: str) -> Tuple[str, str]:
    """Get bucket name and path from s3_uri

    Args:
        s3_uri (str): uri of s3 file

    Returns:
        Tuple[str,str]: bucket name and path
    """
    parse = urlparse(s3_uri, allow_fragments=False)
    print(parse)
    print(type(parse))
    print(type(parse.path))
    print(type(parse.path.lstrip("/")))
    return parse.netloc, parse.path.lstrip("/")


print(get_bucket_path_from_s3_uri("ds"))


def copy_object(source_uri: str, destination_uri: str) -> None:
    """Creates a copy of an object already stored in S3

    Args:
        source_uri (str): full path of the source object
        destination_uri (str): full path of the destination object
    """
    print(f"soure{source_uri}")
    print(type(source_uri))
    parse = urlparse(destination_uri, allow_fragments=False)
    print(f"Parsed {parse}")
    print((parse.path.lstrip("/")))
    (
        destination_bucket_name,
        destination_relative_s3_path,
    ) = get_bucket_path_from_s3_uri(destination_uri)
    if destination_bucket_name == "":
        return

    (
        source_bucket_name,
        source_relative_s3_path,
    ) = get_bucket_path_from_s3_uri(source_uri)

    s3_client = boto3.client("s3")
    s3_client.copy_object(
        Bucket=destination_bucket_name,
        Key=destination_relative_s3_path,
        CopySource={
            "Bucket": source_bucket_name,
            "Key": source_relative_s3_path,
        },
    )

In [ ]:
from core.utils.aws_utils import upload_file
from urllib.parse import urlparse, urlunparse

bucket = "voxel-raw-logs"
ext = "mp4"
video_uuids = []
for incident in incidents:
    video_uuid = (
        f"{incident.camera_uuid}/scenarios/"
        f"{incident.incident_type_id}/"
        f"{incident.scenario_type}/"
        f"{incident.incident_uuid}"
    )
    scenario_s3_uri = f"s3://{bucket}/{video_uuid}.{ext}"
    print(incident)
    copy_object(incident.copied_s3_path, scenario_s3_uri)
    video_uuids.append(video_uuid)
print(video_uuids)

In [ ]:
print(incidents)

In [ ]:
video_uuids = []
for incident in incidents:
    video_uuids.append(
        os.path.splitext(
            get_bucket_path_from_s3_uri(incident.copied_s3_path)[1]
        )[0]
    )
print(video_uuids)

In [ ]:
from core.infra.sematic.perception.data_ingestion.ingest_raw_videos.pipeline import (
    pipeline,
)
from core.infra.sematic.shared.utils import (
    PipelineSetup,
    SematicOptions,
    resolve_sematic_future,
)
from sematic import (
    CloudResolver,
    LocalResolver,
    SilentResolver,
    has_container_image,
)

future = pipeline(
    videos=video_uuids,
    is_test=False,
    fps=-1,
    metaverse_environment="PROD",
    pipeline_setup=PipelineSetup(),
)
future.resolve(SilentResolver())
return 0

In [ ]:
print(os.environ.get("METAVERSE_ENVIRONMENT", "PROD"))

In [ ]:
import os

incident_map = {}
for incident in incidents:
    uuid = os.path.splitext(
        get_bucket_path_from_s3_uri(incident.copied_s3_path)[1]
    )[0]
    if incident.incident_type_id not in incident_map:
        incident_map[incident.incident_type_id] = []
    incident_map[incident.incident_type_id].append(uuid)
print(incident_map)

In [ ]:
COPYRIGHT = (
    "#\n"
    "# Copyright 2023 Voxel Labs, Inc.\n"
    "# All rights reserved.\n"
    "#\n"
    "# This document may not be reproduced, republished, distributed, transmitted,\n"
    "# displayed, broadcast or otherwise exploited in any manner without the express\n"
    "# prior written permission of Voxel Labs, Inc. The receipt or possession of this\n"
    "# document does not convey any rights to reproduce, disclose, or distribute its\n"
    "# contents, or to manufacture, use, or sell anything that it may describe, in\n"
    "# whole or in part.\n"
    "#\n"
    "#\n"
)
from core.utils.video_utils import get_camera_uuid

print(
    get_camera_uuid(
        "ppg/cedar_falls/0011/cha/d8e223bc-4e1a-470d-980e-ad044844124f"
    )
)

In [ ]:
for incident_type_id, incidents in incident_map.items():
    with open(
        f"/home/diksha/workspace2/voxel/data/scenario_sets/regression/{incident_type_id.lower()}.yaml",
        "w",
    ) as f:
        f.write(COPYRIGHT)
        f.write(("scenarios:\n" "  # POSITIVE SCENARIOS\n"))
        for incident in incidents:
            f.write(
                (
                    f"  - camera_uuid: {get_camera_uuid(incident)}\n"
                    f"    incidents: [{incident_type_id}]\n"
                    f"    video_uuid: {incident}_0000\n"
                )
            )

In [ ]:
print(incidents)

In [ ]:
incidents = [
    Incident(
        camera_uuid="uscold/laredo/room_d5p/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/04d89f27-08f1-4723-8f40-597fb778f220_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="04d89f27-08f1-4723-8f40-597fb778f220",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-03-01T15:46:19.526778+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/room_d5p/cha/04d89f27-08f1-4723-8f40-597fb778f220.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/room_d5p/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/2f5c200a-83f0-4154-b41f-a5f5982dc9d2_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="2f5c200a-83f0-4154-b41f-a5f5982dc9d2",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-03-01T17:06:08.865139+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/room_d5p/cha/2f5c200a-83f0-4154-b41f-a5f5982dc9d2.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/room_d4/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/4471ace5-c31e-49b6-aa96-9d150b9875b7_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="4471ace5-c31e-49b6-aa96-9d150b9875b7",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-28T03:27:46.306001+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/room_d4/cha/4471ace5-c31e-49b6-aa96-9d150b9875b7.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/dock01/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/1d20af2a-c642-42b2-9d45-618c48919b48_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="1d20af2a-c642-42b2-9d45-618c48919b48",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-03-02T17:35:26.829060+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/dock01/cha/1d20af2a-c642-42b2-9d45-618c48919b48.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/dock01/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/fcc4a7e7-7ef0-4cec-8ec9-500368127d63_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="fcc4a7e7-7ef0-4cec-8ec9-500368127d63",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-23T16:56:41.037540+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/dock01/cha/fcc4a7e7-7ef0-4cec-8ec9-500368127d63.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/dock01/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/33269097-ef9a-4061-9dec-03ae76946546_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="33269097-ef9a-4061-9dec-03ae76946546",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-20T15:49:01.566317+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/dock01/cha/33269097-ef9a-4061-9dec-03ae76946546.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/dock01/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/23dc083b-edba-41c0-944c-76ae19131e56_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="23dc083b-edba-41c0-944c-76ae19131e56",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-20T17:08:06.084695+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/dock01/cha/23dc083b-edba-41c0-944c-76ae19131e56.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/dock01/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/507e63ca-05ee-4ce8-bc08-396e6a37e47e_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="507e63ca-05ee-4ce8-bc08-396e6a37e47e",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-23T16:56:18.285278+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/dock01/cha/507e63ca-05ee-4ce8-bc08-396e6a37e47e.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/dock01/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/d0d6bed5-3082-45b9-9638-4cb6c5167e15_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="d0d6bed5-3082-45b9-9638-4cb6c5167e15",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-23T17:43:18.874734+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/dock01/cha/d0d6bed5-3082-45b9-9638-4cb6c5167e15.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/room_d2/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/49bb7b40-1621-4a97-ba7b-79c8de65797c_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="49bb7b40-1621-4a97-ba7b-79c8de65797c",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-28T17:07:41.838541+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/room_d2/cha/49bb7b40-1621-4a97-ba7b-79c8de65797c.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/room_d2/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/ca9499a4-649f-49f2-be1b-eaf80ef5b314_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="ca9499a4-649f-49f2-be1b-eaf80ef5b314",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-28T16:37:47.086656+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/room_d2/cha/ca9499a4-649f-49f2-be1b-eaf80ef5b314.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/dock04/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/0e70d5fd-2d39-43be-92bf-1244b7eeba14_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="0e70d5fd-2d39-43be-92bf-1244b7eeba14",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-10T01:43:11.563813+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/dock04/cha/0e70d5fd-2d39-43be-92bf-1244b7eeba14.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/dock04/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/98456299-357d-4cb0-9670-a22dd97fa1bd_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="98456299-357d-4cb0-9670-a22dd97fa1bd",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-20T22:05:28.153132+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/dock04/cha/98456299-357d-4cb0-9670-a22dd97fa1bd.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/dock04/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/c0b13672-460c-43ae-a1c8-4877f9dfaf00_video.mp4",
        incident_type_id="HARD_HAT",
        incident_uuid="c0b13672-460c-43ae-a1c8-4877f9dfaf00",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-20T21:55:55.821100+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/dock04/cha/c0b13672-460c-43ae-a1c8-4877f9dfaf00.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/dock04/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/5c20c2c3-03c3-42b5-9c63-51987e09ab45_video.mp4",
        incident_type_id="HARD_HAT",
        incident_uuid="5c20c2c3-03c3-42b5-9c63-51987e09ab45",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-03-04T07:06:25.032075+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/dock04/cha/5c20c2c3-03c3-42b5-9c63-51987e09ab45.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/dock04/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/4b6db807-6fe2-4ec4-ac76-bd9359ca5c1c_video.mp4",
        incident_type_id="NO_PED_ZONE",
        incident_uuid="4b6db807-6fe2-4ec4-ac76-bd9359ca5c1c",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-24T14:06:50.409322+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/dock04/cha/4b6db807-6fe2-4ec4-ac76-bd9359ca5c1c.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/dock04/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/e17529fb-8c06-4775-b5d7-87c89e01b07a_video.mp4",
        incident_type_id="NO_PED_ZONE",
        incident_uuid="e17529fb-8c06-4775-b5d7-87c89e01b07a",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-03-04T15:11:16.187633+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/dock04/cha/e17529fb-8c06-4775-b5d7-87c89e01b07a.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/dock04/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/04325c95-b126-49fd-9069-1eb00d9dbc4d_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="04325c95-b126-49fd-9069-1eb00d9dbc4d",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-24T20:37:12.093662+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/dock04/cha/04325c95-b126-49fd-9069-1eb00d9dbc4d.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/dock04/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/d1eefaae-e672-4880-bd2c-8db81808fc03_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="d1eefaae-e672-4880-bd2c-8db81808fc03",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-03-06T12:08:55.333124+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/dock04/cha/d1eefaae-e672-4880-bd2c-8db81808fc03.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/doors_14_20/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/5da393dc-7bb0-4e4f-8e9d-93bd9d91635d_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="5da393dc-7bb0-4e4f-8e9d-93bd9d91635d",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-03-04T00:27:45.240284+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/doors_14_20/cha/5da393dc-7bb0-4e4f-8e9d-93bd9d91635d.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/doors_14_20/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/98068733-5d90-422b-be4e-a04653d6c12c_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="98068733-5d90-422b-be4e-a04653d6c12c",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-03-04T00:27:32.977630+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/doors_14_20/cha/98068733-5d90-422b-be4e-a04653d6c12c.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/doors_14_20/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/fd0ed3bb-fff1-497c-9551-f7fd39d45e81_video.mp4",
        incident_type_id="HARD_HAT",
        incident_uuid="fd0ed3bb-fff1-497c-9551-f7fd39d45e81",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-13T17:12:23.234346+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/doors_14_20/cha/fd0ed3bb-fff1-497c-9551-f7fd39d45e81.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/doors_14_20/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/313a928c-e98f-4497-bfd6-27afe26e2713_video.mp4",
        incident_type_id="HARD_HAT",
        incident_uuid="313a928c-e98f-4497-bfd6-27afe26e2713",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-06T23:24:37.500311+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/doors_14_20/cha/313a928c-e98f-4497-bfd6-27afe26e2713.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/doors_14_20/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/84f791cb-d3c1-43c4-8169-cb930f3ed5df_video.mp4",
        incident_type_id="NO_PED_ZONE",
        incident_uuid="84f791cb-d3c1-43c4-8169-cb930f3ed5df",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-24T19:19:13.223814+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/doors_14_20/cha/84f791cb-d3c1-43c4-8169-cb930f3ed5df.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/doors_14_20/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/626f67a3-7c8f-4acc-9b16-7527014eaa1a_video.mp4",
        incident_type_id="NO_PED_ZONE",
        incident_uuid="626f67a3-7c8f-4acc-9b16-7527014eaa1a",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-07T14:05:08.890495+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/doors_14_20/cha/626f67a3-7c8f-4acc-9b16-7527014eaa1a.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/doors_14_20/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/c5c2a92e-9789-49b9-becb-19ec3442d12d_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="c5c2a92e-9789-49b9-becb-19ec3442d12d",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-27T14:13:33.728186+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/doors_14_20/cha/c5c2a92e-9789-49b9-becb-19ec3442d12d.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/doors_14_20/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/2aced1a2-dbc6-4a22-8e19-bcd3adec6261_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="2aced1a2-dbc6-4a22-8e19-bcd3adec6261",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-27T14:33:36.925258+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/doors_14_20/cha/2aced1a2-dbc6-4a22-8e19-bcd3adec6261.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/room_f1/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/74a5849c-78f4-4bd8-bf3d-3fd4e9b63d51_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="74a5849c-78f4-4bd8-bf3d-3fd4e9b63d51",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-21T06:59:02.460558+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/room_f1/cha/74a5849c-78f4-4bd8-bf3d-3fd4e9b63d51.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/room_f1/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/d44fe204-2943-4010-a7d2-30a35a806487_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="d44fe204-2943-4010-a7d2-30a35a806487",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-21T07:35:42.221810+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/room_f1/cha/d44fe204-2943-4010-a7d2-30a35a806487.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/room_f1/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/98d5a2ea-4688-4cde-9aa1-2cceebab9525_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="98d5a2ea-4688-4cde-9aa1-2cceebab9525",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-20T23:46:20.492548+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/room_f1/cha/98d5a2ea-4688-4cde-9aa1-2cceebab9525.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/room_f1/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/13ae5015-ebd2-4a28-83d8-6542a8f67400_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="13ae5015-ebd2-4a28-83d8-6542a8f67400",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-20T23:46:36.001865+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/room_f1/cha/13ae5015-ebd2-4a28-83d8-6542a8f67400.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/room_d1p/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/d2f51f23-3b0b-4e8f-8c35-848a7b93da1a_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="d2f51f23-3b0b-4e8f-8c35-848a7b93da1a",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-25T08:55:32.896517+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/room_d1p/cha/d2f51f23-3b0b-4e8f-8c35-848a7b93da1a.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/room_d1p/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/c25cac36-4e89-436c-a10a-78d17efb0971_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="c25cac36-4e89-436c-a10a-78d17efb0971",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-25T12:11:57.881530+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/room_d1p/cha/c25cac36-4e89-436c-a10a-78d17efb0971.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/dock03/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/96e30425-407a-42d8-8351-4293fcdaa09a_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="96e30425-407a-42d8-8351-4293fcdaa09a",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-10T01:48:30.946416+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/dock03/cha/96e30425-407a-42d8-8351-4293fcdaa09a.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/dock03/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/4d559a8a-5cf9-4492-a915-fd5408d07504_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="4d559a8a-5cf9-4492-a915-fd5408d07504",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-21T20:54:35.432791+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/dock03/cha/4d559a8a-5cf9-4492-a915-fd5408d07504.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/dock03/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/2aa4c685-4989-40b4-9b19-19905e27e4ec_video.mp4",
        incident_type_id="HARD_HAT",
        incident_uuid="2aa4c685-4989-40b4-9b19-19905e27e4ec",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-03-04T05:15:46.036966+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/dock03/cha/2aa4c685-4989-40b4-9b19-19905e27e4ec.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/dock03/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/43066680-c347-4746-8f01-e63c00b6a0f0_video.mp4",
        incident_type_id="HARD_HAT",
        incident_uuid="43066680-c347-4746-8f01-e63c00b6a0f0",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-20T14:43:25.212840+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/dock03/cha/43066680-c347-4746-8f01-e63c00b6a0f0.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/dock03/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/82e4e8c5-02c3-41ee-94d3-92594467576c_video.mp4",
        incident_type_id="NO_PED_ZONE",
        incident_uuid="82e4e8c5-02c3-41ee-94d3-92594467576c",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-21T03:22:04.723064+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/dock03/cha/82e4e8c5-02c3-41ee-94d3-92594467576c.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/dock03/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/6cb88de3-7a7a-445c-8136-163e59da5700_video.mp4",
        incident_type_id="NO_PED_ZONE",
        incident_uuid="6cb88de3-7a7a-445c-8136-163e59da5700",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-24T13:40:12.351438+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/dock03/cha/6cb88de3-7a7a-445c-8136-163e59da5700.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/dock03/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/19589a73-70ac-4d16-9eaf-aa9a198e3adb_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="19589a73-70ac-4d16-9eaf-aa9a198e3adb",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-27T14:41:12.586105+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/dock03/cha/19589a73-70ac-4d16-9eaf-aa9a198e3adb.mp4",
    ),
    Incident(
        camera_uuid="uscold/laredo/dock03/cha",
        video_s3_path="s3://voxel-portal-production/uscold/laredo/incidents/cfd55b5c-0f1c-440a-b257-33fa1ba47732_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="cfd55b5c-0f1c-440a-b257-33fa1ba47732",
        organization="USCOLD",
        zone="LAREDO",
        experimental=False,
        created_at="2023-02-27T20:48:07.788448+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/laredo/dock03/cha/cfd55b5c-0f1c-440a-b257-33fa1ba47732.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0001/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/b0f2ae00-091f-42ee-8030-75c4d2162147_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="b0f2ae00-091f-42ee-8030-75c4d2162147",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-03-01T13:37:00.491526+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0001/cha/b0f2ae00-091f-42ee-8030-75c4d2162147.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0001/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/428e5db8-a973-40db-8ef3-c358043a776e_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="428e5db8-a973-40db-8ef3-c358043a776e",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-03-03T01:11:54.932148+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0001/cha/428e5db8-a973-40db-8ef3-c358043a776e.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0001/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/eaa76935-7ee5-414c-b0c1-5a7bbd81639a_video.mp4",
        incident_type_id="DOOR_VIOLATION",
        incident_uuid="eaa76935-7ee5-414c-b0c1-5a7bbd81639a",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-02-25T08:54:19.339452+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0001/cha/eaa76935-7ee5-414c-b0c1-5a7bbd81639a.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0001/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/34638961-2244-4160-9e7d-4a9dbcba8bff_video.mp4",
        incident_type_id="DOOR_VIOLATION",
        incident_uuid="34638961-2244-4160-9e7d-4a9dbcba8bff",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-02-26T14:32:24.349277+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0001/cha/34638961-2244-4160-9e7d-4a9dbcba8bff.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0001/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/d8360551-d29c-48a8-ba39-5b592706489d_video.mp4",
        incident_type_id="PIGGYBACK",
        incident_uuid="d8360551-d29c-48a8-ba39-5b592706489d",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-03-01T10:27:46.263970+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0001/cha/d8360551-d29c-48a8-ba39-5b592706489d.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0001/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/d0101fe1-2b3e-45bc-9334-bb5a12d23864_video.mp4",
        incident_type_id="PIGGYBACK",
        incident_uuid="d0101fe1-2b3e-45bc-9334-bb5a12d23864",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-03-01T13:34:22.363393+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0001/cha/d0101fe1-2b3e-45bc-9334-bb5a12d23864.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0002/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/a19dd508-2ad2-466e-828c-e7e9a0a0ed7f_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="a19dd508-2ad2-466e-828c-e7e9a0a0ed7f",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-02-24T22:47:34.039527+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0002/cha/a19dd508-2ad2-466e-828c-e7e9a0a0ed7f.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0002/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/fa132036-05a0-448d-adec-29671d765825_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="fa132036-05a0-448d-adec-29671d765825",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-02-27T01:15:48.075156+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0002/cha/fa132036-05a0-448d-adec-29671d765825.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0002/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/7710d4e2-db58-48c2-aea8-b3bada5ab12c_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="7710d4e2-db58-48c2-aea8-b3bada5ab12c",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-03-05T20:25:37.739982+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0002/cha/7710d4e2-db58-48c2-aea8-b3bada5ab12c.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0002/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/15137327-a513-410b-93b8-f2452ff58c45_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="15137327-a513-410b-93b8-f2452ff58c45",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-03-05T20:25:43.571890+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0002/cha/15137327-a513-410b-93b8-f2452ff58c45.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0002/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/7ec81ad1-d5c9-4f02-8b0b-29135ab56ea5_video.mp4",
        incident_type_id="DOOR_VIOLATION",
        incident_uuid="7ec81ad1-d5c9-4f02-8b0b-29135ab56ea5",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-03-04T02:10:52.234039+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0002/cha/7ec81ad1-d5c9-4f02-8b0b-29135ab56ea5.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0002/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/8b4b7e3f-fabd-4928-8d60-019779b23189_video.mp4",
        incident_type_id="DOOR_VIOLATION",
        incident_uuid="8b4b7e3f-fabd-4928-8d60-019779b23189",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-03-04T02:49:17.501267+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0002/cha/8b4b7e3f-fabd-4928-8d60-019779b23189.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0002/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/306fb0ec-7bc9-4d9a-91df-d6671561899d_video.mp4",
        incident_type_id="PIGGYBACK",
        incident_uuid="306fb0ec-7bc9-4d9a-91df-d6671561899d",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-03-04T07:41:07.507225+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0002/cha/306fb0ec-7bc9-4d9a-91df-d6671561899d.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0002/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/4139bc0e-d3dc-4f06-b1ea-3b05e288e05e_video.mp4",
        incident_type_id="PIGGYBACK",
        incident_uuid="4139bc0e-d3dc-4f06-b1ea-3b05e288e05e",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-02-25T06:08:37.853782+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0002/cha/4139bc0e-d3dc-4f06-b1ea-3b05e288e05e.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0003/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/f7e3c0ce-6bd5-4177-8473-80445cfd08c5_video.mp4",
        incident_type_id="DOOR_VIOLATION",
        incident_uuid="f7e3c0ce-6bd5-4177-8473-80445cfd08c5",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-02-25T04:28:35.543782+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0003/cha/f7e3c0ce-6bd5-4177-8473-80445cfd08c5.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0003/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/8b311f6a-3af6-4e4d-b561-13a0f3379d71_video.mp4",
        incident_type_id="DOOR_VIOLATION",
        incident_uuid="8b311f6a-3af6-4e4d-b561-13a0f3379d71",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-02-25T07:12:43.854620+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0003/cha/8b311f6a-3af6-4e4d-b561-13a0f3379d71.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0003/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/09dba059-7030-4a27-8f02-df51f8f4f93a_video.mp4",
        incident_type_id="PIGGYBACK",
        incident_uuid="09dba059-7030-4a27-8f02-df51f8f4f93a",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-02-24T23:16:46.732003+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0003/cha/09dba059-7030-4a27-8f02-df51f8f4f93a.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0003/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/519b4bf7-6122-4aaf-9973-23f43725c4c5_video.mp4",
        incident_type_id="PIGGYBACK",
        incident_uuid="519b4bf7-6122-4aaf-9973-23f43725c4c5",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-02-26T23:40:04.035517+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0003/cha/519b4bf7-6122-4aaf-9973-23f43725c4c5.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0009/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/b3182dfc-67e1-42be-a8bb-7ee3604ccba2_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="b3182dfc-67e1-42be-a8bb-7ee3604ccba2",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-03-01T15:54:23.619456+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0009/cha/b3182dfc-67e1-42be-a8bb-7ee3604ccba2.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0009/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/3f80116f-12d4-45f4-b5f0-8868994d3877_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="3f80116f-12d4-45f4-b5f0-8868994d3877",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-02-28T19:15:36.266584+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0009/cha/3f80116f-12d4-45f4-b5f0-8868994d3877.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0009/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/bb6f089a-1e90-4511-a2ca-b74728868dad_video.mp4",
        incident_type_id="DOOR_VIOLATION",
        incident_uuid="bb6f089a-1e90-4511-a2ca-b74728868dad",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-02-25T01:20:01.314630+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0009/cha/bb6f089a-1e90-4511-a2ca-b74728868dad.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0009/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/f64e3ee5-79a7-460e-b164-9bf890f7a688_video.mp4",
        incident_type_id="DOOR_VIOLATION",
        incident_uuid="f64e3ee5-79a7-460e-b164-9bf890f7a688",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-02-25T01:16:55.003134+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0009/cha/f64e3ee5-79a7-460e-b164-9bf890f7a688.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0010/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/a64f4b9b-1ec1-47b6-b86c-689a1198f3f8_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="a64f4b9b-1ec1-47b6-b86c-689a1198f3f8",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-02-24T23:24:48.981718+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0010/cha/a64f4b9b-1ec1-47b6-b86c-689a1198f3f8.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0010/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/2a72416a-0963-4a1c-b28d-7f5424e2c4d2_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="2a72416a-0963-4a1c-b28d-7f5424e2c4d2",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-02-24T23:24:54.121055+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0010/cha/2a72416a-0963-4a1c-b28d-7f5424e2c4d2.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0010/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/e06ccf7b-3496-47d0-9354-58ed107ce1e5_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="e06ccf7b-3496-47d0-9354-58ed107ce1e5",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-03-02T02:03:33.687754+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0010/cha/e06ccf7b-3496-47d0-9354-58ed107ce1e5.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0010/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/9f67420c-266c-457a-a81e-baa7b61ba6e8_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="9f67420c-266c-457a-a81e-baa7b61ba6e8",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-03-02T02:03:41.864509+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0010/cha/9f67420c-266c-457a-a81e-baa7b61ba6e8.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0011/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/62dde9ff-fb25-41ed-8657-8b13be93f7e3_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="62dde9ff-fb25-41ed-8657-8b13be93f7e3",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-03-06T21:26:48.968552+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0011/cha/62dde9ff-fb25-41ed-8657-8b13be93f7e3.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0011/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/7ecab44c-0f3e-4063-9f37-572a5f0cb3ff_video.mp4",
        incident_type_id="DOOR_VIOLATION",
        incident_uuid="7ecab44c-0f3e-4063-9f37-572a5f0cb3ff",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-02-25T04:16:22.975373+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0011/cha/7ecab44c-0f3e-4063-9f37-572a5f0cb3ff.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0011/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/907b8201-5563-46ab-bb17-31e3900f720a_video.mp4",
        incident_type_id="DOOR_VIOLATION",
        incident_uuid="907b8201-5563-46ab-bb17-31e3900f720a",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-02-25T04:19:42.557935+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0011/cha/907b8201-5563-46ab-bb17-31e3900f720a.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0011/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/8248960b-cf1d-4d62-a1f3-a7e2899766f6_video.mp4",
        incident_type_id="PIGGYBACK",
        incident_uuid="8248960b-cf1d-4d62-a1f3-a7e2899766f6",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-02-25T04:53:58.761369+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0011/cha/8248960b-cf1d-4d62-a1f3-a7e2899766f6.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0011/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/699f278b-e9a9-4aac-bf1f-dfba7d7740bd_video.mp4",
        incident_type_id="PIGGYBACK",
        incident_uuid="699f278b-e9a9-4aac-bf1f-dfba7d7740bd",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-02-25T05:17:07.802342+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0011/cha/699f278b-e9a9-4aac-bf1f-dfba7d7740bd.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0012/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/65df4905-54ba-4c4b-810e-1d57b8556187_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="65df4905-54ba-4c4b-810e-1d57b8556187",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-03-04T02:50:24.846369+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0012/cha/65df4905-54ba-4c4b-810e-1d57b8556187.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0012/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/893fab52-bced-425a-9857-60ed5d97052e_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="893fab52-bced-425a-9857-60ed5d97052e",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-03-04T03:07:09.431620+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0012/cha/893fab52-bced-425a-9857-60ed5d97052e.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0012/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/9843b309-0a8b-4ed3-8dd6-75421715661c_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="9843b309-0a8b-4ed3-8dd6-75421715661c",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-03-04T02:03:58.777828+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0012/cha/9843b309-0a8b-4ed3-8dd6-75421715661c.mp4",
    ),
    Incident(
        camera_uuid="americold/modesto/0012/cha",
        video_s3_path="s3://voxel-portal-production/americold/modesto/incidents/3ee49441-3daa-4a28-b297-89028d97fe12_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="3ee49441-3daa-4a28-b297-89028d97fe12",
        organization="AMERICOLD",
        zone="MODESTO",
        experimental=False,
        created_at="2023-03-04T03:20:48.072771+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/modesto/0012/cha/3ee49441-3daa-4a28-b297-89028d97fe12.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0002/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/72d60faa-1471-442e-b0f9-f9dd1d8e7103_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="72d60faa-1471-442e-b0f9-f9dd1d8e7103",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-03-05T18:11:06.853818+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0002/cha/72d60faa-1471-442e-b0f9-f9dd1d8e7103.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0002/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/11aa1eec-cf46-46c2-941c-6ecd45493e86_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="11aa1eec-cf46-46c2-941c-6ecd45493e86",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-03-05T18:10:59.072274+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0002/cha/11aa1eec-cf46-46c2-941c-6ecd45493e86.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0003/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/f83a3263-0b1b-444e-aeee-afbdea256486_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="f83a3263-0b1b-444e-aeee-afbdea256486",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-14T14:52:27.257222+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0003/cha/f83a3263-0b1b-444e-aeee-afbdea256486.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0003/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/0d6b743d-e058-4e4e-92f6-e8df8e9b8135_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="0d6b743d-e058-4e4e-92f6-e8df8e9b8135",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-17T22:45:23.787235+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0003/cha/0d6b743d-e058-4e4e-92f6-e8df8e9b8135.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0003/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/61b811e2-d652-4008-be09-0920fad78b85_video.mp4",
        incident_type_id="DOOR_VIOLATION",
        incident_uuid="61b811e2-d652-4008-be09-0920fad78b85",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-20T19:43:05.281554+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0003/cha/61b811e2-d652-4008-be09-0920fad78b85.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0003/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/aeebc879-9052-434a-a345-44f374e8501c_video.mp4",
        incident_type_id="DOOR_VIOLATION",
        incident_uuid="aeebc879-9052-434a-a345-44f374e8501c",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-20T19:34:54.771304+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0003/cha/aeebc879-9052-434a-a345-44f374e8501c.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0003/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/3abf68a8-5aad-402a-9da5-8ff3d6acf067_video.mp4",
        incident_type_id="PIGGYBACK",
        incident_uuid="3abf68a8-5aad-402a-9da5-8ff3d6acf067",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-20T19:43:16.872514+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0003/cha/3abf68a8-5aad-402a-9da5-8ff3d6acf067.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0003/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/bdc5a267-3f43-47c5-937e-3efc6b213ae1_video.mp4",
        incident_type_id="PIGGYBACK",
        incident_uuid="bdc5a267-3f43-47c5-937e-3efc6b213ae1",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-12T01:48:25.756453+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0003/cha/bdc5a267-3f43-47c5-937e-3efc6b213ae1.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0004/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/30c636de-eadb-49f6-b9fe-ccf41d9e49dc_video.mp4",
        incident_type_id="DOOR_VIOLATION",
        incident_uuid="30c636de-eadb-49f6-b9fe-ccf41d9e49dc",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-12T05:52:41.795862+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0004/cha/30c636de-eadb-49f6-b9fe-ccf41d9e49dc.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0004/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/d8aa3470-5339-4bd4-97f5-90cf75114445_video.mp4",
        incident_type_id="DOOR_VIOLATION",
        incident_uuid="d8aa3470-5339-4bd4-97f5-90cf75114445",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-11T05:07:22.008285+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0004/cha/d8aa3470-5339-4bd4-97f5-90cf75114445.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0004/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/d7c15aa8-6e3a-4508-bfa5-0c469cccd72a_video.mp4",
        incident_type_id="PIGGYBACK",
        incident_uuid="d7c15aa8-6e3a-4508-bfa5-0c469cccd72a",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-12T18:47:39.473112+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0004/cha/d7c15aa8-6e3a-4508-bfa5-0c469cccd72a.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0004/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/1c1b6f36-f792-4088-b8dc-4b3fbf2a8ff8_video.mp4",
        incident_type_id="PIGGYBACK",
        incident_uuid="1c1b6f36-f792-4088-b8dc-4b3fbf2a8ff8",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-12T05:45:50.323453+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0004/cha/1c1b6f36-f792-4088-b8dc-4b3fbf2a8ff8.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0005/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/58b319f3-9037-4ab9-aec8-07be8566c782_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="58b319f3-9037-4ab9-aec8-07be8566c782",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-14T17:53:23.358707+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0005/cha/58b319f3-9037-4ab9-aec8-07be8566c782.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0005/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/4236fee9-a2e9-420c-a9c0-a86052797e92_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="4236fee9-a2e9-420c-a9c0-a86052797e92",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-03-01T04:07:41.464001+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0005/cha/4236fee9-a2e9-420c-a9c0-a86052797e92.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0005/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/98ecacd4-71ec-4e1b-96fa-5dc27e2b35c4_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="98ecacd4-71ec-4e1b-96fa-5dc27e2b35c4",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-14T18:13:19.666253+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0005/cha/98ecacd4-71ec-4e1b-96fa-5dc27e2b35c4.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0005/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/1a1e7ffa-5cd7-4cad-a343-95779cbd90cd_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="1a1e7ffa-5cd7-4cad-a343-95779cbd90cd",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-14T13:06:25.895379+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0005/cha/1a1e7ffa-5cd7-4cad-a343-95779cbd90cd.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0005/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/343efd03-b02d-4040-b047-8f1d7454e981_video.mp4",
        incident_type_id="DOOR_VIOLATION",
        incident_uuid="343efd03-b02d-4040-b047-8f1d7454e981",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-24T03:54:19.440039+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0005/cha/343efd03-b02d-4040-b047-8f1d7454e981.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0005/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/a20cfd66-4d35-4fac-b8a7-eccdbb3dbd02_video.mp4",
        incident_type_id="DOOR_VIOLATION",
        incident_uuid="a20cfd66-4d35-4fac-b8a7-eccdbb3dbd02",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-22T02:17:14.085215+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0005/cha/a20cfd66-4d35-4fac-b8a7-eccdbb3dbd02.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0005/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/4647f738-77a2-4d62-9720-64d473dee0cf_video.mp4",
        incident_type_id="PIGGYBACK",
        incident_uuid="4647f738-77a2-4d62-9720-64d473dee0cf",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-27T14:43:41.784164+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0005/cha/4647f738-77a2-4d62-9720-64d473dee0cf.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0006/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/bf49adae-6913-4926-a318-debe5fd218fe_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="bf49adae-6913-4926-a318-debe5fd218fe",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-24T04:02:32.941279+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0006/cha/bf49adae-6913-4926-a318-debe5fd218fe.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0006/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/dee40bb7-270d-4cf6-88fd-d72dd9a2cff5_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="dee40bb7-270d-4cf6-88fd-d72dd9a2cff5",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-17T00:58:45.178673+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0006/cha/dee40bb7-270d-4cf6-88fd-d72dd9a2cff5.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0006/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/3fb208cf-67ca-4aae-ae7e-85fdad6fe2a2_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="3fb208cf-67ca-4aae-ae7e-85fdad6fe2a2",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-09T16:35:53.427323+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0006/cha/3fb208cf-67ca-4aae-ae7e-85fdad6fe2a2.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0009/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/027359a7-e4a5-4609-9572-ad72f331bd1c_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="027359a7-e4a5-4609-9572-ad72f331bd1c",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-23T06:01:25.654631+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0009/cha/027359a7-e4a5-4609-9572-ad72f331bd1c.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0009/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/1d6ba248-8580-4b64-a170-9886190c1e04_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="1d6ba248-8580-4b64-a170-9886190c1e04",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-26T12:15:56.244166+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0009/cha/1d6ba248-8580-4b64-a170-9886190c1e04.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0009/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/88f7ab2f-7f90-4300-a403-f444e0c98dd7_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="88f7ab2f-7f90-4300-a403-f444e0c98dd7",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-21T02:52:32.816074+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0009/cha/88f7ab2f-7f90-4300-a403-f444e0c98dd7.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0009/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/c099c516-b95e-4fac-b402-59728eb88d1a_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="c099c516-b95e-4fac-b402-59728eb88d1a",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-21T17:45:39.008283+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0009/cha/c099c516-b95e-4fac-b402-59728eb88d1a.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0011/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/e4d75bb0-0c08-44d8-88da-50b85ba87d46_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="e4d75bb0-0c08-44d8-88da-50b85ba87d46",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-24T14:26:16.350825+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0011/cha/e4d75bb0-0c08-44d8-88da-50b85ba87d46.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0011/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/746cf4e2-93c1-4d47-92c7-15efbcdb0f72_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="746cf4e2-93c1-4d47-92c7-15efbcdb0f72",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-25T13:47:40.976277+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0011/cha/746cf4e2-93c1-4d47-92c7-15efbcdb0f72.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0011/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/b9d049bf-b66d-4234-bd2a-f94e8c4e7159_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="b9d049bf-b66d-4234-bd2a-f94e8c4e7159",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-15T23:35:43.757280+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0011/cha/b9d049bf-b66d-4234-bd2a-f94e8c4e7159.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0011/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/70ed6524-fd04-4345-865a-dce87160e098_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="70ed6524-fd04-4345-865a-dce87160e098",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-05T11:46:33.391399+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0011/cha/70ed6524-fd04-4345-865a-dce87160e098.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0011/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/c9f3ed20-4ffb-461f-bcf1-ef48cc3a7b70_video.mp4",
        incident_type_id="DOOR_VIOLATION",
        incident_uuid="c9f3ed20-4ffb-461f-bcf1-ef48cc3a7b70",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-11T16:32:04.072146+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0011/cha/c9f3ed20-4ffb-461f-bcf1-ef48cc3a7b70.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0011/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/a04134e1-2956-485c-9cac-75b05e00707b_video.mp4",
        incident_type_id="DOOR_VIOLATION",
        incident_uuid="a04134e1-2956-485c-9cac-75b05e00707b",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-07T10:24:54.690460+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0011/cha/a04134e1-2956-485c-9cac-75b05e00707b.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0011/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/202c257b-487c-492d-b8bf-dec47d24523c_video.mp4",
        incident_type_id="PIGGYBACK",
        incident_uuid="202c257b-487c-492d-b8bf-dec47d24523c",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-23T17:43:07.673320+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0011/cha/202c257b-487c-492d-b8bf-dec47d24523c.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0011/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/970859ed-6588-4663-98e2-ef2368e65651_video.mp4",
        incident_type_id="PIGGYBACK",
        incident_uuid="970859ed-6588-4663-98e2-ef2368e65651",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-20T19:49:42.699002+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0011/cha/970859ed-6588-4663-98e2-ef2368e65651.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0012/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/d8637b45-f45b-49d4-b514-48a8ffac37ed_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="d8637b45-f45b-49d4-b514-48a8ffac37ed",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-27T17:56:26.887000+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0012/cha/d8637b45-f45b-49d4-b514-48a8ffac37ed.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0012/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/69be5a72-0276-4315-993e-ea980306b643_video.mp4",
        incident_type_id="PIGGYBACK",
        incident_uuid="69be5a72-0276-4315-993e-ea980306b643",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-20T19:49:02.444585+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0012/cha/69be5a72-0276-4315-993e-ea980306b643.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario/0012/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario/incidents/b2664cb5-b1ce-405c-8300-08f124ec96b4_video.mp4",
        incident_type_id="PIGGYBACK",
        incident_uuid="b2664cb5-b1ce-405c-8300-08f124ec96b4",
        organization="AMERICOLD",
        zone="ONTARIO",
        experimental=False,
        created_at="2023-02-20T19:49:09.273461+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario/0012/cha/b2664cb5-b1ce-405c-8300-08f124ec96b4.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0001/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/74996edd-1b15-40b1-a45b-926b5d1dd7c3_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="74996edd-1b15-40b1-a45b-926b5d1dd7c3",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-20T03:57:58.640520+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0001/cha/74996edd-1b15-40b1-a45b-926b5d1dd7c3.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0001/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/2ed7a04c-badd-42ce-aa56-a3c52ab96a33_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="2ed7a04c-badd-42ce-aa56-a3c52ab96a33",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-03-01T16:45:12.769833+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0001/cha/2ed7a04c-badd-42ce-aa56-a3c52ab96a33.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0001/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/3bf86c4c-5c8f-492f-bbba-207410a61216_video.mp4",
        incident_type_id="HARD_HAT",
        incident_uuid="3bf86c4c-5c8f-492f-bbba-207410a61216",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-08T19:53:02.157530+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0001/cha/3bf86c4c-5c8f-492f-bbba-207410a61216.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0001/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/83a1c0f5-f681-405a-b8dd-4f6b95cf469c_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="83a1c0f5-f681-405a-b8dd-4f6b95cf469c",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-23T04:45:24.733832+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0001/cha/83a1c0f5-f681-405a-b8dd-4f6b95cf469c.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0001/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/b0aafe23-8d52-4f70-ac82-434efee3ff3f_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="b0aafe23-8d52-4f70-ac82-434efee3ff3f",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-23T04:49:01.757987+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0001/cha/b0aafe23-8d52-4f70-ac82-434efee3ff3f.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0001/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/da169464-c885-4005-a227-aa4b9bda4237_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="da169464-c885-4005-a227-aa4b9bda4237",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-03-06T13:53:33.439427+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0001/cha/da169464-c885-4005-a227-aa4b9bda4237.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0001/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/81eb38e8-b41f-4939-9c7b-0e8249063497_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="81eb38e8-b41f-4939-9c7b-0e8249063497",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-03-06T13:57:03.471613+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0001/cha/81eb38e8-b41f-4939-9c7b-0e8249063497.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0001/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/b2bf3522-7f98-4e2f-abe7-2dc68155c36b_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="b2bf3522-7f98-4e2f-abe7-2dc68155c36b",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-03-04T22:05:23.852239+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0001/cha/b2bf3522-7f98-4e2f-abe7-2dc68155c36b.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0001/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/5943bf96-a81f-45fc-ab96-295e625ca39d_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="5943bf96-a81f-45fc-ab96-295e625ca39d",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-26T14:27:29.279256+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0001/cha/5943bf96-a81f-45fc-ab96-295e625ca39d.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0002/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/9317ad37-29e5-498e-82af-0aa2fa294324_video.mp4",
        incident_type_id="HARD_HAT",
        incident_uuid="9317ad37-29e5-498e-82af-0aa2fa294324",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-03-01T20:02:22.532388+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0002/cha/9317ad37-29e5-498e-82af-0aa2fa294324.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0002/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/94e18ea1-c7b5-4636-866a-206836154a5a_video.mp4",
        incident_type_id="HARD_HAT",
        incident_uuid="94e18ea1-c7b5-4636-866a-206836154a5a",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-03-01T19:17:54.852389+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0002/cha/94e18ea1-c7b5-4636-866a-206836154a5a.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0002/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/bd09f98a-2c9e-4f69-9cab-797b2f169445_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="bd09f98a-2c9e-4f69-9cab-797b2f169445",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-03-01T17:15:34.204600+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0002/cha/bd09f98a-2c9e-4f69-9cab-797b2f169445.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0002/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/540c8cd9-80e2-4977-9609-0cff99e9ebad_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="540c8cd9-80e2-4977-9609-0cff99e9ebad",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-03-01T16:04:13.701130+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0002/cha/540c8cd9-80e2-4977-9609-0cff99e9ebad.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0002/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/9d67e09c-aa70-4fe2-8dd9-a229864e9616_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="9d67e09c-aa70-4fe2-8dd9-a229864e9616",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-03-01T17:15:08.710799+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0002/cha/9d67e09c-aa70-4fe2-8dd9-a229864e9616.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0002/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/7d0c6663-9dcf-47d4-a713-d2a5ba709090_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="7d0c6663-9dcf-47d4-a713-d2a5ba709090",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-28T16:52:18.549286+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0002/cha/7d0c6663-9dcf-47d4-a713-d2a5ba709090.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0003/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/5375eb17-97e1-4cb0-b140-3b19a9752a63_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="5375eb17-97e1-4cb0-b140-3b19a9752a63",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-20T10:08:14.282963+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0003/cha/5375eb17-97e1-4cb0-b140-3b19a9752a63.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0003/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/432fa88a-8096-4b0e-857d-7d681de1f7f2_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="432fa88a-8096-4b0e-857d-7d681de1f7f2",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-07T04:19:36.803813+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0003/cha/432fa88a-8096-4b0e-857d-7d681de1f7f2.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0003/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/961b7524-5937-4d31-a6d6-b2a910c175d0_video.mp4",
        incident_type_id="HARD_HAT",
        incident_uuid="961b7524-5937-4d31-a6d6-b2a910c175d0",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-03-05T14:02:40.162069+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0003/cha/961b7524-5937-4d31-a6d6-b2a910c175d0.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0003/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/a0f6ac4a-a098-4155-bba3-c197e3780fac_video.mp4",
        incident_type_id="HARD_HAT",
        incident_uuid="a0f6ac4a-a098-4155-bba3-c197e3780fac",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-03-05T14:04:52.209797+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0003/cha/a0f6ac4a-a098-4155-bba3-c197e3780fac.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0003/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/c4ebe418-a932-4dce-ad3f-dcbcc4ca2458_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="c4ebe418-a932-4dce-ad3f-dcbcc4ca2458",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-22T01:34:29.956581+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0003/cha/c4ebe418-a932-4dce-ad3f-dcbcc4ca2458.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0003/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/1379f403-751e-4aa4-b75e-fb411401414c_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="1379f403-751e-4aa4-b75e-fb411401414c",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-22T02:20:05.453023+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0003/cha/1379f403-751e-4aa4-b75e-fb411401414c.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0003/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/b0c602e7-fdbf-417b-b7bb-f0109f5c56f2_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="b0c602e7-fdbf-417b-b7bb-f0109f5c56f2",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-26T01:12:56.546433+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0003/cha/b0c602e7-fdbf-417b-b7bb-f0109f5c56f2.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0004/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/e749cdb7-4839-49f8-a8cf-1d8e14ce443a_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="e749cdb7-4839-49f8-a8cf-1d8e14ce443a",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-15T16:53:27.908718+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0004/cha/e749cdb7-4839-49f8-a8cf-1d8e14ce443a.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0004/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/2ce6aafe-1018-4e14-b97d-1787e41ae4e7_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="2ce6aafe-1018-4e14-b97d-1787e41ae4e7",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-20T12:20:51.914078+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0004/cha/2ce6aafe-1018-4e14-b97d-1787e41ae4e7.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0004/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/7dab5270-3ba4-434e-bef7-00887f4b9a91_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="7dab5270-3ba4-434e-bef7-00887f4b9a91",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-21T18:26:17.615414+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0004/cha/7dab5270-3ba4-434e-bef7-00887f4b9a91.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0005/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/7fb2af7d-595c-4269-b04c-6f2a774a125a_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="7fb2af7d-595c-4269-b04c-6f2a774a125a",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-21T12:31:44.123397+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0005/cha/7fb2af7d-595c-4269-b04c-6f2a774a125a.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0005/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/ff315d86-077d-4df8-b947-b0c38733d5c2_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="ff315d86-077d-4df8-b947-b0c38733d5c2",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-21T12:34:46.509120+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0005/cha/ff315d86-077d-4df8-b947-b0c38733d5c2.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0005/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/5ae485ee-fdc8-49d8-a304-1c69c1f51798_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="5ae485ee-fdc8-49d8-a304-1c69c1f51798",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-21T08:49:43.205230+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0005/cha/5ae485ee-fdc8-49d8-a304-1c69c1f51798.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0005/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/c491ad9b-4e29-47c5-8b5c-fd8d387bb275_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="c491ad9b-4e29-47c5-8b5c-fd8d387bb275",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-21T08:49:52.188045+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0005/cha/c491ad9b-4e29-47c5-8b5c-fd8d387bb275.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0005/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/2915f1d5-8f55-4104-9d16-838e30f19aee_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="2915f1d5-8f55-4104-9d16-838e30f19aee",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-06T13:13:58.744326+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0005/cha/2915f1d5-8f55-4104-9d16-838e30f19aee.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0005/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/6518f630-2086-4cd9-9607-fbd89ef4c3cc_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="6518f630-2086-4cd9-9607-fbd89ef4c3cc",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-26T13:07:37.743962+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0005/cha/6518f630-2086-4cd9-9607-fbd89ef4c3cc.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0005/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/65f4e609-a1d7-4b6d-95ea-0054dc4f7e54_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="65f4e609-a1d7-4b6d-95ea-0054dc4f7e54",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-26T15:57:33.011845+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0005/cha/65f4e609-a1d7-4b6d-95ea-0054dc4f7e54.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0006/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/f11d408a-cdde-408e-9f27-329a80cb6c1e_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="f11d408a-cdde-408e-9f27-329a80cb6c1e",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-21T02:15:40.941975+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0006/cha/f11d408a-cdde-408e-9f27-329a80cb6c1e.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0006/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/fc711edb-7df2-4742-8bc2-c0034dfb23eb_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="fc711edb-7df2-4742-8bc2-c0034dfb23eb",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-11T19:39:18.499485+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0006/cha/fc711edb-7df2-4742-8bc2-c0034dfb23eb.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0006/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/b4ce4e4f-6ddc-48a8-b97e-4c6eefc6862b_video.mp4",
        incident_type_id="HARD_HAT",
        incident_uuid="b4ce4e4f-6ddc-48a8-b97e-4c6eefc6862b",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-20T19:42:08.377942+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0006/cha/b4ce4e4f-6ddc-48a8-b97e-4c6eefc6862b.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0006/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/70630cf7-5b46-4657-915f-324100049e0f_video.mp4",
        incident_type_id="HARD_HAT",
        incident_uuid="70630cf7-5b46-4657-915f-324100049e0f",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-20T19:40:41.384501+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0006/cha/70630cf7-5b46-4657-915f-324100049e0f.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0006/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/e69b9ca1-9cca-424f-9daa-a85a61ec80ef_video.mp4",
        incident_type_id="NO_PED_ZONE",
        incident_uuid="e69b9ca1-9cca-424f-9daa-a85a61ec80ef",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-03-03T19:29:21.449998+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0006/cha/e69b9ca1-9cca-424f-9daa-a85a61ec80ef.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0006/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/02de9875-e817-4c65-9db5-18a84ac52bd8_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="02de9875-e817-4c65-9db5-18a84ac52bd8",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-10T11:29:21.022752+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0006/cha/02de9875-e817-4c65-9db5-18a84ac52bd8.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0006/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/26be5a4a-c24b-48be-9844-45091c380f16_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="26be5a4a-c24b-48be-9844-45091c380f16",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-09T23:33:31.768392+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0006/cha/26be5a4a-c24b-48be-9844-45091c380f16.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0006/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/fc900667-95c7-4b56-960a-fbadf54d2247_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="fc900667-95c7-4b56-960a-fbadf54d2247",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-09T23:06:17.546072+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0006/cha/fc900667-95c7-4b56-960a-fbadf54d2247.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0006/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/1bdc1bd2-985d-4831-8a6e-39ca33091997_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="1bdc1bd2-985d-4831-8a6e-39ca33091997",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-03-02T05:53:55.972683+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0006/cha/1bdc1bd2-985d-4831-8a6e-39ca33091997.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0007/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/c4679ed2-c10b-452d-967e-d7b070cd50a5_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="c4679ed2-c10b-452d-967e-d7b070cd50a5",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-24T02:24:32.800945+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0007/cha/c4679ed2-c10b-452d-967e-d7b070cd50a5.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0007/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/6ee96036-bfb6-42ce-8792-d941fb6e1718_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="6ee96036-bfb6-42ce-8792-d941fb6e1718",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-24T03:02:12.516963+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0007/cha/6ee96036-bfb6-42ce-8792-d941fb6e1718.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0007/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/b1ba73d0-c2ba-41b5-b615-228c781a585c_video.mp4",
        incident_type_id="HARD_HAT",
        incident_uuid="b1ba73d0-c2ba-41b5-b615-228c781a585c",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-03-06T17:00:23.350852+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0007/cha/b1ba73d0-c2ba-41b5-b615-228c781a585c.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0007/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/0d4327dc-e6f5-468b-a888-6e9ed3556a65_video.mp4",
        incident_type_id="HARD_HAT",
        incident_uuid="0d4327dc-e6f5-468b-a888-6e9ed3556a65",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-09T19:39:44.572807+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0007/cha/0d4327dc-e6f5-468b-a888-6e9ed3556a65.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0007/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/d92443a2-4413-440e-b25e-6864140c2fd0_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="d92443a2-4413-440e-b25e-6864140c2fd0",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-24T00:20:38.369970+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0007/cha/d92443a2-4413-440e-b25e-6864140c2fd0.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0007/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/a460e190-73df-4a8a-96f7-73021135b83d_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="a460e190-73df-4a8a-96f7-73021135b83d",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-24T02:27:47.860059+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0007/cha/a460e190-73df-4a8a-96f7-73021135b83d.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0008/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/971c8230-151a-48ed-9983-66eabcb173a2_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="971c8230-151a-48ed-9983-66eabcb173a2",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-09T11:43:39.434418+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0008/cha/971c8230-151a-48ed-9983-66eabcb173a2.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0008/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/9e8f518c-ef23-43d8-82d1-1338bab33e33_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="9e8f518c-ef23-43d8-82d1-1338bab33e33",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-09T11:43:48.201866+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0008/cha/9e8f518c-ef23-43d8-82d1-1338bab33e33.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0008/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/dcae4fe9-c3b1-429b-8c7b-dabb037e1cca_video.mp4",
        incident_type_id="HARD_HAT",
        incident_uuid="dcae4fe9-c3b1-429b-8c7b-dabb037e1cca",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-03-02T11:27:20.502957+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0008/cha/dcae4fe9-c3b1-429b-8c7b-dabb037e1cca.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0008/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/deb2b59f-5c25-4502-9411-d2a591193160_video.mp4",
        incident_type_id="HARD_HAT",
        incident_uuid="deb2b59f-5c25-4502-9411-d2a591193160",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-03-02T11:27:10.341132+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0008/cha/deb2b59f-5c25-4502-9411-d2a591193160.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0008/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/36320858-781f-4cdf-8104-878a33b391d0_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="36320858-781f-4cdf-8104-878a33b391d0",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-23T11:57:44.687205+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0008/cha/36320858-781f-4cdf-8104-878a33b391d0.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0008/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/64937eec-3ec1-42cd-88ce-79da2c7f535d_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="64937eec-3ec1-42cd-88ce-79da2c7f535d",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-10T07:54:00.991178+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0008/cha/64937eec-3ec1-42cd-88ce-79da2c7f535d.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0008/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/cb510b64-2417-4a50-9598-876ea19f9b9d_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="cb510b64-2417-4a50-9598-876ea19f9b9d",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-11T00:29:39.653943+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0008/cha/cb510b64-2417-4a50-9598-876ea19f9b9d.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0008/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/c00cc99b-cdf4-45c0-b26c-f186b18aaa4b_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="c00cc99b-cdf4-45c0-b26c-f186b18aaa4b",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-16T01:54:27.554038+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0008/cha/c00cc99b-cdf4-45c0-b26c-f186b18aaa4b.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0008/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/c444ab55-51d5-4fb5-bba8-25c2ec9e4cba_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="c444ab55-51d5-4fb5-bba8-25c2ec9e4cba",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-03-02T18:18:58.866692+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0008/cha/c444ab55-51d5-4fb5-bba8-25c2ec9e4cba.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0009/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/6191d533-a115-4e1e-918c-30f0156d7cb7_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="6191d533-a115-4e1e-918c-30f0156d7cb7",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-03-04T18:16:53.219693+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0009/cha/6191d533-a115-4e1e-918c-30f0156d7cb7.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0009/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/eadd2b37-c630-4880-b8da-b62081a84086_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="eadd2b37-c630-4880-b8da-b62081a84086",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-03-04T18:16:45.563428+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0009/cha/eadd2b37-c630-4880-b8da-b62081a84086.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0009/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/01d8a3e3-5d7a-439e-ac1f-724e10d5787d_video.mp4",
        incident_type_id="HARD_HAT",
        incident_uuid="01d8a3e3-5d7a-439e-ac1f-724e10d5787d",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-20T20:10:44.062409+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0009/cha/01d8a3e3-5d7a-439e-ac1f-724e10d5787d.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0009/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/82b987a9-a5f4-43bf-9ada-733500e29227_video.mp4",
        incident_type_id="HARD_HAT",
        incident_uuid="82b987a9-a5f4-43bf-9ada-733500e29227",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-20T20:10:25.083959+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0009/cha/82b987a9-a5f4-43bf-9ada-733500e29227.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0009/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/2c5e74a6-d4ed-45f2-8bc6-bf11da90073e_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="2c5e74a6-d4ed-45f2-8bc6-bf11da90073e",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-24T05:49:37.502543+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0009/cha/2c5e74a6-d4ed-45f2-8bc6-bf11da90073e.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0009/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/7ce4190b-f048-4eff-b029-3fe6cd8c921c_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="7ce4190b-f048-4eff-b029-3fe6cd8c921c",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-24T05:49:12.592794+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0009/cha/7ce4190b-f048-4eff-b029-3fe6cd8c921c.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0009/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/6d0314db-f378-46a0-83c1-37d57616f054_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="6d0314db-f378-46a0-83c1-37d57616f054",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-03-02T05:57:42.127785+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0009/cha/6d0314db-f378-46a0-83c1-37d57616f054.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0009/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/a2e69b81-f83b-4fc7-9840-dce63ab905fb_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="a2e69b81-f83b-4fc7-9840-dce63ab905fb",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-03-02T05:54:01.427864+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0009/cha/a2e69b81-f83b-4fc7-9840-dce63ab905fb.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0010/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/6fd6c17e-13c6-421a-afa3-ef691ec69cee_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="6fd6c17e-13c6-421a-afa3-ef691ec69cee",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-23T13:28:39.314776+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0010/cha/6fd6c17e-13c6-421a-afa3-ef691ec69cee.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0010/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/ada1af72-d686-4dce-b1b0-f9afb8d58a7f_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="ada1af72-d686-4dce-b1b0-f9afb8d58a7f",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-21T21:28:11.352802+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0010/cha/ada1af72-d686-4dce-b1b0-f9afb8d58a7f.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0010/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/d49d8236-4ae4-4fb2-aede-14df77bcedd6_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="d49d8236-4ae4-4fb2-aede-14df77bcedd6",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-20T15:10:14.917023+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0010/cha/d49d8236-4ae4-4fb2-aede-14df77bcedd6.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0010/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/bfe123f9-b9dc-4108-981f-1c1a461aec1a_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="bfe123f9-b9dc-4108-981f-1c1a461aec1a",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-21T12:30:25.587154+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0010/cha/bfe123f9-b9dc-4108-981f-1c1a461aec1a.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0010/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/fbcce731-a0af-4d87-842e-5507c19d9b97_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="fbcce731-a0af-4d87-842e-5507c19d9b97",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-20T21:27:55.955939+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0010/cha/fbcce731-a0af-4d87-842e-5507c19d9b97.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0010/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/82950a73-1eb2-44ff-b829-fe7508e9d29b_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="82950a73-1eb2-44ff-b829-fe7508e9d29b",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-02-22T12:08:55.283801+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0010/cha/82950a73-1eb2-44ff-b829-fe7508e9d29b.mp4",
    ),
    Incident(
        camera_uuid="uscold/quakertown/0010/cha",
        video_s3_path="s3://voxel-portal-production/uscold/quakertown/incidents/81e0febe-4288-48c5-b8a2-e0a2cc34dd03_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="81e0febe-4288-48c5-b8a2-e0a2cc34dd03",
        organization="USCOLD",
        zone="QUAKERTOWN",
        experimental=False,
        created_at="2023-03-01T12:11:57.214632+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/uscold/quakertown/0010/cha/81e0febe-4288-48c5-b8a2-e0a2cc34dd03.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/solanabeach/0001/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/solanabeach/incidents/631db43c-89f2-4be9-a754-a260a7a35523_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="631db43c-89f2-4be9-a754-a260a7a35523",
        organization="BUILDERSFIRSTSOURCE",
        zone="SOLANABEACH",
        experimental=False,
        created_at="2023-02-09T23:20:09.987140+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/solanabeach/0001/cha/631db43c-89f2-4be9-a754-a260a7a35523.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/solanabeach/0001/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/solanabeach/incidents/b3805fa7-7512-4363-9e08-3b5b978f71fd_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="b3805fa7-7512-4363-9e08-3b5b978f71fd",
        organization="BUILDERSFIRSTSOURCE",
        zone="SOLANABEACH",
        experimental=False,
        created_at="2023-02-07T19:51:41.117681+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/solanabeach/0001/cha/b3805fa7-7512-4363-9e08-3b5b978f71fd.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/solanabeach/0001/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/solanabeach/incidents/6eb30864-c824-460e-8346-0e96016fab5a_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="6eb30864-c824-460e-8346-0e96016fab5a",
        organization="BUILDERSFIRSTSOURCE",
        zone="SOLANABEACH",
        experimental=False,
        created_at="2023-02-09T17:35:16.115947+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/solanabeach/0001/cha/6eb30864-c824-460e-8346-0e96016fab5a.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/solanabeach/0001/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/solanabeach/incidents/7f9c886f-c403-4f07-b6a2-68359770b159_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="7f9c886f-c403-4f07-b6a2-68359770b159",
        organization="BUILDERSFIRSTSOURCE",
        zone="SOLANABEACH",
        experimental=False,
        created_at="2023-02-09T22:13:40.800340+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/solanabeach/0001/cha/7f9c886f-c403-4f07-b6a2-68359770b159.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/solanabeach/0002/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/solanabeach/incidents/865156cd-7a26-4239-bdd5-4b9ea73f461a_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="865156cd-7a26-4239-bdd5-4b9ea73f461a",
        organization="BUILDERSFIRSTSOURCE",
        zone="SOLANABEACH",
        experimental=False,
        created_at="2023-02-17T22:29:09.065580+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/solanabeach/0002/cha/865156cd-7a26-4239-bdd5-4b9ea73f461a.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/solanabeach/0002/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/solanabeach/incidents/56ef2c26-9112-4d4c-86d8-928d4d4c3688_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="56ef2c26-9112-4d4c-86d8-928d4d4c3688",
        organization="BUILDERSFIRSTSOURCE",
        zone="SOLANABEACH",
        experimental=False,
        created_at="2023-02-11T21:36:57.198778+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/solanabeach/0002/cha/56ef2c26-9112-4d4c-86d8-928d4d4c3688.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/solanabeach/0002/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/solanabeach/incidents/771bf1f8-15c4-46d6-baf1-1e66785c10bb_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="771bf1f8-15c4-46d6-baf1-1e66785c10bb",
        organization="BUILDERSFIRSTSOURCE",
        zone="SOLANABEACH",
        experimental=False,
        created_at="2023-02-20T20:35:41.870340+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/solanabeach/0002/cha/771bf1f8-15c4-46d6-baf1-1e66785c10bb.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/solanabeach/0002/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/solanabeach/incidents/b0b98fa7-bd8e-4c5a-8803-26226ac32e24_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="b0b98fa7-bd8e-4c5a-8803-26226ac32e24",
        organization="BUILDERSFIRSTSOURCE",
        zone="SOLANABEACH",
        experimental=False,
        created_at="2023-02-14T18:17:18.232744+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/solanabeach/0002/cha/b0b98fa7-bd8e-4c5a-8803-26226ac32e24.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/solanabeach/0004/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/solanabeach/incidents/e1059d97-007e-40ee-a7fe-3de5105a20c9_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="e1059d97-007e-40ee-a7fe-3de5105a20c9",
        organization="BUILDERSFIRSTSOURCE",
        zone="SOLANABEACH",
        experimental=False,
        created_at="2023-02-05T17:54:53.450972+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/solanabeach/0004/cha/e1059d97-007e-40ee-a7fe-3de5105a20c9.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/solanabeach/0004/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/solanabeach/incidents/22b38c1a-5fc3-47d5-9df8-607d8d73f9e3_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="22b38c1a-5fc3-47d5-9df8-607d8d73f9e3",
        organization="BUILDERSFIRSTSOURCE",
        zone="SOLANABEACH",
        experimental=False,
        created_at="2023-02-05T17:51:47.198879+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/solanabeach/0004/cha/22b38c1a-5fc3-47d5-9df8-607d8d73f9e3.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/solanabeach/0004/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/solanabeach/incidents/ab6b54d0-5a9a-431a-bf76-47333bb18bd2_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="ab6b54d0-5a9a-431a-bf76-47333bb18bd2",
        organization="BUILDERSFIRSTSOURCE",
        zone="SOLANABEACH",
        experimental=False,
        created_at="2023-02-21T21:13:23.639782+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/solanabeach/0004/cha/ab6b54d0-5a9a-431a-bf76-47333bb18bd2.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/solanabeach/0004/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/solanabeach/incidents/d0d74ab5-e428-4aa9-9692-a6de77bfed9b_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="d0d74ab5-e428-4aa9-9692-a6de77bfed9b",
        organization="BUILDERSFIRSTSOURCE",
        zone="SOLANABEACH",
        experimental=False,
        created_at="2023-02-21T20:36:56.985497+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/solanabeach/0004/cha/d0d74ab5-e428-4aa9-9692-a6de77bfed9b.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/solanabeach/0005/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/solanabeach/incidents/644abbf8-4bbc-4489-89c4-a40fb484af0a_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="644abbf8-4bbc-4489-89c4-a40fb484af0a",
        organization="BUILDERSFIRSTSOURCE",
        zone="SOLANABEACH",
        experimental=False,
        created_at="2023-02-10T18:03:45.228711+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/solanabeach/0005/cha/644abbf8-4bbc-4489-89c4-a40fb484af0a.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/solanabeach/0005/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/solanabeach/incidents/914fa455-d8b4-4df8-bce2-e4a987c8b4d5_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="914fa455-d8b4-4df8-bce2-e4a987c8b4d5",
        organization="BUILDERSFIRSTSOURCE",
        zone="SOLANABEACH",
        experimental=False,
        created_at="2023-02-10T22:40:34.230083+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/solanabeach/0005/cha/914fa455-d8b4-4df8-bce2-e4a987c8b4d5.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/solanabeach/0005/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/solanabeach/incidents/ad83cddd-56ef-4c4f-958d-1892c4ec8344_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="ad83cddd-56ef-4c4f-958d-1892c4ec8344",
        organization="BUILDERSFIRSTSOURCE",
        zone="SOLANABEACH",
        experimental=False,
        created_at="2023-02-21T21:16:54.872208+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/solanabeach/0005/cha/ad83cddd-56ef-4c4f-958d-1892c4ec8344.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/solanabeach/0005/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/solanabeach/incidents/c107333e-0fe1-4f74-b97b-8c088e6e4340_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="c107333e-0fe1-4f74-b97b-8c088e6e4340",
        organization="BUILDERSFIRSTSOURCE",
        zone="SOLANABEACH",
        experimental=False,
        created_at="2023-02-22T18:50:29.797576+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/solanabeach/0005/cha/c107333e-0fe1-4f74-b97b-8c088e6e4340.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/solanabeach/0006/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/solanabeach/incidents/e5b935d0-4cbb-436b-a516-11de19f2eb13_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="e5b935d0-4cbb-436b-a516-11de19f2eb13",
        organization="BUILDERSFIRSTSOURCE",
        zone="SOLANABEACH",
        experimental=False,
        created_at="2023-02-09T01:26:58.142919+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/solanabeach/0006/cha/e5b935d0-4cbb-436b-a516-11de19f2eb13.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/solanabeach/0006/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/solanabeach/incidents/721a963f-eae2-4aeb-aa90-4714ea42b001_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="721a963f-eae2-4aeb-aa90-4714ea42b001",
        organization="BUILDERSFIRSTSOURCE",
        zone="SOLANABEACH",
        experimental=False,
        created_at="2023-02-10T21:20:23.406026+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/solanabeach/0006/cha/721a963f-eae2-4aeb-aa90-4714ea42b001.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/solanabeach/0006/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/solanabeach/incidents/abef0419-5f38-49c6-a6a3-23cfceb2e312_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="abef0419-5f38-49c6-a6a3-23cfceb2e312",
        organization="BUILDERSFIRSTSOURCE",
        zone="SOLANABEACH",
        experimental=False,
        created_at="2023-02-10T19:23:34.896646+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/solanabeach/0006/cha/abef0419-5f38-49c6-a6a3-23cfceb2e312.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0002/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/94663101-feb0-46f7-98bb-45abe69e7d48_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="94663101-feb0-46f7-98bb-45abe69e7d48",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-20T22:20:50.491383+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0002/cha/94663101-feb0-46f7-98bb-45abe69e7d48.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0002/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/c805c697-0dc2-4b0b-b622-ee95d7c58800_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="c805c697-0dc2-4b0b-b622-ee95d7c58800",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-20T18:32:44.702312+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0002/cha/c805c697-0dc2-4b0b-b622-ee95d7c58800.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0002/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/9af1de8d-beba-4792-a2fe-5d045e1b4698_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="9af1de8d-beba-4792-a2fe-5d045e1b4698",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-13T22:48:19.619354+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0002/cha/9af1de8d-beba-4792-a2fe-5d045e1b4698.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0002/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/b2ff6dd6-f7d2-4c2e-8796-2e895d39e70e_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="b2ff6dd6-f7d2-4c2e-8796-2e895d39e70e",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-06T21:31:42.055508+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0002/cha/b2ff6dd6-f7d2-4c2e-8796-2e895d39e70e.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0002/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/a64ef687-37cf-4723-b6b7-ddb570d4be2d_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="a64ef687-37cf-4723-b6b7-ddb570d4be2d",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-09T21:43:55.510844+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0002/cha/a64ef687-37cf-4723-b6b7-ddb570d4be2d.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0002/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/efff4d0a-fab0-40e7-8039-060a38c94244_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="efff4d0a-fab0-40e7-8039-060a38c94244",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-22T19:34:22.527188+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0002/cha/efff4d0a-fab0-40e7-8039-060a38c94244.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0003/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/023ed780-68ed-41af-ab23-d8a7c24548f9_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="023ed780-68ed-41af-ab23-d8a7c24548f9",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-09T17:51:03.352090+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0003/cha/023ed780-68ed-41af-ab23-d8a7c24548f9.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0003/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/abe34c57-566a-4378-be72-8f5068359230_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="abe34c57-566a-4378-be72-8f5068359230",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-09T11:07:55.259645+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0003/cha/abe34c57-566a-4378-be72-8f5068359230.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0003/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/fca0281c-f269-4475-8bbb-32f1e3a94c0b_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="fca0281c-f269-4475-8bbb-32f1e3a94c0b",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-13T17:42:32.311920+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0003/cha/fca0281c-f269-4475-8bbb-32f1e3a94c0b.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0003/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/10de68a0-f8bf-486c-bbc9-68d801861b78_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="10de68a0-f8bf-486c-bbc9-68d801861b78",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-13T21:12:26.921190+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0003/cha/10de68a0-f8bf-486c-bbc9-68d801861b78.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0003/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/c759d4ca-fd1c-44b1-8f32-e17a9d3243ce_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="c759d4ca-fd1c-44b1-8f32-e17a9d3243ce",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-08T20:51:25.591211+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0003/cha/c759d4ca-fd1c-44b1-8f32-e17a9d3243ce.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0004/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/1712c9b8-d483-46cf-95b8-d324a2c7c97f_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="1712c9b8-d483-46cf-95b8-d324a2c7c97f",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-20T20:52:29.288427+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0004/cha/1712c9b8-d483-46cf-95b8-d324a2c7c97f.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0004/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/475af389-322a-4b97-973e-15a0c0ce424a_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="475af389-322a-4b97-973e-15a0c0ce424a",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-20T21:52:34.267578+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0004/cha/475af389-322a-4b97-973e-15a0c0ce424a.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0005/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/f2680ed6-4bcd-4df9-8ed8-9fb168fca46d_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="f2680ed6-4bcd-4df9-8ed8-9fb168fca46d",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-09T16:31:10.890045+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0005/cha/f2680ed6-4bcd-4df9-8ed8-9fb168fca46d.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0005/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/75570a43-26f0-403d-ba29-261ad5da824d_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="75570a43-26f0-403d-ba29-261ad5da824d",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-20T20:42:51.038431+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0005/cha/75570a43-26f0-403d-ba29-261ad5da824d.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0006/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/e4f7a574-e062-4a19-abf1-1a944472e5ee_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="e4f7a574-e062-4a19-abf1-1a944472e5ee",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-06T17:54:12.147470+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0006/cha/e4f7a574-e062-4a19-abf1-1a944472e5ee.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0006/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/6649d820-7f51-4aeb-84bf-28c36fad91f3_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="6649d820-7f51-4aeb-84bf-28c36fad91f3",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-22T20:41:34.328487+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0006/cha/6649d820-7f51-4aeb-84bf-28c36fad91f3.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0007/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/9b0c9704-cb19-4d26-a3da-abb8f7a05cfa_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="9b0c9704-cb19-4d26-a3da-abb8f7a05cfa",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-09T21:14:48.634861+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0007/cha/9b0c9704-cb19-4d26-a3da-abb8f7a05cfa.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0007/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/c9bd60b0-e038-4e23-a061-ea08d4ec6c59_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="c9bd60b0-e038-4e23-a061-ea08d4ec6c59",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-21T16:29:38.687336+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0007/cha/c9bd60b0-e038-4e23-a061-ea08d4ec6c59.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0007/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/31ba20a4-d2a4-4721-8dbd-a44b972dc8f0_video.mp4",
        incident_type_id="NO_PED_ZONE",
        incident_uuid="31ba20a4-d2a4-4721-8dbd-a44b972dc8f0",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-23T17:02:28.724410+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0007/cha/31ba20a4-d2a4-4721-8dbd-a44b972dc8f0.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0007/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/b0514ddb-d071-4a8e-9966-a2b846f3e553_video.mp4",
        incident_type_id="NO_PED_ZONE",
        incident_uuid="b0514ddb-d071-4a8e-9966-a2b846f3e553",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-23T16:55:46.509406+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0007/cha/b0514ddb-d071-4a8e-9966-a2b846f3e553.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0007/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/20bdd42a-642e-4b77-a6e2-e9b6aa2ee92d_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="20bdd42a-642e-4b77-a6e2-e9b6aa2ee92d",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-09T21:08:12.329186+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0007/cha/20bdd42a-642e-4b77-a6e2-e9b6aa2ee92d.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0007/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/334c748a-5751-40ec-b39b-20f3d732bed2_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="334c748a-5751-40ec-b39b-20f3d732bed2",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-21T19:14:28.765333+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0007/cha/334c748a-5751-40ec-b39b-20f3d732bed2.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0007/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/5f482825-9d25-4cd4-a39a-545b21b4f9ff_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="5f482825-9d25-4cd4-a39a-545b21b4f9ff",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-15T22:22:56.014203+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0007/cha/5f482825-9d25-4cd4-a39a-545b21b4f9ff.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0007/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/8b376308-52ae-4709-b01b-5243cdea1ff5_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="8b376308-52ae-4709-b01b-5243cdea1ff5",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-20T14:04:58.032374+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0007/cha/8b376308-52ae-4709-b01b-5243cdea1ff5.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0008/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/68e1f11f-bc6c-438d-bd51-aa82fe2e1988_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="68e1f11f-bc6c-438d-bd51-aa82fe2e1988",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-16T15:50:11.969058+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0008/cha/68e1f11f-bc6c-438d-bd51-aa82fe2e1988.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0008/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/70125dd5-5462-434f-9b2f-c331634308ad_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="70125dd5-5462-434f-9b2f-c331634308ad",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-16T15:50:02.747390+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0008/cha/70125dd5-5462-434f-9b2f-c331634308ad.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0008/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/72404f22-9c27-4818-82a4-025fe6f47de8_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="72404f22-9c27-4818-82a4-025fe6f47de8",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-21T16:21:54.773007+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0008/cha/72404f22-9c27-4818-82a4-025fe6f47de8.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0008/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/87aba693-9565-4448-9f77-b9583a75e5f9_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="87aba693-9565-4448-9f77-b9583a75e5f9",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-21T19:14:31.891932+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0008/cha/87aba693-9565-4448-9f77-b9583a75e5f9.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0008/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/eeb98fd8-4a63-48e3-9851-4298012c10ed_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="eeb98fd8-4a63-48e3-9851-4298012c10ed",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-21T19:47:57.239646+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0008/cha/eeb98fd8-4a63-48e3-9851-4298012c10ed.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0008/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/98de17fa-abce-4c5c-80bd-f0f5b33e01f6_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="98de17fa-abce-4c5c-80bd-f0f5b33e01f6",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-21T20:24:36.635558+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0008/cha/98de17fa-abce-4c5c-80bd-f0f5b33e01f6.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0010/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/6d50fd55-956a-448e-bb82-9000b46ae76d_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="6d50fd55-956a-448e-bb82-9000b46ae76d",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-09T16:22:58.366955+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0010/cha/6d50fd55-956a-448e-bb82-9000b46ae76d.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0010/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/eae44df8-b70d-4551-8f41-fb459a44b517_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="eae44df8-b70d-4551-8f41-fb459a44b517",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-06T20:13:08.478632+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0010/cha/eae44df8-b70d-4551-8f41-fb459a44b517.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0010/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/90310458-8cc5-4d83-92a6-ba90232a18d2_video.mp4",
        incident_type_id="NO_PED_ZONE",
        incident_uuid="90310458-8cc5-4d83-92a6-ba90232a18d2",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-23T18:02:24.901993+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0010/cha/90310458-8cc5-4d83-92a6-ba90232a18d2.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0010/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/e580d91c-07c5-436b-b27e-948aad6b97fb_video.mp4",
        incident_type_id="NO_PED_ZONE",
        incident_uuid="e580d91c-07c5-436b-b27e-948aad6b97fb",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-09T20:49:37.567807+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0010/cha/e580d91c-07c5-436b-b27e-948aad6b97fb.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0010/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/aea8b6c3-638b-4389-b442-201c7bc5d10c_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="aea8b6c3-638b-4389-b442-201c7bc5d10c",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-23T13:39:33.443374+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0010/cha/aea8b6c3-638b-4389-b442-201c7bc5d10c.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0010/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/5bbdfdee-604a-4a34-af8e-9e934c81f95f_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="5bbdfdee-604a-4a34-af8e-9e934c81f95f",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-10T18:43:11.568787+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0010/cha/5bbdfdee-604a-4a34-af8e-9e934c81f95f.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0010/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/113705e3-7de3-433d-8b0d-1e075371686f_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="113705e3-7de3-433d-8b0d-1e075371686f",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-20T19:43:21.146074+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0010/cha/113705e3-7de3-433d-8b0d-1e075371686f.mp4",
    ),
    Incident(
        camera_uuid="buildersfirstsource/dixieline/0010/cha",
        video_s3_path="s3://voxel-portal-production/buildersfirstsource/dixieline/incidents/d0877ade-4465-4f9d-811e-de2c33acade3_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="d0877ade-4465-4f9d-811e-de2c33acade3",
        organization="BUILDERSFIRSTSOURCE",
        zone="DIXIELINE",
        experimental=False,
        created_at="2023-02-20T22:56:30.919092+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/buildersfirstsource/dixieline/0010/cha/d0877ade-4465-4f9d-811e-de2c33acade3.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0001/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/2b8b0561-fae6-484e-b267-5724826054a6_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="2b8b0561-fae6-484e-b267-5724826054a6",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-08T23:55:51.982968+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0001/cha/2b8b0561-fae6-484e-b267-5724826054a6.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0001/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/aeeb3f7b-82b0-4b93-b232-56d39810c57e_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="aeeb3f7b-82b0-4b93-b232-56d39810c57e",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-23T17:58:25.519365+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0001/cha/aeeb3f7b-82b0-4b93-b232-56d39810c57e.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0001/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/a0829944-29f0-4b3b-8f30-19f2dde0e330_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="a0829944-29f0-4b3b-8f30-19f2dde0e330",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-16T01:59:20.228275+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0001/cha/a0829944-29f0-4b3b-8f30-19f2dde0e330.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0001/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/76392c37-962f-4543-b168-875d55b16f94_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="76392c37-962f-4543-b168-875d55b16f94",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-24T00:17:57.818870+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0001/cha/76392c37-962f-4543-b168-875d55b16f94.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0003/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/c7f257f0-4723-4178-a863-12bfbde6e752_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="c7f257f0-4723-4178-a863-12bfbde6e752",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-23T13:47:41.323497+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0003/cha/c7f257f0-4723-4178-a863-12bfbde6e752.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0003/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/6459d5c3-298c-4cbb-84c5-3d0a4fd258c3_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="6459d5c3-298c-4cbb-84c5-3d0a4fd258c3",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-28T12:28:36.006614+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0003/cha/6459d5c3-298c-4cbb-84c5-3d0a4fd258c3.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0003/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/a11e024d-b201-46c6-9fd0-14eec50efb38_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="a11e024d-b201-46c6-9fd0-14eec50efb38",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-21T11:52:38.281239+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0003/cha/a11e024d-b201-46c6-9fd0-14eec50efb38.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0003/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/4439b733-0cf9-4853-8b76-fe3d0d6b526c_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="4439b733-0cf9-4853-8b76-fe3d0d6b526c",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-24T05:44:18.344629+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0003/cha/4439b733-0cf9-4853-8b76-fe3d0d6b526c.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0004/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/1799490b-8802-42a0-bf9a-22e1b1e82eaf_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="1799490b-8802-42a0-bf9a-22e1b1e82eaf",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-20T19:19:09.113687+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0004/cha/1799490b-8802-42a0-bf9a-22e1b1e82eaf.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0004/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/26e57390-2f63-4526-86b9-b96ca7ba3cb1_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="26e57390-2f63-4526-86b9-b96ca7ba3cb1",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-20T19:18:49.580957+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0004/cha/26e57390-2f63-4526-86b9-b96ca7ba3cb1.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0004/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/e66cd2f1-9c3e-48bf-be58-7e83b08283b4_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="e66cd2f1-9c3e-48bf-be58-7e83b08283b4",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-23T16:47:03.987531+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0004/cha/e66cd2f1-9c3e-48bf-be58-7e83b08283b4.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0004/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/e2bf9f83-0d58-4271-b95b-87be38f5307e_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="e2bf9f83-0d58-4271-b95b-87be38f5307e",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-16T01:47:10.546509+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0004/cha/e2bf9f83-0d58-4271-b95b-87be38f5307e.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0005/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/92efea92-6db4-4696-a0a1-e80ea9479668_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="92efea92-6db4-4696-a0a1-e80ea9479668",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-03-04T00:24:06.372502+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0005/cha/92efea92-6db4-4696-a0a1-e80ea9479668.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0005/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/e75c6681-bd49-44d9-ab27-93b824a80830_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="e75c6681-bd49-44d9-ab27-93b824a80830",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-03-04T00:24:13.378438+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0005/cha/e75c6681-bd49-44d9-ab27-93b824a80830.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0005/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/bca164e9-b7ca-48b8-af9f-8906bdc79fa2_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="bca164e9-b7ca-48b8-af9f-8906bdc79fa2",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-09T18:06:41.269847+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0005/cha/bca164e9-b7ca-48b8-af9f-8906bdc79fa2.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0005/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/3a063d8a-573b-4428-b8bb-e2403143db8a_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="3a063d8a-573b-4428-b8bb-e2403143db8a",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-23T16:55:18.966613+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0005/cha/3a063d8a-573b-4428-b8bb-e2403143db8a.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0007/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/0d7bc0b8-8a4c-4aa7-a544-f1911242a182_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="0d7bc0b8-8a4c-4aa7-a544-f1911242a182",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-24T01:01:21.008956+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0007/cha/0d7bc0b8-8a4c-4aa7-a544-f1911242a182.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0007/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/5c4a5b9f-356b-468b-b368-ea8feded36f2_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="5c4a5b9f-356b-468b-b368-ea8feded36f2",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-21T21:31:37.738936+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0007/cha/5c4a5b9f-356b-468b-b368-ea8feded36f2.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0007/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/1527a756-2d23-4792-aa2d-107ac60b0874_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="1527a756-2d23-4792-aa2d-107ac60b0874",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-07T22:39:26.753893+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0007/cha/1527a756-2d23-4792-aa2d-107ac60b0874.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0007/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/a7390d8c-2e05-4e93-bb79-6662225a8726_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="a7390d8c-2e05-4e93-bb79-6662225a8726",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-25T12:25:07.117684+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0007/cha/a7390d8c-2e05-4e93-bb79-6662225a8726.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0007/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/051bb0ff-3cd9-401d-ba9f-0077fe9d7e49_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="051bb0ff-3cd9-401d-ba9f-0077fe9d7e49",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-27T22:28:02.658936+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0007/cha/051bb0ff-3cd9-401d-ba9f-0077fe9d7e49.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0007/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/27f4e229-07ed-423c-adcf-8ec285e5e100_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="27f4e229-07ed-423c-adcf-8ec285e5e100",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-23T18:34:57.372112+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0007/cha/27f4e229-07ed-423c-adcf-8ec285e5e100.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0007/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/bd981853-3803-402e-a512-a19e7aaae453_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="bd981853-3803-402e-a512-a19e7aaae453",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-09T23:00:20.721994+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0007/cha/bd981853-3803-402e-a512-a19e7aaae453.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0008/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/746daaae-eaa6-4263-84bb-b945d50c83d0_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="746daaae-eaa6-4263-84bb-b945d50c83d0",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-20T19:17:52.160700+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0008/cha/746daaae-eaa6-4263-84bb-b945d50c83d0.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0008/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/4f2f1789-2a16-45cf-bdd5-857243e0c53e_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="4f2f1789-2a16-45cf-bdd5-857243e0c53e",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-23T17:05:31.814652+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0008/cha/4f2f1789-2a16-45cf-bdd5-857243e0c53e.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0008/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/f025dc33-87c3-44c5-a6d4-8ca13738afe5_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="f025dc33-87c3-44c5-a6d4-8ca13738afe5",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-21T18:45:18.961284+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0008/cha/f025dc33-87c3-44c5-a6d4-8ca13738afe5.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0008/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/b1627c4f-f383-4767-92a2-f1cc0d738f6c_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="b1627c4f-f383-4767-92a2-f1cc0d738f6c",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-22T19:10:06.000295+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0008/cha/b1627c4f-f383-4767-92a2-f1cc0d738f6c.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0008/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/98031062-532f-4128-851a-6810b29282a4_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="98031062-532f-4128-851a-6810b29282a4",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-03-04T10:58:45.359371+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0008/cha/98031062-532f-4128-851a-6810b29282a4.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0008/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/5df4fe03-9b77-4a33-ac0f-176d3f5a22cc_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="5df4fe03-9b77-4a33-ac0f-176d3f5a22cc",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-03-04T11:01:57.560484+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0008/cha/5df4fe03-9b77-4a33-ac0f-176d3f5a22cc.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0008/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/a52230d1-901d-4085-a3cb-c8cdcdb16db0_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="a52230d1-901d-4085-a3cb-c8cdcdb16db0",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-23T17:28:53.893021+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0008/cha/a52230d1-901d-4085-a3cb-c8cdcdb16db0.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0008/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/1ade6fca-ebbe-4b8b-a281-0500ccbc846d_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="1ade6fca-ebbe-4b8b-a281-0500ccbc846d",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-23T16:11:31.385790+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0008/cha/1ade6fca-ebbe-4b8b-a281-0500ccbc846d.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0009/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/923c42a8-0bde-40f2-9b95-3de9cf6ba878_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="923c42a8-0bde-40f2-9b95-3de9cf6ba878",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-03-04T09:43:25.227934+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0009/cha/923c42a8-0bde-40f2-9b95-3de9cf6ba878.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0009/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/8fb7375f-8578-4c79-a4ca-4711e2ea293c_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="8fb7375f-8578-4c79-a4ca-4711e2ea293c",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-03-04T09:43:39.486078+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0009/cha/8fb7375f-8578-4c79-a4ca-4711e2ea293c.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0009/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/a15545f2-5082-4975-9be0-6a9ebc9d9d49_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="a15545f2-5082-4975-9be0-6a9ebc9d9d49",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-23T17:31:49.346651+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0009/cha/a15545f2-5082-4975-9be0-6a9ebc9d9d49.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0009/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/f7b1e156-83fc-4370-97ed-8565354c58ec_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="f7b1e156-83fc-4370-97ed-8565354c58ec",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-23T17:28:37.378126+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0009/cha/f7b1e156-83fc-4370-97ed-8565354c58ec.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0009/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/d8238c35-97c2-4e8d-bc52-7d5ae91dce15_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="d8238c35-97c2-4e8d-bc52-7d5ae91dce15",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-03-02T08:30:45.470919+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0009/cha/d8238c35-97c2-4e8d-bc52-7d5ae91dce15.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0009/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/d1690f49-08cd-4f4f-b275-b1239557cbda_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="d1690f49-08cd-4f4f-b275-b1239557cbda",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-03-02T08:10:45.663074+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0009/cha/d1690f49-08cd-4f4f-b275-b1239557cbda.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0010/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/ebd3486c-f5bc-4d38-b254-794e364f0a90_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="ebd3486c-f5bc-4d38-b254-794e364f0a90",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-03-04T09:44:22.952191+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0010/cha/ebd3486c-f5bc-4d38-b254-794e364f0a90.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0010/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/699c4a78-cd3c-4fc3-812b-84913953a54a_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="699c4a78-cd3c-4fc3-812b-84913953a54a",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-03-04T09:44:44.031404+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0010/cha/699c4a78-cd3c-4fc3-812b-84913953a54a.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0010/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/d2d0e483-b898-4fb5-9c2e-937529a4646e_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="d2d0e483-b898-4fb5-9c2e-937529a4646e",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-23T17:31:19.162240+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0010/cha/d2d0e483-b898-4fb5-9c2e-937529a4646e.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0010/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/867d1afa-9471-4c8c-9581-077191336ac3_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="867d1afa-9471-4c8c-9581-077191336ac3",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-02-23T17:31:14.265613+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0010/cha/867d1afa-9471-4c8c-9581-077191336ac3.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0010/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/dcedb10b-ce33-49af-85f2-52631253e735_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="dcedb10b-ce33-49af-85f2-52631253e735",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-03-02T08:00:35.372250+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0010/cha/dcedb10b-ce33-49af-85f2-52631253e735.mp4",
    ),
    Incident(
        camera_uuid="trieagle/ocala/0010/cha",
        video_s3_path="s3://voxel-portal-production/trieagle/ocala/incidents/60edfcca-2ce6-4519-913c-1385240a9c9a_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="60edfcca-2ce6-4519-913c-1385240a9c9a",
        organization="TRIEAGLE",
        zone="OCALA",
        experimental=False,
        created_at="2023-03-02T08:20:32.207328+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/trieagle/ocala/0010/cha/60edfcca-2ce6-4519-913c-1385240a9c9a.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0001/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/4cb51d56-9b76-4a02-85a1-e0ea7d7609e7_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="4cb51d56-9b76-4a02-85a1-e0ea7d7609e7",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-10T12:19:45.887014+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0001/cha/4cb51d56-9b76-4a02-85a1-e0ea7d7609e7.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0002/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/22ed3379-6a66-44ba-95c9-f938ed659dcd_video.mp4",
        incident_type_id="NO_PED_ZONE",
        incident_uuid="22ed3379-6a66-44ba-95c9-f938ed659dcd",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-21T15:54:32.941500+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0002/cha/22ed3379-6a66-44ba-95c9-f938ed659dcd.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0002/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/c5617bdd-f604-46c4-893f-8c510e840944_video.mp4",
        incident_type_id="NO_PED_ZONE",
        incident_uuid="c5617bdd-f604-46c4-893f-8c510e840944",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-09T20:29:35.601921+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0002/cha/c5617bdd-f604-46c4-893f-8c510e840944.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0002/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/ea3c0865-2b0a-4e9c-b413-f476d96ab1b6_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="ea3c0865-2b0a-4e9c-b413-f476d96ab1b6",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-20T12:05:32.866122+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0002/cha/ea3c0865-2b0a-4e9c-b413-f476d96ab1b6.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0002/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/81dfb061-019d-4fa7-b711-9aeaa757c8ed_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="81dfb061-019d-4fa7-b711-9aeaa757c8ed",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-20T12:02:05.503071+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0002/cha/81dfb061-019d-4fa7-b711-9aeaa757c8ed.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0003/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/e3bbf324-58f9-40e6-a211-c5a81ba67f7a_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="e3bbf324-58f9-40e6-a211-c5a81ba67f7a",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-21T04:16:10.733343+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0003/cha/e3bbf324-58f9-40e6-a211-c5a81ba67f7a.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0003/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/4dd72a81-1f38-4b3c-969b-e1a317145cb2_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="4dd72a81-1f38-4b3c-969b-e1a317145cb2",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-24T12:24:29.506689+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0003/cha/4dd72a81-1f38-4b3c-969b-e1a317145cb2.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0003/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/fbd63103-012e-4118-bd49-230641cfecc0_video.mp4",
        incident_type_id="NO_PED_ZONE",
        incident_uuid="fbd63103-012e-4118-bd49-230641cfecc0",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-28T20:08:29.483835+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0003/cha/fbd63103-012e-4118-bd49-230641cfecc0.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0003/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/4979cb2e-9d0c-4893-b404-fda1a9ce8881_video.mp4",
        incident_type_id="NO_PED_ZONE",
        incident_uuid="4979cb2e-9d0c-4893-b404-fda1a9ce8881",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-09T14:21:28.265839+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0003/cha/4979cb2e-9d0c-4893-b404-fda1a9ce8881.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0003/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/1ed9c783-ab7e-4ef6-b738-4eaa1efb5aff_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="1ed9c783-ab7e-4ef6-b738-4eaa1efb5aff",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-09T16:53:38.922292+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0003/cha/1ed9c783-ab7e-4ef6-b738-4eaa1efb5aff.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0003/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/e1a10817-258d-4dba-beb0-22a1b45327c7_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="e1a10817-258d-4dba-beb0-22a1b45327c7",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-09T20:27:41.733824+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0003/cha/e1a10817-258d-4dba-beb0-22a1b45327c7.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0004/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/f5df95b1-4926-4102-904e-45419c6d2545_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="f5df95b1-4926-4102-904e-45419c6d2545",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-09T14:53:29.135959+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0004/cha/f5df95b1-4926-4102-904e-45419c6d2545.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0004/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/89ae9955-4468-4e49-83b5-75a82184a730_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="89ae9955-4468-4e49-83b5-75a82184a730",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-06T20:19:28.500162+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0004/cha/89ae9955-4468-4e49-83b5-75a82184a730.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0004/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/705cb8ed-3d19-4999-87e2-88cf7f7f2771_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="705cb8ed-3d19-4999-87e2-88cf7f7f2771",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-20T12:08:53.163073+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0004/cha/705cb8ed-3d19-4999-87e2-88cf7f7f2771.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0004/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/6241b16d-e8f7-4dc2-a791-88e72e7c1fcf_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="6241b16d-e8f7-4dc2-a791-88e72e7c1fcf",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-21T05:44:48.854192+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0004/cha/6241b16d-e8f7-4dc2-a791-88e72e7c1fcf.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0005/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/bcba5067-1f0c-4acc-80e7-b98739e07cff_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="bcba5067-1f0c-4acc-80e7-b98739e07cff",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-20T19:52:50.957916+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0005/cha/bcba5067-1f0c-4acc-80e7-b98739e07cff.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0005/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/d6075743-6824-4936-a1f2-38ebd9a57dff_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="d6075743-6824-4936-a1f2-38ebd9a57dff",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-20T20:34:41.039104+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0005/cha/d6075743-6824-4936-a1f2-38ebd9a57dff.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0005/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/d25caab7-03ba-4c9e-acfb-757f30da8471_video.mp4",
        incident_type_id="NO_PED_ZONE",
        incident_uuid="d25caab7-03ba-4c9e-acfb-757f30da8471",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-21T02:34:05.410980+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0005/cha/d25caab7-03ba-4c9e-acfb-757f30da8471.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0005/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/0067abc0-0b17-4bc4-a3ca-49ae40a7fd00_video.mp4",
        incident_type_id="NO_PED_ZONE",
        incident_uuid="0067abc0-0b17-4bc4-a3ca-49ae40a7fd00",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-06T20:31:03.604352+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0005/cha/0067abc0-0b17-4bc4-a3ca-49ae40a7fd00.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0005/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/f10d41ef-4656-451f-aaba-3990cdc46410_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="f10d41ef-4656-451f-aaba-3990cdc46410",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-23T16:37:13.035827+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0005/cha/f10d41ef-4656-451f-aaba-3990cdc46410.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0005/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/0b21b31a-bc5e-4a0e-959d-ddc5f205ce48_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="0b21b31a-bc5e-4a0e-959d-ddc5f205ce48",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-23T18:52:13.646070+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0005/cha/0b21b31a-bc5e-4a0e-959d-ddc5f205ce48.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0006/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/fc95686c-400a-4098-8950-8a2d559714f0_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="fc95686c-400a-4098-8950-8a2d559714f0",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-06T13:13:41.577578+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0006/cha/fc95686c-400a-4098-8950-8a2d559714f0.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0006/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/f2fae535-7e78-44e8-84da-1381ceccdc06_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="f2fae535-7e78-44e8-84da-1381ceccdc06",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-06T13:13:48.970992+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0006/cha/f2fae535-7e78-44e8-84da-1381ceccdc06.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0006/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/cbb485b2-9f3a-410e-b637-61a3e40cad7b_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="cbb485b2-9f3a-410e-b637-61a3e40cad7b",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-09T16:53:54.023749+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0006/cha/cbb485b2-9f3a-410e-b637-61a3e40cad7b.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0007/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/7ecb9ad7-7428-4f79-9330-35990171f029_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="7ecb9ad7-7428-4f79-9330-35990171f029",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-15T21:35:46.721078+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0007/cha/7ecb9ad7-7428-4f79-9330-35990171f029.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0007/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/00df0885-7d99-4c63-9d0e-05afc0ff6f2c_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="00df0885-7d99-4c63-9d0e-05afc0ff6f2c",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-24T04:30:45.255890+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0007/cha/00df0885-7d99-4c63-9d0e-05afc0ff6f2c.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0009/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/9651f0e7-0fd5-488d-afce-18680d930c95_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="9651f0e7-0fd5-488d-afce-18680d930c95",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-24T04:29:32.254284+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0009/cha/9651f0e7-0fd5-488d-afce-18680d930c95.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0009/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/01942a85-e5d2-4bf4-b3dd-1373adcaf9de_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="01942a85-e5d2-4bf4-b3dd-1373adcaf9de",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-23T03:02:18.241735+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0009/cha/01942a85-e5d2-4bf4-b3dd-1373adcaf9de.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0010/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/3722781c-12fa-46b9-8b87-94835159a976_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="3722781c-12fa-46b9-8b87-94835159a976",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-10T12:40:16.543865+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0010/cha/3722781c-12fa-46b9-8b87-94835159a976.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0010/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/dd5dc257-3bfc-467a-831a-03e7a0bc4218_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="dd5dc257-3bfc-467a-831a-03e7a0bc4218",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-10T12:40:23.650622+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0010/cha/dd5dc257-3bfc-467a-831a-03e7a0bc4218.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0011/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/923074a0-2fd5-4040-8305-be4a3a2ba729_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="923074a0-2fd5-4040-8305-be4a3a2ba729",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-20T19:29:28.942548+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0011/cha/923074a0-2fd5-4040-8305-be4a3a2ba729.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0011/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/07876c05-88c3-49cc-93dc-d1fcdf32ec4f_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="07876c05-88c3-49cc-93dc-d1fcdf32ec4f",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-20T19:45:39.191976+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0011/cha/07876c05-88c3-49cc-93dc-d1fcdf32ec4f.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0011/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/b50dbd86-5dad-4d34-a9a8-8cfa91bfc6d3_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="b50dbd86-5dad-4d34-a9a8-8cfa91bfc6d3",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-21T11:20:30.895172+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0011/cha/b50dbd86-5dad-4d34-a9a8-8cfa91bfc6d3.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0011/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/dfc10757-a86c-4150-aee5-3d2d5b2b1ac9_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="dfc10757-a86c-4150-aee5-3d2d5b2b1ac9",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-21T11:20:22.009654+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0011/cha/dfc10757-a86c-4150-aee5-3d2d5b2b1ac9.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0013/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/c73143ed-d640-4463-83d1-ba862fbb1f66_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="c73143ed-d640-4463-83d1-ba862fbb1f66",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-09T23:04:24.258940+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0013/cha/c73143ed-d640-4463-83d1-ba862fbb1f66.mp4",
    ),
    Incident(
        camera_uuid="verst/walton/0013/cha",
        video_s3_path="s3://voxel-portal-production/verst/walton/incidents/4c939478-ff35-4dc5-80b9-501300350161_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="4c939478-ff35-4dc5-80b9-501300350161",
        organization="VERST",
        zone="WALTON",
        experimental=False,
        created_at="2023-02-21T00:38:05.118982+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/verst/walton/0013/cha/4c939478-ff35-4dc5-80b9-501300350161.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0001/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/d3532498-edae-4d8a-8660-d8175d5fe78e_video.mp4",
        incident_type_id="PRODUCTION_LINE_DOWN",
        incident_uuid="d3532498-edae-4d8a-8660-d8175d5fe78e",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-02-10T05:22:10.238923+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0001/cha/d3532498-edae-4d8a-8660-d8175d5fe78e.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0001/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/7ac9f584-97ec-4e7b-871d-be6191009b93_video.mp4",
        incident_type_id="PRODUCTION_LINE_DOWN",
        incident_uuid="7ac9f584-97ec-4e7b-871d-be6191009b93",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-02-10T05:32:21.213969+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0001/cha/7ac9f584-97ec-4e7b-871d-be6191009b93.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0003/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/eccd31df-07f9-474b-bb41-f4b26782071b_video.mp4",
        incident_type_id="PRODUCTION_LINE_DOWN",
        incident_uuid="eccd31df-07f9-474b-bb41-f4b26782071b",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-02-23T17:10:58.117477+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0003/cha/eccd31df-07f9-474b-bb41-f4b26782071b.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0003/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/7fd92377-7e55-46dd-b286-2336197367d9_video.mp4",
        incident_type_id="PRODUCTION_LINE_DOWN",
        incident_uuid="7fd92377-7e55-46dd-b286-2336197367d9",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-02-10T08:56:05.548699+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0003/cha/7fd92377-7e55-46dd-b286-2336197367d9.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0004/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/edac3690-c2e1-40ef-9414-eccc517ffcac_video.mp4",
        incident_type_id="PRODUCTION_LINE_DOWN",
        incident_uuid="edac3690-c2e1-40ef-9414-eccc517ffcac",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-02-10T08:29:24.739825+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0004/cha/edac3690-c2e1-40ef-9414-eccc517ffcac.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0004/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/a8653a76-21fd-4f45-9a9e-0fe44ba1774b_video.mp4",
        incident_type_id="PRODUCTION_LINE_DOWN",
        incident_uuid="a8653a76-21fd-4f45-9a9e-0fe44ba1774b",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-02-23T17:29:52.538730+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0004/cha/a8653a76-21fd-4f45-9a9e-0fe44ba1774b.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0006/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/fd9d9d3e-9555-4ba7-9e0d-a21031da3eb0_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="fd9d9d3e-9555-4ba7-9e0d-a21031da3eb0",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-02-09T18:16:25.303187+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0006/cha/fd9d9d3e-9555-4ba7-9e0d-a21031da3eb0.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0006/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/1abe0618-dece-42c0-a518-75ae12eaf3e5_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="1abe0618-dece-42c0-a518-75ae12eaf3e5",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-03-06T18:28:36.292743+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0006/cha/1abe0618-dece-42c0-a518-75ae12eaf3e5.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0006/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/d05f58e6-c6c3-4d9b-b049-fd1a4dd78f0c_video.mp4",
        incident_type_id="PRODUCTION_LINE_DOWN",
        incident_uuid="d05f58e6-c6c3-4d9b-b049-fd1a4dd78f0c",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-02-23T16:15:42.731295+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0006/cha/d05f58e6-c6c3-4d9b-b049-fd1a4dd78f0c.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0006/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/5c67a6c9-064b-4752-b65e-44f9ef9c4090_video.mp4",
        incident_type_id="PRODUCTION_LINE_DOWN",
        incident_uuid="5c67a6c9-064b-4752-b65e-44f9ef9c4090",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-02-10T03:12:30.966874+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0006/cha/5c67a6c9-064b-4752-b65e-44f9ef9c4090.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0007/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/36c4b3db-2d17-4d78-b896-ae8ef7ac52ef_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="36c4b3db-2d17-4d78-b896-ae8ef7ac52ef",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-03-04T16:14:26.394788+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0007/cha/36c4b3db-2d17-4d78-b896-ae8ef7ac52ef.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0007/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/9f85d7ae-cef7-4ef3-95c0-bb3f296f96a3_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="9f85d7ae-cef7-4ef3-95c0-bb3f296f96a3",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-03-04T16:14:32.185650+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0007/cha/9f85d7ae-cef7-4ef3-95c0-bb3f296f96a3.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0007/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/e59b11b7-eed0-45e8-82b4-c59758385d70_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="e59b11b7-eed0-45e8-82b4-c59758385d70",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-02-06T18:41:06.978071+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0007/cha/e59b11b7-eed0-45e8-82b4-c59758385d70.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0008/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/102b75f9-c2d2-4508-b36d-a1b25116a564_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="102b75f9-c2d2-4508-b36d-a1b25116a564",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-02-15T19:49:31.346988+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0008/cha/102b75f9-c2d2-4508-b36d-a1b25116a564.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0008/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/adce3362-3741-431e-aeae-f83cfc29e482_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="adce3362-3741-431e-aeae-f83cfc29e482",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-02-27T18:42:11.738967+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0008/cha/adce3362-3741-431e-aeae-f83cfc29e482.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0008/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/b8de6a17-4843-47ca-add8-f32c02a95fdb_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="b8de6a17-4843-47ca-add8-f32c02a95fdb",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-02-15T19:49:41.805781+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0008/cha/b8de6a17-4843-47ca-add8-f32c02a95fdb.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0008/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/60a6c333-79a6-4ec0-92e4-5ea0f5d57e24_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="60a6c333-79a6-4ec0-92e4-5ea0f5d57e24",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-02-24T08:49:28.326033+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0008/cha/60a6c333-79a6-4ec0-92e4-5ea0f5d57e24.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0008/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/90c40a6f-f3b8-4864-939f-68c880b7cbd1_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="90c40a6f-f3b8-4864-939f-68c880b7cbd1",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-02-24T14:42:45.333360+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0008/cha/90c40a6f-f3b8-4864-939f-68c880b7cbd1.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0009/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/fff0abdf-b83c-4593-8b89-1c061139395b_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="fff0abdf-b83c-4593-8b89-1c061139395b",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-02-24T01:42:07.013066+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0009/cha/fff0abdf-b83c-4593-8b89-1c061139395b.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0009/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/f28ec5cd-5f75-4a16-80f4-efc1a7db39d2_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="f28ec5cd-5f75-4a16-80f4-efc1a7db39d2",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-02-28T00:26:15.612905+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0009/cha/f28ec5cd-5f75-4a16-80f4-efc1a7db39d2.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0009/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/c122d286-c692-4afb-8860-445d610da4b1_video.mp4",
        incident_type_id="DOOR_VIOLATION",
        incident_uuid="c122d286-c692-4afb-8860-445d610da4b1",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-02-10T05:33:43.288809+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0009/cha/c122d286-c692-4afb-8860-445d610da4b1.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0009/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/43255da9-106d-4a2a-acf2-37b5cf082e33_video.mp4",
        incident_type_id="DOOR_VIOLATION",
        incident_uuid="43255da9-106d-4a2a-acf2-37b5cf082e33",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-02-10T05:37:01.358332+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0009/cha/43255da9-106d-4a2a-acf2-37b5cf082e33.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0009/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/bb1bd09d-5b14-4895-99fe-c60153fda9dc_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="bb1bd09d-5b14-4895-99fe-c60153fda9dc",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-02-23T14:05:32.754996+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0009/cha/bb1bd09d-5b14-4895-99fe-c60153fda9dc.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0009/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/3e22e3ae-9149-4560-b488-06925fc95164_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="3e22e3ae-9149-4560-b488-06925fc95164",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-02-16T14:09:10.466351+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0009/cha/3e22e3ae-9149-4560-b488-06925fc95164.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0009/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/27841722-f642-4d1f-9c97-40b1d42f6635_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="27841722-f642-4d1f-9c97-40b1d42f6635",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-03-05T17:28:30.348220+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0009/cha/27841722-f642-4d1f-9c97-40b1d42f6635.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0009/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/6c5a58a1-2ac3-42ee-8a70-8ca675a006a8_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="6c5a58a1-2ac3-42ee-8a70-8ca675a006a8",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-03-05T15:01:54.183294+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0009/cha/6c5a58a1-2ac3-42ee-8a70-8ca675a006a8.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0011/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/c2ce1e09-7288-4d29-ba01-6b8148779da3_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="c2ce1e09-7288-4d29-ba01-6b8148779da3",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-02-09T23:46:55.093534+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0011/cha/c2ce1e09-7288-4d29-ba01-6b8148779da3.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0011/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/999a93b6-3117-4dd3-8c7d-3d30a29d1cff_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="999a93b6-3117-4dd3-8c7d-3d30a29d1cff",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-02-16T00:06:05.321953+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0011/cha/999a93b6-3117-4dd3-8c7d-3d30a29d1cff.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0011/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/1fb86003-8c9a-4ca8-9a94-46e1949e924b_video.mp4",
        incident_type_id="DOOR_VIOLATION",
        incident_uuid="1fb86003-8c9a-4ca8-9a94-46e1949e924b",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-02-23T21:59:42.990455+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0011/cha/1fb86003-8c9a-4ca8-9a94-46e1949e924b.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0011/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/40de59e9-42ad-482e-8e6f-71e52defc5ff_video.mp4",
        incident_type_id="DOOR_VIOLATION",
        incident_uuid="40de59e9-42ad-482e-8e6f-71e52defc5ff",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-02-23T21:59:51.068572+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0011/cha/40de59e9-42ad-482e-8e6f-71e52defc5ff.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0011/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/a500dcbb-e5a8-4ef2-a2e6-64b711860461_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="a500dcbb-e5a8-4ef2-a2e6-64b711860461",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-02-24T01:53:33.363553+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0011/cha/a500dcbb-e5a8-4ef2-a2e6-64b711860461.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0011/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/a26223bd-4990-4aac-8c7b-f750bd32c4c1_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="a26223bd-4990-4aac-8c7b-f750bd32c4c1",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-02-22T04:26:20.306653+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0011/cha/a26223bd-4990-4aac-8c7b-f750bd32c4c1.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0011/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/c98825c2-a6e1-4682-a921-993649ca1f8c_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="c98825c2-a6e1-4682-a921-993649ca1f8c",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-02-25T12:43:08.570866+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0011/cha/c98825c2-a6e1-4682-a921-993649ca1f8c.mp4",
    ),
    Incident(
        camera_uuid="wn_foods/hayward/0011/cha",
        video_s3_path="s3://voxel-portal-production/wn_foods/hayward/incidents/cfebc4f9-2cea-43ed-a7b6-6e1d20c1bd3d_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="cfebc4f9-2cea-43ed-a7b6-6e1d20c1bd3d",
        organization="WN_FOODS",
        zone="HAYWARD",
        experimental=False,
        created_at="2023-03-05T02:03:26.939119+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wn_foods/hayward/0011/cha/cfebc4f9-2cea-43ed-a7b6-6e1d20c1bd3d.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0001/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/e16d4093-c0cd-4578-9f72-e8551cb62260_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="e16d4093-c0cd-4578-9f72-e8551cb62260",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-03-04T05:17:30.363017+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0001/cha/e16d4093-c0cd-4578-9f72-e8551cb62260.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0001/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/effe5656-7097-41ea-8576-02ba1ac5f38e_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="effe5656-7097-41ea-8576-02ba1ac5f38e",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-03-04T09:31:16.672995+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0001/cha/effe5656-7097-41ea-8576-02ba1ac5f38e.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0001/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/ab3f5096-66f9-4389-820a-a0267c31d479_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="ab3f5096-66f9-4389-820a-a0267c31d479",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-02-22T08:18:10.720919+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0001/cha/ab3f5096-66f9-4389-820a-a0267c31d479.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0001/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/87e94dba-1d71-4cb0-beb3-ec91b1c11f6e_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="87e94dba-1d71-4cb0-beb3-ec91b1c11f6e",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-02-22T08:18:48.397808+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0001/cha/87e94dba-1d71-4cb0-beb3-ec91b1c11f6e.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0001/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/e50ee9c4-9c94-4293-833b-d4e2a5bcbe46_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="e50ee9c4-9c94-4293-833b-d4e2a5bcbe46",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-02-24T02:29:23.564781+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0001/cha/e50ee9c4-9c94-4293-833b-d4e2a5bcbe46.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0001/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/d29ffc64-a942-452d-a6e1-ccf0be76abc2_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="d29ffc64-a942-452d-a6e1-ccf0be76abc2",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-03-05T23:04:33.331670+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0001/cha/d29ffc64-a942-452d-a6e1-ccf0be76abc2.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0002/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/c4bcb6e3-ec5f-40f1-a6c4-da732d640721_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="c4bcb6e3-ec5f-40f1-a6c4-da732d640721",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-03-04T05:48:14.828600+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0002/cha/c4bcb6e3-ec5f-40f1-a6c4-da732d640721.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0002/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/107e3a14-004c-4cfc-a77c-8323831ca42b_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="107e3a14-004c-4cfc-a77c-8323831ca42b",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-02-28T23:14:43.059868+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0002/cha/107e3a14-004c-4cfc-a77c-8323831ca42b.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0002/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/55373f87-2d5b-43f7-a3cb-3e1b6f92f0f3_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="55373f87-2d5b-43f7-a3cb-3e1b6f92f0f3",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-02-23T08:32:11.443576+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0002/cha/55373f87-2d5b-43f7-a3cb-3e1b6f92f0f3.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0002/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/a4358ab6-d8a7-41a3-8ae7-48d74457061c_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="a4358ab6-d8a7-41a3-8ae7-48d74457061c",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-02-23T08:19:00.796331+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0002/cha/a4358ab6-d8a7-41a3-8ae7-48d74457061c.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0002/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/711a8273-fdf7-449e-8402-baf75f52887a_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="711a8273-fdf7-449e-8402-baf75f52887a",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-03-05T23:25:19.725072+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0002/cha/711a8273-fdf7-449e-8402-baf75f52887a.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0002/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/a841e4a2-75ec-44de-b981-cdd4cd02884b_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="a841e4a2-75ec-44de-b981-cdd4cd02884b",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-03-05T23:28:36.266625+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0002/cha/a841e4a2-75ec-44de-b981-cdd4cd02884b.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0003/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/794d8d64-1fbe-4cb4-8b9e-427c1ff68d7b_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="794d8d64-1fbe-4cb4-8b9e-427c1ff68d7b",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-03-04T03:44:40.768173+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0003/cha/794d8d64-1fbe-4cb4-8b9e-427c1ff68d7b.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0003/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/8b903052-c629-46e7-8b00-a24364dd41e9_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="8b903052-c629-46e7-8b00-a24364dd41e9",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-03-04T10:59:57.542164+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0003/cha/8b903052-c629-46e7-8b00-a24364dd41e9.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0003/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/62be6b32-dad3-479f-b5d7-7cd30a371dec_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="62be6b32-dad3-479f-b5d7-7cd30a371dec",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-02-23T15:58:58.592690+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0003/cha/62be6b32-dad3-479f-b5d7-7cd30a371dec.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0003/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/9fb2e132-e91d-4efd-a753-7093afac2194_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="9fb2e132-e91d-4efd-a753-7093afac2194",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-02-23T15:52:24.777107+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0003/cha/9fb2e132-e91d-4efd-a753-7093afac2194.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0003/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/077cd034-d7bd-4041-b360-0f629b6774f5_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="077cd034-d7bd-4041-b360-0f629b6774f5",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-02-25T05:09:17.420385+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0003/cha/077cd034-d7bd-4041-b360-0f629b6774f5.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0003/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/daabad8d-4b74-40c2-82c4-8ccddea1d344_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="daabad8d-4b74-40c2-82c4-8ccddea1d344",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-02-27T00:22:05.964197+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0003/cha/daabad8d-4b74-40c2-82c4-8ccddea1d344.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0004/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/5356a386-99c2-4b1d-874c-ef3d53c2f7f2_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="5356a386-99c2-4b1d-874c-ef3d53c2f7f2",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-03-04T01:50:27.654444+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0004/cha/5356a386-99c2-4b1d-874c-ef3d53c2f7f2.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0004/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/df2b7006-de5b-4126-9086-be34c6ccfb82_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="df2b7006-de5b-4126-9086-be34c6ccfb82",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-03-04T04:48:27.458931+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0004/cha/df2b7006-de5b-4126-9086-be34c6ccfb82.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0004/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/fd0a62b2-420f-4c41-a81d-133ccd41deeb_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="fd0a62b2-420f-4c41-a81d-133ccd41deeb",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-03-04T02:40:54.936061+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0004/cha/fd0a62b2-420f-4c41-a81d-133ccd41deeb.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0004/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/d7534f5e-dcb9-4cdd-b076-0d8b39fa7e0d_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="d7534f5e-dcb9-4cdd-b076-0d8b39fa7e0d",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-03-04T02:40:27.412081+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0004/cha/d7534f5e-dcb9-4cdd-b076-0d8b39fa7e0d.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0004/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/e73bfeb3-089d-49fb-b0aa-a6434eafa6a0_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="e73bfeb3-089d-49fb-b0aa-a6434eafa6a0",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-02-26T23:35:27.136358+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0004/cha/e73bfeb3-089d-49fb-b0aa-a6434eafa6a0.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0004/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/83d24b86-e4ee-4111-9c9c-b70638ca5f19_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="83d24b86-e4ee-4111-9c9c-b70638ca5f19",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-02-26T23:38:53.104610+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0004/cha/83d24b86-e4ee-4111-9c9c-b70638ca5f19.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0005/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/aaae43f0-9cef-4ec8-a43d-f23d3b12f55b_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="aaae43f0-9cef-4ec8-a43d-f23d3b12f55b",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-03-01T20:09:33.855563+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0005/cha/aaae43f0-9cef-4ec8-a43d-f23d3b12f55b.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0005/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/02d27dbf-23a0-40b1-9613-163deb9a12b5_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="02d27dbf-23a0-40b1-9613-163deb9a12b5",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-03-06T13:07:46.296536+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0005/cha/02d27dbf-23a0-40b1-9613-163deb9a12b5.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0005/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/b6a997c4-536b-4d6f-93f2-13b43aa2fae0_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="b6a997c4-536b-4d6f-93f2-13b43aa2fae0",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-02-20T19:34:54.770471+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0005/cha/b6a997c4-536b-4d6f-93f2-13b43aa2fae0.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0005/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/af30bd32-951e-455b-9a49-d8889b86a90b_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="af30bd32-951e-455b-9a49-d8889b86a90b",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-02-20T20:03:43.098915+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0005/cha/af30bd32-951e-455b-9a49-d8889b86a90b.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0005/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/a25641c4-1a4a-4f4a-8ca4-8c0ea94b32c2_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="a25641c4-1a4a-4f4a-8ca4-8c0ea94b32c2",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-03-03T21:20:47.174729+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0005/cha/a25641c4-1a4a-4f4a-8ca4-8c0ea94b32c2.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0005/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/fe431eb1-cffa-4b32-b5c9-10db6d6f003b_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="fe431eb1-cffa-4b32-b5c9-10db6d6f003b",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-03-03T21:24:08.404010+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0005/cha/fe431eb1-cffa-4b32-b5c9-10db6d6f003b.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0006/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/40cfba95-5ae2-4c06-8ba8-35906ef67fdf_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="40cfba95-5ae2-4c06-8ba8-35906ef67fdf",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-03-04T05:14:00.346516+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0006/cha/40cfba95-5ae2-4c06-8ba8-35906ef67fdf.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0006/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/9f10e737-f5c3-47df-805d-bcede31fcc42_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="9f10e737-f5c3-47df-805d-bcede31fcc42",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-03-04T05:37:09.819866+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0006/cha/9f10e737-f5c3-47df-805d-bcede31fcc42.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0006/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/2640740d-ccdf-4540-99d2-8a3924783aa9_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="2640740d-ccdf-4540-99d2-8a3924783aa9",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-02-20T19:40:30.482636+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0006/cha/2640740d-ccdf-4540-99d2-8a3924783aa9.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0006/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/1871fcc7-5d1e-4f25-aeb1-858a6c22f3ba_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="1871fcc7-5d1e-4f25-aeb1-858a6c22f3ba",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-02-09T21:55:06.009549+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0006/cha/1871fcc7-5d1e-4f25-aeb1-858a6c22f3ba.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0006/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/c33faaae-b8a2-4911-a445-dcbb381cab7c_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="c33faaae-b8a2-4911-a445-dcbb381cab7c",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-02-25T03:16:19.704646+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0006/cha/c33faaae-b8a2-4911-a445-dcbb381cab7c.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0006/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/61f1a547-3c22-495a-bac7-0af2cee36b3f_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="61f1a547-3c22-495a-bac7-0af2cee36b3f",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-02-25T15:36:06.713549+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0006/cha/61f1a547-3c22-495a-bac7-0af2cee36b3f.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0007/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/1553c06e-9a44-4c17-975a-e10b00c08459_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="1553c06e-9a44-4c17-975a-e10b00c08459",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-03-04T00:54:49.600437+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0007/cha/1553c06e-9a44-4c17-975a-e10b00c08459.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0007/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/054127c3-6685-42ac-be1d-d29e969b116e_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="054127c3-6685-42ac-be1d-d29e969b116e",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-03-04T01:14:46.511411+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0007/cha/054127c3-6685-42ac-be1d-d29e969b116e.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0007/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/02a3c7b1-aa4c-4a63-b7b2-43e19a069cdb_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="02a3c7b1-aa4c-4a63-b7b2-43e19a069cdb",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-02-24T12:17:47.332938+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0007/cha/02a3c7b1-aa4c-4a63-b7b2-43e19a069cdb.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0007/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/520e0abd-88d2-4897-be11-44449d8e13de_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="520e0abd-88d2-4897-be11-44449d8e13de",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-02-22T07:35:23.078225+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0007/cha/520e0abd-88d2-4897-be11-44449d8e13de.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0007/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/d0eb63f0-ebf9-460a-9b64-db8e08c5c433_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="d0eb63f0-ebf9-460a-9b64-db8e08c5c433",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-02-21T14:29:15.974473+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0007/cha/d0eb63f0-ebf9-460a-9b64-db8e08c5c433.mp4",
    ),
    Incident(
        camera_uuid="americold/tacoma/0007/cha",
        video_s3_path="s3://voxel-portal-production/americold/tacoma/incidents/db4c9851-8285-4944-ae6c-e93eba45006d_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="db4c9851-8285-4944-ae6c-e93eba45006d",
        organization="AMERICOLD",
        zone="TACOMA",
        experimental=False,
        created_at="2023-02-28T15:17:49.852759+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/tacoma/0007/cha/db4c9851-8285-4944-ae6c-e93eba45006d.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0001/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/34c2cb43-acd7-4e46-ad95-a046e5ef8003_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="34c2cb43-acd7-4e46-ad95-a046e5ef8003",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-03-04T01:41:13.089427+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0001/cha/34c2cb43-acd7-4e46-ad95-a046e5ef8003.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0001/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/8084d63d-9556-4566-839f-5987c82a75d7_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="8084d63d-9556-4566-839f-5987c82a75d7",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-02-24T22:25:47.490319+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0001/cha/8084d63d-9556-4566-839f-5987c82a75d7.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0001/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/ea96be94-dc67-4e50-9729-05806962f881_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="ea96be94-dc67-4e50-9729-05806962f881",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-02-24T22:43:20.880070+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0001/cha/ea96be94-dc67-4e50-9729-05806962f881.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0003/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/d9d286b4-788d-468a-8ba0-d27e16fdeaae_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="d9d286b4-788d-468a-8ba0-d27e16fdeaae",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-02-21T00:57:36.124912+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0003/cha/d9d286b4-788d-468a-8ba0-d27e16fdeaae.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0003/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/349d20c2-8a42-479d-86be-d941b4d9e5bd_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="349d20c2-8a42-479d-86be-d941b4d9e5bd",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-02-22T18:11:43.241173+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0003/cha/349d20c2-8a42-479d-86be-d941b4d9e5bd.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0003/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/2f58b29b-52ad-49a0-b114-9d7fd38d0235_video.mp4",
        incident_type_id="N_PERSON_PED_ZONE",
        incident_uuid="2f58b29b-52ad-49a0-b114-9d7fd38d0235",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-02-23T18:46:10.139233+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0003/cha/2f58b29b-52ad-49a0-b114-9d7fd38d0235.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0003/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/d810a15f-b6c8-4714-a00a-418fa49cf5d6_video.mp4",
        incident_type_id="N_PERSON_PED_ZONE",
        incident_uuid="d810a15f-b6c8-4714-a00a-418fa49cf5d6",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-02-20T20:35:15.371521+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0003/cha/d810a15f-b6c8-4714-a00a-418fa49cf5d6.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0004/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/6b879b0e-124f-47d8-a142-eea7cedb851c_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="6b879b0e-124f-47d8-a142-eea7cedb851c",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-03-01T16:58:33.413924+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0004/cha/6b879b0e-124f-47d8-a142-eea7cedb851c.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0004/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/989f1765-a155-4321-b494-dd4b20f894ef_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="989f1765-a155-4321-b494-dd4b20f894ef",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-03-02T11:31:01.810660+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0004/cha/989f1765-a155-4321-b494-dd4b20f894ef.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0005/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/1e7599fb-efe4-4b6e-8b49-9e1e3a5f9e71_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="1e7599fb-efe4-4b6e-8b49-9e1e3a5f9e71",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-02-23T18:26:08.951611+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0005/cha/1e7599fb-efe4-4b6e-8b49-9e1e3a5f9e71.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0005/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/d53dee4a-8b68-463e-9f10-5a29ae7eb7b5_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="d53dee4a-8b68-463e-9f10-5a29ae7eb7b5",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-02-23T18:29:33.370215+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0005/cha/d53dee4a-8b68-463e-9f10-5a29ae7eb7b5.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0005/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/8398090b-5a82-4a46-bbc9-e836382eaf76_video.mp4",
        incident_type_id="N_PERSON_PED_ZONE",
        incident_uuid="8398090b-5a82-4a46-bbc9-e836382eaf76",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-02-20T19:17:48.920554+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0005/cha/8398090b-5a82-4a46-bbc9-e836382eaf76.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0005/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/be93daf9-7005-4877-a1c3-d6d93c911c09_video.mp4",
        incident_type_id="N_PERSON_PED_ZONE",
        incident_uuid="be93daf9-7005-4877-a1c3-d6d93c911c09",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-02-23T18:46:09.521135+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0005/cha/be93daf9-7005-4877-a1c3-d6d93c911c09.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0005/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/cfefc5bf-4e99-43de-be9f-80432cddd70c_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="cfefc5bf-4e99-43de-be9f-80432cddd70c",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-03-04T15:03:46.250406+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0005/cha/cfefc5bf-4e99-43de-be9f-80432cddd70c.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0005/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/a835728f-1eba-4929-8429-ded7dd789fe9_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="a835728f-1eba-4929-8429-ded7dd789fe9",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-03-04T15:07:02.755026+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0005/cha/a835728f-1eba-4929-8429-ded7dd789fe9.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0007/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/003ffb3e-beba-4b45-bada-aa92d403a3bf_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="003ffb3e-beba-4b45-bada-aa92d403a3bf",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-02-20T19:51:28.841548+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0007/cha/003ffb3e-beba-4b45-bada-aa92d403a3bf.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0007/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/38de0798-87ed-475d-adc0-703a630b5c6a_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="38de0798-87ed-475d-adc0-703a630b5c6a",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-02-20T19:54:54.722699+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0007/cha/38de0798-87ed-475d-adc0-703a630b5c6a.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0007/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/97ba773b-9487-47a4-9c91-e12ababc34a0_video.mp4",
        incident_type_id="N_PERSON_PED_ZONE",
        incident_uuid="97ba773b-9487-47a4-9c91-e12ababc34a0",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-02-20T19:51:58.361146+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0007/cha/97ba773b-9487-47a4-9c91-e12ababc34a0.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0007/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/672d9cdb-8bd7-4e36-a8a2-039f56f5749e_video.mp4",
        incident_type_id="N_PERSON_PED_ZONE",
        incident_uuid="672d9cdb-8bd7-4e36-a8a2-039f56f5749e",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-02-23T17:50:24.296344+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0007/cha/672d9cdb-8bd7-4e36-a8a2-039f56f5749e.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0007/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/55e1aa8f-e30d-4131-87e0-f1dc1753fcd0_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="55e1aa8f-e30d-4131-87e0-f1dc1753fcd0",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-02-23T20:31:09.292333+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0007/cha/55e1aa8f-e30d-4131-87e0-f1dc1753fcd0.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0007/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/57bc90a2-61a5-4225-b18e-e29f549a733d_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="57bc90a2-61a5-4225-b18e-e29f549a733d",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-02-23T21:51:22.263067+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0007/cha/57bc90a2-61a5-4225-b18e-e29f549a733d.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0008/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/4a04ff42-2498-46a0-8eaa-b81bd2326319_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="4a04ff42-2498-46a0-8eaa-b81bd2326319",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-03-04T03:11:45.800859+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0008/cha/4a04ff42-2498-46a0-8eaa-b81bd2326319.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0008/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/ceafef1e-3136-402c-8ae5-8b35e1f0dd5b_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="ceafef1e-3136-402c-8ae5-8b35e1f0dd5b",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-03-01T23:22:31.721210+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0008/cha/ceafef1e-3136-402c-8ae5-8b35e1f0dd5b.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0010/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/4808733a-5eee-4ba8-9db5-1b9b7c704ff7_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="4808733a-5eee-4ba8-9db5-1b9b7c704ff7",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-02-06T10:46:59.979773+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0010/cha/4808733a-5eee-4ba8-9db5-1b9b7c704ff7.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0010/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/72f64640-7297-4b0b-ad9b-24b3ac529cbb_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="72f64640-7297-4b0b-ad9b-24b3ac529cbb",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-03-03T19:00:01.384076+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0010/cha/72f64640-7297-4b0b-ad9b-24b3ac529cbb.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0010/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/c16aa5d9-93ad-4044-bfbe-69da241eaa07_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="c16aa5d9-93ad-4044-bfbe-69da241eaa07",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-03-03T18:36:39.960551+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0010/cha/c16aa5d9-93ad-4044-bfbe-69da241eaa07.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0011/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/c618ea03-cc9c-4f0d-a644-5a90cacde8f5_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="c618ea03-cc9c-4f0d-a644-5a90cacde8f5",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-03-03T11:52:21.905390+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0011/cha/c618ea03-cc9c-4f0d-a644-5a90cacde8f5.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0011/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/057be853-d235-491c-8011-16cb12d20467_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="057be853-d235-491c-8011-16cb12d20467",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-03-03T11:50:02.690719+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0011/cha/057be853-d235-491c-8011-16cb12d20467.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0011/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/8f623718-dfaf-427e-b975-20aecc146c0e_video.mp4",
        incident_type_id="N_PERSON_PED_ZONE",
        incident_uuid="8f623718-dfaf-427e-b975-20aecc146c0e",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-02-20T19:19:45.753415+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0011/cha/8f623718-dfaf-427e-b975-20aecc146c0e.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0011/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/d8e223bc-4e1a-470d-980e-ad044844124f_video.mp4",
        incident_type_id="N_PERSON_PED_ZONE",
        incident_uuid="d8e223bc-4e1a-470d-980e-ad044844124f",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-02-20T19:46:34.300458+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0011/cha/d8e223bc-4e1a-470d-980e-ad044844124f.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0011/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/d234a4f0-e7a2-4c88-938c-b180650dff25_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="d234a4f0-e7a2-4c88-938c-b180650dff25",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-02-25T07:00:31.942901+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0011/cha/d234a4f0-e7a2-4c88-938c-b180650dff25.mp4",
    ),
    Incident(
        camera_uuid="ppg/cedar_falls/0011/cha",
        video_s3_path="s3://voxel-portal-production/ppg/cedar_falls/incidents/982517a9-b44a-4990-b3f3-bb5ced2ee5c9_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="982517a9-b44a-4990-b3f3-bb5ced2ee5c9",
        organization="PPG",
        zone="CEDAR_FALLS",
        experimental=False,
        created_at="2023-02-25T00:24:59.612847+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ppg/cedar_falls/0011/cha/982517a9-b44a-4990-b3f3-bb5ced2ee5c9.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0001/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/fc7595d4-c57e-426b-a8ad-db885e327e9b_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="fc7595d4-c57e-426b-a8ad-db885e327e9b",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-02-23T13:03:53.221823+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0001/cha/fc7595d4-c57e-426b-a8ad-db885e327e9b.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0001/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/3afcd7ab-dea9-4a53-b809-07e413cee6b6_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="3afcd7ab-dea9-4a53-b809-07e413cee6b6",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-02-09T17:54:15.170986+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0001/cha/3afcd7ab-dea9-4a53-b809-07e413cee6b6.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0002/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/2089ec60-5455-4232-8947-ccae367e1de4_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="2089ec60-5455-4232-8947-ccae367e1de4",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-02-16T01:54:16.176696+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0002/cha/2089ec60-5455-4232-8947-ccae367e1de4.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0002/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/77d081d3-1a99-4562-b49d-09c6c29ea5af_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="77d081d3-1a99-4562-b49d-09c6c29ea5af",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-02-23T21:50:29.866713+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0002/cha/77d081d3-1a99-4562-b49d-09c6c29ea5af.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0002/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/5ea7d42b-56ff-494e-b658-c1e7a1f524d4_video.mp4",
        incident_type_id="NO_PED_ZONE",
        incident_uuid="5ea7d42b-56ff-494e-b658-c1e7a1f524d4",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-02-23T20:06:52.150592+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0002/cha/5ea7d42b-56ff-494e-b658-c1e7a1f524d4.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0002/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/d375f988-c119-4a39-bb0b-8ce61133e117_video.mp4",
        incident_type_id="NO_PED_ZONE",
        incident_uuid="d375f988-c119-4a39-bb0b-8ce61133e117",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-02-23T20:27:16.858641+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0002/cha/d375f988-c119-4a39-bb0b-8ce61133e117.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0002/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/7ab5c9e1-15ee-42f1-9a2d-b99a73f8f539_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="7ab5c9e1-15ee-42f1-9a2d-b99a73f8f539",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-03-04T01:03:35.845853+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0002/cha/7ab5c9e1-15ee-42f1-9a2d-b99a73f8f539.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0002/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/076f7390-f14d-4dbe-b008-576f4892779e_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="076f7390-f14d-4dbe-b008-576f4892779e",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-02-20T23:28:34.709578+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0002/cha/076f7390-f14d-4dbe-b008-576f4892779e.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0002/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/fdd73e01-1aef-4dd3-905e-04692be0d276_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="fdd73e01-1aef-4dd3-905e-04692be0d276",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-02-25T05:23:50.743530+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0002/cha/fdd73e01-1aef-4dd3-905e-04692be0d276.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0002/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/3f35abcd-6469-4555-b310-d69be3868d05_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="3f35abcd-6469-4555-b310-d69be3868d05",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-02-25T08:04:39.252518+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0002/cha/3f35abcd-6469-4555-b310-d69be3868d05.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0003/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/3b118781-74bd-4285-9302-b63dd0202b79_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="3b118781-74bd-4285-9302-b63dd0202b79",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-02-27T16:45:11.062484+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0003/cha/3b118781-74bd-4285-9302-b63dd0202b79.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0003/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/ebbd0436-d5c0-436f-bbda-f8da88c03031_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="ebbd0436-d5c0-436f-bbda-f8da88c03031",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-02-27T18:08:35.961000+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0003/cha/ebbd0436-d5c0-436f-bbda-f8da88c03031.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0003/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/79cad386-cdb5-4a56-aa81-dca7ba7bda2b_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="79cad386-cdb5-4a56-aa81-dca7ba7bda2b",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-02-10T09:55:38.430081+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0003/cha/79cad386-cdb5-4a56-aa81-dca7ba7bda2b.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0003/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/71eac462-193f-4217-8d20-499791f7fd9b_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="71eac462-193f-4217-8d20-499791f7fd9b",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-02-09T09:46:34.419498+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0003/cha/71eac462-193f-4217-8d20-499791f7fd9b.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0004/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/4b385286-4c13-45f3-87e9-3daaf2f1f630_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="4b385286-4c13-45f3-87e9-3daaf2f1f630",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-02-11T14:07:54.940168+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0004/cha/4b385286-4c13-45f3-87e9-3daaf2f1f630.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0004/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/782ae94d-9585-4b92-a846-a298ead5caf6_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="782ae94d-9585-4b92-a846-a298ead5caf6",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-02-09T10:39:55.595837+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0004/cha/782ae94d-9585-4b92-a846-a298ead5caf6.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0005/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/e912d336-2ae0-48ad-a59e-1d5446a89e7e_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="e912d336-2ae0-48ad-a59e-1d5446a89e7e",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-02-24T12:46:37.891867+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0005/cha/e912d336-2ae0-48ad-a59e-1d5446a89e7e.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0005/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/bc3913a8-51bb-4a62-8a17-5a8f137acb7d_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="bc3913a8-51bb-4a62-8a17-5a8f137acb7d",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-03-03T12:28:19.516964+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0005/cha/bc3913a8-51bb-4a62-8a17-5a8f137acb7d.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0005/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/7fadaecd-0b9f-4354-896f-ebb7384744b0_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="7fadaecd-0b9f-4354-896f-ebb7384744b0",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-02-17T08:59:47.609587+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0005/cha/7fadaecd-0b9f-4354-896f-ebb7384744b0.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0006/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/0e2f1393-37e6-489c-8b9e-b3bc673cc169_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="0e2f1393-37e6-489c-8b9e-b3bc673cc169",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-02-23T16:00:34.096200+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0006/cha/0e2f1393-37e6-489c-8b9e-b3bc673cc169.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0006/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/95f76e84-b7df-4976-a314-1e544c23cdd3_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="95f76e84-b7df-4976-a314-1e544c23cdd3",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-03-04T03:02:10.561958+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0006/cha/95f76e84-b7df-4976-a314-1e544c23cdd3.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0006/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/841fdd43-8e85-431a-ae82-dca02c7779cd_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="841fdd43-8e85-431a-ae82-dca02c7779cd",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-02-14T20:08:53.577624+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0006/cha/841fdd43-8e85-431a-ae82-dca02c7779cd.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0007/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/968356c4-6efa-453e-99f7-1bbe202f1ee0_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="968356c4-6efa-453e-99f7-1bbe202f1ee0",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-02-28T12:11:14.113247+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0007/cha/968356c4-6efa-453e-99f7-1bbe202f1ee0.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0007/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/db03a6c1-dc29-4686-9ca7-e4db2253b256_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="db03a6c1-dc29-4686-9ca7-e4db2253b256",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-02-28T12:14:22.616358+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0007/cha/db03a6c1-dc29-4686-9ca7-e4db2253b256.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0009/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/327b3b0a-244a-4561-8737-5e880ff38b61_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="327b3b0a-244a-4561-8737-5e880ff38b61",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-02-22T10:10:37.272511+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0009/cha/327b3b0a-244a-4561-8737-5e880ff38b61.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0009/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/b22d4e29-845e-4be0-bae0-4a8bf6f752f0_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="b22d4e29-845e-4be0-bae0-4a8bf6f752f0",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-02-22T09:20:22.746025+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0009/cha/b22d4e29-845e-4be0-bae0-4a8bf6f752f0.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0009/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/c9f7907b-ad0c-4022-96a3-1e2b4461da52_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="c9f7907b-ad0c-4022-96a3-1e2b4461da52",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-02-25T14:22:25.991214+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0009/cha/c9f7907b-ad0c-4022-96a3-1e2b4461da52.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0009/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/a1808b1f-b4d6-4d07-ad26-743921b0a798_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="a1808b1f-b4d6-4d07-ad26-743921b0a798",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-02-25T16:59:41.387722+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0009/cha/a1808b1f-b4d6-4d07-ad26-743921b0a798.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0009/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/c0c0fbaa-67e2-47e4-88eb-4ad5bd4180ab_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="c0c0fbaa-67e2-47e4-88eb-4ad5bd4180ab",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-02-28T02:45:32.075673+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0009/cha/c0c0fbaa-67e2-47e4-88eb-4ad5bd4180ab.mp4",
    ),
    Incident(
        camera_uuid="piston_automotive/marion/0009/cha",
        video_s3_path="s3://voxel-portal-production/piston_automotive/marion/incidents/dc29d23b-a56c-420b-b663-80fd7147ec3b_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="dc29d23b-a56c-420b-b663-80fd7147ec3b",
        organization="PISTON_AUTOMOTIVE",
        zone="MARION",
        experimental=False,
        created_at="2023-03-03T23:12:12.336285+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/piston_automotive/marion/0009/cha/dc29d23b-a56c-420b-b663-80fd7147ec3b.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0004/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/68abcd5d-dd9d-4b45-ad60-087f884ecddd_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="68abcd5d-dd9d-4b45-ad60-087f884ecddd",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-02-21T16:48:04.400707+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0004/cha/68abcd5d-dd9d-4b45-ad60-087f884ecddd.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0004/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/fc4e1cdd-ff8f-4c6c-939d-d66654e21455_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="fc4e1cdd-ff8f-4c6c-939d-d66654e21455",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-02-24T09:37:46.274292+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0004/cha/fc4e1cdd-ff8f-4c6c-939d-d66654e21455.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0004/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/68024c54-369a-496b-9ffb-aef96b0c3fe6_video.mp4",
        incident_type_id="DOOR_VIOLATION",
        incident_uuid="68024c54-369a-496b-9ffb-aef96b0c3fe6",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-03-04T06:49:10.110701+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0004/cha/68024c54-369a-496b-9ffb-aef96b0c3fe6.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0004/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/c19575bb-a856-40ef-be84-62b20c78f433_video.mp4",
        incident_type_id="DOOR_VIOLATION",
        incident_uuid="c19575bb-a856-40ef-be84-62b20c78f433",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-03-04T06:52:30.317149+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0004/cha/c19575bb-a856-40ef-be84-62b20c78f433.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0004/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/9acc8059-c578-4dab-a252-aee291396fd6_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="9acc8059-c578-4dab-a252-aee291396fd6",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-03-04T01:15:37.302507+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0004/cha/9acc8059-c578-4dab-a252-aee291396fd6.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0004/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/e8b1d13c-776a-44f0-9fed-bea6b6f09947_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="e8b1d13c-776a-44f0-9fed-bea6b6f09947",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-02-21T22:23:32.846042+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0004/cha/e8b1d13c-776a-44f0-9fed-bea6b6f09947.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0005/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/a431f614-6acd-4c33-b0a5-72dce9ffbd31_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="a431f614-6acd-4c33-b0a5-72dce9ffbd31",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-02-10T05:41:14.745826+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0005/cha/a431f614-6acd-4c33-b0a5-72dce9ffbd31.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0005/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/0a0b0e09-7887-4101-bec6-9a81a10384f7_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="0a0b0e09-7887-4101-bec6-9a81a10384f7",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-02-10T03:58:01.319100+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0005/cha/0a0b0e09-7887-4101-bec6-9a81a10384f7.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0005/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/838859f0-5aec-427d-b4f0-500fb6ea5c7f_video.mp4",
        incident_type_id="DOOR_VIOLATION",
        incident_uuid="838859f0-5aec-427d-b4f0-500fb6ea5c7f",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-02-27T16:06:15.435372+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0005/cha/838859f0-5aec-427d-b4f0-500fb6ea5c7f.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0005/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/ea15085d-0e3c-4f03-9da0-ea3a0c192865_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="ea15085d-0e3c-4f03-9da0-ea3a0c192865",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-02-09T10:33:29.223426+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0005/cha/ea15085d-0e3c-4f03-9da0-ea3a0c192865.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0005/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/597ebcd7-3a8d-4ebc-990e-f0926738320c_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="597ebcd7-3a8d-4ebc-990e-f0926738320c",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-02-27T07:21:41.698383+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0005/cha/597ebcd7-3a8d-4ebc-990e-f0926738320c.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0006/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/8b592d7a-40e5-4081-bf51-553984bfff22_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="8b592d7a-40e5-4081-bf51-553984bfff22",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-03-04T01:06:30.059668+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0006/cha/8b592d7a-40e5-4081-bf51-553984bfff22.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0006/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/8b10e6c5-4d0a-4f27-9135-344687c218a1_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="8b10e6c5-4d0a-4f27-9135-344687c218a1",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-02-23T15:44:01.120512+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0006/cha/8b10e6c5-4d0a-4f27-9135-344687c218a1.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0006/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/5b9a439c-748d-46eb-be91-36a8b74e00ef_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="5b9a439c-748d-46eb-be91-36a8b74e00ef",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-02-16T05:06:03.782281+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0006/cha/5b9a439c-748d-46eb-be91-36a8b74e00ef.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0006/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/6a1ffb71-4cd2-4202-9c19-0b7f382dd3bd_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="6a1ffb71-4cd2-4202-9c19-0b7f382dd3bd",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-02-16T05:32:38.907880+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0006/cha/6a1ffb71-4cd2-4202-9c19-0b7f382dd3bd.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0006/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/62df16f9-b3cf-42c0-9bac-d543ab6db4ba_video.mp4",
        incident_type_id="PRODUCTION_LINE_DOWN",
        incident_uuid="62df16f9-b3cf-42c0-9bac-d543ab6db4ba",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-02-12T23:45:38.201657+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0006/cha/62df16f9-b3cf-42c0-9bac-d543ab6db4ba.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0006/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/aab31539-0da5-4f1c-ac1d-50902b9009d3_video.mp4",
        incident_type_id="PRODUCTION_LINE_DOWN",
        incident_uuid="aab31539-0da5-4f1c-ac1d-50902b9009d3",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-02-13T09:53:44.334128+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0006/cha/aab31539-0da5-4f1c-ac1d-50902b9009d3.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0007/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/35b46484-ef7e-4d56-a1fe-ee3b7ca2f88c_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="35b46484-ef7e-4d56-a1fe-ee3b7ca2f88c",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-02-09T11:21:19.404865+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0007/cha/35b46484-ef7e-4d56-a1fe-ee3b7ca2f88c.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0007/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/f1f9ca4a-ba99-4588-be8e-d724a557148c_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="f1f9ca4a-ba99-4588-be8e-d724a557148c",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-02-09T23:22:38.290021+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0007/cha/f1f9ca4a-ba99-4588-be8e-d724a557148c.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0007/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/09c46664-e0db-4d61-bc0d-ffb18fd25f40_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="09c46664-e0db-4d61-bc0d-ffb18fd25f40",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-02-21T16:59:24.352362+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0007/cha/09c46664-e0db-4d61-bc0d-ffb18fd25f40.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0007/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/0fff1082-8293-4218-90e8-acd94d6596d8_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="0fff1082-8293-4218-90e8-acd94d6596d8",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-02-08T17:58:31.701578+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0007/cha/0fff1082-8293-4218-90e8-acd94d6596d8.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0009/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/448e6c7e-8065-4b12-a471-5996c807bbdb_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="448e6c7e-8065-4b12-a471-5996c807bbdb",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-03-04T03:37:21.097682+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0009/cha/448e6c7e-8065-4b12-a471-5996c807bbdb.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0009/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/95f520fc-33e1-4a71-a0bf-e87ed6cbc01b_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="95f520fc-33e1-4a71-a0bf-e87ed6cbc01b",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-02-23T19:21:14.322677+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0009/cha/95f520fc-33e1-4a71-a0bf-e87ed6cbc01b.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0009/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/3770b085-1fe3-4e70-83d8-0b6b586b50f0_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="3770b085-1fe3-4e70-83d8-0b6b586b50f0",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-03-04T05:24:14.117768+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0009/cha/3770b085-1fe3-4e70-83d8-0b6b586b50f0.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0009/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/7c1a1bcf-bc9b-415a-b580-2919bca14a25_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="7c1a1bcf-bc9b-415a-b580-2919bca14a25",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-03-04T05:24:25.072169+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0009/cha/7c1a1bcf-bc9b-415a-b580-2919bca14a25.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0010/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/f42df378-897c-4a5d-a011-eda932ac6501_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="f42df378-897c-4a5d-a011-eda932ac6501",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-02-27T07:51:46.326329+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0010/cha/f42df378-897c-4a5d-a011-eda932ac6501.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0010/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/1b1c997f-1b0b-4534-a5ab-d2be88733644_video.mp4",
        incident_type_id="NO_PED_ZONE",
        incident_uuid="1b1c997f-1b0b-4534-a5ab-d2be88733644",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-03-04T09:56:20.363595+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0010/cha/1b1c997f-1b0b-4534-a5ab-d2be88733644.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0010/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/12d87f0b-98e1-47fb-803e-b3438d1a0d3d_video.mp4",
        incident_type_id="NO_PED_ZONE",
        incident_uuid="12d87f0b-98e1-47fb-803e-b3438d1a0d3d",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-02-24T11:03:06.170955+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0010/cha/12d87f0b-98e1-47fb-803e-b3438d1a0d3d.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0013/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/98f18844-c2e1-401d-8ed2-5683fdf3ea67_video.mp4",
        incident_type_id="PRODUCTION_LINE_DOWN",
        incident_uuid="98f18844-c2e1-401d-8ed2-5683fdf3ea67",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-02-13T09:01:33.745309+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0013/cha/98f18844-c2e1-401d-8ed2-5683fdf3ea67.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0013/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/c8d74253-b9a7-4a77-93c4-92e7b7b03c5c_video.mp4",
        incident_type_id="PRODUCTION_LINE_DOWN",
        incident_uuid="c8d74253-b9a7-4a77-93c4-92e7b7b03c5c",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-02-13T23:06:44.375662+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0013/cha/c8d74253-b9a7-4a77-93c4-92e7b7b03c5c.mp4",
    ),
    Incident(
        camera_uuid="innovate_manufacturing/knoxville/0014/cha",
        video_s3_path="s3://voxel-portal-production/innovate_manufacturing/knoxville/incidents/f25476d6-49b0-4ceb-a685-aa265343ecc8_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="f25476d6-49b0-4ceb-a685-aa265343ecc8",
        organization="INNOVATE_MANUFACTURING",
        zone="KNOXVILLE",
        experimental=False,
        created_at="2023-02-20T01:43:04.060367+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/innovate_manufacturing/knoxville/0014/cha/f25476d6-49b0-4ceb-a685-aa265343ecc8.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0001/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/292c350a-afeb-4138-966e-aff832862dd4_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="292c350a-afeb-4138-966e-aff832862dd4",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-03-04T03:33:22.916942+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0001/cha/292c350a-afeb-4138-966e-aff832862dd4.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0001/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/f7b028c6-7eec-4658-a106-1cdcd6c6cde2_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="f7b028c6-7eec-4658-a106-1cdcd6c6cde2",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-03-04T03:36:46.889131+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0001/cha/f7b028c6-7eec-4658-a106-1cdcd6c6cde2.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0001/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/3bdedecf-23bf-4a02-8310-8d0e912b0f89_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="3bdedecf-23bf-4a02-8310-8d0e912b0f89",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-24T23:45:52.473463+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0001/cha/3bdedecf-23bf-4a02-8310-8d0e912b0f89.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0001/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/736777ea-9fe0-40af-bb83-462a181be690_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="736777ea-9fe0-40af-bb83-462a181be690",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-28T01:03:30.164745+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0001/cha/736777ea-9fe0-40af-bb83-462a181be690.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0001/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/12d6015e-da17-4dc5-b26a-5028464285f6_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="12d6015e-da17-4dc5-b26a-5028464285f6",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-23T17:04:25.518929+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0001/cha/12d6015e-da17-4dc5-b26a-5028464285f6.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0001/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/59764276-1da0-470b-be04-35b85ce88ae1_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="59764276-1da0-470b-be04-35b85ce88ae1",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-03-04T01:08:03.614102+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0001/cha/59764276-1da0-470b-be04-35b85ce88ae1.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0002/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/8ea6d628-2aa9-47cb-a718-b747bdb71a01_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="8ea6d628-2aa9-47cb-a718-b747bdb71a01",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-03-04T02:52:53.024353+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0002/cha/8ea6d628-2aa9-47cb-a718-b747bdb71a01.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0002/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/ea74af84-40f7-4c90-b1b8-49962dd70508_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="ea74af84-40f7-4c90-b1b8-49962dd70508",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-16T03:31:50.486615+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0002/cha/ea74af84-40f7-4c90-b1b8-49962dd70508.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0002/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/6e7d6810-04cd-4333-b0e1-070cdb701d45_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="6e7d6810-04cd-4333-b0e1-070cdb701d45",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-10T03:22:10.943632+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0002/cha/6e7d6810-04cd-4333-b0e1-070cdb701d45.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0002/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/68376c13-5079-4bbd-b36e-4af20362fb88_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="68376c13-5079-4bbd-b36e-4af20362fb88",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-03-03T16:59:31.970584+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0002/cha/68376c13-5079-4bbd-b36e-4af20362fb88.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0002/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/6e047a66-a6d2-4c5f-b4e8-49d73cae3b4c_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="6e047a66-a6d2-4c5f-b4e8-49d73cae3b4c",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-23T17:11:08.628034+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0002/cha/6e047a66-a6d2-4c5f-b4e8-49d73cae3b4c.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0002/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/3883191e-04bc-4d30-92ec-96564bbfd2af_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="3883191e-04bc-4d30-92ec-96564bbfd2af",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-10T05:48:16.210154+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0002/cha/3883191e-04bc-4d30-92ec-96564bbfd2af.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0003/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/5aad81bb-5cb0-4b97-a92f-898f112518a6_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="5aad81bb-5cb0-4b97-a92f-898f112518a6",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-21T23:00:20.989775+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0003/cha/5aad81bb-5cb0-4b97-a92f-898f112518a6.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0003/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/c4f301c1-2a0f-4740-aced-2769dc45fcbd_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="c4f301c1-2a0f-4740-aced-2769dc45fcbd",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-22T01:06:42.967011+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0003/cha/c4f301c1-2a0f-4740-aced-2769dc45fcbd.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0003/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/febfc222-1645-43fd-9df6-28850954f638_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="febfc222-1645-43fd-9df6-28850954f638",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-03-04T17:33:57.288877+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0003/cha/febfc222-1645-43fd-9df6-28850954f638.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0003/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/c43b2b30-d7b5-4b02-accc-22fb2196d26d_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="c43b2b30-d7b5-4b02-accc-22fb2196d26d",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-03-04T17:33:47.439851+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0003/cha/c43b2b30-d7b5-4b02-accc-22fb2196d26d.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0004/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/ee74070d-740b-4208-bf10-ff85efb63cc4_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="ee74070d-740b-4208-bf10-ff85efb63cc4",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-16T03:34:17.113469+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0004/cha/ee74070d-740b-4208-bf10-ff85efb63cc4.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0004/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/04c8260a-0d8b-4fd0-8f6d-1ef2e6fa4984_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="04c8260a-0d8b-4fd0-8f6d-1ef2e6fa4984",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-16T03:41:08.425204+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0004/cha/04c8260a-0d8b-4fd0-8f6d-1ef2e6fa4984.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0004/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/189d795e-72aa-4548-b2ad-a39757b92cb8_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="189d795e-72aa-4548-b2ad-a39757b92cb8",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-23T19:43:49.541439+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0004/cha/189d795e-72aa-4548-b2ad-a39757b92cb8.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0004/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/c619e89a-eaf5-4d62-b9fd-8c7927537061_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="c619e89a-eaf5-4d62-b9fd-8c7927537061",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-22T00:45:08.798724+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0004/cha/c619e89a-eaf5-4d62-b9fd-8c7927537061.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0004/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/fdee68ed-c681-44e3-9c12-a38b06da4957_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="fdee68ed-c681-44e3-9c12-a38b06da4957",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-10T23:54:22.908993+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0004/cha/fdee68ed-c681-44e3-9c12-a38b06da4957.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0004/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/21af0a0d-0ce7-4a3c-8f12-dbf4525e7633_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="21af0a0d-0ce7-4a3c-8f12-dbf4525e7633",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-27T16:32:24.630142+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0004/cha/21af0a0d-0ce7-4a3c-8f12-dbf4525e7633.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0005/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/4e927c97-9c87-471f-9576-8d4ccc51725f_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="4e927c97-9c87-471f-9576-8d4ccc51725f",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-03-04T01:10:52.534790+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0005/cha/4e927c97-9c87-471f-9576-8d4ccc51725f.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0005/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/d09363e0-41e3-4971-a490-1ca3f910cfbb_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="d09363e0-41e3-4971-a490-1ca3f910cfbb",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-03-04T02:39:05.325600+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0005/cha/d09363e0-41e3-4971-a490-1ca3f910cfbb.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0005/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/438e8f8e-2db1-4875-87e1-085a344ccdca_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="438e8f8e-2db1-4875-87e1-085a344ccdca",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-16T00:24:19.488408+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0005/cha/438e8f8e-2db1-4875-87e1-085a344ccdca.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0005/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/6ca5469a-4235-405a-9c3d-ae81810bb49a_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="6ca5469a-4235-405a-9c3d-ae81810bb49a",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-16T01:52:37.333060+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0005/cha/6ca5469a-4235-405a-9c3d-ae81810bb49a.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0005/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/64dac84b-1a79-4ab6-8e1d-c106336dede7_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="64dac84b-1a79-4ab6-8e1d-c106336dede7",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-23T15:51:15.163276+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0005/cha/64dac84b-1a79-4ab6-8e1d-c106336dede7.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0005/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/bd853f0b-9799-4a14-9b63-af44d73638ca_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="bd853f0b-9799-4a14-9b63-af44d73638ca",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-23T17:34:01.293999+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0005/cha/bd853f0b-9799-4a14-9b63-af44d73638ca.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0006/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/0b95a465-a0b7-4434-aab4-fdd6a8611b86_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="0b95a465-a0b7-4434-aab4-fdd6a8611b86",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-03-02T17:51:23.950206+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0006/cha/0b95a465-a0b7-4434-aab4-fdd6a8611b86.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0006/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/c776c4e3-a686-47c6-8da7-5040d1cc97e1_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="c776c4e3-a686-47c6-8da7-5040d1cc97e1",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-10T17:08:44.707072+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0006/cha/c776c4e3-a686-47c6-8da7-5040d1cc97e1.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0007/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/426b7886-0aed-46e2-ac53-7e6a21a033a1_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="426b7886-0aed-46e2-ac53-7e6a21a033a1",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-09T23:36:39.296441+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0007/cha/426b7886-0aed-46e2-ac53-7e6a21a033a1.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0007/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/178941ae-c548-4cd3-acca-a854fad8d844_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="178941ae-c548-4cd3-acca-a854fad8d844",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-07T06:32:04.411357+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0007/cha/178941ae-c548-4cd3-acca-a854fad8d844.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0008/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/0ec2fede-177c-4e58-87a6-0ec89bae8e3e_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="0ec2fede-177c-4e58-87a6-0ec89bae8e3e",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-23T19:01:32.848579+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0008/cha/0ec2fede-177c-4e58-87a6-0ec89bae8e3e.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0008/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/f1b9a27e-d34c-467a-bb43-dd64a3ef55b9_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="f1b9a27e-d34c-467a-bb43-dd64a3ef55b9",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-23T07:18:13.849492+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0008/cha/f1b9a27e-d34c-467a-bb43-dd64a3ef55b9.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0009/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/f8e79842-cda7-4e5f-8087-7f54fe9ec484_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="f8e79842-cda7-4e5f-8087-7f54fe9ec484",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-03-04T02:32:00.134074+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0009/cha/f8e79842-cda7-4e5f-8087-7f54fe9ec484.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0009/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/0e39a2a6-8c1e-498a-a867-11fd098aeddd_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="0e39a2a6-8c1e-498a-a867-11fd098aeddd",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-03-04T03:23:28.885758+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0009/cha/0e39a2a6-8c1e-498a-a867-11fd098aeddd.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0009/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/c19c680c-5cc0-44c7-9bc5-b66602aa0a23_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="c19c680c-5cc0-44c7-9bc5-b66602aa0a23",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-24T18:30:03.812027+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0009/cha/c19c680c-5cc0-44c7-9bc5-b66602aa0a23.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0009/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/e95957d1-f1fe-43bd-a62c-73cacda71405_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="e95957d1-f1fe-43bd-a62c-73cacda71405",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-23T17:13:39.277646+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0009/cha/e95957d1-f1fe-43bd-a62c-73cacda71405.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0009/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/3d50cf64-af09-4bfe-88c4-cb2cb6494a59_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="3d50cf64-af09-4bfe-88c4-cb2cb6494a59",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-23T19:40:03.156972+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0009/cha/3d50cf64-af09-4bfe-88c4-cb2cb6494a59.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0010/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/624103b4-12af-4976-b2a8-39ed54cd1131_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="624103b4-12af-4976-b2a8-39ed54cd1131",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-24T03:16:47.461092+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0010/cha/624103b4-12af-4976-b2a8-39ed54cd1131.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0010/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/5dd7be8f-83d6-43f3-97b3-e4acf3f79f32_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="5dd7be8f-83d6-43f3-97b3-e4acf3f79f32",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-25T09:15:49.268054+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0010/cha/5dd7be8f-83d6-43f3-97b3-e4acf3f79f32.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0011/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/224dd2b2-3779-4b69-84ac-9158ffe5f3c7_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="224dd2b2-3779-4b69-84ac-9158ffe5f3c7",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-10T07:41:59.597212+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0011/cha/224dd2b2-3779-4b69-84ac-9158ffe5f3c7.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0011/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/a4928fb7-acc8-41c9-994e-1912419bccf0_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="a4928fb7-acc8-41c9-994e-1912419bccf0",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-23T18:05:07.242891+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0011/cha/a4928fb7-acc8-41c9-994e-1912419bccf0.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0011/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/f1250a84-72c3-4d44-b84a-fdd0059012c6_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="f1250a84-72c3-4d44-b84a-fdd0059012c6",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-28T02:46:32.517717+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0011/cha/f1250a84-72c3-4d44-b84a-fdd0059012c6.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0012/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/0f281695-e89a-4686-ae98-dfc0f7f411d5_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="0f281695-e89a-4686-ae98-dfc0f7f411d5",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-23T21:15:26.085324+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0012/cha/0f281695-e89a-4686-ae98-dfc0f7f411d5.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0012/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/7b6b254a-04b0-43f6-9c23-2c7260f9cb71_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="7b6b254a-04b0-43f6-9c23-2c7260f9cb71",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-24T00:29:21.245015+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0012/cha/7b6b254a-04b0-43f6-9c23-2c7260f9cb71.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0014/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/7ea0db68-f0da-4789-874d-4f9cc2f0f87d_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="7ea0db68-f0da-4789-874d-4f9cc2f0f87d",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-03-04T17:34:27.235049+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0014/cha/7ea0db68-f0da-4789-874d-4f9cc2f0f87d.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0014/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/2280d4f9-1574-4bbc-97fb-167901d18eb2_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="2280d4f9-1574-4bbc-97fb-167901d18eb2",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-24T20:59:49.714337+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0014/cha/2280d4f9-1574-4bbc-97fb-167901d18eb2.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0015/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/9ce4a62f-7aab-47ba-af25-55f1e580eae5_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="9ce4a62f-7aab-47ba-af25-55f1e580eae5",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-13T16:34:57.895678+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0015/cha/9ce4a62f-7aab-47ba-af25-55f1e580eae5.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0015/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/b4baf119-4eb6-4b58-9813-97b354719a0d_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="b4baf119-4eb6-4b58-9813-97b354719a0d",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-22T23:59:48.281026+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0015/cha/b4baf119-4eb6-4b58-9813-97b354719a0d.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0015/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/1032edac-e308-4458-b646-796c0c07fed6_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="1032edac-e308-4458-b646-796c0c07fed6",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-03-03T16:37:13.440678+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0015/cha/1032edac-e308-4458-b646-796c0c07fed6.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0015/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/379202ad-4eb9-4207-9e64-0672ac9376db_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="379202ad-4eb9-4207-9e64-0672ac9376db",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-03-03T16:37:02.654280+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0015/cha/379202ad-4eb9-4207-9e64-0672ac9376db.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0016/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/680838d0-b818-4b4e-92aa-8324054a0c66_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="680838d0-b818-4b4e-92aa-8324054a0c66",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-09T17:44:47.869393+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0016/cha/680838d0-b818-4b4e-92aa-8324054a0c66.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0016/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/570d2491-f3af-4367-b322-252773899124_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="570d2491-f3af-4367-b322-252773899124",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-09T19:04:38.237786+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0016/cha/570d2491-f3af-4367-b322-252773899124.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0016/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/91620d4e-a0aa-4e10-871f-f9e53e70218b_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="91620d4e-a0aa-4e10-871f-f9e53e70218b",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-20T15:32:38.191490+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0016/cha/91620d4e-a0aa-4e10-871f-f9e53e70218b.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0016/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/d988caf8-a5d2-40fa-9e06-f2f60d63941b_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="d988caf8-a5d2-40fa-9e06-f2f60d63941b",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-22T01:40:40.052420+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0016/cha/d988caf8-a5d2-40fa-9e06-f2f60d63941b.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0016/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/653b5988-1974-4b98-8325-6bf6dbe85c8a_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="653b5988-1974-4b98-8325-6bf6dbe85c8a",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-02-23T16:42:45.422756+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0016/cha/653b5988-1974-4b98-8325-6bf6dbe85c8a.mp4",
    ),
    Incident(
        camera_uuid="wesco/reno/0016/cha",
        video_s3_path="s3://voxel-portal-production/wesco/reno/incidents/eebf5c11-4ad1-4fe9-b553-ae9c2c659cb7_video.mp4",
        incident_type_id="SAFETY_VEST",
        incident_uuid="eebf5c11-4ad1-4fe9-b553-ae9c2c659cb7",
        organization="WESCO",
        zone="RENO",
        experimental=False,
        created_at="2023-03-04T15:19:42.756042+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/wesco/reno/0016/cha/eebf5c11-4ad1-4fe9-b553-ae9c2c659cb7.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0001/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/17d40aef-2ec3-4f89-bce4-183041e16b35_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="17d40aef-2ec3-4f89-bce4-183041e16b35",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-20T17:20:05.283163+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0001/cha/17d40aef-2ec3-4f89-bce4-183041e16b35.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0001/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/8d53a9b4-e490-4e51-936e-cfa954f8bc4f_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="8d53a9b4-e490-4e51-936e-cfa954f8bc4f",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-21T23:22:03.021131+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0001/cha/8d53a9b4-e490-4e51-936e-cfa954f8bc4f.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0001/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/2c67fc8b-84f1-4141-b5da-45055a08c265_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="2c67fc8b-84f1-4141-b5da-45055a08c265",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-07T02:56:38.133008+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0001/cha/2c67fc8b-84f1-4141-b5da-45055a08c265.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0001/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/21031d57-8bfb-488e-bc2e-ad41a55dc2fb_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="21031d57-8bfb-488e-bc2e-ad41a55dc2fb",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-07T00:51:42.301616+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0001/cha/21031d57-8bfb-488e-bc2e-ad41a55dc2fb.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0002/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/29a6457d-29e6-43b8-bdef-ad75417e25ad_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="29a6457d-29e6-43b8-bdef-ad75417e25ad",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-22T01:01:28.575962+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0002/cha/29a6457d-29e6-43b8-bdef-ad75417e25ad.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0002/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/7e4d0f61-fab2-41ca-9eb1-a3504d4a698f_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="7e4d0f61-fab2-41ca-9eb1-a3504d4a698f",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-22T00:24:50.550131+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0002/cha/7e4d0f61-fab2-41ca-9eb1-a3504d4a698f.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0002/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/5e2f7d31-0888-4dda-9561-6f15777f0fa1_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="5e2f7d31-0888-4dda-9561-6f15777f0fa1",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-06T23:39:54.192901+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0002/cha/5e2f7d31-0888-4dda-9561-6f15777f0fa1.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0002/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/5faf7dbd-0dd7-4c94-bf2f-cebfe343066d_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="5faf7dbd-0dd7-4c94-bf2f-cebfe343066d",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-07T17:11:51.736371+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0002/cha/5faf7dbd-0dd7-4c94-bf2f-cebfe343066d.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0003/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/ebbda12b-44f4-420e-9047-eddbbf438c60_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="ebbda12b-44f4-420e-9047-eddbbf438c60",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-22T00:22:37.109192+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0003/cha/ebbda12b-44f4-420e-9047-eddbbf438c60.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0003/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/88e41f96-08a0-4a90-921f-9c0f01bb4304_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="88e41f96-08a0-4a90-921f-9c0f01bb4304",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-22T00:19:18.803369+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0003/cha/88e41f96-08a0-4a90-921f-9c0f01bb4304.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0003/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/1734b486-301d-4b6c-950d-c5e9dc09b360_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="1734b486-301d-4b6c-950d-c5e9dc09b360",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-08T16:22:53.242343+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0003/cha/1734b486-301d-4b6c-950d-c5e9dc09b360.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0003/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/db79330b-a279-4901-b219-8d79a5fb7725_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="db79330b-a279-4901-b219-8d79a5fb7725",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-22T01:33:14.933258+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0003/cha/db79330b-a279-4901-b219-8d79a5fb7725.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0004/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/6abe392f-68a9-4b98-85ed-a996bc132154_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="6abe392f-68a9-4b98-85ed-a996bc132154",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-22T15:49:13.860675+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0004/cha/6abe392f-68a9-4b98-85ed-a996bc132154.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0004/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/bbc3b6b4-07fc-47d5-9886-3b15ed33e4f9_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="bbc3b6b4-07fc-47d5-9886-3b15ed33e4f9",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-06T15:53:55.131187+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0004/cha/bbc3b6b4-07fc-47d5-9886-3b15ed33e4f9.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0004/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/1282ed22-71f4-42d0-9729-62e541d295c1_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="1282ed22-71f4-42d0-9729-62e541d295c1",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-22T17:55:54.390170+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0004/cha/1282ed22-71f4-42d0-9729-62e541d295c1.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0004/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/aa71b998-0bef-4504-986b-8bf2ba814eb4_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="aa71b998-0bef-4504-986b-8bf2ba814eb4",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-07T21:31:23.476420+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0004/cha/aa71b998-0bef-4504-986b-8bf2ba814eb4.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0005/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/6aec93af-96a9-47d6-a751-4713bc0d8d69_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="6aec93af-96a9-47d6-a751-4713bc0d8d69",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-22T00:15:14.719120+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0005/cha/6aec93af-96a9-47d6-a751-4713bc0d8d69.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0005/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/8a5ea95f-7cb9-4347-b809-1e966368671f_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="8a5ea95f-7cb9-4347-b809-1e966368671f",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-22T00:18:34.779765+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0005/cha/8a5ea95f-7cb9-4347-b809-1e966368671f.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0005/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/38428e9b-db4f-4291-a416-d0b5631c3254_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="38428e9b-db4f-4291-a416-d0b5631c3254",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-07T02:49:42.469334+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0005/cha/38428e9b-db4f-4291-a416-d0b5631c3254.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0005/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/7f448188-722f-49dc-9617-3dc2954c5721_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="7f448188-722f-49dc-9617-3dc2954c5721",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-14T01:57:23.093837+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0005/cha/7f448188-722f-49dc-9617-3dc2954c5721.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0007/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/ab5cc8f0-2b40-45dc-9289-0e8758fe8f62_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="ab5cc8f0-2b40-45dc-9289-0e8758fe8f62",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-08T16:32:28.192177+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0007/cha/ab5cc8f0-2b40-45dc-9289-0e8758fe8f62.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0007/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/a964c1df-2876-4079-b509-762b4c3c308b_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="a964c1df-2876-4079-b509-762b4c3c308b",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-08T23:42:14.449519+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0007/cha/a964c1df-2876-4079-b509-762b4c3c308b.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0007/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/d81ed104-8fb3-45c3-86f7-0ae79c1f2c82_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="d81ed104-8fb3-45c3-86f7-0ae79c1f2c82",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-08T17:12:26.561848+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0007/cha/d81ed104-8fb3-45c3-86f7-0ae79c1f2c82.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0007/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/c54db749-859d-442b-b6b5-464ce75679fa_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="c54db749-859d-442b-b6b5-464ce75679fa",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-22T00:41:31.715282+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0007/cha/c54db749-859d-442b-b6b5-464ce75679fa.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0008/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/5db471fc-8b55-4f1f-a851-58214c1dfddb_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="5db471fc-8b55-4f1f-a851-58214c1dfddb",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-15T23:59:50.724586+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0008/cha/5db471fc-8b55-4f1f-a851-58214c1dfddb.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0008/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/558ad4fc-10ca-46eb-aaa9-0aa6ee6657b7_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="558ad4fc-10ca-46eb-aaa9-0aa6ee6657b7",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-13T17:07:12.426847+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0008/cha/558ad4fc-10ca-46eb-aaa9-0aa6ee6657b7.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0008/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/a48e7573-82af-4f1c-9647-c8d0786bd054_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="a48e7573-82af-4f1c-9647-c8d0786bd054",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-15T23:59:58.640493+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0008/cha/a48e7573-82af-4f1c-9647-c8d0786bd054.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0008/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/9c8b0fe1-6220-4a8a-a611-6cacc102dd4b_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="9c8b0fe1-6220-4a8a-a611-6cacc102dd4b",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-15T17:44:08.762719+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0008/cha/9c8b0fe1-6220-4a8a-a611-6cacc102dd4b.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0009/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/1c78b488-4577-4b79-8027-cea2313f8b9d_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="1c78b488-4577-4b79-8027-cea2313f8b9d",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-06T16:39:29.274744+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0009/cha/1c78b488-4577-4b79-8027-cea2313f8b9d.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0009/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/216480f2-0dd8-4503-aa9c-99dd5b4bda21_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="216480f2-0dd8-4503-aa9c-99dd5b4bda21",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-16T17:50:21.520955+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0009/cha/216480f2-0dd8-4503-aa9c-99dd5b4bda21.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0010/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/23e403a4-5721-44d7-9d1b-b95544038b92_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="23e403a4-5721-44d7-9d1b-b95544038b92",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-20T15:35:57.522717+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0010/cha/23e403a4-5721-44d7-9d1b-b95544038b92.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0010/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/983e52d1-2e1b-4ee1-b9d8-f96f46109b31_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="983e52d1-2e1b-4ee1-b9d8-f96f46109b31",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-21T23:59:33.065741+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0010/cha/983e52d1-2e1b-4ee1-b9d8-f96f46109b31.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0010/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/0a7f5d43-409b-4f85-b888-72014dedaaa1_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="0a7f5d43-409b-4f85-b888-72014dedaaa1",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-07T02:25:11.469988+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0010/cha/0a7f5d43-409b-4f85-b888-72014dedaaa1.mp4",
    ),
    Incident(
        camera_uuid="michaels/tracy/0010/cha",
        video_s3_path="s3://voxel-portal-production/michaels/tracy/incidents/2d592902-5a77-49a4-9e3b-c3afe9bab939_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="2d592902-5a77-49a4-9e3b-c3afe9bab939",
        organization="MICHAELS",
        zone="TRACY",
        experimental=False,
        created_at="2023-02-07T00:19:59.459713+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/michaels/tracy/0010/cha/2d592902-5a77-49a4-9e3b-c3afe9bab939.mp4",
    ),
    Incident(
        camera_uuid="office_depot/dallas/0001/cha",
        video_s3_path="s3://voxel-portal-production/office_depot/dallas/incidents/1cfcdb49-9083-49dd-9e42-82f73af0d820_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="1cfcdb49-9083-49dd-9e42-82f73af0d820",
        organization="OFFICE_DEPOT",
        zone="DALLAS",
        experimental=False,
        created_at="2023-02-09T22:28:29.856585+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/office_depot/dallas/0001/cha/1cfcdb49-9083-49dd-9e42-82f73af0d820.mp4",
    ),
    Incident(
        camera_uuid="office_depot/dallas/0001/cha",
        video_s3_path="s3://voxel-portal-production/office_depot/dallas/incidents/11f79a57-aa2b-4648-af34-c64923259996_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="11f79a57-aa2b-4648-af34-c64923259996",
        organization="OFFICE_DEPOT",
        zone="DALLAS",
        experimental=False,
        created_at="2023-02-09T18:42:15.172054+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/office_depot/dallas/0001/cha/11f79a57-aa2b-4648-af34-c64923259996.mp4",
    ),
    Incident(
        camera_uuid="office_depot/dallas/0001/cha",
        video_s3_path="s3://voxel-portal-production/office_depot/dallas/incidents/4b4d52b4-9482-4925-9ffa-d62c4e7b5143_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="4b4d52b4-9482-4925-9ffa-d62c4e7b5143",
        organization="OFFICE_DEPOT",
        zone="DALLAS",
        experimental=False,
        created_at="2023-02-24T19:13:22.441123+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/office_depot/dallas/0001/cha/4b4d52b4-9482-4925-9ffa-d62c4e7b5143.mp4",
    ),
    Incident(
        camera_uuid="office_depot/dallas/0001/cha",
        video_s3_path="s3://voxel-portal-production/office_depot/dallas/incidents/e43ff591-1a47-47ea-8c34-3a06d59d6f14_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="e43ff591-1a47-47ea-8c34-3a06d59d6f14",
        organization="OFFICE_DEPOT",
        zone="DALLAS",
        experimental=False,
        created_at="2023-02-23T01:27:16.113336+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/office_depot/dallas/0001/cha/e43ff591-1a47-47ea-8c34-3a06d59d6f14.mp4",
    ),
    Incident(
        camera_uuid="office_depot/dallas/0002/cha",
        video_s3_path="s3://voxel-portal-production/office_depot/dallas/incidents/64ffdbab-82bb-460d-8804-7ae0722ea22a_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="64ffdbab-82bb-460d-8804-7ae0722ea22a",
        organization="OFFICE_DEPOT",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T01:06:11.710956+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/office_depot/dallas/0002/cha/64ffdbab-82bb-460d-8804-7ae0722ea22a.mp4",
    ),
    Incident(
        camera_uuid="office_depot/dallas/0002/cha",
        video_s3_path="s3://voxel-portal-production/office_depot/dallas/incidents/7b900986-1e16-4550-b34e-de351fa139f2_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="7b900986-1e16-4550-b34e-de351fa139f2",
        organization="OFFICE_DEPOT",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T01:16:10.790856+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/office_depot/dallas/0002/cha/7b900986-1e16-4550-b34e-de351fa139f2.mp4",
    ),
    Incident(
        camera_uuid="office_depot/dallas/0002/cha",
        video_s3_path="s3://voxel-portal-production/office_depot/dallas/incidents/d01ea9b0-4e11-4a19-8283-81559890d613_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="d01ea9b0-4e11-4a19-8283-81559890d613",
        organization="OFFICE_DEPOT",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T01:49:25.229100+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/office_depot/dallas/0002/cha/d01ea9b0-4e11-4a19-8283-81559890d613.mp4",
    ),
    Incident(
        camera_uuid="office_depot/dallas/0002/cha",
        video_s3_path="s3://voxel-portal-production/office_depot/dallas/incidents/493c4c4d-b06e-4244-a0a1-8122e856de8c_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="493c4c4d-b06e-4244-a0a1-8122e856de8c",
        organization="OFFICE_DEPOT",
        zone="DALLAS",
        experimental=False,
        created_at="2023-02-10T00:41:28.753643+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/office_depot/dallas/0002/cha/493c4c4d-b06e-4244-a0a1-8122e856de8c.mp4",
    ),
    Incident(
        camera_uuid="office_depot/dallas/0003/cha",
        video_s3_path="s3://voxel-portal-production/office_depot/dallas/incidents/510c9923-c11f-4819-8be6-e67b64c07cf8_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="510c9923-c11f-4819-8be6-e67b64c07cf8",
        organization="OFFICE_DEPOT",
        zone="DALLAS",
        experimental=False,
        created_at="2023-02-09T20:36:01.358587+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/office_depot/dallas/0003/cha/510c9923-c11f-4819-8be6-e67b64c07cf8.mp4",
    ),
    Incident(
        camera_uuid="office_depot/dallas/0003/cha",
        video_s3_path="s3://voxel-portal-production/office_depot/dallas/incidents/149146ef-fa54-4d09-a667-90a3effb46ac_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="149146ef-fa54-4d09-a667-90a3effb46ac",
        organization="OFFICE_DEPOT",
        zone="DALLAS",
        experimental=False,
        created_at="2023-02-24T01:17:45.906881+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/office_depot/dallas/0003/cha/149146ef-fa54-4d09-a667-90a3effb46ac.mp4",
    ),
    Incident(
        camera_uuid="office_depot/dallas/0003/cha",
        video_s3_path="s3://voxel-portal-production/office_depot/dallas/incidents/cf9723d9-5c09-4bed-b8d0-aa830febf78c_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="cf9723d9-5c09-4bed-b8d0-aa830febf78c",
        organization="OFFICE_DEPOT",
        zone="DALLAS",
        experimental=False,
        created_at="2023-02-24T01:18:00.738857+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/office_depot/dallas/0003/cha/cf9723d9-5c09-4bed-b8d0-aa830febf78c.mp4",
    ),
    Incident(
        camera_uuid="office_depot/dallas/0003/cha",
        video_s3_path="s3://voxel-portal-production/office_depot/dallas/incidents/443bc078-986d-43b3-8220-3cb0572013d8_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="443bc078-986d-43b3-8220-3cb0572013d8",
        organization="OFFICE_DEPOT",
        zone="DALLAS",
        experimental=False,
        created_at="2023-02-25T01:15:02.642182+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/office_depot/dallas/0003/cha/443bc078-986d-43b3-8220-3cb0572013d8.mp4",
    ),
    Incident(
        camera_uuid="office_depot/dallas/0004/cha",
        video_s3_path="s3://voxel-portal-production/office_depot/dallas/incidents/be233282-492d-4d6a-aba1-133c5d794150_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="be233282-492d-4d6a-aba1-133c5d794150",
        organization="OFFICE_DEPOT",
        zone="DALLAS",
        experimental=False,
        created_at="2023-02-20T19:48:06.708516+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/office_depot/dallas/0004/cha/be233282-492d-4d6a-aba1-133c5d794150.mp4",
    ),
    Incident(
        camera_uuid="office_depot/dallas/0004/cha",
        video_s3_path="s3://voxel-portal-production/office_depot/dallas/incidents/18ce82e9-3c99-40d6-b537-b8afcd12ccbb_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="18ce82e9-3c99-40d6-b537-b8afcd12ccbb",
        organization="OFFICE_DEPOT",
        zone="DALLAS",
        experimental=False,
        created_at="2023-02-20T19:51:22.190591+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/office_depot/dallas/0004/cha/18ce82e9-3c99-40d6-b537-b8afcd12ccbb.mp4",
    ),
    Incident(
        camera_uuid="office_depot/dallas/0004/cha",
        video_s3_path="s3://voxel-portal-production/office_depot/dallas/incidents/09b73b20-13f5-4a01-a7a4-8d1c08b1b62b_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="09b73b20-13f5-4a01-a7a4-8d1c08b1b62b",
        organization="OFFICE_DEPOT",
        zone="DALLAS",
        experimental=False,
        created_at="2023-02-24T18:10:56.745954+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/office_depot/dallas/0004/cha/09b73b20-13f5-4a01-a7a4-8d1c08b1b62b.mp4",
    ),
    Incident(
        camera_uuid="office_depot/dallas/0004/cha",
        video_s3_path="s3://voxel-portal-production/office_depot/dallas/incidents/5fdd2ff4-4871-4835-801f-cc6217040afc_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="5fdd2ff4-4871-4835-801f-cc6217040afc",
        organization="OFFICE_DEPOT",
        zone="DALLAS",
        experimental=False,
        created_at="2023-02-15T00:29:13.676425+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/office_depot/dallas/0004/cha/5fdd2ff4-4871-4835-801f-cc6217040afc.mp4",
    ),
    Incident(
        camera_uuid="office_depot/dallas/0005/cha",
        video_s3_path="s3://voxel-portal-production/office_depot/dallas/incidents/f5f3b306-33f8-417a-8cca-34384ddba05e_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="f5f3b306-33f8-417a-8cca-34384ddba05e",
        organization="OFFICE_DEPOT",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T01:06:16.122913+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/office_depot/dallas/0005/cha/f5f3b306-33f8-417a-8cca-34384ddba05e.mp4",
    ),
    Incident(
        camera_uuid="office_depot/dallas/0005/cha",
        video_s3_path="s3://voxel-portal-production/office_depot/dallas/incidents/9a42710b-b8b3-4d9c-a3cb-bc61f009b560_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="9a42710b-b8b3-4d9c-a3cb-bc61f009b560",
        organization="OFFICE_DEPOT",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T01:20:11.030614+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/office_depot/dallas/0005/cha/9a42710b-b8b3-4d9c-a3cb-bc61f009b560.mp4",
    ),
    Incident(
        camera_uuid="office_depot/dallas/0005/cha",
        video_s3_path="s3://voxel-portal-production/office_depot/dallas/incidents/f3b26c07-54ca-40b6-8865-4de4d99f9e48_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="f3b26c07-54ca-40b6-8865-4de4d99f9e48",
        organization="OFFICE_DEPOT",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T02:06:42.676268+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/office_depot/dallas/0005/cha/f3b26c07-54ca-40b6-8865-4de4d99f9e48.mp4",
    ),
    Incident(
        camera_uuid="office_depot/dallas/0005/cha",
        video_s3_path="s3://voxel-portal-production/office_depot/dallas/incidents/c1c732dc-23e7-4268-b623-5b480b9ec21c_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="c1c732dc-23e7-4268-b623-5b480b9ec21c",
        organization="OFFICE_DEPOT",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T02:37:05.408191+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/office_depot/dallas/0005/cha/c1c732dc-23e7-4268-b623-5b480b9ec21c.mp4",
    ),
    Incident(
        camera_uuid="office_depot/dallas/0006/cha",
        video_s3_path="s3://voxel-portal-production/office_depot/dallas/incidents/fbb1ec16-4c68-4cd6-ac00-369fc8de5320_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="fbb1ec16-4c68-4cd6-ac00-369fc8de5320",
        organization="OFFICE_DEPOT",
        zone="DALLAS",
        experimental=False,
        created_at="2023-02-10T07:54:11.537930+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/office_depot/dallas/0006/cha/fbb1ec16-4c68-4cd6-ac00-369fc8de5320.mp4",
    ),
    Incident(
        camera_uuid="office_depot/dallas/0006/cha",
        video_s3_path="s3://voxel-portal-production/office_depot/dallas/incidents/dcf7fe20-f5a6-44d3-a9b9-db9d4e9d3c53_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="dcf7fe20-f5a6-44d3-a9b9-db9d4e9d3c53",
        organization="OFFICE_DEPOT",
        zone="DALLAS",
        experimental=False,
        created_at="2023-02-10T08:27:32.756150+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/office_depot/dallas/0006/cha/dcf7fe20-f5a6-44d3-a9b9-db9d4e9d3c53.mp4",
    ),
    Incident(
        camera_uuid="office_depot/dallas/0006/cha",
        video_s3_path="s3://voxel-portal-production/office_depot/dallas/incidents/dde2e1fa-c8cb-465d-943b-c38cfa2815d6_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="dde2e1fa-c8cb-465d-943b-c38cfa2815d6",
        organization="OFFICE_DEPOT",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-03T11:45:19.635821+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/office_depot/dallas/0006/cha/dde2e1fa-c8cb-465d-943b-c38cfa2815d6.mp4",
    ),
    Incident(
        camera_uuid="office_depot/dallas/0006/cha",
        video_s3_path="s3://voxel-portal-production/office_depot/dallas/incidents/afb317ab-4493-4d64-a6d7-451cc6879335_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="afb317ab-4493-4d64-a6d7-451cc6879335",
        organization="OFFICE_DEPOT",
        zone="DALLAS",
        experimental=False,
        created_at="2023-02-10T07:44:19.292238+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/office_depot/dallas/0006/cha/afb317ab-4493-4d64-a6d7-451cc6879335.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0001/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/f2d30af1-f19b-45c2-b9e2-6394e47545a7_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="f2d30af1-f19b-45c2-b9e2-6394e47545a7",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T14:40:41.019992+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0001/cha/f2d30af1-f19b-45c2-b9e2-6394e47545a7.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0001/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/d6526dc2-d26f-4a60-a5bc-4abc4705227d_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="d6526dc2-d26f-4a60-a5bc-4abc4705227d",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T15:00:39.883264+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0001/cha/d6526dc2-d26f-4a60-a5bc-4abc4705227d.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0001/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/6813cf4f-5ab9-46c1-8831-e70548c470a5_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="6813cf4f-5ab9-46c1-8831-e70548c470a5",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-03T12:40:31.926745+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0001/cha/6813cf4f-5ab9-46c1-8831-e70548c470a5.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0001/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/40ee0e96-612c-460a-a4f9-defcb47ced48_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="40ee0e96-612c-460a-a4f9-defcb47ced48",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-03T12:17:03.324671+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0001/cha/40ee0e96-612c-460a-a4f9-defcb47ced48.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0004/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/90ed0e9c-ea02-4f5d-b1a2-ff25869c56c4_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="90ed0e9c-ea02-4f5d-b1a2-ff25869c56c4",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-03T04:20:26.406734+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0004/cha/90ed0e9c-ea02-4f5d-b1a2-ff25869c56c4.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0004/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/b85b9102-e5bc-4d5f-b47b-2195966ae0ab_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="b85b9102-e5bc-4d5f-b47b-2195966ae0ab",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-03T04:17:32.787434+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0004/cha/b85b9102-e5bc-4d5f-b47b-2195966ae0ab.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0004/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/3e8e5099-9332-4198-854f-b06a19174f3d_video.mp4",
        incident_type_id="NO_PED_ZONE",
        incident_uuid="3e8e5099-9332-4198-854f-b06a19174f3d",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T01:55:22.742436+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0004/cha/3e8e5099-9332-4198-854f-b06a19174f3d.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0004/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/b2c551d5-c291-4b30-aee5-904439274188_video.mp4",
        incident_type_id="NO_PED_ZONE",
        incident_uuid="b2c551d5-c291-4b30-aee5-904439274188",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-03T06:46:30.119755+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0004/cha/b2c551d5-c291-4b30-aee5-904439274188.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0004/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/ab9fa668-3a47-4c91-887a-390137139fec_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="ab9fa668-3a47-4c91-887a-390137139fec",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-03T12:59:37.313221+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0004/cha/ab9fa668-3a47-4c91-887a-390137139fec.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0004/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/9df01a8b-93ec-4fb7-8f30-b483fe327f8d_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="9df01a8b-93ec-4fb7-8f30-b483fe327f8d",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T14:29:52.796114+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0004/cha/9df01a8b-93ec-4fb7-8f30-b483fe327f8d.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0005/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/caa69fbd-1542-47db-b9f2-7f5e0dd341d2_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="caa69fbd-1542-47db-b9f2-7f5e0dd341d2",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-05T03:26:16.629855+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0005/cha/caa69fbd-1542-47db-b9f2-7f5e0dd341d2.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0005/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/b90bfcb4-064a-400a-85ac-35dbda01e59b_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="b90bfcb4-064a-400a-85ac-35dbda01e59b",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-05T03:26:25.379409+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0005/cha/b90bfcb4-064a-400a-85ac-35dbda01e59b.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0005/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/e17eae28-ff26-426d-8fce-a510e14b8b92_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="e17eae28-ff26-426d-8fce-a510e14b8b92",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T09:42:47.802213+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0005/cha/e17eae28-ff26-426d-8fce-a510e14b8b92.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0005/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/6b1655cb-3a3c-426e-8fb1-7740a881fa3e_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="6b1655cb-3a3c-426e-8fb1-7740a881fa3e",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T09:42:55.657967+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0005/cha/6b1655cb-3a3c-426e-8fb1-7740a881fa3e.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0006/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/c5caf04c-d897-4a52-8df4-590b805eb4d9_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="c5caf04c-d897-4a52-8df4-590b805eb4d9",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T13:17:15.536235+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0006/cha/c5caf04c-d897-4a52-8df4-590b805eb4d9.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0006/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/8401417e-70c6-4118-bdb0-18670fdc262f_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="8401417e-70c6-4118-bdb0-18670fdc262f",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T13:17:23.880111+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0006/cha/8401417e-70c6-4118-bdb0-18670fdc262f.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0006/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/b5cabdb0-2689-4bbf-9c11-444a1739e43e_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="b5cabdb0-2689-4bbf-9c11-444a1739e43e",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T02:40:45.006169+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0006/cha/b5cabdb0-2689-4bbf-9c11-444a1739e43e.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0006/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/0014c309-8f01-44ce-a940-26814d9f85df_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="0014c309-8f01-44ce-a940-26814d9f85df",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-03T23:24:32.909937+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0006/cha/0014c309-8f01-44ce-a940-26814d9f85df.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0009/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/d5986859-8857-4cb4-b61a-f98c82669809_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="d5986859-8857-4cb4-b61a-f98c82669809",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-02-26T13:19:20.724294+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0009/cha/d5986859-8857-4cb4-b61a-f98c82669809.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0009/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/ef204804-690e-47cd-b1d7-74bc72fa0554_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="ef204804-690e-47cd-b1d7-74bc72fa0554",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-02-26T13:45:50.945797+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0009/cha/ef204804-690e-47cd-b1d7-74bc72fa0554.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0009/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/b48d30a9-db18-4edf-b826-8ad63fda7aa1_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="b48d30a9-db18-4edf-b826-8ad63fda7aa1",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-03T16:27:24.098647+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0009/cha/b48d30a9-db18-4edf-b826-8ad63fda7aa1.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0009/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/e58591c2-fd0d-42ac-99ef-f7f2a8506dbb_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="e58591c2-fd0d-42ac-99ef-f7f2a8506dbb",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-03T16:27:05.921752+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0009/cha/e58591c2-fd0d-42ac-99ef-f7f2a8506dbb.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0010/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/90ae7aab-a1a2-4d0a-9ff6-6c594e8ff429_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="90ae7aab-a1a2-4d0a-9ff6-6c594e8ff429",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T12:59:46.060696+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0010/cha/90ae7aab-a1a2-4d0a-9ff6-6c594e8ff429.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0010/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/578d7fca-c1bc-408e-b150-03002fa4b548_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="578d7fca-c1bc-408e-b150-03002fa4b548",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T13:00:00.101857+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0010/cha/578d7fca-c1bc-408e-b150-03002fa4b548.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0010/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/5068ec8f-dbae-435f-80c2-c81efa047bbd_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="5068ec8f-dbae-435f-80c2-c81efa047bbd",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-03T23:53:20.861745+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0010/cha/5068ec8f-dbae-435f-80c2-c81efa047bbd.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0010/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/70b44f85-7573-43e0-bfdd-ffd52c484b58_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="70b44f85-7573-43e0-bfdd-ffd52c484b58",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T13:00:21.538414+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0010/cha/70b44f85-7573-43e0-bfdd-ffd52c484b58.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0011/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/bb4037a6-9196-4314-94a4-2375a583e504_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="bb4037a6-9196-4314-94a4-2375a583e504",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T12:56:12.056310+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0011/cha/bb4037a6-9196-4314-94a4-2375a583e504.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0011/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/ba7a8cf9-73cd-4faf-bcc1-daef7151885d_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="ba7a8cf9-73cd-4faf-bcc1-daef7151885d",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T12:12:50.395989+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0011/cha/ba7a8cf9-73cd-4faf-bcc1-daef7151885d.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0011/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/f0ed4397-970c-4d42-8230-6b88960fd013_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="f0ed4397-970c-4d42-8230-6b88960fd013",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T14:06:15.603498+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0011/cha/f0ed4397-970c-4d42-8230-6b88960fd013.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0011/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/be1d9eb8-128a-4b7c-8a6c-94bb769543ef_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="be1d9eb8-128a-4b7c-8a6c-94bb769543ef",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T14:19:38.572772+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0011/cha/be1d9eb8-128a-4b7c-8a6c-94bb769543ef.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0012/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/b3419a5e-266a-4d63-9e0e-089ace410baf_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="b3419a5e-266a-4d63-9e0e-089ace410baf",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-02-27T13:35:31.583861+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0012/cha/b3419a5e-266a-4d63-9e0e-089ace410baf.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0012/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/ed7ba3e4-5f3e-4e93-8b83-c88726421f68_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="ed7ba3e4-5f3e-4e93-8b83-c88726421f68",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-02-27T09:19:15.877632+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0012/cha/ed7ba3e4-5f3e-4e93-8b83-c88726421f68.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0012/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/a8e6a876-620e-4b29-b13e-cd96e2174ce7_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="a8e6a876-620e-4b29-b13e-cd96e2174ce7",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T11:26:28.131664+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0012/cha/a8e6a876-620e-4b29-b13e-cd96e2174ce7.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0012/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/46e1d391-17fc-4b51-9ecd-7a0b401d0209_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="46e1d391-17fc-4b51-9ecd-7a0b401d0209",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-02-26T12:15:34.385423+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0012/cha/46e1d391-17fc-4b51-9ecd-7a0b401d0209.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0014/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/70fc86ac-d15c-4c8a-86e4-be167b54b33e_video.mp4",
        incident_type_id="NO_PED_ZONE",
        incident_uuid="70fc86ac-d15c-4c8a-86e4-be167b54b33e",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T01:07:22.751258+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0014/cha/70fc86ac-d15c-4c8a-86e4-be167b54b33e.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0014/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/f113c143-8008-40a0-a696-295d22dc4ca9_video.mp4",
        incident_type_id="NO_PED_ZONE",
        incident_uuid="f113c143-8008-40a0-a696-295d22dc4ca9",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T01:07:11.561671+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0014/cha/f113c143-8008-40a0-a696-295d22dc4ca9.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0015/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/f0aabc1f-0396-46b8-9449-a3770bbe696f_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="f0aabc1f-0396-46b8-9449-a3770bbe696f",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-02-28T04:21:15.296552+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0015/cha/f0aabc1f-0396-46b8-9449-a3770bbe696f.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0015/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/0bda718e-8a47-4f81-b1c3-ba3877fefb77_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="0bda718e-8a47-4f81-b1c3-ba3877fefb77",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-01T00:37:12.835345+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0015/cha/0bda718e-8a47-4f81-b1c3-ba3877fefb77.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0015/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/5c2d2552-380d-41e2-a51c-e65f6fb3625d_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="5c2d2552-380d-41e2-a51c-e65f6fb3625d",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-02T03:27:59.042358+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0015/cha/5c2d2552-380d-41e2-a51c-e65f6fb3625d.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0015/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/b6b8662a-37bb-4cc5-b12a-7e4acde888a6_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="b6b8662a-37bb-4cc5-b12a-7e4acde888a6",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-02T01:18:06.599563+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0015/cha/b6b8662a-37bb-4cc5-b12a-7e4acde888a6.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0016/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/ed58710c-da69-4607-b3c8-9c07dd775e5b_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="ed58710c-da69-4607-b3c8-9c07dd775e5b",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T01:53:09.693991+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0016/cha/ed58710c-da69-4607-b3c8-9c07dd775e5b.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0016/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/e94a6794-327a-4113-82b3-877a2f6cee1c_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="e94a6794-327a-4113-82b3-877a2f6cee1c",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T14:46:35.119463+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0016/cha/e94a6794-327a-4113-82b3-877a2f6cee1c.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0016/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/bd0f4907-b848-46ef-8ed3-bc8e1627d1d3_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="bd0f4907-b848-46ef-8ed3-bc8e1627d1d3",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T03:16:25.049623+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0016/cha/bd0f4907-b848-46ef-8ed3-bc8e1627d1d3.mp4",
    ),
    Incident(
        camera_uuid="ulta/dallas/0016/cha",
        video_s3_path="s3://voxel-portal-production/ulta/dallas/incidents/1ad40732-b825-4b3c-bb11-832a39496008_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="1ad40732-b825-4b3c-bb11-832a39496008",
        organization="ULTA",
        zone="DALLAS",
        experimental=False,
        created_at="2023-03-04T14:46:53.141591+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/ulta/dallas/0016/cha/1ad40732-b825-4b3c-bb11-832a39496008.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario_bldg_2/0002/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario_bldg_2/incidents/07e2adb4-4b8f-4d8f-aecb-2e012995ddc4_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="07e2adb4-4b8f-4d8f-aecb-2e012995ddc4",
        organization="AMERICOLD",
        zone="ONTARIO_BLDG_2",
        experimental=False,
        created_at="2023-03-06T15:21:12.907398+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario_bldg_2/0002/cha/07e2adb4-4b8f-4d8f-aecb-2e012995ddc4.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario_bldg_2/0002/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario_bldg_2/incidents/9a29e062-26c2-49f9-b1e5-cd1ce97ff0ad_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="9a29e062-26c2-49f9-b1e5-cd1ce97ff0ad",
        organization="AMERICOLD",
        zone="ONTARIO_BLDG_2",
        experimental=False,
        created_at="2023-03-06T23:11:24.878929+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario_bldg_2/0002/cha/9a29e062-26c2-49f9-b1e5-cd1ce97ff0ad.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario_bldg_2/0002/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario_bldg_2/incidents/e3266543-b649-4f6d-af97-162859e2ab78_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="e3266543-b649-4f6d-af97-162859e2ab78",
        organization="AMERICOLD",
        zone="ONTARIO_BLDG_2",
        experimental=False,
        created_at="2023-03-06T23:14:43.381545+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario_bldg_2/0002/cha/e3266543-b649-4f6d-af97-162859e2ab78.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario_bldg_2/0002/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario_bldg_2/incidents/b893c8b3-722c-4657-b68c-c6268d76a05f_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="b893c8b3-722c-4657-b68c-c6268d76a05f",
        organization="AMERICOLD",
        zone="ONTARIO_BLDG_2",
        experimental=False,
        created_at="2023-03-06T21:24:35.486423+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario_bldg_2/0002/cha/b893c8b3-722c-4657-b68c-c6268d76a05f.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario_bldg_2/0003/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario_bldg_2/incidents/16fe35b5-91d2-4b16-a823-1be9efad3ff0_video.mp4",
        incident_type_id="DOOR_VIOLATION",
        incident_uuid="16fe35b5-91d2-4b16-a823-1be9efad3ff0",
        organization="AMERICOLD",
        zone="ONTARIO_BLDG_2",
        experimental=False,
        created_at="2023-03-06T19:06:55.056267+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario_bldg_2/0003/cha/16fe35b5-91d2-4b16-a823-1be9efad3ff0.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario_bldg_2/0003/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario_bldg_2/incidents/91d463f7-e87f-4a00-a411-f12e1f75867d_video.mp4",
        incident_type_id="DOOR_VIOLATION",
        incident_uuid="91d463f7-e87f-4a00-a411-f12e1f75867d",
        organization="AMERICOLD",
        zone="ONTARIO_BLDG_2",
        experimental=False,
        created_at="2023-03-06T19:10:15.492357+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario_bldg_2/0003/cha/91d463f7-e87f-4a00-a411-f12e1f75867d.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario_bldg_2/0003/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario_bldg_2/incidents/7b1a9a67-782c-4663-a941-c5b04de0620e_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="7b1a9a67-782c-4663-a941-c5b04de0620e",
        organization="AMERICOLD",
        zone="ONTARIO_BLDG_2",
        experimental=False,
        created_at="2023-03-06T16:57:55.676161+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario_bldg_2/0003/cha/7b1a9a67-782c-4663-a941-c5b04de0620e.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario_bldg_2/0003/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario_bldg_2/incidents/87940eef-b208-4598-859e-d92f35b5bf03_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="87940eef-b208-4598-859e-d92f35b5bf03",
        organization="AMERICOLD",
        zone="ONTARIO_BLDG_2",
        experimental=False,
        created_at="2023-03-06T16:59:59.527067+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario_bldg_2/0003/cha/87940eef-b208-4598-859e-d92f35b5bf03.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario_bldg_2/0004/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario_bldg_2/incidents/98b91156-e904-4114-929a-de0a81649d3b_video.mp4",
        incident_type_id="BAD_POSTURE",
        incident_uuid="98b91156-e904-4114-929a-de0a81649d3b",
        organization="AMERICOLD",
        zone="ONTARIO_BLDG_2",
        experimental=False,
        created_at="2023-03-06T04:04:27.387173+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario_bldg_2/0004/cha/98b91156-e904-4114-929a-de0a81649d3b.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario_bldg_2/0004/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario_bldg_2/incidents/67451fc6-1e56-4c57-b5f8-326f67badb30_video.mp4",
        incident_type_id="DOOR_VIOLATION",
        incident_uuid="67451fc6-1e56-4c57-b5f8-326f67badb30",
        organization="AMERICOLD",
        zone="ONTARIO_BLDG_2",
        experimental=False,
        created_at="2023-03-06T19:34:21.163376+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario_bldg_2/0004/cha/67451fc6-1e56-4c57-b5f8-326f67badb30.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario_bldg_2/0004/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario_bldg_2/incidents/ec4635f2-b027-407e-8742-e539d4482faa_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="ec4635f2-b027-407e-8742-e539d4482faa",
        organization="AMERICOLD",
        zone="ONTARIO_BLDG_2",
        experimental=False,
        created_at="2023-03-06T12:39:39.458791+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario_bldg_2/0004/cha/ec4635f2-b027-407e-8742-e539d4482faa.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario_bldg_2/0006/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario_bldg_2/incidents/c57f0606-daac-4d4b-8636-06291041c30e_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="c57f0606-daac-4d4b-8636-06291041c30e",
        organization="AMERICOLD",
        zone="ONTARIO_BLDG_2",
        experimental=False,
        created_at="2023-03-06T03:35:41.393909+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario_bldg_2/0006/cha/c57f0606-daac-4d4b-8636-06291041c30e.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario_bldg_2/0006/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario_bldg_2/incidents/66e81b00-6ed9-4a8b-ad9a-34c8fb70a20b_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="66e81b00-6ed9-4a8b-ad9a-34c8fb70a20b",
        organization="AMERICOLD",
        zone="ONTARIO_BLDG_2",
        experimental=False,
        created_at="2023-03-06T03:35:49.272514+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario_bldg_2/0006/cha/66e81b00-6ed9-4a8b-ad9a-34c8fb70a20b.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario_bldg_2/0007/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario_bldg_2/incidents/1d5be8cd-d84f-4544-acb1-bce6099a1d63_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="1d5be8cd-d84f-4544-acb1-bce6099a1d63",
        organization="AMERICOLD",
        zone="ONTARIO_BLDG_2",
        experimental=False,
        created_at="2023-03-06T17:11:59.524822+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario_bldg_2/0007/cha/1d5be8cd-d84f-4544-acb1-bce6099a1d63.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario_bldg_2/0007/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario_bldg_2/incidents/b43b43fc-2384-404c-a2e5-9d24dafff8c1_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="b43b43fc-2384-404c-a2e5-9d24dafff8c1",
        organization="AMERICOLD",
        zone="ONTARIO_BLDG_2",
        experimental=False,
        created_at="2023-03-06T17:45:22.972862+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario_bldg_2/0007/cha/b43b43fc-2384-404c-a2e5-9d24dafff8c1.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario_bldg_2/0008/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario_bldg_2/incidents/2c0ad085-d76f-4f2c-a887-ee069130adfe_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="2c0ad085-d76f-4f2c-a887-ee069130adfe",
        organization="AMERICOLD",
        zone="ONTARIO_BLDG_2",
        experimental=False,
        created_at="2023-03-06T13:23:10.412923+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario_bldg_2/0008/cha/2c0ad085-d76f-4f2c-a887-ee069130adfe.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario_bldg_2/0008/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario_bldg_2/incidents/98f6857c-ae0e-47e6-bfa0-5684a3359fee_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="98f6857c-ae0e-47e6-bfa0-5684a3359fee",
        organization="AMERICOLD",
        zone="ONTARIO_BLDG_2",
        experimental=False,
        created_at="2023-03-06T18:10:00.287138+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario_bldg_2/0008/cha/98f6857c-ae0e-47e6-bfa0-5684a3359fee.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario_bldg_2/0009/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario_bldg_2/incidents/cde79ee3-6ffc-48ab-94f1-869da0506b69_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="cde79ee3-6ffc-48ab-94f1-869da0506b69",
        organization="AMERICOLD",
        zone="ONTARIO_BLDG_2",
        experimental=False,
        created_at="2023-03-06T15:40:34.841083+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario_bldg_2/0009/cha/cde79ee3-6ffc-48ab-94f1-869da0506b69.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario_bldg_2/0009/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario_bldg_2/incidents/a8700568-7e25-4233-a9b7-0866ab273d59_video.mp4",
        incident_type_id="OVERREACHING",
        incident_uuid="a8700568-7e25-4233-a9b7-0866ab273d59",
        organization="AMERICOLD",
        zone="ONTARIO_BLDG_2",
        experimental=False,
        created_at="2023-03-06T17:00:41.750522+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario_bldg_2/0009/cha/a8700568-7e25-4233-a9b7-0866ab273d59.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario_bldg_2/0010/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario_bldg_2/incidents/3e162d24-dce6-4f85-b62a-deed2d7c7470_video.mp4",
        incident_type_id="NO_PED_ZONE",
        incident_uuid="3e162d24-dce6-4f85-b62a-deed2d7c7470",
        organization="AMERICOLD",
        zone="ONTARIO_BLDG_2",
        experimental=False,
        created_at="2023-03-06T16:50:24.749711+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario_bldg_2/0010/cha/3e162d24-dce6-4f85-b62a-deed2d7c7470.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario_bldg_2/0010/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario_bldg_2/incidents/c55883e7-d6ce-4fb9-a9e5-36c29c21ad65_video.mp4",
        incident_type_id="NO_PED_ZONE",
        incident_uuid="c55883e7-d6ce-4fb9-a9e5-36c29c21ad65",
        organization="AMERICOLD",
        zone="ONTARIO_BLDG_2",
        experimental=False,
        created_at="2023-03-06T19:44:04.857850+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario_bldg_2/0010/cha/c55883e7-d6ce-4fb9-a9e5-36c29c21ad65.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario_bldg_2/0010/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario_bldg_2/incidents/926d1a97-2c0d-4979-b31b-772cd194c75f_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="926d1a97-2c0d-4979-b31b-772cd194c75f",
        organization="AMERICOLD",
        zone="ONTARIO_BLDG_2",
        experimental=False,
        created_at="2023-03-06T12:09:20.512171+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario_bldg_2/0010/cha/926d1a97-2c0d-4979-b31b-772cd194c75f.mp4",
    ),
    Incident(
        camera_uuid="americold/ontario_bldg_2/0010/cha",
        video_s3_path="s3://voxel-portal-production/americold/ontario_bldg_2/incidents/29329a03-ec7a-4926-979d-aaf553a3557c_video.mp4",
        incident_type_id="SPILL",
        incident_uuid="29329a03-ec7a-4926-979d-aaf553a3557c",
        organization="AMERICOLD",
        zone="ONTARIO_BLDG_2",
        experimental=False,
        created_at="2023-03-06T15:43:43.242685+00:00",
        feedback_type="valid",
        scenario_type="POSITIVE",
        copied_s3_path="s3://voxel-raw-logs/americold/ontario_bldg_2/0010/cha/29329a03-ec7a-4926-979d-aaf553a3557c.mp4",
    ),
]

In [ ]:
print(len(incidents))

## Scenarios

In [3]:
from core.structs.incident import Incident

scenarios = [
    {
        "camera_uuid": "uscold/laredo/dock01/cha",
        "incidents": [
            "BAD_POSTURE",
            "BAD_POSTURE",
            "BAD_POSTURE",
            "BAD_POSTURE",
        ],
        "video_uuid": "uscold/laredo/dock01/cha/d0d6bed5-3082-45b9-9638-4cb6c5167e15_0000",
        "inferred_incidents": [
            Incident(
                uuid="01295daa-2856-49e1-ad48-b1aebbeb47a6",
                camera_uuid="uscold/laredo/dock01/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["5"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896710332,
                end_frame_relative_ms=1678896710515,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="e854f330-b740-49fa-8ed6-c7bca3ccf207",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            ),
            Incident(
                uuid="41b988c2-137c-4a5f-b99b-a5f0490423f3",
                camera_uuid="uscold/laredo/dock01/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["5"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896710921,
                end_frame_relative_ms=1678896711114,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=True,
                track_uuid="e854f330-b740-49fa-8ed6-c7bca3ccf207",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            ),
            Incident(
                uuid="eb2a3f7d-b122-4e80-aa24-becf699a609f",
                camera_uuid="uscold/laredo/dock01/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["5"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896717871,
                end_frame_relative_ms=1678896718074,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=True,
                track_uuid="e854f330-b740-49fa-8ed6-c7bca3ccf207",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            ),
            Incident(
                uuid="e5b271a6-e6d7-45c1-b157-a298c47987a8",
                camera_uuid="uscold/laredo/dock01/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["5"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896718257,
                end_frame_relative_ms=1678896718869,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=True,
                track_uuid="e854f330-b740-49fa-8ed6-c7bca3ccf207",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            ),
        ],
        "config": {
            "camera_uuid": "uscold/laredo/dock01/cha",
            "ray": {"dashboard": {"port": 5001}},
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/uscold-laredo-0001/1651957466439",
                "version": 5,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "perception": {
                "enabled": True,
                "acausal_layer": {
                    "PPESmoothener": {
                        "time_interval_ms": 10000,
                        "pose_points_min_confidence_threshold": 0.5,
                    }
                },
                "detector_tracker": {
                    "model_path": "artifacts_2021-12-06-00-00-00-0000-yolo/best_608_1024.engine",
                    "vest_model_path": "artifacts_vest_classifier-9-19/vest_classifier_resnet18_traced_model.pth",
                    "height": 608,
                    "width": 1024,
                    "actor2class": {
                        "PERSON": 0,
                        "PIT": 1,
                        "HARD_HAT": 2,
                        "SAFETY_VEST": 3,
                    },
                    "should_do_temporal_propagation": 0,
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_type": "DL",
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                },
                "door_classifier": {
                    "enabled": False,
                    "model_path": "artifacts_door_classifier_resnet50_dataset4/voxel_door_classifier_dataset4_resnet50_traced_model.pth",
                },
                "vest_classifier": {
                    "enabled": False,
                    "model_type": "Transformers",
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit-uscold-laredo_2022-09-15/voxel_safetyvest_vit_laredo-2_laredo_2022-09-15.pth",
                },
                "hat_classifier": {
                    "enabled": False,
                    "model_path": "artifacts_hard_hat_occlusion_vit_v4_2022-11-18-jit/hard_hat_occlusion_vit_v4_2022-11-18-jit.pt",
                    "is_classification_by_detection": False,
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
            },
            "incident": {
                "should_generate_on_cooldown": True,
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "overreaching",
                    "spill",
                    "bad_posture",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "uscold/laredo/dock01/cha/d0d6bed5-3082-45b9-9638-4cb6c5167e15_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/modesto/0001/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "americold/modesto/0001/cha/428e5db8-a973-40db-8ef3-c358043a776e_0000",
        "inferred_incidents": [
            Incident(
                uuid="39283309-a873-4653-8e15-c4b6ea98fe4a",
                camera_uuid="americold/modesto/0001/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["15"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896704305,
                end_frame_relative_ms=1678896704487,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="bf5f222c-b033-4478-a7f1-e10318664a85",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-modesto-0001/1652139618645",
                "version": 3,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/modesto/0001/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 300}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "intersection",
                    "parking",
                    "overreaching",
                    "door_intersection",
                    "door_violation",
                    "open_door",
                    "piggyback",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 480,
                    "model_path": "artifacts_2021-12-06-00-00-00-0000-yolo/best_480_960.engine",
                    "width": 960,
                },
                "door_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_2022-10-26_americold_modesto_0001_cha/2022-10-26_americold_modesto_0001_cha.pt",
                    "model_type": "vanilla_resnet",
                },
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/modesto/0001/cha/428e5db8-a973-40db-8ef3-c358043a776e_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/modesto/0002/cha",
        "incidents": ["BAD_POSTURE", "BAD_POSTURE"],
        "video_uuid": "americold/modesto/0002/cha/a19dd508-2ad2-466e-828c-e7e9a0a0ed7f_0000",
        "inferred_incidents": [
            Incident(
                uuid="8134e4ca-57a5-4459-a5dc-56f0f33864ee",
                camera_uuid="americold/modesto/0002/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["2"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896701930,
                end_frame_relative_ms=1678896702130,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="f91a6dc1-b92b-4690-91b5-5a12c89b098e",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            ),
            Incident(
                uuid="a931c019-496c-4b49-90f7-a6ff268ac4ae",
                camera_uuid="americold/modesto/0002/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["2"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896702501,
                end_frame_relative_ms=1678896702889,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=True,
                track_uuid="f91a6dc1-b92b-4690-91b5-5a12c89b098e",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            ),
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-modesto-0002/1652139759124",
                "version": 3,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/modesto/0002/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 300}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "parking",
                    "overreaching",
                    "door_intersection",
                    "door_violation",
                    "open_door",
                    "piggyback",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 480,
                    "model_path": "artifacts_2021-12-06-00-00-00-0000-yolo/best_480_960.engine",
                    "width": 960,
                },
                "door_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_2022-10-26_americold_modesto_0002_cha/2022-10-26_americold_modesto_0002_cha.pt",
                    "model_type": "vanilla_resnet",
                },
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/modesto/0002/cha/a19dd508-2ad2-466e-828c-e7e9a0a0ed7f_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/modesto/0002/cha",
        "incidents": ["BAD_POSTURE", "BAD_POSTURE", "BAD_POSTURE"],
        "video_uuid": "americold/modesto/0002/cha/fa132036-05a0-448d-adec-29671d765825_0000",
        "inferred_incidents": [
            Incident(
                uuid="4a95e29f-f780-4354-8a28-5ee66734535f",
                camera_uuid="americold/modesto/0002/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896703752,
                end_frame_relative_ms=1678896703972,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="e6b997b4-334a-4a08-9b5d-d6de04c3015f",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            ),
            Incident(
                uuid="88d600b6-1ccd-47ca-9441-3f265ba73fe4",
                camera_uuid="americold/modesto/0002/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896704152,
                end_frame_relative_ms=1678896704376,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=True,
                track_uuid="e6b997b4-334a-4a08-9b5d-d6de04c3015f",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            ),
            Incident(
                uuid="e6e93011-0302-415a-98b5-bea49dc75225",
                camera_uuid="americold/modesto/0002/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896704911,
                end_frame_relative_ms=1678896705122,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=True,
                track_uuid="e6b997b4-334a-4a08-9b5d-d6de04c3015f",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            ),
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-modesto-0002/1652139759124",
                "version": 3,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/modesto/0002/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 300}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "parking",
                    "overreaching",
                    "door_intersection",
                    "door_violation",
                    "open_door",
                    "piggyback",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 480,
                    "model_path": "artifacts_2021-12-06-00-00-00-0000-yolo/best_480_960.engine",
                    "width": 960,
                },
                "door_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_2022-10-26_americold_modesto_0002_cha/2022-10-26_americold_modesto_0002_cha.pt",
                    "model_type": "vanilla_resnet",
                },
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/modesto/0002/cha/fa132036-05a0-448d-adec-29671d765825_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/modesto/0009/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "americold/modesto/0009/cha/b3182dfc-67e1-42be-a8bb-7ee3604ccba2_0000",
        "inferred_incidents": [
            Incident(
                uuid="e18be288-4d9d-4e31-b1ce-16fc78bf7f04",
                camera_uuid="americold/modesto/0009/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896707730,
                end_frame_relative_ms=1678896707919,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="1d1d1d2f-75ea-4fa4-9482-24ba3df4ded4",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-modesto-0009/1652139880560",
                "version": 3,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/modesto/0009/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "overreaching",
                    "door_intersection",
                    "door_violation",
                    "open_door",
                    "piggyback",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 480,
                    "model_path": "artifacts_2021-12-06-00-00-00-0000-yolo/best_480_960.engine",
                    "width": 960,
                },
                "door_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_2022-10-26_americold_modesto_0009_cha/2022-10-26_americold_modesto_0009_cha.pt",
                    "model_type": "vanilla_resnet",
                },
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/modesto/0009/cha/b3182dfc-67e1-42be-a8bb-7ee3604ccba2_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/modesto/0009/cha",
        "incidents": ["BAD_POSTURE", "BAD_POSTURE"],
        "video_uuid": "americold/modesto/0009/cha/3f80116f-12d4-45f4-b5f0-8868994d3877_0000",
        "inferred_incidents": [
            Incident(
                uuid="93b2e80c-79ee-4ee0-abe3-7b16db78f0e5",
                camera_uuid="americold/modesto/0009/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["3"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896733043,
                end_frame_relative_ms=1678896733254,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="90ea3c38-0fdc-425f-9320-876bf9187ed2",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            ),
            Incident(
                uuid="39033d68-8686-4565-b949-7cfcd1085ceb",
                camera_uuid="americold/modesto/0009/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["3"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896738674,
                end_frame_relative_ms=1678896738804,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=True,
                track_uuid="90ea3c38-0fdc-425f-9320-876bf9187ed2",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            ),
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-modesto-0009/1652139880560",
                "version": 3,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/modesto/0009/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "overreaching",
                    "door_intersection",
                    "door_violation",
                    "open_door",
                    "piggyback",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 480,
                    "model_path": "artifacts_2021-12-06-00-00-00-0000-yolo/best_480_960.engine",
                    "width": 960,
                },
                "door_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_2022-10-26_americold_modesto_0009_cha/2022-10-26_americold_modesto_0009_cha.pt",
                    "model_type": "vanilla_resnet",
                },
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/modesto/0009/cha/3f80116f-12d4-45f4-b5f0-8868994d3877_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/modesto/0010/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "americold/modesto/0010/cha/a64f4b9b-1ec1-47b6-b86c-689a1198f3f8_0000",
        "inferred_incidents": [
            Incident(
                uuid="86b649ce-4fd9-4650-b184-9c8e4c2142e7",
                camera_uuid="americold/modesto/0010/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["2"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896740828,
                end_frame_relative_ms=1678896741028,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="2be6d400-02b6-4c22-aa8a-4b6801c1b5a2",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-modesto-0010/1652139922677",
                "version": 5,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/modesto/0010/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 300}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "parking",
                    "overreaching",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 480,
                    "model_path": "artifacts_2021-12-06-00-00-00-0000-yolo/best_480_960.engine",
                    "width": 960,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/modesto/0010/cha/a64f4b9b-1ec1-47b6-b86c-689a1198f3f8_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/modesto/0010/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "americold/modesto/0010/cha/2a72416a-0963-4a1c-b28d-7f5424e2c4d2_0000",
        "inferred_incidents": [
            Incident(
                uuid="456bd334-6e83-488f-ba62-a279b1a65721",
                camera_uuid="americold/modesto/0010/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["2"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896741894,
                end_frame_relative_ms=1678896742094,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="4b184a77-e70e-4348-bc20-ce101ddf0518",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-modesto-0010/1652139922677",
                "version": 5,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/modesto/0010/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 300}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "parking",
                    "overreaching",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 480,
                    "model_path": "artifacts_2021-12-06-00-00-00-0000-yolo/best_480_960.engine",
                    "width": 960,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/modesto/0010/cha/2a72416a-0963-4a1c-b28d-7f5424e2c4d2_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/modesto/0012/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "americold/modesto/0012/cha/65df4905-54ba-4c4b-810e-1d57b8556187_0000",
        "inferred_incidents": [
            Incident(
                uuid="f4dbc028-69db-45dc-a515-6db995a45755",
                camera_uuid="americold/modesto/0012/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["3"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896744707,
                end_frame_relative_ms=1678896744907,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="b87624d7-30db-41f5-ad93-b41181e977f7",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-modesto-0012/1652139947892",
                "version": 4,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/modesto/0012/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 300}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "no_stop_at_aisle_end",
                    "parking",
                    "overreaching",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 480,
                    "model_path": "artifacts_2021-12-06-00-00-00-0000-yolo/best_480_960.engine",
                    "width": 960,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/modesto/0012/cha/65df4905-54ba-4c4b-810e-1d57b8556187_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/modesto/0012/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "americold/modesto/0012/cha/893fab52-bced-425a-9857-60ed5d97052e_0000",
        "inferred_incidents": [
            Incident(
                uuid="e41d37d4-6e26-465d-9f2e-aa86eebcde02",
                camera_uuid="americold/modesto/0012/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["6"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896756521,
                end_frame_relative_ms=1678896756729,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="e1a0853b-2d1d-4a72-b933-49493f9de8ea",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-modesto-0012/1652139947892",
                "version": 4,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/modesto/0012/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 300}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "no_stop_at_aisle_end",
                    "parking",
                    "overreaching",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 480,
                    "model_path": "artifacts_2021-12-06-00-00-00-0000-yolo/best_480_960.engine",
                    "width": 960,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/modesto/0012/cha/893fab52-bced-425a-9857-60ed5d97052e_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/ontario/0003/cha",
        "incidents": ["BAD_POSTURE", "BAD_POSTURE"],
        "video_uuid": "americold/ontario/0003/cha/f83a3263-0b1b-444e-aeee-afbdea256486_0000",
        "inferred_incidents": [
            Incident(
                uuid="c1c99a16-126f-4fab-817b-638b1769898e",
                camera_uuid="americold/ontario/0003/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896769946,
                end_frame_relative_ms=1678896770342,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="3cecd0a6-d308-48bd-b7de-8ec535faa6e6",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            ),
            Incident(
                uuid="4e58758c-de56-4c8e-aa3d-10669189d33e",
                camera_uuid="americold/ontario/0003/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896770540,
                end_frame_relative_ms=1678896770738,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=True,
                track_uuid="3cecd0a6-d308-48bd-b7de-8ec535faa6e6",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            ),
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-ontario-0003/1650319163293",
                "version": 3,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/ontario/0003/cha",
            "incident": {
                "incident_machine_params": {
                    "parking": {"per_camera_cooldown_s": 720},
                    "piggyback": {"max_pit_crossing_time_differences_s": 10},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "door_intersection",
                    "door_violation",
                    "open_door",
                    "overreaching",
                    "parking",
                    "piggyback",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_2022-10-26_americold_ontario_0003_cha/2022-10-26_americold_ontario_0003_cha.pt",
                    "model_type": "vanilla_resnet",
                },
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/ontario/0003/cha/f83a3263-0b1b-444e-aeee-afbdea256486_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/ontario/0003/cha",
        "incidents": ["BAD_POSTURE", "BAD_POSTURE"],
        "video_uuid": "americold/ontario/0003/cha/0d6b743d-e058-4e4e-92f6-e8df8e9b8135_0000",
        "inferred_incidents": [
            Incident(
                uuid="9e5118a8-befe-445f-9e41-44296e6efe2f",
                camera_uuid="americold/ontario/0003/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896769648,
                end_frame_relative_ms=1678896769846,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="7148a839-3fb9-4542-b3fc-bd3fc0adfa87",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            ),
            Incident(
                uuid="9c619ccd-6bee-4390-bd59-052bd224c5b2",
                camera_uuid="americold/ontario/0003/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896777964,
                end_frame_relative_ms=1678896778162,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=True,
                track_uuid="7148a839-3fb9-4542-b3fc-bd3fc0adfa87",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            ),
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-ontario-0003/1650319163293",
                "version": 3,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/ontario/0003/cha",
            "incident": {
                "incident_machine_params": {
                    "parking": {"per_camera_cooldown_s": 720},
                    "piggyback": {"max_pit_crossing_time_differences_s": 10},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "door_intersection",
                    "door_violation",
                    "open_door",
                    "overreaching",
                    "parking",
                    "piggyback",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_2022-10-26_americold_ontario_0003_cha/2022-10-26_americold_ontario_0003_cha.pt",
                    "model_type": "vanilla_resnet",
                },
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/ontario/0003/cha/0d6b743d-e058-4e4e-92f6-e8df8e9b8135_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/ontario/0005/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "americold/ontario/0005/cha/58b319f3-9037-4ab9-aec8-07be8566c782_0000",
        "inferred_incidents": [
            Incident(
                uuid="9bf691f1-20d2-49f2-ba7e-f0d368dd82ee",
                camera_uuid="americold/ontario/0005/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896772380,
                end_frame_relative_ms=1678896772578,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="62ed9d7d-0c0e-4747-8b33-64c991b41a6b",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-ontario-0005/1650319192897",
                "version": 9,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/ontario/0005/cha",
            "incident": {
                "incident_machine_params": {
                    "door_intersection": {"per_camera_cooldown_s": 720},
                    "door_violation": {"per_camera_cooldown_s": 720},
                    "open_door": {"per_camera_cooldown_s": 720},
                    "parking": {"per_camera_cooldown_s": 720},
                    "piggyback": {"max_pit_crossing_time_differences_s": 10},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "overreaching",
                    "door_intersection",
                    "door_violation",
                    "open_door",
                    "piggyback",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_02_15_2023_americold_ontario_0005_cha/54dedcff-fcc9-4521-927b-9c9f3114b320.pt",
                    "model_type": "vanilla_resnet",
                },
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/ontario/0005/cha/58b319f3-9037-4ab9-aec8-07be8566c782_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/ontario/0005/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "americold/ontario/0005/cha/4236fee9-a2e9-420c-a9c0-a86052797e92_0000",
        "inferred_incidents": [
            Incident(
                uuid="40c685ee-664f-47b9-a3cc-224770b03fd4",
                camera_uuid="americold/ontario/0005/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["2"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896792693,
                end_frame_relative_ms=1678896793089,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="34f5f1a5-e178-450b-960b-d0fa8192facd",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-ontario-0005/1650319192897",
                "version": 9,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/ontario/0005/cha",
            "incident": {
                "incident_machine_params": {
                    "door_intersection": {"per_camera_cooldown_s": 720},
                    "door_violation": {"per_camera_cooldown_s": 720},
                    "open_door": {"per_camera_cooldown_s": 720},
                    "parking": {"per_camera_cooldown_s": 720},
                    "piggyback": {"max_pit_crossing_time_differences_s": 10},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "overreaching",
                    "door_intersection",
                    "door_violation",
                    "open_door",
                    "piggyback",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_02_15_2023_americold_ontario_0005_cha/54dedcff-fcc9-4521-927b-9c9f3114b320.pt",
                    "model_type": "vanilla_resnet",
                },
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/ontario/0005/cha/4236fee9-a2e9-420c-a9c0-a86052797e92_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/ontario/0006/cha",
        "incidents": ["BAD_POSTURE", "BAD_POSTURE"],
        "video_uuid": "americold/ontario/0006/cha/bf49adae-6913-4926-a318-debe5fd218fe_0000",
        "inferred_incidents": [
            Incident(
                uuid="77020290-37cc-496a-9245-ea058a134f0f",
                camera_uuid="americold/ontario/0006/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896805730,
                end_frame_relative_ms=1678896805928,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="3385ae26-536f-4c34-b2b4-f43da6482729",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            ),
            Incident(
                uuid="367a0c6c-63e5-4c2e-a5f6-f1734a05384d",
                camera_uuid="americold/ontario/0006/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896806720,
                end_frame_relative_ms=1678896806918,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=True,
                track_uuid="3385ae26-536f-4c34-b2b4-f43da6482729",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            ),
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-ontario-0006/1650390478159",
                "version": 12,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/ontario/0006/cha",
            "incident": {
                "incident_machine_params": {
                    "door_intersection": {"per_camera_cooldown_s": 720},
                    "door_violation": {"per_camera_cooldown_s": 720},
                    "open_door": {"per_camera_cooldown_s": 720},
                    "parking": {"per_camera_cooldown_s": 720},
                    "piggyback": {"max_pit_crossing_time_differences_s": 10},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "parking",
                    "overreaching",
                    "door_intersection",
                    "door_violation",
                    "open_door",
                    "piggyback",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_02_15_2023_americold_ontario_0006_cha/b623a0be-0dbf-40c9-ab03-e3c6f1324d21.pt",
                    "model_type": "vanilla_resnet",
                },
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/ontario/0006/cha/bf49adae-6913-4926-a318-debe5fd218fe_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/ontario/0009/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "americold/ontario/0009/cha/027359a7-e4a5-4609-9572-ad72f331bd1c_0000",
        "inferred_incidents": [
            Incident(
                uuid="926a59ca-b79e-4aa1-b042-a397a73da4ea",
                camera_uuid="americold/ontario/0009/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["4"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896821400,
                end_frame_relative_ms=1678896821600,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="9cb44739-6933-4338-aecd-547d55499f92",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-ontario-0009/1658189363930",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/ontario/0009/cha",
            "incident": {
                "incident_machine_params": {
                    "door_intersection": {"per_camera_cooldown_s": 720},
                    "door_violation": {"per_camera_cooldown_s": 720},
                    "open_door": {"per_camera_cooldown_s": 720},
                    "parking": {"per_camera_cooldown_s": 720},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "parking",
                    "overreaching",
                    "door_violation",
                    "open_door",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_2022-10-26_americold_ontario_0009_cha/2022-10-26_americold_ontario_0009_cha.pt",
                    "model_type": "vanilla_resnet",
                },
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/ontario/0009/cha/027359a7-e4a5-4609-9572-ad72f331bd1c_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/ontario/0009/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "americold/ontario/0009/cha/1d6ba248-8580-4b64-a170-9886190c1e04_0000",
        "inferred_incidents": [
            Incident(
                uuid="baecfb41-d1f6-4f6a-aa1b-ef2bf96374ae",
                camera_uuid="americold/ontario/0009/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896824408,
                end_frame_relative_ms=1678896824608,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="5808dad1-47b1-47b9-8d19-3e6f534f9ece",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-ontario-0009/1658189363930",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/ontario/0009/cha",
            "incident": {
                "incident_machine_params": {
                    "door_intersection": {"per_camera_cooldown_s": 720},
                    "door_violation": {"per_camera_cooldown_s": 720},
                    "open_door": {"per_camera_cooldown_s": 720},
                    "parking": {"per_camera_cooldown_s": 720},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "parking",
                    "overreaching",
                    "door_violation",
                    "open_door",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_2022-10-26_americold_ontario_0009_cha/2022-10-26_americold_ontario_0009_cha.pt",
                    "model_type": "vanilla_resnet",
                },
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/ontario/0009/cha/1d6ba248-8580-4b64-a170-9886190c1e04_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/ontario/0011/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "americold/ontario/0011/cha/e4d75bb0-0c08-44d8-88da-50b85ba87d46_0000",
        "inferred_incidents": [
            Incident(
                uuid="c748e8e7-0d9a-4069-be77-327a952ddc19",
                camera_uuid="americold/ontario/0011/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896810265,
                end_frame_relative_ms=1678896810465,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="b6ff7c57-1ee2-4a9a-b394-5b659ee710d5",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-ontario-0011/1658189408400",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/ontario/0011/cha",
            "incident": {
                "incident_machine_params": {
                    "door_intersection": {"per_camera_cooldown_s": 720},
                    "door_violation": {"per_camera_cooldown_s": 720},
                    "open_door": {"per_camera_cooldown_s": 720},
                    "parking": {"per_camera_cooldown_s": 720},
                    "piggyback": {"max_pit_crossing_time_differences_s": 10},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "parking",
                    "overreaching",
                    "door_intersection",
                    "door_violation",
                    "open_door",
                    "piggyback",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_2022-10-26_americold_ontario_0011_cha/2022-10-26_americold_ontario_0011_cha.pt",
                    "model_type": "vanilla_resnet",
                },
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/ontario/0011/cha/e4d75bb0-0c08-44d8-88da-50b85ba87d46_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/ontario/0011/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "americold/ontario/0011/cha/746cf4e2-93c1-4d47-92c7-15efbcdb0f72_0000",
        "inferred_incidents": [
            Incident(
                uuid="81340e24-aac3-4f7c-ad09-5248be90598f",
                camera_uuid="americold/ontario/0011/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["13"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896839939,
                end_frame_relative_ms=1678896840539,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="8cca5ae4-01f3-41a7-82e2-008f2bc49024",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-ontario-0011/1658189408400",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/ontario/0011/cha",
            "incident": {
                "incident_machine_params": {
                    "door_intersection": {"per_camera_cooldown_s": 720},
                    "door_violation": {"per_camera_cooldown_s": 720},
                    "open_door": {"per_camera_cooldown_s": 720},
                    "parking": {"per_camera_cooldown_s": 720},
                    "piggyback": {"max_pit_crossing_time_differences_s": 10},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "parking",
                    "overreaching",
                    "door_intersection",
                    "door_violation",
                    "open_door",
                    "piggyback",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_2022-10-26_americold_ontario_0011_cha/2022-10-26_americold_ontario_0011_cha.pt",
                    "model_type": "vanilla_resnet",
                },
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/ontario/0011/cha/746cf4e2-93c1-4d47-92c7-15efbcdb0f72_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/ontario/0012/cha",
        "incidents": ["BAD_POSTURE", "BAD_POSTURE", "BAD_POSTURE"],
        "video_uuid": "americold/ontario/0012/cha/d8637b45-f45b-49d4-b514-48a8ffac37ed_0000",
        "inferred_incidents": [
            Incident(
                uuid="2b51e446-82c7-4407-8483-b83833e2748a",
                camera_uuid="americold/ontario/0012/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896841745,
                end_frame_relative_ms=1678896842145,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="1a9b7a41-39b5-4208-aa6e-e68dcd1fa2a2",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            ),
            Incident(
                uuid="4f68675a-7b72-41e9-bae7-917ef441a1bf",
                camera_uuid="americold/ontario/0012/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896857545,
                end_frame_relative_ms=1678896857945,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=True,
                track_uuid="1a9b7a41-39b5-4208-aa6e-e68dcd1fa2a2",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            ),
            Incident(
                uuid="76cfec47-094c-44a0-b7bb-baa72ff2f18e",
                camera_uuid="americold/ontario/0012/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896859145,
                end_frame_relative_ms=1678896859345,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=True,
                track_uuid="1a9b7a41-39b5-4208-aa6e-e68dcd1fa2a2",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            ),
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-ontario-0012/1658189423824",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/ontario/0012/cha",
            "incident": {
                "incident_machine_params": {
                    "door_intersection": {"per_camera_cooldown_s": 720},
                    "door_violation": {"per_camera_cooldown_s": 720},
                    "open_door": {"per_camera_cooldown_s": 720},
                    "parking": {"per_camera_cooldown_s": 720},
                    "piggyback": {"max_pit_crossing_time_differences_s": 10},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "parking",
                    "overreaching",
                    "door_intersection",
                    "door_violation",
                    "open_door",
                    "piggyback",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_2022-10-26_americold_ontario_0012_cha/2022-10-26_americold_ontario_0012_cha.pt",
                    "model_type": "vanilla_resnet",
                },
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/ontario/0012/cha/d8637b45-f45b-49d4-b514-48a8ffac37ed_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "uscold/quakertown/0003/cha",
        "incidents": ["BAD_POSTURE", "BAD_POSTURE"],
        "video_uuid": "uscold/quakertown/0003/cha/5375eb17-97e1-4cb0-b140-3b19a9752a63_0000",
        "inferred_incidents": [
            Incident(
                uuid="10a6fb5e-1eb9-4b7e-ac3b-6e3915d41c3b",
                camera_uuid="uscold/quakertown/0003/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["6"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896873882,
                end_frame_relative_ms=1678896874117,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="8a9b6cb2-6da3-4421-803e-ce71dbf3cbe2",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            ),
            Incident(
                uuid="58d4753b-7711-4dfb-8909-62a421d054ac",
                camera_uuid="uscold/quakertown/0003/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["6"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896875117,
                end_frame_relative_ms=1678896875282,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=True,
                track_uuid="8a9b6cb2-6da3-4421-803e-ce71dbf3cbe2",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            ),
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/uscold-quakertown-0003/1660237262679",
                "version": 7,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "uscold/quakertown/0003/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "hard_hat": {"max_no_hat_s": 3},
                    "parking": {"max_parked_duration_s": 300},
                    "safety_vest": {"per_camera_cooldown_s": 900},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "hard_hat",
                    "overreaching",
                    "parking",
                    "safety_vest",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "enabled": True,
                "acausal_layer": {
                    "PPESmoothener": {
                        "pose_points_min_confidence_threshold": 0.5
                    }
                },
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-10-04-18-20-08-f6eb-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {
                    "enabled": True,
                    "is_classification_by_detection": False,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_hard_hat_occlusion_vit_v4_2022-11-18-jit/hard_hat_occlusion_vit_v4_2022-11-18-jit.pt",
                },
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit_quakertown_laredo_2022-08-24/voxel_safetyvest_vit_quakertown_laredo_2022-08-24.pth",
                    "model_type": "Transformers",
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "uscold/quakertown/0003/cha/5375eb17-97e1-4cb0-b140-3b19a9752a63_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "uscold/quakertown/0003/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "uscold/quakertown/0003/cha/432fa88a-8096-4b0e-857d-7d681de1f7f2_0000",
        "inferred_incidents": [
            Incident(
                uuid="e3b1eafc-c533-42e7-af71-ac2216d72e65",
                camera_uuid="uscold/quakertown/0003/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["6"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896875138,
                end_frame_relative_ms=1678896875371,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="ebf841ac-1903-4347-b6bb-41657ca5efde",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/uscold-quakertown-0003/1660237262679",
                "version": 7,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "uscold/quakertown/0003/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "hard_hat": {"max_no_hat_s": 3},
                    "parking": {"max_parked_duration_s": 300},
                    "safety_vest": {"per_camera_cooldown_s": 900},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "hard_hat",
                    "overreaching",
                    "parking",
                    "safety_vest",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "enabled": True,
                "acausal_layer": {
                    "PPESmoothener": {
                        "pose_points_min_confidence_threshold": 0.5
                    }
                },
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-10-04-18-20-08-f6eb-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {
                    "enabled": True,
                    "is_classification_by_detection": False,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_hard_hat_occlusion_vit_v4_2022-11-18-jit/hard_hat_occlusion_vit_v4_2022-11-18-jit.pt",
                },
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit_quakertown_laredo_2022-08-24/voxel_safetyvest_vit_quakertown_laredo_2022-08-24.pth",
                    "model_type": "Transformers",
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "uscold/quakertown/0003/cha/432fa88a-8096-4b0e-857d-7d681de1f7f2_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "uscold/quakertown/0006/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "uscold/quakertown/0006/cha/f11d408a-cdde-408e-9f27-329a80cb6c1e_0000",
        "inferred_incidents": [
            Incident(
                uuid="54e42cc8-e2c5-4e37-bd4a-59ed0bbc2c08",
                camera_uuid="uscold/quakertown/0006/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["7"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896869232,
                end_frame_relative_ms=1678896869428,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="d214fc83-c155-4255-a1b3-2cf18433439d",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/uscold-quakertown-0006/1660237363650",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "uscold/quakertown/0006/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "hard_hat": {"max_no_hat_s": 3},
                    "safety_vest": {"per_camera_cooldown_s": 900},
                    "no_ped_zone": {
                        "per_camera_cooldown_s": 120,
                        "max_ped_zone_duration_s": 120,
                    },
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "hard_hat",
                    "no_ped_zone",
                    "overreaching",
                    "safety_vest",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "enabled": True,
                "acausal_layer": {
                    "PPESmoothener": {
                        "pose_points_min_confidence_threshold": 0.5
                    }
                },
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-10-04-18-20-08-f6eb-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {
                    "enabled": True,
                    "is_classification_by_detection": False,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_hard_hat_occlusion_vit_v4_2022-11-18-jit/hard_hat_occlusion_vit_v4_2022-11-18-jit.pt",
                },
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit_quakertown_laredo_2022-08-24/voxel_safetyvest_vit_quakertown_laredo_2022-08-24.pth",
                    "model_type": "Transformers",
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "uscold/quakertown/0006/cha/f11d408a-cdde-408e-9f27-329a80cb6c1e_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "uscold/quakertown/0006/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "uscold/quakertown/0006/cha/fc711edb-7df2-4742-8bc2-c0034dfb23eb_0000",
        "inferred_incidents": [
            Incident(
                uuid="35c1535b-e07c-4a49-8d50-af9b8845a41e",
                camera_uuid="uscold/quakertown/0006/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896889854,
                end_frame_relative_ms=1678896890053,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="32f9ad59-9981-40d6-ae6f-afd7a1d47b6c",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/uscold-quakertown-0006/1660237363650",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "uscold/quakertown/0006/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "hard_hat": {"max_no_hat_s": 3},
                    "safety_vest": {"per_camera_cooldown_s": 900},
                    "no_ped_zone": {
                        "per_camera_cooldown_s": 120,
                        "max_ped_zone_duration_s": 120,
                    },
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "hard_hat",
                    "no_ped_zone",
                    "overreaching",
                    "safety_vest",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "enabled": True,
                "acausal_layer": {
                    "PPESmoothener": {
                        "pose_points_min_confidence_threshold": 0.5
                    }
                },
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-10-04-18-20-08-f6eb-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {
                    "enabled": True,
                    "is_classification_by_detection": False,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_hard_hat_occlusion_vit_v4_2022-11-18-jit/hard_hat_occlusion_vit_v4_2022-11-18-jit.pt",
                },
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit_quakertown_laredo_2022-08-24/voxel_safetyvest_vit_quakertown_laredo_2022-08-24.pth",
                    "model_type": "Transformers",
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "uscold/quakertown/0006/cha/fc711edb-7df2-4742-8bc2-c0034dfb23eb_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "uscold/quakertown/0008/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "uscold/quakertown/0008/cha/971c8230-151a-48ed-9983-66eabcb173a2_0000",
        "inferred_incidents": [
            Incident(
                uuid="6e8a0f8c-0f6d-4685-bf5c-97e945f7b8cd",
                camera_uuid="uscold/quakertown/0008/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["21"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896899436,
                end_frame_relative_ms=1678896899634,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="df040723-53ac-4a04-98f8-e7b73e580feb",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/uscold-quakertown-0008/1660237412601",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "uscold/quakertown/0008/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "hard_hat": {"max_no_hat_s": 3},
                    "parking": {"max_parked_duration_s": 300},
                    "safety_vest": {"per_camera_cooldown_s": 900},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "hard_hat",
                    "overreaching",
                    "parking",
                    "safety_vest",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "enabled": True,
                "acausal_layer": {
                    "PPESmoothener": {
                        "pose_points_min_confidence_threshold": 0.5
                    }
                },
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-10-04-18-20-08-f6eb-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {
                    "enabled": True,
                    "is_classification_by_detection": False,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_hard_hat_occlusion_vit_v4_2022-11-18-jit/hard_hat_occlusion_vit_v4_2022-11-18-jit.pt",
                },
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit_quakertown_laredo_2022-08-24/voxel_safetyvest_vit_quakertown_laredo_2022-08-24.pth",
                    "model_type": "Transformers",
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "uscold/quakertown/0008/cha/971c8230-151a-48ed-9983-66eabcb173a2_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "buildersfirstsource/solanabeach/0001/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "buildersfirstsource/solanabeach/0001/cha/b3805fa7-7512-4363-9e08-3b5b978f71fd_0000",
        "inferred_incidents": [
            Incident(
                uuid="c6c2c02f-01d5-470d-9eea-103a62c73689",
                camera_uuid="buildersfirstsource/solanabeach/0001/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896901287,
                end_frame_relative_ms=1678896901487,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="f5520b16-c08f-49a7-a01d-129c874ced04",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/buildersfirstsource-solanabeach-0001/1660612076456",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "buildersfirstsource/solanabeach/0001/cha",
            "incident": {
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 300},
                    "safety_vest": {"per_camera_cooldown_s": 1800},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "intersection",
                    "parking",
                    "overreaching",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-11-09-22-58-28-2d35-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 800,
                    "model_path": "artifacts_voxel_safetyvest_vit_2022-08-30_solana_beach/voxel_safetyvest_vit_2022-08-30.pth",
                    "model_type": "Transformers",
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "buildersfirstsource/solanabeach/0001/cha/b3805fa7-7512-4363-9e08-3b5b978f71fd_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "buildersfirstsource/solanabeach/0002/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "buildersfirstsource/solanabeach/0002/cha/56ef2c26-9112-4d4c-86d8-928d4d4c3688_0000",
        "inferred_incidents": [
            Incident(
                uuid="e078e4f7-f937-42e7-ad55-f9e48a906f49",
                camera_uuid="buildersfirstsource/solanabeach/0002/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["5"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896905835,
                end_frame_relative_ms=1678896906035,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="c7e16665-df01-4f4e-9f74-273efcb16878",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/buildersfirstsource-solanabeach-0002/1660612143858",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "buildersfirstsource/solanabeach/0002/cha",
            "incident": {
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 300},
                    "safety_vest": {"per_camera_cooldown_s": 1800},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "intersection",
                    "parking",
                    "overreaching",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-11-09-22-58-28-2d35-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 800,
                    "model_path": "artifacts_voxel_safetyvest_vit_2022-08-30_solana_beach/voxel_safetyvest_vit_2022-08-30.pth",
                    "model_type": "Transformers",
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "buildersfirstsource/solanabeach/0002/cha/56ef2c26-9112-4d4c-86d8-928d4d4c3688_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "buildersfirstsource/solanabeach/0005/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "buildersfirstsource/solanabeach/0005/cha/644abbf8-4bbc-4489-89c4-a40fb484af0a_0000",
        "inferred_incidents": [
            Incident(
                uuid="3ed2aa65-ffd3-4e6f-9649-fffb037a2ee3",
                camera_uuid="buildersfirstsource/solanabeach/0005/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["4"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896918014,
                end_frame_relative_ms=1678896918214,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="985e1234-b156-4587-91bc-419152842fa2",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/buildersfirstsource-solanabeach-0005/1660612146266",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "buildersfirstsource/solanabeach/0005/cha",
            "incident": {
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 300},
                    "safety_vest": {"per_camera_cooldown_s": 1800},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "intersection",
                    "parking",
                    "overreaching",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-11-09-22-58-28-2d35-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 800,
                    "model_path": "artifacts_voxel_safetyvest_vit_2022-08-30_solana_beach/voxel_safetyvest_vit_2022-08-30.pth",
                    "model_type": "Transformers",
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "buildersfirstsource/solanabeach/0005/cha/644abbf8-4bbc-4489-89c4-a40fb484af0a_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "buildersfirstsource/solanabeach/0006/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "buildersfirstsource/solanabeach/0006/cha/e5b935d0-4cbb-436b-a516-11de19f2eb13_0000",
        "inferred_incidents": [
            Incident(
                uuid="69a02e12-6f62-4c27-89df-a6b67da3a237",
                camera_uuid="buildersfirstsource/solanabeach/0006/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896933686,
                end_frame_relative_ms=1678896933886,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="21cccf24-b80a-41d0-afb0-ef694297cc50",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/buildersfirstsource-solanabeach-0006/1660612147036",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "buildersfirstsource/solanabeach/0006/cha",
            "incident": {
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 300},
                    "safety_vest": {"per_camera_cooldown_s": 1800},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "intersection",
                    "parking",
                    "overreaching",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-11-09-22-58-28-2d35-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 800,
                    "model_path": "artifacts_voxel_safetyvest_vit_2022-08-30_solana_beach/voxel_safetyvest_vit_2022-08-30.pth",
                    "model_type": "Transformers",
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "buildersfirstsource/solanabeach/0006/cha/e5b935d0-4cbb-436b-a516-11de19f2eb13_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "buildersfirstsource/dixieline/0010/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "buildersfirstsource/dixieline/0010/cha/eae44df8-b70d-4551-8f41-fb459a44b517_0000",
        "inferred_incidents": [
            Incident(
                uuid="a5f1f4b3-834d-48fc-89cc-e61651494061",
                camera_uuid="buildersfirstsource/dixieline/0010/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["4"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896941715,
                end_frame_relative_ms=1678896941915,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="1f1b6952-3ec9-4809-9272-71bf29fe631b",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/buildersfirstsource-dixieline-0010/1660685959139",
                "version": 3,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "buildersfirstsource/dixieline/0010/cha",
            "incident": {
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 900},
                    "safety_vest": {"per_camera_cooldown_s": 1800},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "no_ped_zone",
                    "overreaching",
                    "safety_vest",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-11-09-22-58-28-2d35-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit-dixieline_2022-09-16/voxel_safetyvest_vit_dixieline_2022-09-16.pth",
                    "model_type": "Transformers",
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "buildersfirstsource/dixieline/0010/cha/eae44df8-b70d-4551-8f41-fb459a44b517_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "trieagle/ocala/0001/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "trieagle/ocala/0001/cha/2b8b0561-fae6-484e-b267-5724826054a6_0000",
        "inferred_incidents": [
            Incident(
                uuid="9715694f-0506-414e-8c6b-80cec6aaf525",
                camera_uuid="trieagle/ocala/0001/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896935780,
                end_frame_relative_ms=1678896935880,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="d6f25819-2bb4-4447-9cdd-7a9a861e64dc",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/trieagle-ocala-0001/1662508837901",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "trieagle/ocala/0001/cha",
            "incident": {
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "open_door",
                    "overreaching",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-11-09-22-58-28-2d35-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_2022-10-28_trieagle_ocala_0001_cha/2022-10-28_trieagle_ocala_0001_cha.pt",
                    "model_type": "vanilla_resnet",
                },
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "trieagle/ocala/0001/cha/2b8b0561-fae6-484e-b267-5724826054a6_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "trieagle/ocala/0001/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "trieagle/ocala/0001/cha/aeeb3f7b-82b0-4b93-b232-56d39810c57e_0000",
        "inferred_incidents": [
            Incident(
                uuid="4d2f217e-4662-462f-abc5-83119b897eda",
                camera_uuid="trieagle/ocala/0001/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["9"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896941278,
                end_frame_relative_ms=1678896941544,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="5c58074d-ca9d-48ca-9829-6d565a9e07da",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/trieagle-ocala-0001/1662508837901",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "trieagle/ocala/0001/cha",
            "incident": {
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "open_door",
                    "overreaching",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-11-09-22-58-28-2d35-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_2022-10-28_trieagle_ocala_0001_cha/2022-10-28_trieagle_ocala_0001_cha.pt",
                    "model_type": "vanilla_resnet",
                },
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "trieagle/ocala/0001/cha/aeeb3f7b-82b0-4b93-b232-56d39810c57e_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "trieagle/ocala/0003/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "trieagle/ocala/0003/cha/4439b733-0cf9-4853-8b76-fe3d0d6b526c_0000",
        "inferred_incidents": [
            Incident(
                uuid="f8a80d34-ce06-44dd-88f6-6ea74884ec51",
                camera_uuid="trieagle/ocala/0003/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["3"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896942986,
                end_frame_relative_ms=1678896943086,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="5c982b83-ec68-49cf-bd69-a5aa028c403b",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/trieagle-ocala-0003/1662508839362",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "trieagle/ocala/0003/cha",
            "incident": {
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "overreaching",
                    "bad_posture",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-11-09-22-58-28-2d35-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "trieagle/ocala/0003/cha/4439b733-0cf9-4853-8b76-fe3d0d6b526c_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "trieagle/ocala/0004/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "trieagle/ocala/0004/cha/26e57390-2f63-4526-86b9-b96ca7ba3cb1_0000",
        "inferred_incidents": [
            Incident(
                uuid="b8b5e05c-716f-4c78-aace-06cf32fe6651",
                camera_uuid="trieagle/ocala/0004/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["3"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896963896,
                end_frame_relative_ms=1678896963996,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="2ba57610-5d3b-44b4-807e-899656635c96",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/trieagle-ocala-0004/1662508840078",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "trieagle/ocala/0004/cha",
            "incident": {
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "overreaching",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-11-09-22-58-28-2d35-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "trieagle/ocala/0004/cha/26e57390-2f63-4526-86b9-b96ca7ba3cb1_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "trieagle/ocala/0007/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "trieagle/ocala/0007/cha/0d7bc0b8-8a4c-4aa7-a544-f1911242a182_0000",
        "inferred_incidents": [
            Incident(
                uuid="a624c779-5cc8-487f-bf7f-6d3f095e111a",
                camera_uuid="trieagle/ocala/0007/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896979409,
                end_frame_relative_ms=1678896979676,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="f640c66d-1dc6-49ac-8b84-25f698624c9a",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/trieagle-ocala-0007/1668723303083",
                "version": 6,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "trieagle/ocala/0007/cha",
            "incident": {
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 120}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "no_stop_at_aisle_end",
                    "overreaching",
                    "parking",
                    "spill",
                    "safety_vest",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-11-09-22-58-28-2d35-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 800,
                    "model_path": "artifacts_voxel_safetyvest_vit-lakeshore-arlington_20022-09-08/voxel_safetyvest_vit_arlington_2022-09-08.pth",
                    "model_type": "Transformers",
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "trieagle/ocala/0007/cha/0d7bc0b8-8a4c-4aa7-a544-f1911242a182_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "trieagle/ocala/0008/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "trieagle/ocala/0008/cha/746daaae-eaa6-4263-84bb-b945d50c83d0_0000",
        "inferred_incidents": [
            Incident(
                uuid="6ff1d17b-313c-4894-97ec-f0bd7d595e8d",
                camera_uuid="trieagle/ocala/0008/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["2"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896979759,
                end_frame_relative_ms=1678896979993,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="2a158de0-85ae-48b9-a426-1cf90920654a",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/trieagle-ocala-0008/1668723343331",
                "version": 6,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "trieagle/ocala/0008/cha",
            "incident": {
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 120}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "no_stop_at_aisle_end",
                    "overreaching",
                    "parking",
                    "spill",
                    "safety_vest",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-11-09-22-58-28-2d35-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 800,
                    "model_path": "artifacts_voxel_safetyvest_vit-lakeshore-arlington_20022-09-08/voxel_safetyvest_vit_arlington_2022-09-08.pth",
                    "model_type": "Transformers",
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "trieagle/ocala/0008/cha/746daaae-eaa6-4263-84bb-b945d50c83d0_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "verst/walton/0003/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "verst/walton/0003/cha/e3bbf324-58f9-40e6-a211-c5a81ba67f7a_0000",
        "inferred_incidents": [
            Incident(
                uuid="86d7c197-0c7c-49e0-900d-f2a758256ea6",
                camera_uuid="verst/walton/0003/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["5"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896981027,
                end_frame_relative_ms=1678896981217,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="ba5ad7d9-304c-487e-ba4d-4f7dc1a86f03",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/verst-walton-0003/1663349394497",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "verst/walton/0003/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "safety_vest": {"per_camera_cooldown_s": 1800}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "intersection",
                    "no_ped_zone",
                    "safety_vest",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-10-04-18-20-08-f6eb-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vverst-walton_2022-10-06/voxel_safetyvest_vit_laredo_walton_2022-10-05.pth",
                    "model_type": "Transformers",
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "verst/walton/0003/cha/e3bbf324-58f9-40e6-a211-c5a81ba67f7a_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "verst/walton/0004/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "verst/walton/0004/cha/f5df95b1-4926-4102-904e-45419c6d2545_0000",
        "inferred_incidents": [
            Incident(
                uuid="8d986c36-af4c-4a8b-adbb-aa98bad0d054",
                camera_uuid="verst/walton/0004/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["2"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896987626,
                end_frame_relative_ms=1678896987787,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="b9712c0e-26ea-40fa-9149-d2db63529e07",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/verst-walton-0004/1663349395203",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "verst/walton/0004/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "monitors_requested": [],
                "incident_machine_params": {
                    "safety_vest": {"per_camera_cooldown_s": 1800}
                },
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "intersection",
                    "safety_vest",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-10-04-18-20-08-f6eb-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vverst-walton_2022-10-06/voxel_safetyvest_vit_laredo_walton_2022-10-05.pth",
                    "model_type": "Transformers",
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "verst/walton/0004/cha/f5df95b1-4926-4102-904e-45419c6d2545_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "verst/walton/0004/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "verst/walton/0004/cha/89ae9955-4468-4e49-83b5-75a82184a730_0000",
        "inferred_incidents": [
            Incident(
                uuid="fb428d53-2c3f-4e40-a893-2d6bff02e945",
                camera_uuid="verst/walton/0004/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["7"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678896997979,
                end_frame_relative_ms=1678896998149,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="f7d27ca0-bafb-4fea-9563-f6437dc799b0",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/verst-walton-0004/1663349395203",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "verst/walton/0004/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "monitors_requested": [],
                "incident_machine_params": {
                    "safety_vest": {"per_camera_cooldown_s": 1800}
                },
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "intersection",
                    "safety_vest",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-10-04-18-20-08-f6eb-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vverst-walton_2022-10-06/voxel_safetyvest_vit_laredo_walton_2022-10-05.pth",
                    "model_type": "Transformers",
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "verst/walton/0004/cha/89ae9955-4468-4e49-83b5-75a82184a730_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "verst/walton/0005/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "verst/walton/0005/cha/bcba5067-1f0c-4acc-80e7-b98739e07cff_0000",
        "inferred_incidents": [
            Incident(
                uuid="e9b15d22-853e-46f0-9ed3-5209dd2c7a1f",
                camera_uuid="verst/walton/0005/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["9"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897016735,
                end_frame_relative_ms=1678897016935,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="97877c38-c412-4571-872c-1fea8808318c",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/verst-walton-0005/1663349395917",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "verst/walton/0005/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "monitors_requested": [],
                "incident_machine_params": {
                    "safety_vest": {"per_camera_cooldown_s": 1800}
                },
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "no_ped_zone",
                    "safety_vest",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-10-04-18-20-08-f6eb-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vverst-walton_2022-10-06/voxel_safetyvest_vit_laredo_walton_2022-10-05.pth",
                    "model_type": "Transformers",
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "verst/walton/0005/cha/bcba5067-1f0c-4acc-80e7-b98739e07cff_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "verst/walton/0005/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "verst/walton/0005/cha/d6075743-6824-4936-a1f2-38ebd9a57dff_0000",
        "inferred_incidents": [
            Incident(
                uuid="0a8431b1-8a67-4262-9b33-6fe731731157",
                camera_uuid="verst/walton/0005/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["10"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897013236,
                end_frame_relative_ms=1678897013436,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="daf899dd-3fd3-4754-8f3c-6d8c47da21da",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/verst-walton-0005/1663349395917",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "verst/walton/0005/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "monitors_requested": [],
                "incident_machine_params": {
                    "safety_vest": {"per_camera_cooldown_s": 1800}
                },
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "no_ped_zone",
                    "safety_vest",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-10-04-18-20-08-f6eb-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vverst-walton_2022-10-06/voxel_safetyvest_vit_laredo_walton_2022-10-05.pth",
                    "model_type": "Transformers",
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "verst/walton/0005/cha/d6075743-6824-4936-a1f2-38ebd9a57dff_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "verst/walton/0011/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "verst/walton/0011/cha/923074a0-2fd5-4040-8305-be4a3a2ba729_0000",
        "inferred_incidents": [
            Incident(
                uuid="5a6ce2e1-790d-489e-8390-46478f0e2b0d",
                camera_uuid="verst/walton/0011/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897014964,
                end_frame_relative_ms=1678897015165,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="de87ebf8-5bcf-4997-93ce-adaae376ff1f",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/verst-walton-0011/1663349400258",
                "version": 7,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "verst/walton/0011/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "safety_vest": {"per_camera_cooldown_s": 1800}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "intersection",
                    "no_stop_at_aisle_end",
                    "safety_vest",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-10-04-18-20-08-f6eb-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vverst-walton_2022-10-06/voxel_safetyvest_vit_laredo_walton_2022-10-05.pth",
                    "model_type": "Transformers",
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "verst/walton/0011/cha/923074a0-2fd5-4040-8305-be4a3a2ba729_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "verst/walton/0011/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "verst/walton/0011/cha/07876c05-88c3-49cc-93dc-d1fcdf32ec4f_0000",
        "inferred_incidents": [
            Incident(
                uuid="64cd50a7-6a43-4654-9707-8294d5838b9d",
                camera_uuid="verst/walton/0011/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["2"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897021141,
                end_frame_relative_ms=1678897021530,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="93940a7e-853b-4698-94b8-04e3b341a0be",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/verst-walton-0011/1663349400258",
                "version": 7,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "verst/walton/0011/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "safety_vest": {"per_camera_cooldown_s": 1800}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "intersection",
                    "no_stop_at_aisle_end",
                    "safety_vest",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-10-04-18-20-08-f6eb-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vverst-walton_2022-10-06/voxel_safetyvest_vit_laredo_walton_2022-10-05.pth",
                    "model_type": "Transformers",
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "verst/walton/0011/cha/07876c05-88c3-49cc-93dc-d1fcdf32ec4f_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "wn_foods/hayward/0006/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "wn_foods/hayward/0006/cha/fd9d9d3e-9555-4ba7-9e0d-a21031da3eb0_0000",
        "inferred_incidents": [
            Incident(
                uuid="4d889043-8354-4a72-8fd2-628bc353aaec",
                camera_uuid="wn_foods/hayward/0006/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["3"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897039783,
                end_frame_relative_ms=1678897039982,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="9e1f84fe-79c6-4467-ae5b-efde6b7de643",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/wn_foods-hayward-0006/1668290512116",
                "version": 3,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "wn_foods/hayward/0006/cha",
            "incident": {
                "incident_machine_params": {
                    "production_line_down": {"per_actor_cooldown_s": 600}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "overreaching",
                    "production_line_down",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-11-15-03-38-16-91db-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "motion_zone_detection": {
                    "enabled": True,
                    "gaussian_kernel_size": 3,
                    "variance_threshold": 60,
                    "bg_frames_history": 5,
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "wn_foods/hayward/0006/cha/fd9d9d3e-9555-4ba7-9e0d-a21031da3eb0_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "wn_foods/hayward/0006/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "wn_foods/hayward/0006/cha/1abe0618-dece-42c0-a518-75ae12eaf3e5_0000",
        "inferred_incidents": [
            Incident(
                uuid="1771dcc9-f8a1-495e-af1c-577f9217c246",
                camera_uuid="wn_foods/hayward/0006/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["6"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897048709,
                end_frame_relative_ms=1678897048909,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="c6cfd185-3532-4846-8dc7-7699c2166d64",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/wn_foods-hayward-0006/1668290512116",
                "version": 3,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "wn_foods/hayward/0006/cha",
            "incident": {
                "incident_machine_params": {
                    "production_line_down": {"per_actor_cooldown_s": 600}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "overreaching",
                    "production_line_down",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-11-15-03-38-16-91db-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "motion_zone_detection": {
                    "enabled": True,
                    "gaussian_kernel_size": 3,
                    "variance_threshold": 60,
                    "bg_frames_history": 5,
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "wn_foods/hayward/0006/cha/1abe0618-dece-42c0-a518-75ae12eaf3e5_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "wn_foods/hayward/0007/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "wn_foods/hayward/0007/cha/9f85d7ae-cef7-4ef3-95c0-bb3f296f96a3_0000",
        "inferred_incidents": [
            Incident(
                uuid="6d6b8fe8-cd5e-4093-8361-06b0f2a54ddd",
                camera_uuid="wn_foods/hayward/0007/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["3"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897049919,
                end_frame_relative_ms=1678897050325,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="88105cf8-2f19-4d72-bef6-c649a2553443",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/wn_foods-hayward-0007/1671170934391",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "wn_foods/hayward/0007/cha",
            "incident": {
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "no_stop_at_aisle_end",
                    "overreaching",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-11-15-03-38-16-91db-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "wn_foods/hayward/0007/cha/9f85d7ae-cef7-4ef3-95c0-bb3f296f96a3_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "wn_foods/hayward/0009/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "wn_foods/hayward/0009/cha/fff0abdf-b83c-4593-8b89-1c061139395b_0000",
        "inferred_incidents": [
            Incident(
                uuid="dae6dd7e-8878-4e80-b159-224219ea72b9",
                camera_uuid="wn_foods/hayward/0009/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897065028,
                end_frame_relative_ms=1678897065595,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="fdf17d60-e1ef-4800-840c-43d20ca13de5",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/wn_foods-hayward-0009/1671170935813",
                "version": 7,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "wn_foods/hayward/0009/cha",
            "incident": {
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 300}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "door_violation",
                    "open_door",
                    "overreaching",
                    "parking",
                    "spill",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-11-15-03-38-16-91db-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_01_09_2023_wn_foods_hayward_0009_cha/8380802c-969f-44a0-b909-72c83fa2bb66.pt",
                    "model_type": "vanilla_resnet",
                },
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "wn_foods/hayward/0009/cha/fff0abdf-b83c-4593-8b89-1c061139395b_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/tacoma/0001/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "americold/tacoma/0001/cha/e16d4093-c0cd-4578-9f72-e8551cb62260_0000",
        "inferred_incidents": [
            Incident(
                uuid="88ff3243-4ba1-438b-83f4-1893f4e73d62",
                camera_uuid="americold/tacoma/0001/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897057170,
                end_frame_relative_ms=1678897057337,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="e9803770-8152-42c0-9187-c487e7895695",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-tacoma-0001/1665468395801",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/tacoma/0001/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "overreaching",
                    "parking",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/tacoma/0001/cha/e16d4093-c0cd-4578-9f72-e8551cb62260_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/tacoma/0001/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "americold/tacoma/0001/cha/effe5656-7097-41ea-8576-02ba1ac5f38e_0000",
        "inferred_incidents": [
            Incident(
                uuid="b5f1cd2d-2180-40db-922d-c5e722184e99",
                camera_uuid="americold/tacoma/0001/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897073065,
                end_frame_relative_ms=1678897073265,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="7e24542d-2073-4653-9890-2045882b15a6",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-tacoma-0001/1665468395801",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/tacoma/0001/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "overreaching",
                    "parking",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/tacoma/0001/cha/effe5656-7097-41ea-8576-02ba1ac5f38e_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/tacoma/0002/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "americold/tacoma/0002/cha/c4bcb6e3-ec5f-40f1-a6c4-da732d640721_0000",
        "inferred_incidents": [
            Incident(
                uuid="213313ac-08b8-4c54-a482-2adbf0b59cb5",
                camera_uuid="americold/tacoma/0002/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897079283,
                end_frame_relative_ms=1678897079483,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="5abd39ea-eb65-4703-bc9e-058f638179eb",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-tacoma-0002/1665468396516",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/tacoma/0002/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "intersection",
                    "overreaching",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/tacoma/0002/cha/c4bcb6e3-ec5f-40f1-a6c4-da732d640721_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/tacoma/0002/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "americold/tacoma/0002/cha/107e3a14-004c-4cfc-a77c-8323831ca42b_0000",
        "inferred_incidents": [
            Incident(
                uuid="ec27aa48-9ad2-4372-9f8a-a9060c9d71f5",
                camera_uuid="americold/tacoma/0002/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["19"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897086326,
                end_frame_relative_ms=1678897086526,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="a2829a8f-b17a-46e9-832c-098c56975f30",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-tacoma-0002/1665468396516",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/tacoma/0002/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "intersection",
                    "overreaching",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/tacoma/0002/cha/107e3a14-004c-4cfc-a77c-8323831ca42b_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/tacoma/0003/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "americold/tacoma/0003/cha/794d8d64-1fbe-4cb4-8b9e-427c1ff68d7b_0000",
        "inferred_incidents": [
            Incident(
                uuid="0e8f2abc-2a5d-40a8-8604-94ab33af4587",
                camera_uuid="americold/tacoma/0003/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["9"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897087784,
                end_frame_relative_ms=1678897087984,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="17d883cb-bc80-44c8-9590-c70cc8a2a0ab",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-tacoma-0003/1665468397231",
                "version": 7,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/tacoma/0003/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 300}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "intersection",
                    "overreaching",
                    "parking",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/tacoma/0003/cha/794d8d64-1fbe-4cb4-8b9e-427c1ff68d7b_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/tacoma/0003/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "americold/tacoma/0003/cha/8b903052-c629-46e7-8b00-a24364dd41e9_0000",
        "inferred_incidents": [
            Incident(
                uuid="56050bea-108f-4692-abb0-0288944147e4",
                camera_uuid="americold/tacoma/0003/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["13"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897100006,
                end_frame_relative_ms=1678897100206,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="66a9fb4e-33f7-48f2-8cb8-8ae41eb5448e",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-tacoma-0003/1665468397231",
                "version": 7,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/tacoma/0003/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 300}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "intersection",
                    "overreaching",
                    "parking",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/tacoma/0003/cha/8b903052-c629-46e7-8b00-a24364dd41e9_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/tacoma/0004/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "americold/tacoma/0004/cha/5356a386-99c2-4b1d-874c-ef3d53c2f7f2_0000",
        "inferred_incidents": [
            Incident(
                uuid="8e79cbb4-3319-41f9-82cd-b49fde877634",
                camera_uuid="americold/tacoma/0004/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897106690,
                end_frame_relative_ms=1678897106890,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="b319914c-f2ff-4e48-aca8-04ef4828ecd4",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-tacoma-0004/1665468397945",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/tacoma/0004/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "open_door",
                    "overreaching",
                    "parking",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_2022-10-25-T19-22americold_tacoma_0004_cha/2022-10-25-T19-22americold_tacoma_0004_cha.pt",
                    "model_type": "vanilla_resnet",
                },
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/tacoma/0004/cha/5356a386-99c2-4b1d-874c-ef3d53c2f7f2_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/tacoma/0005/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "americold/tacoma/0005/cha/aaae43f0-9cef-4ec8-a43d-f23d3b12f55b_0000",
        "inferred_incidents": [
            Incident(
                uuid="952b5e25-e35c-415e-a255-248023501173",
                camera_uuid="americold/tacoma/0005/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["2"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897106061,
                end_frame_relative_ms=1678897106261,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="7152832b-9327-4741-9174-e44a168d40b0",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-tacoma-0005/1665468398648",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/tacoma/0005/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "door_intersection",
                    "open_door",
                    "overreaching",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_02_15_2023_americold_tacoma_0005_cha/8fcfdb4e-f2aa-476d-b2e5-f7347755d1f2.pt",
                    "model_type": "vanilla_resnet",
                },
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/tacoma/0005/cha/aaae43f0-9cef-4ec8-a43d-f23d3b12f55b_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/tacoma/0005/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "americold/tacoma/0005/cha/02d27dbf-23a0-40b1-9613-163deb9a12b5_0000",
        "inferred_incidents": [
            Incident(
                uuid="42311060-a376-41b9-ba97-4052bcac5416",
                camera_uuid="americold/tacoma/0005/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["4"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897114326,
                end_frame_relative_ms=1678897114526,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="bff45934-f914-4e66-9b52-b2331b79246c",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-tacoma-0005/1665468398648",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/tacoma/0005/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "door_intersection",
                    "open_door",
                    "overreaching",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_02_15_2023_americold_tacoma_0005_cha/8fcfdb4e-f2aa-476d-b2e5-f7347755d1f2.pt",
                    "model_type": "vanilla_resnet",
                },
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/tacoma/0005/cha/02d27dbf-23a0-40b1-9613-163deb9a12b5_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/tacoma/0006/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "americold/tacoma/0006/cha/40cfba95-5ae2-4c06-8ba8-35906ef67fdf_0000",
        "inferred_incidents": [
            Incident(
                uuid="1c03996b-9347-4043-8a22-82239d6f193d",
                camera_uuid="americold/tacoma/0006/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["6"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897114286,
                end_frame_relative_ms=1678897114486,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="205ac707-a2bf-4b1e-9979-b57221209eda",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-tacoma-0006/1665468399350",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/tacoma/0006/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "overreaching",
                    "parking",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/tacoma/0006/cha/40cfba95-5ae2-4c06-8ba8-35906ef67fdf_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/tacoma/0007/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "americold/tacoma/0007/cha/1553c06e-9a44-4c17-975a-e10b00c08459_0000",
        "inferred_incidents": [
            Incident(
                uuid="239788ac-bcae-4f1b-a9e8-7c921db93250",
                camera_uuid="americold/tacoma/0007/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897129470,
                end_frame_relative_ms=1678897129670,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="152421a9-913c-4f23-a549-9363f52ebe83",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-tacoma-0007/1667239958547",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/tacoma/0007/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "overreaching",
                    "parking",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/tacoma/0007/cha/1553c06e-9a44-4c17-975a-e10b00c08459_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "americold/tacoma/0007/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "americold/tacoma/0007/cha/054127c3-6685-42ac-be1d-d29e969b116e_0000",
        "inferred_incidents": [
            Incident(
                uuid="5734d1ea-bb12-4f92-bbcb-35b38bf6f91b",
                camera_uuid="americold/tacoma/0007/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["4"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897136813,
                end_frame_relative_ms=1678897137013,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="8c27cb94-53ca-4ad9-b97d-fa50e42ece1e",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/americold-tacoma-0007/1667239958547",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "americold/tacoma/0007/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "overreaching",
                    "parking",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "americold/tacoma/0007/cha/054127c3-6685-42ac-be1d-d29e969b116e_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "ppg/cedar_falls/0003/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "ppg/cedar_falls/0003/cha/d9d286b4-788d-468a-8ba0-d27e16fdeaae_0000",
        "inferred_incidents": [
            Incident(
                uuid="8a36100e-0ec8-447b-9817-b79df9914d92",
                camera_uuid="ppg/cedar_falls/0003/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["2"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897150884,
                end_frame_relative_ms=1678897151151,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="4c8e295c-57db-4343-93fa-8c965ca53e4d",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/ppg-cedar_falls-0003/1668827596086",
                "version": 6,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "ppg/cedar_falls/0003/cha",
            "incident": {
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 600}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "n_person_ped_zone",
                    "parking",
                    "spill",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_01_18_2023_piston-automotive-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {
                    "enabled": False,
                    "is_classification_by_detection": False,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_bumpcap_vit_99_46_34k_2022-12-20-jit/bumpcap_vit_99_46_34k_2022-12-20-jit.pt",
                },
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": False,
                    "min_actor_pixel_area": 800,
                    "model_path": "artifacts_voxel_safetyvest_vit_general_2022-09-21/voxel_safetyvest_vit_general_2022-09-21.pth",
                    "model_type": "Transformers",
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "ppg/cedar_falls/0003/cha/d9d286b4-788d-468a-8ba0-d27e16fdeaae_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "ppg/cedar_falls/0003/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "ppg/cedar_falls/0003/cha/349d20c2-8a42-479d-86be-d941b4d9e5bd_0000",
        "inferred_incidents": [
            Incident(
                uuid="c534f72b-a397-43a3-b7dc-8587a229e25d",
                camera_uuid="ppg/cedar_falls/0003/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["2"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897159098,
                end_frame_relative_ms=1678897159263,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="f98d308b-df59-4524-a93c-15c502838472",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/ppg-cedar_falls-0003/1668827596086",
                "version": 6,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "ppg/cedar_falls/0003/cha",
            "incident": {
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 600}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "n_person_ped_zone",
                    "parking",
                    "spill",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_01_18_2023_piston-automotive-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {
                    "enabled": False,
                    "is_classification_by_detection": False,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_bumpcap_vit_99_46_34k_2022-12-20-jit/bumpcap_vit_99_46_34k_2022-12-20-jit.pt",
                },
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": False,
                    "min_actor_pixel_area": 800,
                    "model_path": "artifacts_voxel_safetyvest_vit_general_2022-09-21/voxel_safetyvest_vit_general_2022-09-21.pth",
                    "model_type": "Transformers",
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "ppg/cedar_falls/0003/cha/349d20c2-8a42-479d-86be-d941b4d9e5bd_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "ppg/cedar_falls/0004/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "ppg/cedar_falls/0004/cha/6b879b0e-124f-47d8-a142-eea7cedb851c_0000",
        "inferred_incidents": [
            Incident(
                uuid="01cfa58a-e969-4d13-87ae-e02543121323",
                camera_uuid="ppg/cedar_falls/0004/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["10"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897157940,
                end_frame_relative_ms=1678897158175,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="c3162670-6ef3-450c-ba22-a1892cf15be0",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/ppg-cedar_falls-0004/1668827596827",
                "version": 5,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "ppg/cedar_falls/0004/cha",
            "incident": {
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 600}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "n_person_ped_zone",
                    "parking",
                    "spill",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_01_18_2023_piston-automotive-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {
                    "enabled": False,
                    "is_classification_by_detection": False,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_bumpcap_vit_99_46_34k_2022-12-20-jit/bumpcap_vit_99_46_34k_2022-12-20-jit.pt",
                },
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": False,
                    "min_actor_pixel_area": 800,
                    "model_path": "artifacts_voxel_safetyvest_vit_general_2022-09-21/voxel_safetyvest_vit_general_2022-09-21.pth",
                    "model_type": "Transformers",
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "ppg/cedar_falls/0004/cha/6b879b0e-124f-47d8-a142-eea7cedb851c_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "ppg/cedar_falls/0004/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "ppg/cedar_falls/0004/cha/989f1765-a155-4321-b494-dd4b20f894ef_0000",
        "inferred_incidents": [
            Incident(
                uuid="2ffed215-49c4-4a64-8372-28fc014e6656",
                camera_uuid="ppg/cedar_falls/0004/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["4"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897161186,
                end_frame_relative_ms=1678897161452,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="516213c2-e69d-4fb7-9a0b-ce0a333ae8ab",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/ppg-cedar_falls-0004/1668827596827",
                "version": 5,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "ppg/cedar_falls/0004/cha",
            "incident": {
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 600}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "n_person_ped_zone",
                    "parking",
                    "spill",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_01_18_2023_piston-automotive-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {
                    "enabled": False,
                    "is_classification_by_detection": False,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_bumpcap_vit_99_46_34k_2022-12-20-jit/bumpcap_vit_99_46_34k_2022-12-20-jit.pt",
                },
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": False,
                    "min_actor_pixel_area": 800,
                    "model_path": "artifacts_voxel_safetyvest_vit_general_2022-09-21/voxel_safetyvest_vit_general_2022-09-21.pth",
                    "model_type": "Transformers",
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "ppg/cedar_falls/0004/cha/989f1765-a155-4321-b494-dd4b20f894ef_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "ppg/cedar_falls/0005/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "ppg/cedar_falls/0005/cha/1e7599fb-efe4-4b6e-8b49-9e1e3a5f9e71_0000",
        "inferred_incidents": [
            Incident(
                uuid="c48d84b7-6bf3-4d7e-a10e-e06afac9fa6f",
                camera_uuid="ppg/cedar_falls/0005/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["6"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897173113,
                end_frame_relative_ms=1678897173279,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="a3e913fc-444c-4fa7-9f6c-8ad429d86c89",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/ppg-cedar_falls-0005/1668827597571",
                "version": 6,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "ppg/cedar_falls/0005/cha",
            "incident": {
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 600}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "n_person_ped_zone",
                    "parking",
                    "spill",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_01_18_2023_piston-automotive-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {
                    "enabled": False,
                    "is_classification_by_detection": False,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_bumpcap_vit_99_46_34k_2022-12-20-jit/bumpcap_vit_99_46_34k_2022-12-20-jit.pt",
                },
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": False,
                    "min_actor_pixel_area": 800,
                    "model_path": "artifacts_voxel_safetyvest_vit_general_2022-09-21/voxel_safetyvest_vit_general_2022-09-21.pth",
                    "model_type": "Transformers",
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "ppg/cedar_falls/0005/cha/1e7599fb-efe4-4b6e-8b49-9e1e3a5f9e71_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "ppg/cedar_falls/0005/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "ppg/cedar_falls/0005/cha/d53dee4a-8b68-463e-9f10-5a29ae7eb7b5_0000",
        "inferred_incidents": [
            Incident(
                uuid="6d63a629-ce22-40ff-a388-edbc5ced50d6",
                camera_uuid="ppg/cedar_falls/0005/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897182347,
                end_frame_relative_ms=1678897182614,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="a55e44f2-e9ed-451d-8fc3-a5fba114f363",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/ppg-cedar_falls-0005/1668827597571",
                "version": 6,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "ppg/cedar_falls/0005/cha",
            "incident": {
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 600}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "n_person_ped_zone",
                    "parking",
                    "spill",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_01_18_2023_piston-automotive-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {
                    "enabled": False,
                    "is_classification_by_detection": False,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_bumpcap_vit_99_46_34k_2022-12-20-jit/bumpcap_vit_99_46_34k_2022-12-20-jit.pt",
                },
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": False,
                    "min_actor_pixel_area": 800,
                    "model_path": "artifacts_voxel_safetyvest_vit_general_2022-09-21/voxel_safetyvest_vit_general_2022-09-21.pth",
                    "model_type": "Transformers",
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "ppg/cedar_falls/0005/cha/d53dee4a-8b68-463e-9f10-5a29ae7eb7b5_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "ppg/cedar_falls/0007/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "ppg/cedar_falls/0007/cha/003ffb3e-beba-4b45-bada-aa92d403a3bf_0000",
        "inferred_incidents": [
            Incident(
                uuid="b823f8f8-6711-4132-a351-bc25673fb953",
                camera_uuid="ppg/cedar_falls/0007/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["15"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897187591,
                end_frame_relative_ms=1678897187758,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="78c82145-ee1c-4440-a1a4-48ca09a7f28c",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/ppg-cedar_falls-0007/1668827599054",
                "version": 5,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "ppg/cedar_falls/0007/cha",
            "incident": {
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 600}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "n_person_ped_zone",
                    "parking",
                    "spill",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_01_18_2023_piston-automotive-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {
                    "enabled": False,
                    "is_classification_by_detection": False,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_bumpcap_vit_99_46_34k_2022-12-20-jit/bumpcap_vit_99_46_34k_2022-12-20-jit.pt",
                },
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": False,
                    "min_actor_pixel_area": 800,
                    "model_path": "artifacts_voxel_safetyvest_vit_general_2022-09-21/voxel_safetyvest_vit_general_2022-09-21.pth",
                    "model_type": "Transformers",
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "ppg/cedar_falls/0007/cha/003ffb3e-beba-4b45-bada-aa92d403a3bf_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "ppg/cedar_falls/0007/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "ppg/cedar_falls/0007/cha/38de0798-87ed-475d-adc0-703a630b5c6a_0000",
        "inferred_incidents": [
            Incident(
                uuid="19d422d8-252c-43a4-965d-5106bab6bc6d",
                camera_uuid="ppg/cedar_falls/0007/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897184320,
                end_frame_relative_ms=1678897184753,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="9a306a19-1d42-48d6-bb10-efc589c4a077",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/ppg-cedar_falls-0007/1668827599054",
                "version": 5,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "ppg/cedar_falls/0007/cha",
            "incident": {
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 600}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "n_person_ped_zone",
                    "parking",
                    "spill",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_01_18_2023_piston-automotive-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {
                    "enabled": False,
                    "is_classification_by_detection": False,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_bumpcap_vit_99_46_34k_2022-12-20-jit/bumpcap_vit_99_46_34k_2022-12-20-jit.pt",
                },
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": False,
                    "min_actor_pixel_area": 800,
                    "model_path": "artifacts_voxel_safetyvest_vit_general_2022-09-21/voxel_safetyvest_vit_general_2022-09-21.pth",
                    "model_type": "Transformers",
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "ppg/cedar_falls/0007/cha/38de0798-87ed-475d-adc0-703a630b5c6a_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "ppg/cedar_falls/0008/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "ppg/cedar_falls/0008/cha/4a04ff42-2498-46a0-8eaa-b81bd2326319_0000",
        "inferred_incidents": [
            Incident(
                uuid="9860b341-0ddd-44ba-8d8c-9e6bdfeb04ba",
                camera_uuid="ppg/cedar_falls/0008/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["6"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897190380,
                end_frame_relative_ms=1678897190647,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="7f308ea5-1073-4c47-be81-20ede23ecf47",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/ppg-cedar_falls-0008/1668827599797",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "ppg/cedar_falls/0008/cha",
            "incident": {
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 600}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": ["bad_posture"],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_01_18_2023_piston-automotive-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {
                    "enabled": False,
                    "is_classification_by_detection": False,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_bumpcap_vit_99_46_34k_2022-12-20-jit/bumpcap_vit_99_46_34k_2022-12-20-jit.pt",
                },
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": False,
                    "min_actor_pixel_area": 800,
                    "model_path": "artifacts_voxel_safetyvest_vit_general_2022-09-21/voxel_safetyvest_vit_general_2022-09-21.pth",
                    "model_type": "Transformers",
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "ppg/cedar_falls/0008/cha/4a04ff42-2498-46a0-8eaa-b81bd2326319_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "ppg/cedar_falls/0008/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "ppg/cedar_falls/0008/cha/ceafef1e-3136-402c-8ae5-8b35e1f0dd5b_0000",
        "inferred_incidents": [
            Incident(
                uuid="d4b6c01f-4629-4f55-a263-5061804559fa",
                camera_uuid="ppg/cedar_falls/0008/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["8"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897195679,
                end_frame_relative_ms=1678897195846,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="21b3ce55-9439-4cbf-9ba9-c9b50053b4e2",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/ppg-cedar_falls-0008/1668827599797",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "ppg/cedar_falls/0008/cha",
            "incident": {
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 600}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": ["bad_posture"],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_01_18_2023_piston-automotive-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {
                    "enabled": False,
                    "is_classification_by_detection": False,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_bumpcap_vit_99_46_34k_2022-12-20-jit/bumpcap_vit_99_46_34k_2022-12-20-jit.pt",
                },
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": False,
                    "min_actor_pixel_area": 800,
                    "model_path": "artifacts_voxel_safetyvest_vit_general_2022-09-21/voxel_safetyvest_vit_general_2022-09-21.pth",
                    "model_type": "Transformers",
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "ppg/cedar_falls/0008/cha/ceafef1e-3136-402c-8ae5-8b35e1f0dd5b_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "piston_automotive/marion/0001/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "piston_automotive/marion/0001/cha/fc7595d4-c57e-426b-a8ad-db885e327e9b_0000",
        "inferred_incidents": [
            Incident(
                uuid="373c9ee1-0e3c-4df1-8cc5-014710c6cc32",
                camera_uuid="piston_automotive/marion/0001/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["9"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897209938,
                end_frame_relative_ms=1678897210205,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="e833c89e-93eb-45ef-9ecc-32f9bad67271",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/piston_automotive-marion-0001/1670373576615",
                "version": 3,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "piston_automotive/marion/0001/cha",
            "incident": {
                "incident_machine_params": {
                    "intersection": {
                        "incident_thresh_low": 0.4,
                        "incident_thresh_med": 0.7,
                        "incident_thresh_high": 0.9,
                    },
                    "no_stop_at_aisle_end": {
                        "incident_thresh_low": 0.5,
                        "incident_thresh_med": 0.9,
                        "incident_thresh_high": 1.2,
                    },
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "intersection",
                    "no_stop_at_aisle_end",
                    "overreaching",
                    "parking",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_01_18_2023_piston-automotive-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "piston_automotive/marion/0001/cha/fc7595d4-c57e-426b-a8ad-db885e327e9b_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "piston_automotive/marion/0001/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "piston_automotive/marion/0001/cha/3afcd7ab-dea9-4a53-b809-07e413cee6b6_0000",
        "inferred_incidents": [
            Incident(
                uuid="f9aeb5f8-e21c-4bc8-899d-b734e4364412",
                camera_uuid="piston_automotive/marion/0001/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897225191,
                end_frame_relative_ms=1678897225291,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="54a30d8f-e973-455d-be07-4696f946fb0d",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/piston_automotive-marion-0001/1670373576615",
                "version": 3,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "piston_automotive/marion/0001/cha",
            "incident": {
                "incident_machine_params": {
                    "intersection": {
                        "incident_thresh_low": 0.4,
                        "incident_thresh_med": 0.7,
                        "incident_thresh_high": 0.9,
                    },
                    "no_stop_at_aisle_end": {
                        "incident_thresh_low": 0.5,
                        "incident_thresh_med": 0.9,
                        "incident_thresh_high": 1.2,
                    },
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "intersection",
                    "no_stop_at_aisle_end",
                    "overreaching",
                    "parking",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_01_18_2023_piston-automotive-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "piston_automotive/marion/0001/cha/3afcd7ab-dea9-4a53-b809-07e413cee6b6_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "piston_automotive/marion/0002/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "piston_automotive/marion/0002/cha/2089ec60-5455-4232-8947-ccae367e1de4_0000",
        "inferred_incidents": [
            Incident(
                uuid="da60bbb6-0f7b-4603-84db-5b3772f2bbe5",
                camera_uuid="piston_automotive/marion/0002/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["5"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897224204,
                end_frame_relative_ms=1678897224337,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="7f135eb8-fe2c-4fb7-9342-7eb004f1ee26",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/piston_automotive-marion-0002/1670373577339",
                "version": 6,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "piston_automotive/marion/0002/cha",
            "incident": {
                "incident_machine_params": {
                    "no_ped_zone": {"max_ped_zone_duration_s": 3}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "no_ped_zone",
                    "overreaching",
                    "spill",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_01_18_2023_piston-automotive-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "piston_automotive/marion/0002/cha/2089ec60-5455-4232-8947-ccae367e1de4_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "piston_automotive/marion/0004/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "piston_automotive/marion/0004/cha/4b385286-4c13-45f3-87e9-3daaf2f1f630_0000",
        "inferred_incidents": [
            Incident(
                uuid="34fe37fd-d12c-4403-8b62-85c6752d3be5",
                camera_uuid="piston_automotive/marion/0004/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["6"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897229050,
                end_frame_relative_ms=1678897229317,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="9f444df4-2e77-484d-8a7b-59db65cee5f9",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/piston_automotive-marion-0004/1670373578838",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "piston_automotive/marion/0004/cha",
            "incident": {
                "incident_machine_params": {
                    "intersection": {"incident_thresh_low": 0.9}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "intersection",
                    "overreaching",
                    "parking",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_01_18_2023_piston-automotive-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "piston_automotive/marion/0004/cha/4b385286-4c13-45f3-87e9-3daaf2f1f630_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "piston_automotive/marion/0004/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "piston_automotive/marion/0004/cha/782ae94d-9585-4b92-a846-a298ead5caf6_0000",
        "inferred_incidents": [
            Incident(
                uuid="a26fd5ca-a14b-44f3-a8f3-ca5a63c5199c",
                camera_uuid="piston_automotive/marion/0004/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["2"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897238309,
                end_frame_relative_ms=1678897238542,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="4c1d861a-91f6-436c-949b-15a8543d48af",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/piston_automotive-marion-0004/1670373578838",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "piston_automotive/marion/0004/cha",
            "incident": {
                "incident_machine_params": {
                    "intersection": {"incident_thresh_low": 0.9}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "intersection",
                    "overreaching",
                    "parking",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_01_18_2023_piston-automotive-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "piston_automotive/marion/0004/cha/782ae94d-9585-4b92-a846-a298ead5caf6_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "piston_automotive/marion/0005/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "piston_automotive/marion/0005/cha/e912d336-2ae0-48ad-a59e-1d5446a89e7e_0000",
        "inferred_incidents": [
            Incident(
                uuid="c2fa9cc4-15b2-406e-9a9b-5991a5eb37a0",
                camera_uuid="piston_automotive/marion/0005/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897249143,
                end_frame_relative_ms=1678897249343,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="435820f0-a9c6-48bf-a250-0f844105a91a",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/piston_automotive-marion-0005/1670373579569",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "piston_automotive/marion/0005/cha",
            "incident": {
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "overreaching",
                    "parking",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_01_18_2023_piston-automotive-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "piston_automotive/marion/0005/cha/e912d336-2ae0-48ad-a59e-1d5446a89e7e_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "piston_automotive/marion/0006/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "piston_automotive/marion/0006/cha/95f76e84-b7df-4976-a314-1e544c23cdd3_0000",
        "inferred_incidents": [
            Incident(
                uuid="7e4fd48f-2f19-4b7d-9278-d566a7380786",
                camera_uuid="piston_automotive/marion/0006/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["11"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897251243,
                end_frame_relative_ms=1678897251377,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="a3a3ac0a-ab46-4211-8189-bd51eef38144",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/piston_automotive-marion-0006/1670373580284",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "piston_automotive/marion/0006/cha",
            "incident": {
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "overreaching",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_01_18_2023_piston-automotive-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "piston_automotive/marion/0006/cha/95f76e84-b7df-4976-a314-1e544c23cdd3_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "piston_automotive/marion/0007/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "piston_automotive/marion/0007/cha/db03a6c1-dc29-4686-9ca7-e4db2253b256_0000",
        "inferred_incidents": [
            Incident(
                uuid="808e8e61-8f5b-47d5-8ea9-eb5472c52e2a",
                camera_uuid="piston_automotive/marion/0007/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["12"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897251025,
                end_frame_relative_ms=1678897251125,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="e7fabf4b-cd48-4f7d-9a7a-d030da6a824b",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/piston_automotive-marion-0007/1670373581004",
                "version": 3,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "piston_automotive/marion/0007/cha",
            "incident": {
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "overreaching",
                    "parking",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_01_18_2023_piston-automotive-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "piston_automotive/marion/0007/cha/db03a6c1-dc29-4686-9ca7-e4db2253b256_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "innovate_manufacturing/knoxville/0004/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "innovate_manufacturing/knoxville/0004/cha/68abcd5d-dd9d-4b45-ad60-087f884ecddd_0000",
        "inferred_incidents": [
            Incident(
                uuid="b3768688-40f9-4904-b68f-79f7cf2709ad",
                camera_uuid="innovate_manufacturing/knoxville/0004/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["4"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897262638,
                end_frame_relative_ms=1678897262838,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="f443ea98-0828-40a2-8478-09c07968e61b",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/innovate_manufacturing-knoxville-0004/1670226821828",
                "version": 4,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "innovate_manufacturing/knoxville/0004/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "door_violation",
                    "open_door",
                    "overreaching",
                    "parking",
                    "piggyback",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_01_18_2023_piston-automotive-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_02_15_2023_innovate_manufacturing_knoxville_0004_cha/6ec9304f-e1be-4731-b687-82c318bfbf6d.pt",
                    "model_type": "vanilla_resnet",
                },
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "spill": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "innovate_manufacturing/knoxville/0004/cha/68abcd5d-dd9d-4b45-ad60-087f884ecddd_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "innovate_manufacturing/knoxville/0005/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "innovate_manufacturing/knoxville/0005/cha/0a0b0e09-7887-4101-bec6-9a81a10384f7_0000",
        "inferred_incidents": [
            Incident(
                uuid="72158725-5444-434b-83ba-39592e6af3d4",
                camera_uuid="innovate_manufacturing/knoxville/0005/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897266138,
                end_frame_relative_ms=1678897266538,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="74f976b3-bc7e-4927-95b4-cf16e05c6669",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/innovate_manufacturing-knoxville-0005/1670226823357",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "innovate_manufacturing/knoxville/0005/cha",
            "incident": {
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "door_violation",
                    "open_door",
                    "overreaching",
                    "parking",
                    "piggyback",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_01_18_2023_piston-automotive-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_01_18_2023_generalized-front-door/3f11d513-b9bd-471e-862e-4d2474fc443c.pt",
                    "model_type": "vanilla_resnet",
                },
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "spill": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "innovate_manufacturing/knoxville/0005/cha/0a0b0e09-7887-4101-bec6-9a81a10384f7_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "innovate_manufacturing/knoxville/0006/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "innovate_manufacturing/knoxville/0006/cha/8b10e6c5-4d0a-4f27-9135-344687c218a1_0000",
        "inferred_incidents": [
            Incident(
                uuid="309ffb64-38fc-4512-b1f7-b411472e8589",
                camera_uuid="innovate_manufacturing/knoxville/0006/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["9"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897272561,
                end_frame_relative_ms=1678897272761,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="13b54c8a-6eee-4d08-9ebd-fe1d2a07bbab",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/innovate_manufacturing-knoxville-0006/1670226825336",
                "version": 3,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "innovate_manufacturing/knoxville/0006/cha",
            "incident": {
                "incident_machine_params": {
                    "production_line_down": {
                        "max_no_motion_detection_s": 300,
                        "per_actor_cooldown_s": 600,
                    }
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "overreaching",
                    "production_line_down",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_01_18_2023_piston-automotive-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "motion_zone_detection": {
                    "enabled": True,
                    "gaussian_kernel_size": 5,
                    "variance_threshold": 1500,
                    "bg_frames_history": 500,
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "spill": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "innovate_manufacturing/knoxville/0006/cha/8b10e6c5-4d0a-4f27-9135-344687c218a1_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "innovate_manufacturing/knoxville/0007/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "innovate_manufacturing/knoxville/0007/cha/f1f9ca4a-ba99-4588-be8e-d724a557148c_0000",
        "inferred_incidents": [
            Incident(
                uuid="483cec37-7cdd-4ee7-8810-37d4ec977e7e",
                camera_uuid="innovate_manufacturing/knoxville/0007/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["8"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897285997,
                end_frame_relative_ms=1678897286197,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="fbc5fe74-909c-4f37-b765-99cd06ab83f8",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/innovate_manufacturing-knoxville-0007/1670226826725",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "innovate_manufacturing/knoxville/0007/cha",
            "incident": {
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "no_ped_zone",
                    "overreaching",
                    "parking",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_01_18_2023_piston-automotive-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "spill": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "innovate_manufacturing/knoxville/0007/cha/f1f9ca4a-ba99-4588-be8e-d724a557148c_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "innovate_manufacturing/knoxville/0010/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "innovate_manufacturing/knoxville/0010/cha/f42df378-897c-4a5d-a011-eda932ac6501_0000",
        "inferred_incidents": [
            Incident(
                uuid="315b1c07-3e2f-4997-afe7-f1007c34b22a",
                camera_uuid="innovate_manufacturing/knoxville/0010/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897290952,
                end_frame_relative_ms=1678897291152,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="13876e20-ae8e-4e64-844d-6e4aca35277d",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/innovate_manufacturing-knoxville-0010/1670226831117",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "innovate_manufacturing/knoxville/0010/cha",
            "incident": {
                "incident_machine_params": {
                    "no_ped_zone": {
                        "active_hours_start_utc": "23:00",
                        "active_hours_end_utc": "11:00",
                    }
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "no_ped_zone",
                    "overreaching",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_01_18_2023_piston-automotive-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "innovate_manufacturing/knoxville/0010/cha/f42df378-897c-4a5d-a011-eda932ac6501_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "innovate_manufacturing/knoxville/0014/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "innovate_manufacturing/knoxville/0014/cha/f25476d6-49b0-4ceb-a685-aa265343ecc8_0000",
        "inferred_incidents": [
            Incident(
                uuid="946624a8-990c-491d-93fb-1714be3fff35",
                camera_uuid="innovate_manufacturing/knoxville/0014/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["3"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897295580,
                end_frame_relative_ms=1678897295780,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="bdbed8d0-77d8-4781-8e8f-0ae23da9a49b",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/innovate_manufacturing-knoxville-0014/1672262326898",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "innovate_manufacturing/knoxville/0014/cha",
            "incident": {
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "no_ped_zone",
                    "overreaching",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_01_18_2023_piston-automotive-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "innovate_manufacturing/knoxville/0014/cha/f25476d6-49b0-4ceb-a685-aa265343ecc8_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "wesco/reno/0001/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "wesco/reno/0001/cha/292c350a-afeb-4138-966e-aff832862dd4_0000",
        "inferred_incidents": [
            Incident(
                uuid="7e8169e1-7c34-4c8c-a36f-90e8eb10fd43",
                camera_uuid="wesco/reno/0001/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897302189,
                end_frame_relative_ms=1678897302389,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="b3638ed5-7139-4347-9442-aaa76c3404b1",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/wesco-reno-0001/1673393061370",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "wesco/reno/0001/cha",
            "incident": {
                "incident_machine_params": {
                    "safety_vest": {"per_camera_cooldown_s": 3600}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "overreaching",
                    "safety_vest",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit_general_2022-09-21/voxel_safetyvest_vit_general_2022-09-21.pth",
                    "model_type": "Transformers",
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "wesco/reno/0001/cha/292c350a-afeb-4138-966e-aff832862dd4_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "wesco/reno/0001/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "wesco/reno/0001/cha/f7b028c6-7eec-4658-a106-1cdcd6c6cde2_0000",
        "inferred_incidents": [
            Incident(
                uuid="b6dc3261-2aad-47a8-8291-88eb737a9dde",
                camera_uuid="wesco/reno/0001/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897311463,
                end_frame_relative_ms=1678897311863,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="e7b0f40c-e805-413b-b353-0fefb633e08c",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/wesco-reno-0001/1673393061370",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "wesco/reno/0001/cha",
            "incident": {
                "incident_machine_params": {
                    "safety_vest": {"per_camera_cooldown_s": 3600}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "overreaching",
                    "safety_vest",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit_general_2022-09-21/voxel_safetyvest_vit_general_2022-09-21.pth",
                    "model_type": "Transformers",
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "wesco/reno/0001/cha/f7b028c6-7eec-4658-a106-1cdcd6c6cde2_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "wesco/reno/0004/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "wesco/reno/0004/cha/04c8260a-0d8b-4fd0-8f6d-1ef2e6fa4984_0000",
        "inferred_incidents": [
            Incident(
                uuid="4eb2013c-8cd9-40e3-9507-50d2598aba44",
                camera_uuid="wesco/reno/0004/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["2"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897312308,
                end_frame_relative_ms=1678897312558,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="7ad9f5dc-1144-4b22-be9f-a4c494ca2bc6",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/wesco-reno-0004/1673393063250",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "wesco/reno/0004/cha",
            "incident": {
                "incident_machine_params": {
                    "no_stop_at_aisle_end": {"incident_thresh_high": 1.2},
                    "parking": {"max_parked_duration_s": 300},
                    "safety_vest": {"per_camera_cooldown_s": 3600},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "no_stop_at_aisle_end",
                    "overreaching",
                    "parking",
                    "safety_vest",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit_general_2022-09-21/voxel_safetyvest_vit_general_2022-09-21.pth",
                    "model_type": "Transformers",
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "wesco/reno/0004/cha/04c8260a-0d8b-4fd0-8f6d-1ef2e6fa4984_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "wesco/reno/0005/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "wesco/reno/0005/cha/4e927c97-9c87-471f-9576-8d4ccc51725f_0000",
        "inferred_incidents": [
            Incident(
                uuid="58d9fd0a-e3b0-4deb-b00d-6849b2a8f010",
                camera_uuid="wesco/reno/0005/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897310863,
                end_frame_relative_ms=1678897311196,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="8397f9bd-d598-465c-a1f1-b5ee75786d2f",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/wesco-reno-0005/1673393064165",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "wesco/reno/0005/cha",
            "incident": {
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 300},
                    "safety_vest": {"per_camera_cooldown_s": 3600},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "overreaching",
                    "parking",
                    "safety_vest",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit_general_2022-09-21/voxel_safetyvest_vit_general_2022-09-21.pth",
                    "model_type": "Transformers",
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "wesco/reno/0005/cha/4e927c97-9c87-471f-9576-8d4ccc51725f_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "wesco/reno/0005/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "wesco/reno/0005/cha/d09363e0-41e3-4971-a490-1ca3f910cfbb_0000",
        "inferred_incidents": [
            Incident(
                uuid="ca300bfd-5c5e-482f-a391-96220be2c13a",
                camera_uuid="wesco/reno/0005/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897316883,
                end_frame_relative_ms=1678897317133,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="e5d7e8b8-545f-404e-baa1-8fb63a786649",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/wesco-reno-0005/1673393064165",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "wesco/reno/0005/cha",
            "incident": {
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 300},
                    "safety_vest": {"per_camera_cooldown_s": 3600},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "overreaching",
                    "parking",
                    "safety_vest",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit_general_2022-09-21/voxel_safetyvest_vit_general_2022-09-21.pth",
                    "model_type": "Transformers",
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "wesco/reno/0005/cha/d09363e0-41e3-4971-a490-1ca3f910cfbb_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "wesco/reno/0011/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "wesco/reno/0011/cha/a4928fb7-acc8-41c9-994e-1912419bccf0_0000",
        "inferred_incidents": [
            Incident(
                uuid="24ea2c42-2aca-4c42-a291-1b64c61a88c2",
                camera_uuid="wesco/reno/0011/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897338815,
                end_frame_relative_ms=1678897339014,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="97f1abbf-373c-4c21-b9c7-a12c30977273",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/wesco-reno-0011/1673393183357",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "wesco/reno/0011/cha",
            "incident": {
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 300},
                    "safety_vest": {"per_camera_cooldown_s": 3600},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "overreaching",
                    "parking",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": False,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit_general_2022-09-21/voxel_safetyvest_vit_general_2022-09-21.pth",
                    "model_type": "Transformers",
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "wesco/reno/0011/cha/a4928fb7-acc8-41c9-994e-1912419bccf0_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "wesco/reno/0015/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "wesco/reno/0015/cha/b4baf119-4eb6-4b58-9813-97b354719a0d_0000",
        "inferred_incidents": [
            Incident(
                uuid="60261910-bfe5-4a0e-be97-e67d03858ae8",
                camera_uuid="wesco/reno/0015/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["2"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897337883,
                end_frame_relative_ms=1678897338083,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="420b0b05-2951-42f2-afa4-0a99885d0275",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/wesco-reno-0015/1673393072374",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "wesco/reno/0015/cha",
            "incident": {
                "incident_machine_params": {
                    "open_door": {"max_open_door_s": 30},
                    "parking": {"max_parked_duration_s": 300},
                    "safety_vest": {"per_camera_cooldown_s": 3600},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "open_door",
                    "overreaching",
                    "parking",
                    "safety_vest",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_02_06_2023_generalized_EXIT_Door_new/7547a635-1ae9-4b9a-b44a-4e8991269ead.pt",
                    "model_type": "vanilla_resnet",
                },
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit_general_2022-09-21/voxel_safetyvest_vit_general_2022-09-21.pth",
                    "model_type": "Transformers",
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "wesco/reno/0015/cha/b4baf119-4eb6-4b58-9813-97b354719a0d_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "wesco/reno/0016/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "wesco/reno/0016/cha/680838d0-b818-4b4e-92aa-8324054a0c66_0000",
        "inferred_incidents": [
            Incident(
                uuid="5ab36c87-b22a-498b-9d09-1f97d748118f",
                camera_uuid="wesco/reno/0016/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897338295,
                end_frame_relative_ms=1678897338495,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="6864257d-9cd1-4032-8dcf-f5113db527ad",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/wesco-reno-0016/1673393073271",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "wesco/reno/0016/cha",
            "incident": {
                "incident_machine_params": {
                    "no_stop_at_aisle_end": {"incident_thresh_high": 1.2},
                    "parking": {"max_parked_duration_s": 300},
                    "safety_vest": {"per_camera_cooldown_s": 3600},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "no_stop_at_aisle_end",
                    "overreaching",
                    "parking",
                    "safety_vest",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit_general_2022-09-21/voxel_safetyvest_vit_general_2022-09-21.pth",
                    "model_type": "Transformers",
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "wesco/reno/0016/cha/680838d0-b818-4b4e-92aa-8324054a0c66_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "wesco/reno/0016/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "wesco/reno/0016/cha/570d2491-f3af-4367-b322-252773899124_0000",
        "inferred_incidents": [
            Incident(
                uuid="37193a23-bf7c-4dbf-807b-cb530142236d",
                camera_uuid="wesco/reno/0016/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["2"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897344654,
                end_frame_relative_ms=1678897344854,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="619150ef-d119-48ca-8f03-b1d115dc844f",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/wesco-reno-0016/1673393073271",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "wesco/reno/0016/cha",
            "incident": {
                "incident_machine_params": {
                    "no_stop_at_aisle_end": {"incident_thresh_high": 1.2},
                    "parking": {"max_parked_duration_s": 300},
                    "safety_vest": {"per_camera_cooldown_s": 3600},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "no_stop_at_aisle_end",
                    "overreaching",
                    "parking",
                    "safety_vest",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit_general_2022-09-21/voxel_safetyvest_vit_general_2022-09-21.pth",
                    "model_type": "Transformers",
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "wesco/reno/0016/cha/570d2491-f3af-4367-b322-252773899124_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "michaels/tracy/0005/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "michaels/tracy/0005/cha/6aec93af-96a9-47d6-a751-4713bc0d8d69_0000",
        "inferred_incidents": [
            Incident(
                uuid="3e0de00c-4099-45a3-b532-40bab1dde578",
                camera_uuid="michaels/tracy/0005/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["5"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897351554,
                end_frame_relative_ms=1678897351804,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="ed6be700-5fd8-4d08-a80d-d8717fe11b30",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/michaels-tracy-0005/1674507659913",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "michaels/tracy/0005/cha",
            "incident": {
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 600}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "open_door",
                    "overreaching",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_02_06_2023_generalized_EXIT_Door_new/7547a635-1ae9-4b9a-b44a-4e8991269ead.pt",
                    "model_type": "vanilla_resnet",
                },
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "michaels/tracy/0005/cha/6aec93af-96a9-47d6-a751-4713bc0d8d69_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "ulta/dallas/0010/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "ulta/dallas/0010/cha/578d7fca-c1bc-408e-b150-03002fa4b548_0000",
        "inferred_incidents": [
            Incident(
                uuid="c986ad3c-c88d-4244-9411-39b4a1ecac80",
                camera_uuid="ulta/dallas/0010/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897361992,
                end_frame_relative_ms=1678897362257,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="88213fca-d8ff-4ee7-a80b-de905ece3174",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/ulta-dallas-0010/1677204275372",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "ulta/dallas/0010/cha",
            "incident": {
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "overreaching",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "motion_zone_detection": {"enabled": False},
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "ulta/dallas/0010/cha/578d7fca-c1bc-408e-b150-03002fa4b548_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "ulta/dallas/0011/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "ulta/dallas/0011/cha/bb4037a6-9196-4314-94a4-2375a583e504_0000",
        "inferred_incidents": [
            Incident(
                uuid="0b32782f-287e-4cad-b687-0db45d18efd4",
                camera_uuid="ulta/dallas/0011/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897362358,
                end_frame_relative_ms=1678897362591,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="07f8fcb2-8c2c-4bde-8545-ddd48d1d2209",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/ulta-dallas-0011/1677204276077",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "ulta/dallas/0011/cha",
            "incident": {
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "intersection",
                    "overreaching",
                    "parking",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "motion_zone_detection": {"enabled": False},
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "ulta/dallas/0011/cha/bb4037a6-9196-4314-94a4-2375a583e504_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "ulta/dallas/0011/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "ulta/dallas/0011/cha/ba7a8cf9-73cd-4faf-bcc1-daef7151885d_0000",
        "inferred_incidents": [
            Incident(
                uuid="cea7b3e3-85e7-49e2-9c69-eca65faa1bac",
                camera_uuid="ulta/dallas/0011/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["6"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897375587,
                end_frame_relative_ms=1678897375855,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="1d1da131-9ad2-49c9-b9ac-33dd05e5014a",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/ulta-dallas-0011/1677204276077",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "ulta/dallas/0011/cha",
            "incident": {
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "intersection",
                    "overreaching",
                    "parking",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "motion_zone_detection": {"enabled": False},
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "ulta/dallas/0011/cha/ba7a8cf9-73cd-4faf-bcc1-daef7151885d_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "ulta/dallas/0012/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "ulta/dallas/0012/cha/b3419a5e-266a-4d63-9e0e-089ace410baf_0000",
        "inferred_incidents": [
            Incident(
                uuid="b1b4cd5c-9afc-40c3-add3-d5564688bc32",
                camera_uuid="ulta/dallas/0012/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897386211,
                end_frame_relative_ms=1678897386811,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="e26fa0a5-467a-4e61-add2-b4b60fa1aa9f",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/ulta-dallas-0012/1677204276762",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "ulta/dallas/0012/cha",
            "incident": {
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "overreaching",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "motion_zone_detection": {"enabled": False},
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "ulta/dallas/0012/cha/b3419a5e-266a-4d63-9e0e-089ace410baf_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "ulta/dallas/0012/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "ulta/dallas/0012/cha/ed7ba3e4-5f3e-4e93-8b83-c88726421f68_0000",
        "inferred_incidents": [
            Incident(
                uuid="b8f0ff00-f24e-4e30-9557-343e293f0cbf",
                camera_uuid="ulta/dallas/0012/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897381750,
                end_frame_relative_ms=1678897382015,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="66cf522b-d3a4-411a-bdc2-150f71ab79b5",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/ulta-dallas-0012/1677204276762",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "ulta/dallas/0012/cha",
            "incident": {
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "overreaching",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "motion_zone_detection": {"enabled": False},
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "ulta/dallas/0012/cha/ed7ba3e4-5f3e-4e93-8b83-c88726421f68_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "ulta/dallas/0015/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "ulta/dallas/0015/cha/f0aabc1f-0396-46b8-9449-a3770bbe696f_0000",
        "inferred_incidents": [
            Incident(
                uuid="091354fc-a136-418e-841c-3a4d8a283cec",
                camera_uuid="ulta/dallas/0015/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897384564,
                end_frame_relative_ms=1678897385464,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="af8fb25d-1efe-4589-87bd-808f616c04bd",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/ulta-dallas-0015/1677204278932",
                "version": 1,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "ulta/dallas/0015/cha",
            "incident": {
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 300}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "intersection",
                    "no_stop_at_aisle_end",
                    "overreaching",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "motion_zone_detection": {"enabled": False},
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "ulta/dallas/0015/cha/f0aabc1f-0396-46b8-9449-a3770bbe696f_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "ulta/dallas/0016/cha",
        "incidents": ["BAD_POSTURE"],
        "video_uuid": "ulta/dallas/0016/cha/ed58710c-da69-4607-b3c8-9c07dd775e5b_0000",
        "inferred_incidents": [
            Incident(
                uuid="4176ca85-bdeb-4a36-bee8-3fe74389f060",
                camera_uuid="ulta/dallas/0016/cha",
                camera_config_version=None,
                organization_key=None,
                title="Bad Posture",
                incident_type_id="BAD_POSTURE",
                incident_version="1.2",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897405030,
                end_frame_relative_ms=1678897405130,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="ec382bc8-79c4-4c22-8012-7302391d7372",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/ulta-dallas-0016/1677204279641",
                "version": 4,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "ulta/dallas/0016/cha",
            "incident": {
                "incident_machine_params": {},
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "no_stop_at_aisle_end",
                    "overreaching",
                    "parking",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "motion_zone_detection": {"enabled": False},
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "ulta/dallas/0016/cha/ed58710c-da69-4607-b3c8-9c07dd775e5b_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "uscold/laredo/dock04/cha",
        "incidents": ["HARD_HAT"],
        "video_uuid": "uscold/laredo/dock04/cha/c0b13672-460c-43ae-a1c8-4877f9dfaf00_0000",
        "inferred_incidents": [
            Incident(
                uuid="0e962f84-9c6e-4ee5-a65f-e481d5773362",
                camera_uuid="uscold/laredo/dock04/cha",
                camera_config_version=None,
                organization_key=None,
                title="Hard hat",
                incident_type_id="HARD_HAT",
                incident_version="1.0",
                priority="low",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897423666,
                end_frame_relative_ms=1678897430064,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=9000,
                post_end_buffer_ms=6000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="f3e4eb82-50a8-4ef0-8494-9c0386233e8d",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera_uuid": "uscold/laredo/dock04/cha",
            "ray": {"dashboard": {"port": 5003}},
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/uscold-laredo-0003/1651957485510",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "perception": {
                "enabled": True,
                "acausal_layer": {
                    "PPESmoothener": {
                        "safety_vest_time_interval_ms": 10000,
                        "pose_points_min_confidence_threshold": 0.5,
                    }
                },
                "detector_tracker": {
                    "model_path": "artifacts_2021-12-06-00-00-00-0000-yolo/best_608_1024.engine",
                    "vest_model_path": "artifacts_vest_classifier-9-19/vest_classifier_resnet18_traced_model.pth",
                    "height": 608,
                    "width": 1024,
                    "actor2class": {
                        "PERSON": 0,
                        "PIT": 1,
                        "HARD_HAT": 2,
                        "SAFETY_VEST": 3,
                    },
                    "should_do_temporal_propagation": 0,
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "lift_classifier": {
                    "enabled": False,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "door_classifier": {
                    "enabled": False,
                    "model_path": "artifacts_door_classifier_resnet50_dataset4/voxel_door_classifier_dataset4_resnet50_traced_model.pth",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit-uscold-laredo_2022-09-15/voxel_safetyvest_vit_laredo-2_laredo_2022-09-15.pth",
                    "model_type": "Transformers",
                },
                "hat_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_hard_hat_occlusion_vit_v4_2022-11-18-jit/hard_hat_occlusion_vit_v4_2022-11-18-jit.pt",
                    "is_classification_by_detection": False,
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "no_ped_zone": {
                        "per_camera_cooldown_s": 120,
                        "max_ped_zone_duration_s": 120,
                    },
                    "hard_hat": {"per_camera_cooldown_s": 120},
                },
                "monitors_requested": [],
                "incident_machine_enabled": False,
                "state_machine_monitors_requested": [
                    "safety_vest",
                    "hard_hat",
                    "no_ped_zone",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "uscold/laredo/dock04/cha/c0b13672-460c-43ae-a1c8-4877f9dfaf00_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "uscold/laredo/doors_14_20/cha",
        "incidents": ["HARD_HAT"],
        "video_uuid": "uscold/laredo/doors_14_20/cha/fd0ed3bb-fff1-497c-9551-f7fd39d45e81_0000",
        "inferred_incidents": [
            Incident(
                uuid="cf46294d-efca-4d09-a09a-56f01aac49cc",
                camera_uuid="uscold/laredo/doors_14_20/cha",
                camera_config_version=None,
                organization_key=None,
                title="Hard hat",
                incident_type_id="HARD_HAT",
                incident_version="1.0",
                priority="low",
                actor_ids=["40"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897419585,
                end_frame_relative_ms=1678897425584,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=9000,
                post_end_buffer_ms=6000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="04feb55b-5cf0-47c6-91ca-e39911b9a233",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera_uuid": "uscold/laredo/doors_14_20/cha",
            "ray": {"dashboard": {"port": 5004}},
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/uscold-laredo-0004/1651957564634",
                "version": 7,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "perception": {
                "enabled": True,
                "acausal_layer": {
                    "PPESmoothener": {
                        "safety_vest_time_interval_ms": 10000,
                        "pose_points_min_confidence_threshold": 0.5,
                    }
                },
                "detector_tracker": {
                    "model_path": "artifacts_2021-12-06-00-00-00-0000-yolo/best_608_1024.engine",
                    "vest_model_path": "artifacts_vest_classifier-9-19/vest_classifier_resnet18_traced_model.pth",
                    "height": 608,
                    "width": 1024,
                    "actor2class": {
                        "PERSON": 0,
                        "PIT": 1,
                        "HARD_HAT": 2,
                        "SAFETY_VEST": 3,
                    },
                    "should_do_temporal_propagation": 0,
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "lift_classifier": {
                    "enabled": False,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "door_classifier": {
                    "enabled": False,
                    "model_path": "artifacts_door_classifier_resnet50_dataset4/voxel_door_classifier_dataset4_resnet50_traced_model.pth",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit-uscold-laredo_2022-09-15/voxel_safetyvest_vit_laredo-2_laredo_2022-09-15.pth",
                    "model_type": "Transformers",
                },
                "hat_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_hard_hat_occlusion_vit_v4_2022-11-18-jit/hard_hat_occlusion_vit_v4_2022-11-18-jit.pt",
                    "is_classification_by_detection": False,
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "no_ped_zone": {
                        "per_camera_cooldown_s": 120,
                        "max_ped_zone_duration_s": 120,
                    },
                    "hard_hat": {"per_camera_cooldown_s": 120},
                },
                "monitors_requested": [],
                "incident_machine_enabled": False,
                "state_machine_monitors_requested": [
                    "safety_vest",
                    "hard_hat",
                    "no_ped_zone",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "uscold/laredo/doors_14_20/cha/fd0ed3bb-fff1-497c-9551-f7fd39d45e81_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "uscold/laredo/dock03/cha",
        "incidents": ["HARD_HAT"],
        "video_uuid": "uscold/laredo/dock03/cha/43066680-c347-4746-8f01-e63c00b6a0f0_0000",
        "inferred_incidents": [
            Incident(
                uuid="97a7f0b8-fa97-4883-beed-b1f501e7abfc",
                camera_uuid="uscold/laredo/dock03/cha",
                camera_config_version=None,
                organization_key=None,
                title="Hard hat",
                incident_type_id="HARD_HAT",
                incident_version="1.0",
                priority="low",
                actor_ids=["22"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897420116,
                end_frame_relative_ms=1678897430718,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=9000,
                post_end_buffer_ms=6000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="4ea9e61d-12c4-406d-aac7-d5dd353ec023",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera_uuid": "uscold/laredo/dock03/cha",
            "ray": {"dashboard": {"port": 5002}},
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/uscold-laredo-0002/1651957479094",
                "version": 4,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "perception": {
                "enabled": True,
                "acausal_layer": {
                    "PPESmoothener": {
                        "safety_vest_time_interval_ms": 10000,
                        "pose_points_min_confidence_threshold": 0.5,
                    }
                },
                "detector_tracker": {
                    "model_path": "artifacts_2021-12-06-00-00-00-0000-yolo/best_608_1024.engine",
                    "vest_model_path": "artifacts_vest_classifier-9-19/vest_classifier_resnet18_traced_model.pth",
                    "height": 608,
                    "width": 1024,
                    "actor2class": {
                        "PERSON": 0,
                        "PIT": 1,
                        "HARD_HAT": 2,
                        "SAFETY_VEST": 3,
                    },
                    "should_do_temporal_propagation": 0,
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "lift_classifier": {
                    "enabled": False,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "door_classifier": {
                    "enabled": False,
                    "model_path": "artifacts_door_classifier_resnet50_dataset4/voxel_door_classifier_dataset4_resnet50_traced_model.pth",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit-uscold-laredo_2022-09-15/voxel_safetyvest_vit_laredo-2_laredo_2022-09-15.pth",
                    "model_type": "Transformers",
                },
                "hat_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_hard_hat_occlusion_vit_v4_2022-11-18-jit/hard_hat_occlusion_vit_v4_2022-11-18-jit.pt",
                    "is_classification_by_detection": False,
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "no_ped_zone": {
                        "per_camera_cooldown_s": 120,
                        "max_ped_zone_duration_s": 120,
                    },
                    "hard_hat": {"per_camera_cooldown_s": 120},
                },
                "monitors_requested": [],
                "incident_machine_enabled": False,
                "state_machine_monitors_requested": [
                    "safety_vest",
                    "hard_hat",
                    "no_ped_zone",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "uscold/laredo/dock03/cha/43066680-c347-4746-8f01-e63c00b6a0f0_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "uscold/quakertown/0001/cha",
        "incidents": ["HARD_HAT"],
        "video_uuid": "uscold/quakertown/0001/cha/3bf86c4c-5c8f-492f-bbba-207410a61216_0000",
        "inferred_incidents": [
            Incident(
                uuid="3a05d63a-9db7-425c-afef-689c73eb298b",
                camera_uuid="uscold/quakertown/0001/cha",
                camera_config_version=None,
                organization_key=None,
                title="Hard hat",
                incident_type_id="HARD_HAT",
                incident_version="1.0",
                priority="low",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897419630,
                end_frame_relative_ms=1678897423196,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=9000,
                post_end_buffer_ms=6000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="1c9b73c9-4569-4c44-bb31-4ad979ea4612",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/uscold-quakertown-0001/1660236732478",
                "version": 8,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "uscold/quakertown/0001/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "hard_hat": {"max_no_hat_s": 3},
                    "parking": {"max_parked_duration_s": 300},
                    "safety_vest": {"per_camera_cooldown_s": 900},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "hard_hat",
                    "overreaching",
                    "parking",
                    "safety_vest",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "enabled": True,
                "acausal_layer": {
                    "PPESmoothener": {
                        "pose_points_min_confidence_threshold": 0.5
                    }
                },
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-10-04-18-20-08-f6eb-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {
                    "enabled": True,
                    "is_classification_by_detection": False,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_hard_hat_occlusion_vit_v4_2022-11-18-jit/hard_hat_occlusion_vit_v4_2022-11-18-jit.pt",
                },
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit_quakertown_laredo_2022-08-24/voxel_safetyvest_vit_quakertown_laredo_2022-08-24.pth",
                    "model_type": "Transformers",
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "uscold/quakertown/0001/cha/3bf86c4c-5c8f-492f-bbba-207410a61216_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "uscold/quakertown/0002/cha",
        "incidents": ["HARD_HAT", "HARD_HAT"],
        "video_uuid": "uscold/quakertown/0002/cha/9317ad37-29e5-498e-82af-0aa2fa294324_0000",
        "inferred_incidents": [
            Incident(
                uuid="c73c3361-b192-4394-a5df-6d890896506f",
                camera_uuid="uscold/quakertown/0002/cha",
                camera_config_version=None,
                organization_key=None,
                title="Hard hat",
                incident_type_id="HARD_HAT",
                incident_version="1.0",
                priority="low",
                actor_ids=["2"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897440129,
                end_frame_relative_ms=1678897444128,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=9000,
                post_end_buffer_ms=6000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="04152e58-331e-43c4-b3db-446aa175a751",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            ),
            Incident(
                uuid="b0092501-6277-463e-9090-7b2c73593271",
                camera_uuid="uscold/quakertown/0002/cha",
                camera_config_version=None,
                organization_key=None,
                title="Hard hat",
                incident_type_id="HARD_HAT",
                incident_version="1.0",
                priority="low",
                actor_ids=["2"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897444128,
                end_frame_relative_ms=1678897453526,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=9000,
                post_end_buffer_ms=6000,
                docker_image_tag=None,
                cooldown_tag=True,
                track_uuid="04152e58-331e-43c4-b3db-446aa175a751",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            ),
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/uscold-quakertown-0002/1660237237942",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "uscold/quakertown/0002/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "hard_hat": {"max_no_hat_s": 3},
                    "open_door": {"max_open_door_s": 1200},
                    "parking": {"max_parked_duration_s": 300},
                    "safety_vest": {"per_camera_cooldown_s": 900},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "door_violation",
                    "hard_hat",
                    "open_door",
                    "overreaching",
                    "parking",
                    "safety_vest",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "enabled": True,
                "acausal_layer": {
                    "PPESmoothener": {
                        "pose_points_min_confidence_threshold": 0.5
                    }
                },
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-10-04-18-20-08-f6eb-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_2022-10-28_uscold_quakertown_0002_cha/2022-10-28_uscold_quakertown_0002_cha.pt",
                    "model_type": "vanilla_resnet",
                },
                "hat_classifier": {
                    "enabled": True,
                    "is_classification_by_detection": False,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_hard_hat_occlusion_vit_v4_2022-11-18-jit/hard_hat_occlusion_vit_v4_2022-11-18-jit.pt",
                },
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit_quakertown_laredo_2022-08-24/voxel_safetyvest_vit_quakertown_laredo_2022-08-24.pth",
                    "model_type": "Transformers",
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "uscold/quakertown/0002/cha/9317ad37-29e5-498e-82af-0aa2fa294324_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "uscold/quakertown/0003/cha",
        "incidents": ["HARD_HAT"],
        "video_uuid": "uscold/quakertown/0003/cha/961b7524-5937-4d31-a6d6-b2a910c175d0_0000",
        "inferred_incidents": [
            Incident(
                uuid="5181d27e-5e80-4202-a17d-d32d11805a67",
                camera_uuid="uscold/quakertown/0003/cha",
                camera_config_version=None,
                organization_key=None,
                title="Hard hat",
                incident_type_id="HARD_HAT",
                incident_version="1.0",
                priority="low",
                actor_ids=["3"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897453948,
                end_frame_relative_ms=1678897457147,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=9000,
                post_end_buffer_ms=6000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="ae62d6eb-05f7-405e-afa6-903b15efdde9",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/uscold-quakertown-0003/1660237262679",
                "version": 7,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "uscold/quakertown/0003/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "hard_hat": {"max_no_hat_s": 3},
                    "parking": {"max_parked_duration_s": 300},
                    "safety_vest": {"per_camera_cooldown_s": 900},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "hard_hat",
                    "overreaching",
                    "parking",
                    "safety_vest",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "enabled": True,
                "acausal_layer": {
                    "PPESmoothener": {
                        "pose_points_min_confidence_threshold": 0.5
                    }
                },
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-10-04-18-20-08-f6eb-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {
                    "enabled": True,
                    "is_classification_by_detection": False,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_hard_hat_occlusion_vit_v4_2022-11-18-jit/hard_hat_occlusion_vit_v4_2022-11-18-jit.pt",
                },
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit_quakertown_laredo_2022-08-24/voxel_safetyvest_vit_quakertown_laredo_2022-08-24.pth",
                    "model_type": "Transformers",
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "uscold/quakertown/0003/cha/961b7524-5937-4d31-a6d6-b2a910c175d0_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "uscold/quakertown/0007/cha",
        "incidents": ["HARD_HAT"],
        "video_uuid": "uscold/quakertown/0007/cha/b1ba73d0-c2ba-41b5-b615-228c781a585c_0000",
        "inferred_incidents": [
            Incident(
                uuid="79936428-d846-4942-9f16-e1975c2e5598",
                camera_uuid="uscold/quakertown/0007/cha",
                camera_config_version=None,
                organization_key=None,
                title="Hard hat",
                incident_type_id="HARD_HAT",
                incident_version="1.0",
                priority="low",
                actor_ids=["2"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897467657,
                end_frame_relative_ms=1678897473574,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=9000,
                post_end_buffer_ms=6000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="f59f5623-1220-4ff3-bbfe-7623a57b9032",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/uscold-quakertown-0007/1660237383976",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "uscold/quakertown/0007/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "hard_hat": {"max_no_hat_s": 3},
                    "parking": {"max_parked_duration_s": 300},
                    "safety_vest": {"per_camera_cooldown_s": 900},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "hard_hat",
                    "overreaching",
                    "parking",
                    "safety_vest",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "enabled": True,
                "acausal_layer": {
                    "PPESmoothener": {
                        "pose_points_min_confidence_threshold": 0.5
                    }
                },
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-10-04-18-20-08-f6eb-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {
                    "enabled": True,
                    "is_classification_by_detection": False,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_hard_hat_occlusion_vit_v4_2022-11-18-jit/hard_hat_occlusion_vit_v4_2022-11-18-jit.pt",
                },
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit_quakertown_laredo_2022-08-24/voxel_safetyvest_vit_quakertown_laredo_2022-08-24.pth",
                    "model_type": "Transformers",
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "uscold/quakertown/0007/cha/b1ba73d0-c2ba-41b5-b615-228c781a585c_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "uscold/quakertown/0007/cha",
        "incidents": ["HARD_HAT"],
        "video_uuid": "uscold/quakertown/0007/cha/0d4327dc-e6f5-468b-a888-6e9ed3556a65_0000",
        "inferred_incidents": [
            Incident(
                uuid="6b1ec0c2-70eb-45c2-8f10-3fd46c619681",
                camera_uuid="uscold/quakertown/0007/cha",
                camera_config_version=None,
                organization_key=None,
                title="Hard hat",
                incident_type_id="HARD_HAT",
                incident_version="1.0",
                priority="low",
                actor_ids=["15"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897474015,
                end_frame_relative_ms=1678897477367,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=9000,
                post_end_buffer_ms=6000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="63f24641-7850-47bc-ac53-f29313361ecf",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/uscold-quakertown-0007/1660237383976",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "uscold/quakertown/0007/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "hard_hat": {"max_no_hat_s": 3},
                    "parking": {"max_parked_duration_s": 300},
                    "safety_vest": {"per_camera_cooldown_s": 900},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "hard_hat",
                    "overreaching",
                    "parking",
                    "safety_vest",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "enabled": True,
                "acausal_layer": {
                    "PPESmoothener": {
                        "pose_points_min_confidence_threshold": 0.5
                    }
                },
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-10-04-18-20-08-f6eb-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {
                    "enabled": True,
                    "is_classification_by_detection": False,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_hard_hat_occlusion_vit_v4_2022-11-18-jit/hard_hat_occlusion_vit_v4_2022-11-18-jit.pt",
                },
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit_quakertown_laredo_2022-08-24/voxel_safetyvest_vit_quakertown_laredo_2022-08-24.pth",
                    "model_type": "Transformers",
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "uscold/quakertown/0007/cha/0d4327dc-e6f5-468b-a888-6e9ed3556a65_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "uscold/laredo/dock04/cha",
        "incidents": ["NO_PED_ZONE"],
        "video_uuid": "uscold/laredo/dock04/cha/4b6db807-6fe2-4ec4-ac76-bd9359ca5c1c_0000",
        "inferred_incidents": [
            Incident(
                uuid="0b2edf56-a138-4d15-b884-6a6a6a1be2d6",
                camera_uuid="uscold/laredo/dock04/cha",
                camera_config_version=None,
                organization_key=None,
                title="Pedestrian in No-Ped Zone",
                incident_type_id="NO_PED_ZONE",
                incident_version="1.0",
                priority="medium",
                actor_ids=["26"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897482987,
                end_frame_relative_ms=1678897622533,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="a45bda48-a91c-490e-b71e-5a4350385787",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera_uuid": "uscold/laredo/dock04/cha",
            "ray": {"dashboard": {"port": 5003}},
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/uscold-laredo-0003/1651957485510",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "perception": {
                "enabled": True,
                "acausal_layer": {
                    "PPESmoothener": {
                        "safety_vest_time_interval_ms": 10000,
                        "pose_points_min_confidence_threshold": 0.5,
                    }
                },
                "detector_tracker": {
                    "model_path": "artifacts_2021-12-06-00-00-00-0000-yolo/best_608_1024.engine",
                    "vest_model_path": "artifacts_vest_classifier-9-19/vest_classifier_resnet18_traced_model.pth",
                    "height": 608,
                    "width": 1024,
                    "actor2class": {
                        "PERSON": 0,
                        "PIT": 1,
                        "HARD_HAT": 2,
                        "SAFETY_VEST": 3,
                    },
                    "should_do_temporal_propagation": 0,
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "lift_classifier": {
                    "enabled": False,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "door_classifier": {
                    "enabled": False,
                    "model_path": "artifacts_door_classifier_resnet50_dataset4/voxel_door_classifier_dataset4_resnet50_traced_model.pth",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit-uscold-laredo_2022-09-15/voxel_safetyvest_vit_laredo-2_laredo_2022-09-15.pth",
                    "model_type": "Transformers",
                },
                "hat_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_hard_hat_occlusion_vit_v4_2022-11-18-jit/hard_hat_occlusion_vit_v4_2022-11-18-jit.pt",
                    "is_classification_by_detection": False,
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "no_ped_zone": {
                        "per_camera_cooldown_s": 120,
                        "max_ped_zone_duration_s": 120,
                    },
                    "hard_hat": {"per_camera_cooldown_s": 120},
                },
                "monitors_requested": [],
                "incident_machine_enabled": False,
                "state_machine_monitors_requested": [
                    "safety_vest",
                    "hard_hat",
                    "no_ped_zone",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "uscold/laredo/dock04/cha/4b6db807-6fe2-4ec4-ac76-bd9359ca5c1c_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "uscold/laredo/dock04/cha",
        "incidents": ["NO_PED_ZONE"],
        "video_uuid": "uscold/laredo/dock04/cha/e17529fb-8c06-4775-b5d7-87c89e01b07a_0000",
        "inferred_incidents": [
            Incident(
                uuid="6f6bc061-8455-4db1-9f9e-d7db04c7e38d",
                camera_uuid="uscold/laredo/dock04/cha",
                camera_config_version=None,
                organization_key=None,
                title="Pedestrian in No-Ped Zone",
                incident_type_id="NO_PED_ZONE",
                incident_version="1.0",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897500741,
                end_frame_relative_ms=1678897631199,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="5ea16609-d32a-4ffa-86c7-e75622e925a2",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera_uuid": "uscold/laredo/dock04/cha",
            "ray": {"dashboard": {"port": 5003}},
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/uscold-laredo-0003/1651957485510",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "perception": {
                "enabled": True,
                "acausal_layer": {
                    "PPESmoothener": {
                        "safety_vest_time_interval_ms": 10000,
                        "pose_points_min_confidence_threshold": 0.5,
                    }
                },
                "detector_tracker": {
                    "model_path": "artifacts_2021-12-06-00-00-00-0000-yolo/best_608_1024.engine",
                    "vest_model_path": "artifacts_vest_classifier-9-19/vest_classifier_resnet18_traced_model.pth",
                    "height": 608,
                    "width": 1024,
                    "actor2class": {
                        "PERSON": 0,
                        "PIT": 1,
                        "HARD_HAT": 2,
                        "SAFETY_VEST": 3,
                    },
                    "should_do_temporal_propagation": 0,
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "lift_classifier": {
                    "enabled": False,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "door_classifier": {
                    "enabled": False,
                    "model_path": "artifacts_door_classifier_resnet50_dataset4/voxel_door_classifier_dataset4_resnet50_traced_model.pth",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit-uscold-laredo_2022-09-15/voxel_safetyvest_vit_laredo-2_laredo_2022-09-15.pth",
                    "model_type": "Transformers",
                },
                "hat_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_hard_hat_occlusion_vit_v4_2022-11-18-jit/hard_hat_occlusion_vit_v4_2022-11-18-jit.pt",
                    "is_classification_by_detection": False,
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "no_ped_zone": {
                        "per_camera_cooldown_s": 120,
                        "max_ped_zone_duration_s": 120,
                    },
                    "hard_hat": {"per_camera_cooldown_s": 120},
                },
                "monitors_requested": [],
                "incident_machine_enabled": False,
                "state_machine_monitors_requested": [
                    "safety_vest",
                    "hard_hat",
                    "no_ped_zone",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "uscold/laredo/dock04/cha/e17529fb-8c06-4775-b5d7-87c89e01b07a_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "uscold/laredo/doors_14_20/cha",
        "incidents": ["NO_PED_ZONE"],
        "video_uuid": "uscold/laredo/doors_14_20/cha/84f791cb-d3c1-43c4-8169-cb930f3ed5df_0000",
        "inferred_incidents": [
            Incident(
                uuid="8910cdc2-d2c5-406b-bec4-37737d96b283",
                camera_uuid="uscold/laredo/doors_14_20/cha",
                camera_config_version=None,
                organization_key=None,
                title="Pedestrian in No-Ped Zone",
                incident_type_id="NO_PED_ZONE",
                incident_version="1.0",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897492204,
                end_frame_relative_ms=1678897612764,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="69c2f690-b6d8-4b0f-861d-ceaeb4616ee3",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera_uuid": "uscold/laredo/doors_14_20/cha",
            "ray": {"dashboard": {"port": 5004}},
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/uscold-laredo-0004/1651957564634",
                "version": 7,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "perception": {
                "enabled": True,
                "acausal_layer": {
                    "PPESmoothener": {
                        "safety_vest_time_interval_ms": 10000,
                        "pose_points_min_confidence_threshold": 0.5,
                    }
                },
                "detector_tracker": {
                    "model_path": "artifacts_2021-12-06-00-00-00-0000-yolo/best_608_1024.engine",
                    "vest_model_path": "artifacts_vest_classifier-9-19/vest_classifier_resnet18_traced_model.pth",
                    "height": 608,
                    "width": 1024,
                    "actor2class": {
                        "PERSON": 0,
                        "PIT": 1,
                        "HARD_HAT": 2,
                        "SAFETY_VEST": 3,
                    },
                    "should_do_temporal_propagation": 0,
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "lift_classifier": {
                    "enabled": False,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "door_classifier": {
                    "enabled": False,
                    "model_path": "artifacts_door_classifier_resnet50_dataset4/voxel_door_classifier_dataset4_resnet50_traced_model.pth",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit-uscold-laredo_2022-09-15/voxel_safetyvest_vit_laredo-2_laredo_2022-09-15.pth",
                    "model_type": "Transformers",
                },
                "hat_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_hard_hat_occlusion_vit_v4_2022-11-18-jit/hard_hat_occlusion_vit_v4_2022-11-18-jit.pt",
                    "is_classification_by_detection": False,
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "no_ped_zone": {
                        "per_camera_cooldown_s": 120,
                        "max_ped_zone_duration_s": 120,
                    },
                    "hard_hat": {"per_camera_cooldown_s": 120},
                },
                "monitors_requested": [],
                "incident_machine_enabled": False,
                "state_machine_monitors_requested": [
                    "safety_vest",
                    "hard_hat",
                    "no_ped_zone",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "uscold/laredo/doors_14_20/cha/84f791cb-d3c1-43c4-8169-cb930f3ed5df_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "uscold/laredo/doors_14_20/cha",
        "incidents": ["NO_PED_ZONE"],
        "video_uuid": "uscold/laredo/doors_14_20/cha/626f67a3-7c8f-4acc-9b16-7527014eaa1a_0000",
        "inferred_incidents": [
            Incident(
                uuid="17e8bb63-36ea-4576-8bd5-44fef490660c",
                camera_uuid="uscold/laredo/doors_14_20/cha",
                camera_config_version=None,
                organization_key=None,
                title="Pedestrian in No-Ped Zone",
                incident_type_id="NO_PED_ZONE",
                incident_version="1.0",
                priority="medium",
                actor_ids=["12"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897508150,
                end_frame_relative_ms=1678897701340,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="47fede34-5f4c-47f9-8f8d-4aff5afda838",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera_uuid": "uscold/laredo/doors_14_20/cha",
            "ray": {"dashboard": {"port": 5004}},
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/uscold-laredo-0004/1651957564634",
                "version": 7,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "perception": {
                "enabled": True,
                "acausal_layer": {
                    "PPESmoothener": {
                        "safety_vest_time_interval_ms": 10000,
                        "pose_points_min_confidence_threshold": 0.5,
                    }
                },
                "detector_tracker": {
                    "model_path": "artifacts_2021-12-06-00-00-00-0000-yolo/best_608_1024.engine",
                    "vest_model_path": "artifacts_vest_classifier-9-19/vest_classifier_resnet18_traced_model.pth",
                    "height": 608,
                    "width": 1024,
                    "actor2class": {
                        "PERSON": 0,
                        "PIT": 1,
                        "HARD_HAT": 2,
                        "SAFETY_VEST": 3,
                    },
                    "should_do_temporal_propagation": 0,
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "lift_classifier": {
                    "enabled": False,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "door_classifier": {
                    "enabled": False,
                    "model_path": "artifacts_door_classifier_resnet50_dataset4/voxel_door_classifier_dataset4_resnet50_traced_model.pth",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit-uscold-laredo_2022-09-15/voxel_safetyvest_vit_laredo-2_laredo_2022-09-15.pth",
                    "model_type": "Transformers",
                },
                "hat_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_hard_hat_occlusion_vit_v4_2022-11-18-jit/hard_hat_occlusion_vit_v4_2022-11-18-jit.pt",
                    "is_classification_by_detection": False,
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "no_ped_zone": {
                        "per_camera_cooldown_s": 120,
                        "max_ped_zone_duration_s": 120,
                    },
                    "hard_hat": {"per_camera_cooldown_s": 120},
                },
                "monitors_requested": [],
                "incident_machine_enabled": False,
                "state_machine_monitors_requested": [
                    "safety_vest",
                    "hard_hat",
                    "no_ped_zone",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "uscold/laredo/doors_14_20/cha/626f67a3-7c8f-4acc-9b16-7527014eaa1a_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "uscold/laredo/dock03/cha",
        "incidents": ["NO_PED_ZONE"],
        "video_uuid": "uscold/laredo/dock03/cha/82e4e8c5-02c3-41ee-94d3-92594467576c_0000",
        "inferred_incidents": [
            Incident(
                uuid="97f4a355-c4fe-49d1-8c8d-d3ba08e008b0",
                camera_uuid="uscold/laredo/dock03/cha",
                camera_config_version=None,
                organization_key=None,
                title="Pedestrian in No-Ped Zone",
                incident_type_id="NO_PED_ZONE",
                incident_version="1.0",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897520427,
                end_frame_relative_ms=1678897663610,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="f6df61e9-b04e-454a-a5c5-af5dcb03ae80",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera_uuid": "uscold/laredo/dock03/cha",
            "ray": {"dashboard": {"port": 5002}},
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/uscold-laredo-0002/1651957479094",
                "version": 4,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "perception": {
                "enabled": True,
                "acausal_layer": {
                    "PPESmoothener": {
                        "safety_vest_time_interval_ms": 10000,
                        "pose_points_min_confidence_threshold": 0.5,
                    }
                },
                "detector_tracker": {
                    "model_path": "artifacts_2021-12-06-00-00-00-0000-yolo/best_608_1024.engine",
                    "vest_model_path": "artifacts_vest_classifier-9-19/vest_classifier_resnet18_traced_model.pth",
                    "height": 608,
                    "width": 1024,
                    "actor2class": {
                        "PERSON": 0,
                        "PIT": 1,
                        "HARD_HAT": 2,
                        "SAFETY_VEST": 3,
                    },
                    "should_do_temporal_propagation": 0,
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "lift_classifier": {
                    "enabled": False,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "door_classifier": {
                    "enabled": False,
                    "model_path": "artifacts_door_classifier_resnet50_dataset4/voxel_door_classifier_dataset4_resnet50_traced_model.pth",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit-uscold-laredo_2022-09-15/voxel_safetyvest_vit_laredo-2_laredo_2022-09-15.pth",
                    "model_type": "Transformers",
                },
                "hat_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_hard_hat_occlusion_vit_v4_2022-11-18-jit/hard_hat_occlusion_vit_v4_2022-11-18-jit.pt",
                    "is_classification_by_detection": False,
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "no_ped_zone": {
                        "per_camera_cooldown_s": 120,
                        "max_ped_zone_duration_s": 120,
                    },
                    "hard_hat": {"per_camera_cooldown_s": 120},
                },
                "monitors_requested": [],
                "incident_machine_enabled": False,
                "state_machine_monitors_requested": [
                    "safety_vest",
                    "hard_hat",
                    "no_ped_zone",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "uscold/laredo/dock03/cha/82e4e8c5-02c3-41ee-94d3-92594467576c_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "uscold/laredo/dock03/cha",
        "incidents": ["NO_PED_ZONE"],
        "video_uuid": "uscold/laredo/dock03/cha/6cb88de3-7a7a-445c-8136-163e59da5700_0000",
        "inferred_incidents": [
            Incident(
                uuid="995f9b49-0d54-455e-a000-c4843d498589",
                camera_uuid="uscold/laredo/dock03/cha",
                camera_config_version=None,
                organization_key=None,
                title="Pedestrian in No-Ped Zone",
                incident_type_id="NO_PED_ZONE",
                incident_version="1.0",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897568885,
                end_frame_relative_ms=1678897738563,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="865d6cfd-aa24-4f7e-af2b-dba03aa32dd5",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera_uuid": "uscold/laredo/dock03/cha",
            "ray": {"dashboard": {"port": 5002}},
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/uscold-laredo-0002/1651957479094",
                "version": 4,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "perception": {
                "enabled": True,
                "acausal_layer": {
                    "PPESmoothener": {
                        "safety_vest_time_interval_ms": 10000,
                        "pose_points_min_confidence_threshold": 0.5,
                    }
                },
                "detector_tracker": {
                    "model_path": "artifacts_2021-12-06-00-00-00-0000-yolo/best_608_1024.engine",
                    "vest_model_path": "artifacts_vest_classifier-9-19/vest_classifier_resnet18_traced_model.pth",
                    "height": 608,
                    "width": 1024,
                    "actor2class": {
                        "PERSON": 0,
                        "PIT": 1,
                        "HARD_HAT": 2,
                        "SAFETY_VEST": 3,
                    },
                    "should_do_temporal_propagation": 0,
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "lift_classifier": {
                    "enabled": False,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "door_classifier": {
                    "enabled": False,
                    "model_path": "artifacts_door_classifier_resnet50_dataset4/voxel_door_classifier_dataset4_resnet50_traced_model.pth",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit-uscold-laredo_2022-09-15/voxel_safetyvest_vit_laredo-2_laredo_2022-09-15.pth",
                    "model_type": "Transformers",
                },
                "hat_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_hard_hat_occlusion_vit_v4_2022-11-18-jit/hard_hat_occlusion_vit_v4_2022-11-18-jit.pt",
                    "is_classification_by_detection": False,
                },
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "incident": {
                "should_generate_on_cooldown": True,
                "incident_machine_params": {
                    "no_ped_zone": {
                        "per_camera_cooldown_s": 120,
                        "max_ped_zone_duration_s": 120,
                    },
                    "hard_hat": {"per_camera_cooldown_s": 120},
                },
                "monitors_requested": [],
                "incident_machine_enabled": False,
                "state_machine_monitors_requested": [
                    "safety_vest",
                    "hard_hat",
                    "no_ped_zone",
                    "spill",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "uscold/laredo/dock03/cha/6cb88de3-7a7a-445c-8136-163e59da5700_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "buildersfirstsource/dixieline/0007/cha",
        "incidents": ["NO_PED_ZONE"],
        "video_uuid": "buildersfirstsource/dixieline/0007/cha/b0514ddb-d071-4a8e-9966-a2b846f3e553_0000",
        "inferred_incidents": [
            Incident(
                uuid="99d621eb-e013-437c-9de8-184817aeef94",
                camera_uuid="buildersfirstsource/dixieline/0007/cha",
                camera_config_version=None,
                organization_key=None,
                title="Pedestrian in No-Ped Zone",
                incident_type_id="NO_PED_ZONE",
                incident_version="1.0",
                priority="medium",
                actor_ids=["4"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897695024,
                end_frame_relative_ms=1678897711432,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="073f8f6e-7ebf-4756-9800-1bcc3ca2961c",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/buildersfirstsource-dixieline-0007/1660685956922",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "buildersfirstsource/dixieline/0007/cha",
            "incident": {
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 900},
                    "safety_vest": {"per_camera_cooldown_s": 1800},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "no_ped_zone",
                    "overreaching",
                    "safety_vest",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-11-09-22-58-28-2d35-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit-dixieline_2022-09-16/voxel_safetyvest_vit_dixieline_2022-09-16.pth",
                    "model_type": "Transformers",
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "buildersfirstsource/dixieline/0007/cha/b0514ddb-d071-4a8e-9966-a2b846f3e553_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "buildersfirstsource/dixieline/0010/cha",
        "incidents": ["NO_PED_ZONE"],
        "video_uuid": "buildersfirstsource/dixieline/0010/cha/90310458-8cc5-4d83-92a6-ba90232a18d2_0000",
        "inferred_incidents": [
            Incident(
                uuid="bd5c5134-177e-4c07-ba7a-092baa9c7a6d",
                camera_uuid="buildersfirstsource/dixieline/0010/cha",
                camera_config_version=None,
                organization_key=None,
                title="Pedestrian in No-Ped Zone",
                incident_type_id="NO_PED_ZONE",
                incident_version="1.0",
                priority="medium",
                actor_ids=["1"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897689076,
                end_frame_relative_ms=1678897705487,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="005d5ebd-e63c-4811-a5dd-1d14ac3a8581",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/buildersfirstsource-dixieline-0010/1660685959139",
                "version": 3,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "buildersfirstsource/dixieline/0010/cha",
            "incident": {
                "incident_machine_params": {
                    "parking": {"max_parked_duration_s": 900},
                    "safety_vest": {"per_camera_cooldown_s": 1800},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "no_ped_zone",
                    "overreaching",
                    "safety_vest",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-11-09-22-58-28-2d35-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vit-dixieline_2022-09-16/voxel_safetyvest_vit_dixieline_2022-09-16.pth",
                    "model_type": "Transformers",
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "buildersfirstsource/dixieline/0010/cha/90310458-8cc5-4d83-92a6-ba90232a18d2_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "verst/walton/0005/cha",
        "incidents": ["NO_PED_ZONE"],
        "video_uuid": "verst/walton/0005/cha/0067abc0-0b17-4bc4-a3ca-49ae40a7fd00_0000",
        "inferred_incidents": [
            Incident(
                uuid="9b3fc035-ce83-4e0d-8533-e073cea9222c",
                camera_uuid="verst/walton/0005/cha",
                camera_config_version=None,
                organization_key=None,
                title="Pedestrian in No-Ped Zone",
                incident_type_id="NO_PED_ZONE",
                incident_version="1.0",
                priority="medium",
                actor_ids=["2"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897694662,
                end_frame_relative_ms=1678897705054,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="16515c1d-9a68-406a-b796-eaf017315ad8",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/verst-walton-0005/1663349395917",
                "version": 2,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "verst/walton/0005/cha",
            "incident": {
                "should_generate_on_cooldown": True,
                "monitors_requested": [],
                "incident_machine_params": {
                    "safety_vest": {"per_camera_cooldown_s": 1800}
                },
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "no_ped_zone",
                    "safety_vest",
                ],
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_2022-10-04-18-20-08-f6eb-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_type": "HM",
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {
                    "enabled": True,
                    "min_actor_pixel_area": 2000,
                    "model_path": "artifacts_voxel_safetyvest_vverst-walton_2022-10-06/voxel_safetyvest_vit_laredo_walton_2022-10-05.pth",
                    "model_type": "Transformers",
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "verst/walton/0005/cha/0067abc0-0b17-4bc4-a3ca-49ae40a7fd00_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "piston_automotive/marion/0002/cha",
        "incidents": ["NO_PED_ZONE"],
        "video_uuid": "piston_automotive/marion/0002/cha/5ea7d42b-56ff-494e-b658-c1e7a1f524d4_0000",
        "inferred_incidents": [
            Incident(
                uuid="97246ad4-6f48-4d50-8b25-abb9d4019218",
                camera_uuid="piston_automotive/marion/0002/cha",
                camera_config_version=None,
                organization_key=None,
                title="Pedestrian in No-Ped Zone",
                incident_type_id="NO_PED_ZONE",
                incident_version="1.0",
                priority="medium",
                actor_ids=["3"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897716652,
                end_frame_relative_ms=1678897719851,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="92e3cbe5-e562-4897-a274-3aeb1b8c3381",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/piston_automotive-marion-0002/1670373577339",
                "version": 6,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "piston_automotive/marion/0002/cha",
            "incident": {
                "incident_machine_params": {
                    "no_ped_zone": {"max_ped_zone_duration_s": 3}
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "no_ped_zone",
                    "overreaching",
                    "spill",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_01_18_2023_piston-automotive-yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {"enabled": False},
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "spill": {
                    "enabled": True,
                    "model_path": "artifacts_02_05_2023_spill_generalized_jit/smp-bd6b62dfd5b64dcfb4970102e2c9b2aa-jit.pt",
                    "min_run_time_difference_ms": 15000,
                    "min_pixel_size": 200,
                    "max_consecutive_runs": 1,
                    "post_process_enabled": True,
                    "frame_segment2class": {"UNKNOWN": 0, "SPILL": 1},
                },
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "temporal": {
                "max_frame_segments": 9,
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "piston_automotive/marion/0002/cha/5ea7d42b-56ff-494e-b658-c1e7a1f524d4_0000",
            "enable_video_writer": False,
        },
    },
    {
        "camera_uuid": "ulta/dallas/0004/cha",
        "incidents": ["NO_PED_ZONE"],
        "video_uuid": "ulta/dallas/0004/cha/3e8e5099-9332-4198-854f-b06a19174f3d_0000",
        "inferred_incidents": [
            Incident(
                uuid="bde3e8ef-2d1d-4462-afc6-a2bdd82687d0",
                camera_uuid="ulta/dallas/0004/cha",
                camera_config_version=None,
                organization_key=None,
                title="Pedestrian in No-Ped Zone",
                incident_type_id="NO_PED_ZONE",
                incident_version="1.0",
                priority="medium",
                actor_ids=["9"],
                video_thumbnail_gcs_path=None,
                video_gcs_path=None,
                original_video_gcs_path=None,
                annotations_gcs_path=None,
                video_thumbnail_s3_path=None,
                video_s3_path=None,
                original_video_s3_path=None,
                annotations_s3_path=None,
                start_frame_relative_ms=1678897722471,
                end_frame_relative_ms=1678897733207,
                incident_group_start_time_ms=None,
                pre_start_buffer_ms=10000,
                post_end_buffer_ms=10000,
                docker_image_tag=None,
                cooldown_tag=False,
                track_uuid="825e0570-325a-45dc-b306-fd93b1e8ee5f",
                sequence_id=None,
                run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
            )
        ],
        "config": {
            "camera": {
                "arn": "arn:aws:kinesisvideo:us-west-2:360054435465:stream/ulta-dallas-0004/1677204271161",
                "version": 5,
                "fps": 5,
                "min_frame_difference_ms": 85,
            },
            "camera_uuid": "ulta/dallas/0004/cha",
            "incident": {
                "incident_machine_params": {
                    "open_door": {"max_open_door_s": 2},
                    "parking": {"max_parked_duration_s": 300},
                },
                "monitors_requested": [],
                "state_machine_monitors_requested": [
                    "bad_posture",
                    "no_ped_zone",
                    "open_door",
                    "overreaching",
                    "parking",
                ],
                "should_generate_on_cooldown": True,
                "dry_run": True,
                "temp_directory": "/var/tmp/voxel/incidents",
                "generate_temp_subdirs": True,
            },
            "perception": {
                "detector_tracker": {
                    "actor2class": {"PERSON": 0, "PIT": 1},
                    "height": 736,
                    "model_path": "artifacts_02_27_2023_michaels_wesco_office_yolo/best_736_1280.engine",
                    "width": 1280,
                },
                "door_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_03_2023_ulta_dallas_0004_cha/eacec874-5f92-43b6-8eb8-3fb0162ed60d.pt",
                    "model_type": "vanilla_resnet",
                },
                "hat_classifier": {"enabled": False},
                "lift_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_lift_classifier_01282022/lift_classifer_01282022.sav",
                },
                "motion_zone_detection": {"enabled": False},
                "pose": {
                    "enabled": True,
                    "model_path": "artifacts_03_21_2023_pose_0630_jit_update/fast_res50_256x192.pt",
                },
                "reach_classifier": {
                    "enabled": True,
                    "model_path": "artifacts_03_23_2023_overreaching_model_jit/voxel_ergo_ml_overreaching_2022-05-23-jit.pt",
                    "model_type": "DL",
                },
                "vest_classifier": {"enabled": False},
                "enabled": True,
            },
            "publisher": {
                "organization_key": "VOXEL_SANDBOX",
                "enabled": False,
                "portal_host": "https://app.voxelai.com",
                "auth_token": "6bdeb3310546f9cae57c87604f80ece37ba6da43",
            },
            "state": {
                "add_epoch_time": True,
                "frames_to_buffer": 1000,
                "publisher": {
                    "batch_max_bytes": 1000000,
                    "batch_max_latency_seconds": 60,
                    "batch_max_message": 1000,
                    "retry_deadline_seconds": 600,
                    "state_topic": "projects/sodium-carving-227300/topics/voxel-internal-state-messages",
                    "event_topic": "projects/sodium-carving-227300/topics/voxel-internal-event-messages",
                    "emulator_host": None,
                },
                "enabled": False,
            },
            "temporal": {
                "expire_threshold_ms": 120000,
                "max_future_frames": 10,
                "max_past_frames": 60,
            },
            "video_stream": {"video_source_bucket": "voxel-logs"},
            "cache_key": "ed3c1d68-8ba9-46b2-8025-2344c1689218",
            "log_key": "",
            "run_uuid": "ea4336cd-2782-41b6-8399-77d0856003c4",
            "video_uuid": "ulta/dallas/0004/cha/3e8e5099-9332-4198-854f-b06a19174f3d_0000",
            "enable_video_writer": False,
        },
    },
]

In [4]:
detected_incidents = [
    [
        Incident(
            uuid="01295daa-2856-49e1-ad48-b1aebbeb47a6",
            camera_uuid="uscold/laredo/dock01/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["5"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896710332,
            end_frame_relative_ms=1678896710515,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="e854f330-b740-49fa-8ed6-c7bca3ccf207",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        ),
        Incident(
            uuid="41b988c2-137c-4a5f-b99b-a5f0490423f3",
            camera_uuid="uscold/laredo/dock01/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["5"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896710921,
            end_frame_relative_ms=1678896711114,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=True,
            track_uuid="e854f330-b740-49fa-8ed6-c7bca3ccf207",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        ),
        Incident(
            uuid="eb2a3f7d-b122-4e80-aa24-becf699a609f",
            camera_uuid="uscold/laredo/dock01/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["5"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896717871,
            end_frame_relative_ms=1678896718074,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=True,
            track_uuid="e854f330-b740-49fa-8ed6-c7bca3ccf207",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        ),
        Incident(
            uuid="e5b271a6-e6d7-45c1-b157-a298c47987a8",
            camera_uuid="uscold/laredo/dock01/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["5"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896718257,
            end_frame_relative_ms=1678896718869,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=True,
            track_uuid="e854f330-b740-49fa-8ed6-c7bca3ccf207",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        ),
    ],
    [
        Incident(
            uuid="39283309-a873-4653-8e15-c4b6ea98fe4a",
            camera_uuid="americold/modesto/0001/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["15"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896704305,
            end_frame_relative_ms=1678896704487,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="bf5f222c-b033-4478-a7f1-e10318664a85",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="8134e4ca-57a5-4459-a5dc-56f0f33864ee",
            camera_uuid="americold/modesto/0002/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["2"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896701930,
            end_frame_relative_ms=1678896702130,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="f91a6dc1-b92b-4690-91b5-5a12c89b098e",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        ),
        Incident(
            uuid="a931c019-496c-4b49-90f7-a6ff268ac4ae",
            camera_uuid="americold/modesto/0002/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["2"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896702501,
            end_frame_relative_ms=1678896702889,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=True,
            track_uuid="f91a6dc1-b92b-4690-91b5-5a12c89b098e",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        ),
    ],
    [
        Incident(
            uuid="4a95e29f-f780-4354-8a28-5ee66734535f",
            camera_uuid="americold/modesto/0002/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896703752,
            end_frame_relative_ms=1678896703972,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="e6b997b4-334a-4a08-9b5d-d6de04c3015f",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        ),
        Incident(
            uuid="88d600b6-1ccd-47ca-9441-3f265ba73fe4",
            camera_uuid="americold/modesto/0002/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896704152,
            end_frame_relative_ms=1678896704376,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=True,
            track_uuid="e6b997b4-334a-4a08-9b5d-d6de04c3015f",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        ),
        Incident(
            uuid="e6e93011-0302-415a-98b5-bea49dc75225",
            camera_uuid="americold/modesto/0002/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896704911,
            end_frame_relative_ms=1678896705122,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=True,
            track_uuid="e6b997b4-334a-4a08-9b5d-d6de04c3015f",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        ),
    ],
    [
        Incident(
            uuid="e18be288-4d9d-4e31-b1ce-16fc78bf7f04",
            camera_uuid="americold/modesto/0009/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896707730,
            end_frame_relative_ms=1678896707919,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="1d1d1d2f-75ea-4fa4-9482-24ba3df4ded4",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="93b2e80c-79ee-4ee0-abe3-7b16db78f0e5",
            camera_uuid="americold/modesto/0009/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["3"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896733043,
            end_frame_relative_ms=1678896733254,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="90ea3c38-0fdc-425f-9320-876bf9187ed2",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        ),
        Incident(
            uuid="39033d68-8686-4565-b949-7cfcd1085ceb",
            camera_uuid="americold/modesto/0009/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["3"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896738674,
            end_frame_relative_ms=1678896738804,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=True,
            track_uuid="90ea3c38-0fdc-425f-9320-876bf9187ed2",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        ),
    ],
    [
        Incident(
            uuid="86b649ce-4fd9-4650-b184-9c8e4c2142e7",
            camera_uuid="americold/modesto/0010/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["2"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896740828,
            end_frame_relative_ms=1678896741028,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="2be6d400-02b6-4c22-aa8a-4b6801c1b5a2",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="456bd334-6e83-488f-ba62-a279b1a65721",
            camera_uuid="americold/modesto/0010/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["2"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896741894,
            end_frame_relative_ms=1678896742094,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="4b184a77-e70e-4348-bc20-ce101ddf0518",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="f4dbc028-69db-45dc-a515-6db995a45755",
            camera_uuid="americold/modesto/0012/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["3"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896744707,
            end_frame_relative_ms=1678896744907,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="b87624d7-30db-41f5-ad93-b41181e977f7",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="e41d37d4-6e26-465d-9f2e-aa86eebcde02",
            camera_uuid="americold/modesto/0012/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["6"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896756521,
            end_frame_relative_ms=1678896756729,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="e1a0853b-2d1d-4a72-b933-49493f9de8ea",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="c1c99a16-126f-4fab-817b-638b1769898e",
            camera_uuid="americold/ontario/0003/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896769946,
            end_frame_relative_ms=1678896770342,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="3cecd0a6-d308-48bd-b7de-8ec535faa6e6",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        ),
        Incident(
            uuid="4e58758c-de56-4c8e-aa3d-10669189d33e",
            camera_uuid="americold/ontario/0003/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896770540,
            end_frame_relative_ms=1678896770738,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=True,
            track_uuid="3cecd0a6-d308-48bd-b7de-8ec535faa6e6",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        ),
    ],
    [
        Incident(
            uuid="9e5118a8-befe-445f-9e41-44296e6efe2f",
            camera_uuid="americold/ontario/0003/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896769648,
            end_frame_relative_ms=1678896769846,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="7148a839-3fb9-4542-b3fc-bd3fc0adfa87",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        ),
        Incident(
            uuid="9c619ccd-6bee-4390-bd59-052bd224c5b2",
            camera_uuid="americold/ontario/0003/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896777964,
            end_frame_relative_ms=1678896778162,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=True,
            track_uuid="7148a839-3fb9-4542-b3fc-bd3fc0adfa87",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        ),
    ],
    [
        Incident(
            uuid="9bf691f1-20d2-49f2-ba7e-f0d368dd82ee",
            camera_uuid="americold/ontario/0005/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896772380,
            end_frame_relative_ms=1678896772578,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="62ed9d7d-0c0e-4747-8b33-64c991b41a6b",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="40c685ee-664f-47b9-a3cc-224770b03fd4",
            camera_uuid="americold/ontario/0005/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["2"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896792693,
            end_frame_relative_ms=1678896793089,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="34f5f1a5-e178-450b-960b-d0fa8192facd",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="77020290-37cc-496a-9245-ea058a134f0f",
            camera_uuid="americold/ontario/0006/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896805730,
            end_frame_relative_ms=1678896805928,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="3385ae26-536f-4c34-b2b4-f43da6482729",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        ),
        Incident(
            uuid="367a0c6c-63e5-4c2e-a5f6-f1734a05384d",
            camera_uuid="americold/ontario/0006/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896806720,
            end_frame_relative_ms=1678896806918,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=True,
            track_uuid="3385ae26-536f-4c34-b2b4-f43da6482729",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        ),
    ],
    [
        Incident(
            uuid="926a59ca-b79e-4aa1-b042-a397a73da4ea",
            camera_uuid="americold/ontario/0009/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["4"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896821400,
            end_frame_relative_ms=1678896821600,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="9cb44739-6933-4338-aecd-547d55499f92",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="baecfb41-d1f6-4f6a-aa1b-ef2bf96374ae",
            camera_uuid="americold/ontario/0009/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896824408,
            end_frame_relative_ms=1678896824608,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="5808dad1-47b1-47b9-8d19-3e6f534f9ece",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="c748e8e7-0d9a-4069-be77-327a952ddc19",
            camera_uuid="americold/ontario/0011/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896810265,
            end_frame_relative_ms=1678896810465,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="b6ff7c57-1ee2-4a9a-b394-5b659ee710d5",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="81340e24-aac3-4f7c-ad09-5248be90598f",
            camera_uuid="americold/ontario/0011/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["13"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896839939,
            end_frame_relative_ms=1678896840539,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="8cca5ae4-01f3-41a7-82e2-008f2bc49024",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="2b51e446-82c7-4407-8483-b83833e2748a",
            camera_uuid="americold/ontario/0012/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896841745,
            end_frame_relative_ms=1678896842145,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="1a9b7a41-39b5-4208-aa6e-e68dcd1fa2a2",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        ),
        Incident(
            uuid="4f68675a-7b72-41e9-bae7-917ef441a1bf",
            camera_uuid="americold/ontario/0012/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896857545,
            end_frame_relative_ms=1678896857945,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=True,
            track_uuid="1a9b7a41-39b5-4208-aa6e-e68dcd1fa2a2",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        ),
        Incident(
            uuid="76cfec47-094c-44a0-b7bb-baa72ff2f18e",
            camera_uuid="americold/ontario/0012/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896859145,
            end_frame_relative_ms=1678896859345,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=True,
            track_uuid="1a9b7a41-39b5-4208-aa6e-e68dcd1fa2a2",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        ),
    ],
    [
        Incident(
            uuid="10a6fb5e-1eb9-4b7e-ac3b-6e3915d41c3b",
            camera_uuid="uscold/quakertown/0003/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["6"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896873882,
            end_frame_relative_ms=1678896874117,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="8a9b6cb2-6da3-4421-803e-ce71dbf3cbe2",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        ),
        Incident(
            uuid="58d4753b-7711-4dfb-8909-62a421d054ac",
            camera_uuid="uscold/quakertown/0003/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["6"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896875117,
            end_frame_relative_ms=1678896875282,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=True,
            track_uuid="8a9b6cb2-6da3-4421-803e-ce71dbf3cbe2",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        ),
    ],
    [
        Incident(
            uuid="e3b1eafc-c533-42e7-af71-ac2216d72e65",
            camera_uuid="uscold/quakertown/0003/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["6"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896875138,
            end_frame_relative_ms=1678896875371,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="ebf841ac-1903-4347-b6bb-41657ca5efde",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="54e42cc8-e2c5-4e37-bd4a-59ed0bbc2c08",
            camera_uuid="uscold/quakertown/0006/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["7"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896869232,
            end_frame_relative_ms=1678896869428,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="d214fc83-c155-4255-a1b3-2cf18433439d",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="35c1535b-e07c-4a49-8d50-af9b8845a41e",
            camera_uuid="uscold/quakertown/0006/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896889854,
            end_frame_relative_ms=1678896890053,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="32f9ad59-9981-40d6-ae6f-afd7a1d47b6c",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="6e8a0f8c-0f6d-4685-bf5c-97e945f7b8cd",
            camera_uuid="uscold/quakertown/0008/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["21"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896899436,
            end_frame_relative_ms=1678896899634,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="df040723-53ac-4a04-98f8-e7b73e580feb",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="c6c2c02f-01d5-470d-9eea-103a62c73689",
            camera_uuid="buildersfirstsource/solanabeach/0001/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896901287,
            end_frame_relative_ms=1678896901487,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="f5520b16-c08f-49a7-a01d-129c874ced04",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="e078e4f7-f937-42e7-ad55-f9e48a906f49",
            camera_uuid="buildersfirstsource/solanabeach/0002/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["5"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896905835,
            end_frame_relative_ms=1678896906035,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="c7e16665-df01-4f4e-9f74-273efcb16878",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="3ed2aa65-ffd3-4e6f-9649-fffb037a2ee3",
            camera_uuid="buildersfirstsource/solanabeach/0005/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["4"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896918014,
            end_frame_relative_ms=1678896918214,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="985e1234-b156-4587-91bc-419152842fa2",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="69a02e12-6f62-4c27-89df-a6b67da3a237",
            camera_uuid="buildersfirstsource/solanabeach/0006/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896933686,
            end_frame_relative_ms=1678896933886,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="21cccf24-b80a-41d0-afb0-ef694297cc50",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="a5f1f4b3-834d-48fc-89cc-e61651494061",
            camera_uuid="buildersfirstsource/dixieline/0010/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["4"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896941715,
            end_frame_relative_ms=1678896941915,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="1f1b6952-3ec9-4809-9272-71bf29fe631b",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="9715694f-0506-414e-8c6b-80cec6aaf525",
            camera_uuid="trieagle/ocala/0001/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896935780,
            end_frame_relative_ms=1678896935880,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="d6f25819-2bb4-4447-9cdd-7a9a861e64dc",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="4d2f217e-4662-462f-abc5-83119b897eda",
            camera_uuid="trieagle/ocala/0001/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["9"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896941278,
            end_frame_relative_ms=1678896941544,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="5c58074d-ca9d-48ca-9829-6d565a9e07da",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="f8a80d34-ce06-44dd-88f6-6ea74884ec51",
            camera_uuid="trieagle/ocala/0003/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["3"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896942986,
            end_frame_relative_ms=1678896943086,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="5c982b83-ec68-49cf-bd69-a5aa028c403b",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="b8b5e05c-716f-4c78-aace-06cf32fe6651",
            camera_uuid="trieagle/ocala/0004/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["3"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896963896,
            end_frame_relative_ms=1678896963996,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="2ba57610-5d3b-44b4-807e-899656635c96",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="a624c779-5cc8-487f-bf7f-6d3f095e111a",
            camera_uuid="trieagle/ocala/0007/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896979409,
            end_frame_relative_ms=1678896979676,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="f640c66d-1dc6-49ac-8b84-25f698624c9a",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="6ff1d17b-313c-4894-97ec-f0bd7d595e8d",
            camera_uuid="trieagle/ocala/0008/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["2"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896979759,
            end_frame_relative_ms=1678896979993,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="2a158de0-85ae-48b9-a426-1cf90920654a",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="86d7c197-0c7c-49e0-900d-f2a758256ea6",
            camera_uuid="verst/walton/0003/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["5"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896981027,
            end_frame_relative_ms=1678896981217,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="ba5ad7d9-304c-487e-ba4d-4f7dc1a86f03",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="8d986c36-af4c-4a8b-adbb-aa98bad0d054",
            camera_uuid="verst/walton/0004/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["2"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896987626,
            end_frame_relative_ms=1678896987787,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="b9712c0e-26ea-40fa-9149-d2db63529e07",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="fb428d53-2c3f-4e40-a893-2d6bff02e945",
            camera_uuid="verst/walton/0004/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["7"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678896997979,
            end_frame_relative_ms=1678896998149,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="f7d27ca0-bafb-4fea-9563-f6437dc799b0",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="e9b15d22-853e-46f0-9ed3-5209dd2c7a1f",
            camera_uuid="verst/walton/0005/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["9"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897016735,
            end_frame_relative_ms=1678897016935,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="97877c38-c412-4571-872c-1fea8808318c",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="0a8431b1-8a67-4262-9b33-6fe731731157",
            camera_uuid="verst/walton/0005/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["10"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897013236,
            end_frame_relative_ms=1678897013436,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="daf899dd-3fd3-4754-8f3c-6d8c47da21da",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="5a6ce2e1-790d-489e-8390-46478f0e2b0d",
            camera_uuid="verst/walton/0011/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897014964,
            end_frame_relative_ms=1678897015165,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="de87ebf8-5bcf-4997-93ce-adaae376ff1f",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="64cd50a7-6a43-4654-9707-8294d5838b9d",
            camera_uuid="verst/walton/0011/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["2"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897021141,
            end_frame_relative_ms=1678897021530,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="93940a7e-853b-4698-94b8-04e3b341a0be",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="4d889043-8354-4a72-8fd2-628bc353aaec",
            camera_uuid="wn_foods/hayward/0006/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["3"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897039783,
            end_frame_relative_ms=1678897039982,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="9e1f84fe-79c6-4467-ae5b-efde6b7de643",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="1771dcc9-f8a1-495e-af1c-577f9217c246",
            camera_uuid="wn_foods/hayward/0006/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["6"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897048709,
            end_frame_relative_ms=1678897048909,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="c6cfd185-3532-4846-8dc7-7699c2166d64",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="6d6b8fe8-cd5e-4093-8361-06b0f2a54ddd",
            camera_uuid="wn_foods/hayward/0007/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["3"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897049919,
            end_frame_relative_ms=1678897050325,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="88105cf8-2f19-4d72-bef6-c649a2553443",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="dae6dd7e-8878-4e80-b159-224219ea72b9",
            camera_uuid="wn_foods/hayward/0009/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897065028,
            end_frame_relative_ms=1678897065595,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="fdf17d60-e1ef-4800-840c-43d20ca13de5",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="88ff3243-4ba1-438b-83f4-1893f4e73d62",
            camera_uuid="americold/tacoma/0001/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897057170,
            end_frame_relative_ms=1678897057337,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="e9803770-8152-42c0-9187-c487e7895695",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="b5f1cd2d-2180-40db-922d-c5e722184e99",
            camera_uuid="americold/tacoma/0001/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897073065,
            end_frame_relative_ms=1678897073265,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="7e24542d-2073-4653-9890-2045882b15a6",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="213313ac-08b8-4c54-a482-2adbf0b59cb5",
            camera_uuid="americold/tacoma/0002/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897079283,
            end_frame_relative_ms=1678897079483,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="5abd39ea-eb65-4703-bc9e-058f638179eb",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="ec27aa48-9ad2-4372-9f8a-a9060c9d71f5",
            camera_uuid="americold/tacoma/0002/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["19"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897086326,
            end_frame_relative_ms=1678897086526,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="a2829a8f-b17a-46e9-832c-098c56975f30",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="0e8f2abc-2a5d-40a8-8604-94ab33af4587",
            camera_uuid="americold/tacoma/0003/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["9"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897087784,
            end_frame_relative_ms=1678897087984,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="17d883cb-bc80-44c8-9590-c70cc8a2a0ab",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="56050bea-108f-4692-abb0-0288944147e4",
            camera_uuid="americold/tacoma/0003/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["13"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897100006,
            end_frame_relative_ms=1678897100206,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="66a9fb4e-33f7-48f2-8cb8-8ae41eb5448e",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="8e79cbb4-3319-41f9-82cd-b49fde877634",
            camera_uuid="americold/tacoma/0004/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897106690,
            end_frame_relative_ms=1678897106890,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="b319914c-f2ff-4e48-aca8-04ef4828ecd4",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="952b5e25-e35c-415e-a255-248023501173",
            camera_uuid="americold/tacoma/0005/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["2"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897106061,
            end_frame_relative_ms=1678897106261,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="7152832b-9327-4741-9174-e44a168d40b0",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="42311060-a376-41b9-ba97-4052bcac5416",
            camera_uuid="americold/tacoma/0005/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["4"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897114326,
            end_frame_relative_ms=1678897114526,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="bff45934-f914-4e66-9b52-b2331b79246c",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="1c03996b-9347-4043-8a22-82239d6f193d",
            camera_uuid="americold/tacoma/0006/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["6"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897114286,
            end_frame_relative_ms=1678897114486,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="205ac707-a2bf-4b1e-9979-b57221209eda",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="239788ac-bcae-4f1b-a9e8-7c921db93250",
            camera_uuid="americold/tacoma/0007/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897129470,
            end_frame_relative_ms=1678897129670,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="152421a9-913c-4f23-a549-9363f52ebe83",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="5734d1ea-bb12-4f92-bbcb-35b38bf6f91b",
            camera_uuid="americold/tacoma/0007/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["4"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897136813,
            end_frame_relative_ms=1678897137013,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="8c27cb94-53ca-4ad9-b97d-fa50e42ece1e",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="8a36100e-0ec8-447b-9817-b79df9914d92",
            camera_uuid="ppg/cedar_falls/0003/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["2"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897150884,
            end_frame_relative_ms=1678897151151,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="4c8e295c-57db-4343-93fa-8c965ca53e4d",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="c534f72b-a397-43a3-b7dc-8587a229e25d",
            camera_uuid="ppg/cedar_falls/0003/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["2"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897159098,
            end_frame_relative_ms=1678897159263,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="f98d308b-df59-4524-a93c-15c502838472",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="01cfa58a-e969-4d13-87ae-e02543121323",
            camera_uuid="ppg/cedar_falls/0004/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["10"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897157940,
            end_frame_relative_ms=1678897158175,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="c3162670-6ef3-450c-ba22-a1892cf15be0",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="2ffed215-49c4-4a64-8372-28fc014e6656",
            camera_uuid="ppg/cedar_falls/0004/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["4"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897161186,
            end_frame_relative_ms=1678897161452,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="516213c2-e69d-4fb7-9a0b-ce0a333ae8ab",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="c48d84b7-6bf3-4d7e-a10e-e06afac9fa6f",
            camera_uuid="ppg/cedar_falls/0005/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["6"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897173113,
            end_frame_relative_ms=1678897173279,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="a3e913fc-444c-4fa7-9f6c-8ad429d86c89",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="6d63a629-ce22-40ff-a388-edbc5ced50d6",
            camera_uuid="ppg/cedar_falls/0005/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897182347,
            end_frame_relative_ms=1678897182614,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="a55e44f2-e9ed-451d-8fc3-a5fba114f363",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="b823f8f8-6711-4132-a351-bc25673fb953",
            camera_uuid="ppg/cedar_falls/0007/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["15"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897187591,
            end_frame_relative_ms=1678897187758,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="78c82145-ee1c-4440-a1a4-48ca09a7f28c",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="19d422d8-252c-43a4-965d-5106bab6bc6d",
            camera_uuid="ppg/cedar_falls/0007/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897184320,
            end_frame_relative_ms=1678897184753,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="9a306a19-1d42-48d6-bb10-efc589c4a077",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="9860b341-0ddd-44ba-8d8c-9e6bdfeb04ba",
            camera_uuid="ppg/cedar_falls/0008/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["6"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897190380,
            end_frame_relative_ms=1678897190647,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="7f308ea5-1073-4c47-be81-20ede23ecf47",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="d4b6c01f-4629-4f55-a263-5061804559fa",
            camera_uuid="ppg/cedar_falls/0008/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["8"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897195679,
            end_frame_relative_ms=1678897195846,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="21b3ce55-9439-4cbf-9ba9-c9b50053b4e2",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="373c9ee1-0e3c-4df1-8cc5-014710c6cc32",
            camera_uuid="piston_automotive/marion/0001/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["9"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897209938,
            end_frame_relative_ms=1678897210205,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="e833c89e-93eb-45ef-9ecc-32f9bad67271",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="f9aeb5f8-e21c-4bc8-899d-b734e4364412",
            camera_uuid="piston_automotive/marion/0001/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897225191,
            end_frame_relative_ms=1678897225291,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="54a30d8f-e973-455d-be07-4696f946fb0d",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="da60bbb6-0f7b-4603-84db-5b3772f2bbe5",
            camera_uuid="piston_automotive/marion/0002/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["5"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897224204,
            end_frame_relative_ms=1678897224337,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="7f135eb8-fe2c-4fb7-9342-7eb004f1ee26",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="34fe37fd-d12c-4403-8b62-85c6752d3be5",
            camera_uuid="piston_automotive/marion/0004/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["6"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897229050,
            end_frame_relative_ms=1678897229317,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="9f444df4-2e77-484d-8a7b-59db65cee5f9",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="a26fd5ca-a14b-44f3-a8f3-ca5a63c5199c",
            camera_uuid="piston_automotive/marion/0004/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["2"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897238309,
            end_frame_relative_ms=1678897238542,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="4c1d861a-91f6-436c-949b-15a8543d48af",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="c2fa9cc4-15b2-406e-9a9b-5991a5eb37a0",
            camera_uuid="piston_automotive/marion/0005/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897249143,
            end_frame_relative_ms=1678897249343,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="435820f0-a9c6-48bf-a250-0f844105a91a",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="7e4fd48f-2f19-4b7d-9278-d566a7380786",
            camera_uuid="piston_automotive/marion/0006/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["11"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897251243,
            end_frame_relative_ms=1678897251377,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="a3a3ac0a-ab46-4211-8189-bd51eef38144",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="808e8e61-8f5b-47d5-8ea9-eb5472c52e2a",
            camera_uuid="piston_automotive/marion/0007/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["12"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897251025,
            end_frame_relative_ms=1678897251125,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="e7fabf4b-cd48-4f7d-9a7a-d030da6a824b",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="b3768688-40f9-4904-b68f-79f7cf2709ad",
            camera_uuid="innovate_manufacturing/knoxville/0004/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["4"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897262638,
            end_frame_relative_ms=1678897262838,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="f443ea98-0828-40a2-8478-09c07968e61b",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="72158725-5444-434b-83ba-39592e6af3d4",
            camera_uuid="innovate_manufacturing/knoxville/0005/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897266138,
            end_frame_relative_ms=1678897266538,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="74f976b3-bc7e-4927-95b4-cf16e05c6669",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="309ffb64-38fc-4512-b1f7-b411472e8589",
            camera_uuid="innovate_manufacturing/knoxville/0006/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["9"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897272561,
            end_frame_relative_ms=1678897272761,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="13b54c8a-6eee-4d08-9ebd-fe1d2a07bbab",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="483cec37-7cdd-4ee7-8810-37d4ec977e7e",
            camera_uuid="innovate_manufacturing/knoxville/0007/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["8"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897285997,
            end_frame_relative_ms=1678897286197,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="fbc5fe74-909c-4f37-b765-99cd06ab83f8",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="315b1c07-3e2f-4997-afe7-f1007c34b22a",
            camera_uuid="innovate_manufacturing/knoxville/0010/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897290952,
            end_frame_relative_ms=1678897291152,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="13876e20-ae8e-4e64-844d-6e4aca35277d",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="946624a8-990c-491d-93fb-1714be3fff35",
            camera_uuid="innovate_manufacturing/knoxville/0014/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["3"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897295580,
            end_frame_relative_ms=1678897295780,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="bdbed8d0-77d8-4781-8e8f-0ae23da9a49b",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="7e8169e1-7c34-4c8c-a36f-90e8eb10fd43",
            camera_uuid="wesco/reno/0001/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897302189,
            end_frame_relative_ms=1678897302389,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="b3638ed5-7139-4347-9442-aaa76c3404b1",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="b6dc3261-2aad-47a8-8291-88eb737a9dde",
            camera_uuid="wesco/reno/0001/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897311463,
            end_frame_relative_ms=1678897311863,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="e7b0f40c-e805-413b-b353-0fefb633e08c",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="4eb2013c-8cd9-40e3-9507-50d2598aba44",
            camera_uuid="wesco/reno/0004/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["2"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897312308,
            end_frame_relative_ms=1678897312558,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="7ad9f5dc-1144-4b22-be9f-a4c494ca2bc6",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="58d9fd0a-e3b0-4deb-b00d-6849b2a8f010",
            camera_uuid="wesco/reno/0005/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897310863,
            end_frame_relative_ms=1678897311196,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="8397f9bd-d598-465c-a1f1-b5ee75786d2f",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="ca300bfd-5c5e-482f-a391-96220be2c13a",
            camera_uuid="wesco/reno/0005/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897316883,
            end_frame_relative_ms=1678897317133,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="e5d7e8b8-545f-404e-baa1-8fb63a786649",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="24ea2c42-2aca-4c42-a291-1b64c61a88c2",
            camera_uuid="wesco/reno/0011/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897338815,
            end_frame_relative_ms=1678897339014,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="97f1abbf-373c-4c21-b9c7-a12c30977273",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="60261910-bfe5-4a0e-be97-e67d03858ae8",
            camera_uuid="wesco/reno/0015/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["2"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897337883,
            end_frame_relative_ms=1678897338083,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="420b0b05-2951-42f2-afa4-0a99885d0275",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="5ab36c87-b22a-498b-9d09-1f97d748118f",
            camera_uuid="wesco/reno/0016/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897338295,
            end_frame_relative_ms=1678897338495,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="6864257d-9cd1-4032-8dcf-f5113db527ad",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="37193a23-bf7c-4dbf-807b-cb530142236d",
            camera_uuid="wesco/reno/0016/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["2"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897344654,
            end_frame_relative_ms=1678897344854,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="619150ef-d119-48ca-8f03-b1d115dc844f",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="3e0de00c-4099-45a3-b532-40bab1dde578",
            camera_uuid="michaels/tracy/0005/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["5"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897351554,
            end_frame_relative_ms=1678897351804,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="ed6be700-5fd8-4d08-a80d-d8717fe11b30",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="c986ad3c-c88d-4244-9411-39b4a1ecac80",
            camera_uuid="ulta/dallas/0010/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897361992,
            end_frame_relative_ms=1678897362257,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="88213fca-d8ff-4ee7-a80b-de905ece3174",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="0b32782f-287e-4cad-b687-0db45d18efd4",
            camera_uuid="ulta/dallas/0011/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897362358,
            end_frame_relative_ms=1678897362591,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="07f8fcb2-8c2c-4bde-8545-ddd48d1d2209",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="cea7b3e3-85e7-49e2-9c69-eca65faa1bac",
            camera_uuid="ulta/dallas/0011/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["6"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897375587,
            end_frame_relative_ms=1678897375855,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="1d1da131-9ad2-49c9-b9ac-33dd05e5014a",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="b1b4cd5c-9afc-40c3-add3-d5564688bc32",
            camera_uuid="ulta/dallas/0012/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897386211,
            end_frame_relative_ms=1678897386811,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="e26fa0a5-467a-4e61-add2-b4b60fa1aa9f",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="b8f0ff00-f24e-4e30-9557-343e293f0cbf",
            camera_uuid="ulta/dallas/0012/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897381750,
            end_frame_relative_ms=1678897382015,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="66cf522b-d3a4-411a-bdc2-150f71ab79b5",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="091354fc-a136-418e-841c-3a4d8a283cec",
            camera_uuid="ulta/dallas/0015/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897384564,
            end_frame_relative_ms=1678897385464,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="af8fb25d-1efe-4589-87bd-808f616c04bd",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="4176ca85-bdeb-4a36-bee8-3fe74389f060",
            camera_uuid="ulta/dallas/0016/cha",
            camera_config_version=None,
            organization_key=None,
            title="Bad Posture",
            incident_type_id="BAD_POSTURE",
            incident_version="1.2",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897405030,
            end_frame_relative_ms=1678897405130,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="ec382bc8-79c4-4c22-8012-7302391d7372",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="0e962f84-9c6e-4ee5-a65f-e481d5773362",
            camera_uuid="uscold/laredo/dock04/cha",
            camera_config_version=None,
            organization_key=None,
            title="Hard hat",
            incident_type_id="HARD_HAT",
            incident_version="1.0",
            priority="low",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897423666,
            end_frame_relative_ms=1678897430064,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=9000,
            post_end_buffer_ms=6000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="f3e4eb82-50a8-4ef0-8494-9c0386233e8d",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="cf46294d-efca-4d09-a09a-56f01aac49cc",
            camera_uuid="uscold/laredo/doors_14_20/cha",
            camera_config_version=None,
            organization_key=None,
            title="Hard hat",
            incident_type_id="HARD_HAT",
            incident_version="1.0",
            priority="low",
            actor_ids=["40"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897419585,
            end_frame_relative_ms=1678897425584,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=9000,
            post_end_buffer_ms=6000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="04feb55b-5cf0-47c6-91ca-e39911b9a233",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="97a7f0b8-fa97-4883-beed-b1f501e7abfc",
            camera_uuid="uscold/laredo/dock03/cha",
            camera_config_version=None,
            organization_key=None,
            title="Hard hat",
            incident_type_id="HARD_HAT",
            incident_version="1.0",
            priority="low",
            actor_ids=["22"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897420116,
            end_frame_relative_ms=1678897430718,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=9000,
            post_end_buffer_ms=6000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="4ea9e61d-12c4-406d-aac7-d5dd353ec023",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="3a05d63a-9db7-425c-afef-689c73eb298b",
            camera_uuid="uscold/quakertown/0001/cha",
            camera_config_version=None,
            organization_key=None,
            title="Hard hat",
            incident_type_id="HARD_HAT",
            incident_version="1.0",
            priority="low",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897419630,
            end_frame_relative_ms=1678897423196,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=9000,
            post_end_buffer_ms=6000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="1c9b73c9-4569-4c44-bb31-4ad979ea4612",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="c73c3361-b192-4394-a5df-6d890896506f",
            camera_uuid="uscold/quakertown/0002/cha",
            camera_config_version=None,
            organization_key=None,
            title="Hard hat",
            incident_type_id="HARD_HAT",
            incident_version="1.0",
            priority="low",
            actor_ids=["2"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897440129,
            end_frame_relative_ms=1678897444128,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=9000,
            post_end_buffer_ms=6000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="04152e58-331e-43c4-b3db-446aa175a751",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        ),
        Incident(
            uuid="b0092501-6277-463e-9090-7b2c73593271",
            camera_uuid="uscold/quakertown/0002/cha",
            camera_config_version=None,
            organization_key=None,
            title="Hard hat",
            incident_type_id="HARD_HAT",
            incident_version="1.0",
            priority="low",
            actor_ids=["2"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897444128,
            end_frame_relative_ms=1678897453526,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=9000,
            post_end_buffer_ms=6000,
            docker_image_tag=None,
            cooldown_tag=True,
            track_uuid="04152e58-331e-43c4-b3db-446aa175a751",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        ),
    ],
    [
        Incident(
            uuid="5181d27e-5e80-4202-a17d-d32d11805a67",
            camera_uuid="uscold/quakertown/0003/cha",
            camera_config_version=None,
            organization_key=None,
            title="Hard hat",
            incident_type_id="HARD_HAT",
            incident_version="1.0",
            priority="low",
            actor_ids=["3"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897453948,
            end_frame_relative_ms=1678897457147,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=9000,
            post_end_buffer_ms=6000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="ae62d6eb-05f7-405e-afa6-903b15efdde9",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="79936428-d846-4942-9f16-e1975c2e5598",
            camera_uuid="uscold/quakertown/0007/cha",
            camera_config_version=None,
            organization_key=None,
            title="Hard hat",
            incident_type_id="HARD_HAT",
            incident_version="1.0",
            priority="low",
            actor_ids=["2"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897467657,
            end_frame_relative_ms=1678897473574,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=9000,
            post_end_buffer_ms=6000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="f59f5623-1220-4ff3-bbfe-7623a57b9032",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="6b1ec0c2-70eb-45c2-8f10-3fd46c619681",
            camera_uuid="uscold/quakertown/0007/cha",
            camera_config_version=None,
            organization_key=None,
            title="Hard hat",
            incident_type_id="HARD_HAT",
            incident_version="1.0",
            priority="low",
            actor_ids=["15"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897474015,
            end_frame_relative_ms=1678897477367,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=9000,
            post_end_buffer_ms=6000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="63f24641-7850-47bc-ac53-f29313361ecf",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="0b2edf56-a138-4d15-b884-6a6a6a1be2d6",
            camera_uuid="uscold/laredo/dock04/cha",
            camera_config_version=None,
            organization_key=None,
            title="Pedestrian in No-Ped Zone",
            incident_type_id="NO_PED_ZONE",
            incident_version="1.0",
            priority="medium",
            actor_ids=["26"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897482987,
            end_frame_relative_ms=1678897622533,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="a45bda48-a91c-490e-b71e-5a4350385787",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="6f6bc061-8455-4db1-9f9e-d7db04c7e38d",
            camera_uuid="uscold/laredo/dock04/cha",
            camera_config_version=None,
            organization_key=None,
            title="Pedestrian in No-Ped Zone",
            incident_type_id="NO_PED_ZONE",
            incident_version="1.0",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897500741,
            end_frame_relative_ms=1678897631199,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="5ea16609-d32a-4ffa-86c7-e75622e925a2",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="8910cdc2-d2c5-406b-bec4-37737d96b283",
            camera_uuid="uscold/laredo/doors_14_20/cha",
            camera_config_version=None,
            organization_key=None,
            title="Pedestrian in No-Ped Zone",
            incident_type_id="NO_PED_ZONE",
            incident_version="1.0",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897492204,
            end_frame_relative_ms=1678897612764,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="69c2f690-b6d8-4b0f-861d-ceaeb4616ee3",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="17e8bb63-36ea-4576-8bd5-44fef490660c",
            camera_uuid="uscold/laredo/doors_14_20/cha",
            camera_config_version=None,
            organization_key=None,
            title="Pedestrian in No-Ped Zone",
            incident_type_id="NO_PED_ZONE",
            incident_version="1.0",
            priority="medium",
            actor_ids=["12"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897508150,
            end_frame_relative_ms=1678897701340,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="47fede34-5f4c-47f9-8f8d-4aff5afda838",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="97f4a355-c4fe-49d1-8c8d-d3ba08e008b0",
            camera_uuid="uscold/laredo/dock03/cha",
            camera_config_version=None,
            organization_key=None,
            title="Pedestrian in No-Ped Zone",
            incident_type_id="NO_PED_ZONE",
            incident_version="1.0",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897520427,
            end_frame_relative_ms=1678897663610,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="f6df61e9-b04e-454a-a5c5-af5dcb03ae80",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="995f9b49-0d54-455e-a000-c4843d498589",
            camera_uuid="uscold/laredo/dock03/cha",
            camera_config_version=None,
            organization_key=None,
            title="Pedestrian in No-Ped Zone",
            incident_type_id="NO_PED_ZONE",
            incident_version="1.0",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897568885,
            end_frame_relative_ms=1678897738563,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="865d6cfd-aa24-4f7e-af2b-dba03aa32dd5",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="99d621eb-e013-437c-9de8-184817aeef94",
            camera_uuid="buildersfirstsource/dixieline/0007/cha",
            camera_config_version=None,
            organization_key=None,
            title="Pedestrian in No-Ped Zone",
            incident_type_id="NO_PED_ZONE",
            incident_version="1.0",
            priority="medium",
            actor_ids=["4"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897695024,
            end_frame_relative_ms=1678897711432,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="073f8f6e-7ebf-4756-9800-1bcc3ca2961c",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="bd5c5134-177e-4c07-ba7a-092baa9c7a6d",
            camera_uuid="buildersfirstsource/dixieline/0010/cha",
            camera_config_version=None,
            organization_key=None,
            title="Pedestrian in No-Ped Zone",
            incident_type_id="NO_PED_ZONE",
            incident_version="1.0",
            priority="medium",
            actor_ids=["1"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897689076,
            end_frame_relative_ms=1678897705487,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="005d5ebd-e63c-4811-a5dd-1d14ac3a8581",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="9b3fc035-ce83-4e0d-8533-e073cea9222c",
            camera_uuid="verst/walton/0005/cha",
            camera_config_version=None,
            organization_key=None,
            title="Pedestrian in No-Ped Zone",
            incident_type_id="NO_PED_ZONE",
            incident_version="1.0",
            priority="medium",
            actor_ids=["2"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897694662,
            end_frame_relative_ms=1678897705054,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="16515c1d-9a68-406a-b796-eaf017315ad8",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="97246ad4-6f48-4d50-8b25-abb9d4019218",
            camera_uuid="piston_automotive/marion/0002/cha",
            camera_config_version=None,
            organization_key=None,
            title="Pedestrian in No-Ped Zone",
            incident_type_id="NO_PED_ZONE",
            incident_version="1.0",
            priority="medium",
            actor_ids=["3"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897716652,
            end_frame_relative_ms=1678897719851,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="92e3cbe5-e562-4897-a274-3aeb1b8c3381",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
    [
        Incident(
            uuid="bde3e8ef-2d1d-4462-afc6-a2bdd82687d0",
            camera_uuid="ulta/dallas/0004/cha",
            camera_config_version=None,
            organization_key=None,
            title="Pedestrian in No-Ped Zone",
            incident_type_id="NO_PED_ZONE",
            incident_version="1.0",
            priority="medium",
            actor_ids=["9"],
            video_thumbnail_gcs_path=None,
            video_gcs_path=None,
            original_video_gcs_path=None,
            annotations_gcs_path=None,
            video_thumbnail_s3_path=None,
            video_s3_path=None,
            original_video_s3_path=None,
            annotations_s3_path=None,
            start_frame_relative_ms=1678897722471,
            end_frame_relative_ms=1678897733207,
            incident_group_start_time_ms=None,
            pre_start_buffer_ms=10000,
            post_end_buffer_ms=10000,
            docker_image_tag=None,
            cooldown_tag=False,
            track_uuid="825e0570-325a-45dc-b306-fd93b1e8ee5f",
            sequence_id=None,
            run_uuid="ea4336cd-2782-41b6-8399-77d0856003c4",
        )
    ],
]

In [17]:
from typing import List, Dict
from collections import Counter


def count_ground_truth_incidents(monitors_requested, ground_truth):
    ground_truth_incident_count = {}

    if ground_truth:
        for incident_type in ground_truth:
            if incident_type not in ground_truth_incident_count:
                ground_truth_incident_count[incident_type] = 1
            else:
                ground_truth_incident_count[incident_type] += 1
    else:
        for monitor in monitors_requested:
            if monitor in monitor_incident_map:
                ground_truth_incident_count[monitor_incident_map[monitor]] = 0

    return ground_truth_incident_count


def count_detected_incidents(
    monitors_requested: List[str],
    detected_incidents: List[Incident],
) -> Dict[str, int]:
    """Count incidents by incident type id

    Args:
        monitors_requested: A list of incident type ids that should have been monitored
        detected_incidents: A list of detected incidents of type Incident.

    Returns:
        A dictionary mapping incident type id to number of such incidents
        identified
    Raises:
        TypeError: if the detected incident is not of type Incident
    """
    detected_incident_count = Counter()

    # This takes care of the case where there are multiple requested incidents
    # but incidents aren't generated for all of them. Since we're using Counter,
    # we don't technically need to initialize the counts to 0, but this will
    # ensure calling .keys() on the return value gives all incident types.
    for monitor in monitors_requested:
        if monitor in monitor_incident_map:
            detected_incident_count[monitor_incident_map[monitor]] = 0

    if detected_incidents:
        for incident in detected_incidents:
            if not isinstance(incident, Incident):
                raise TypeError("incidents should all be of type Incident.")

            incident_type_id = incident.incident_type_id
            if incident.cooldown_tag is False:
                detected_incident_count[incident_type_id] += 1

    return detected_incident_count


monitor_incident_map = {
    "piggyback": "PIGGYBACK",
    "door_violation": "DOOR_VIOLATION",
    "bad_posture": "BAD_POSTURE",
    "door_intersection": "NO_STOP_AT_DOOR_INTERSECTION",
    "ergo2_bad_posture": "BAD_POSTURE",
    "hard_hat": "HARD_HAT",
    "safety_vest": "SAFETY_VEST",
    "parking": "PARKING_DURATION",
    "open_door": "OPEN_DOOR_DURATION",
    "no_stop_at_aisle_end": "NO_STOP_AT_END_OF_AISLE",
    "no_ped_zone": "NO_PED_ZONE",
    "intersection": "NO_STOP_AT_INTERSECTION",
    "overreaching": "OVERREACHING",
    "spill": "SPILL",
    "random_spill": "SPILL",
    "production_line_down": "PRODUCTION_LINE_DOWN",
    "high_vis_hat_or_vest": "HIGH_VIS_HAT_OR_VEST",
    "bump_cap": "BUMP_CAP",
    "bad_posture_with_uniform": "BAD_POSTURE_WITH_SAFETY_UNIFORM",
    "overreaching_with_uniform": "OVERREACHING_WITH_SAFETY_UNIFORM",
    "n_person_ped_zone": "N_PERSON_PED_ZONE",
}

In [24]:
for i, detected_incident in enumerate(detected_incidents):
    detected_incident = [
        inc.incident_type_id
        for inc in detected_incident
        if inc.cooldown_tag == False
    ]
    if scenarios[i]["incidents"] != detected_incident:
        print(
            f'{scenarios[i]["video_uuid"]} {scenarios[i]["incidents"]} {detected_incident}'
        )
#     else:
#         print(f"{scenarios[i]['incidents']} {detected_incident}True")

uscold/laredo/dock01/cha/d0d6bed5-3082-45b9-9638-4cb6c5167e15_0000 ['BAD_POSTURE', 'BAD_POSTURE', 'BAD_POSTURE', 'BAD_POSTURE'] ['BAD_POSTURE']
americold/modesto/0002/cha/a19dd508-2ad2-466e-828c-e7e9a0a0ed7f_0000 ['BAD_POSTURE', 'BAD_POSTURE'] ['BAD_POSTURE']
americold/modesto/0002/cha/fa132036-05a0-448d-adec-29671d765825_0000 ['BAD_POSTURE', 'BAD_POSTURE', 'BAD_POSTURE'] ['BAD_POSTURE']
americold/modesto/0009/cha/3f80116f-12d4-45f4-b5f0-8868994d3877_0000 ['BAD_POSTURE', 'BAD_POSTURE'] ['BAD_POSTURE']
americold/ontario/0003/cha/f83a3263-0b1b-444e-aeee-afbdea256486_0000 ['BAD_POSTURE', 'BAD_POSTURE'] ['BAD_POSTURE']
americold/ontario/0003/cha/0d6b743d-e058-4e4e-92f6-e8df8e9b8135_0000 ['BAD_POSTURE', 'BAD_POSTURE'] ['BAD_POSTURE']
americold/ontario/0006/cha/bf49adae-6913-4926-a318-debe5fd218fe_0000 ['BAD_POSTURE', 'BAD_POSTURE'] ['BAD_POSTURE']
americold/ontario/0012/cha/d8637b45-f45b-49d4-b514-48a8ffac37ed_0000 ['BAD_POSTURE', 'BAD_POSTURE', 'BAD_POSTURE'] ['BAD_POSTURE']
uscold/quakert

In [20]:
for i, scenario in enumerate(scenarios):
    monitors_requested = (
        scenario.get("config", {})
        .get("incident", {})
        .get("state_machine_monitors_requested", [])
    )
    print(monitors_requested)
    ground_truth = scenario["incidents"]
    print(ground_truth)
    # Count each type of ground truth incident
    ground_truth_incident_count = count_ground_truth_incidents(
        monitors_requested, ground_truth
    )
    print(ground_truth_incident_count)
    print([incident.incident_type_id for incident in detected_incidents[i]])
    detected_incident_count = count_detected_incidents(
        monitors_requested, detected_incidents[i]
    )
    print(detected_incident_count)
    break

['overreaching', 'spill', 'bad_posture']
['BAD_POSTURE', 'BAD_POSTURE', 'BAD_POSTURE', 'BAD_POSTURE']
{'BAD_POSTURE': 4}
['BAD_POSTURE', 'BAD_POSTURE', 'BAD_POSTURE', 'BAD_POSTURE']
Counter({'BAD_POSTURE': 1, 'OVERREACHING': 0, 'SPILL': 0})


In [ ]:
active_cameras = [
    camera["uuid"]
    for camera in json.loads(response.text)["data"]["cameras"]
    if camera["zone"]["isActive"]
]
print(len(active_cameras))
print(len(json.loads(response.text)["data"]["cameras"]))

In [ ]:
def _get_camera_incident_map(cameras):
    camera_incident_map = {}
    for camera_uuid in cameras:
        config = load_yaml_with_jinja(f"configs/cameras/{camera_uuid}.yaml")
        camera_incident_map[config["camera_uuid"]] = config["incident"][
            "state_machine_monitors_requested"
        ]
    return camera_incident_map


live_cameras_incident_map = _get_camera_incident_map(active_cameras)